<a href="https://colab.research.google.com/github/deltorobarba/sciences/blob/master/chemistry.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# <font color="blue">**Natural Sciences 🦋**

![ggg](https://raw.githubusercontent.com/deltorobarba/repo/master/sciences_0000.png)

### <font color="blue">**Chemische Elemente**

##### <font color="blue">*Periodensystem und Häufigkeit chemischer Elemente*

###### *Gruppen des Periodensystem*

![ggg](https://raw.githubusercontent.com/deltorobarba/repo/master/sciences_0678.jpg)

*Wegen ihrer ähnlichen chemischen Eigenschaften werden oft auch die Lanthanoide und die Actinoide als Gruppe betrachtet. Bei ihnen wird sukzessive das f-Orbital gefüllt.*

*Die noch nicht gefundenen Elemente mit der Ordnungszahl 122 bis 153 bilden eine Gruppe, die nach Glenn T. Seaborg Superactinoide genannt wird. Bei ihnen werden die 5g- und 6f-Orbitale aufgefüllt. Alle diese Elemente sind wahrscheinlich instabil und radioaktiv.*

*Siehe hier eine optische Trennung zwischen Hauptgruppen (A) und Nebengruppen (B nach CAS):*

![ggg](https://raw.githubusercontent.com/deltorobarba/repo/master/sciences_0677.jpg)

https://de.m.wikipedia.org/wiki/Gruppe_des_Periodensystems

###### *Elemente des Periodensystems (Gruppen und Perioden)*

![ggg](https://raw.githubusercontent.com/deltorobarba/repo/master/sciences_0749.PNG)

![ggg](https://raw.githubusercontent.com/deltorobarba/repo/master/sciences_0781.jpeg)

* **Chemische Elemente = Atomsorten**

* **Atome sind elektrisch neutral: Anzahl der Protonen und Elektronen ist gleich**

* **Gruppe**: lese senkrecht von oben nach unten: ahnliche chemische (Gruppen)Eigenschaften. zB Edelgase (8) sehr traege, und Halogene (7) sehr reaktionsfreudig. Gleiche Anzahl an Valenzelektronen (in äußerster Schale)

* **Periode**: lese waagerecht von links nach rechts: gleiche Anzahl an Orbitalen (Energieniveaus). Alle Elemente einer Periode haben dieselbe Anzahl an Atomorbitalen, wo die Elektronen sich befinden = Die Anzahl der Orbitale entspricht der Zahl der Periode. Es gibt sieben Reihen, was bedeutet, dass es sieben Perioden gibt.

  * Die Elemente in der ersten Periode haben zum Beispiel ein Orbital, während die Elemente in der siebten Periode sieben Orbitale haben.

  * Innerhalb einer Periode steigt die Kernladungszahl, also die Anzahl der Protonen im Kern von links nach rechts.

* Anzahl der Protonen und Neutronen idR auch gleich. Gibt es mehr Neutronen als Protonen, spricht man von **Isotopen**.

> Die verschiedenen Atomsorten (Elemente) unterscheiden sich in der Anzahl der Protonen, die im Atomkern stecken.

* **Die Ordnungszahl im Periodensystem sagt uns, wie viele Protonen im Kern vorhanden sind**.

* Daneben findet man noch die Zahl für die Atommasse u. Wasserstoff zB 1,01. Anzahl der Neutronen: gerundete Atommasse u - Anzahl der Protonen: 1-1=0 im Wasserstoffatom. Bei C Kohlenstoff ist es 12-6= 6 Neutronen.

* Es heisst Periodensystem, weil Elemente unterhalb in einer Spalte immer die gleichen chemischen Verhaltensweisen haben, zB H und Li.

https://en.m.wikipedia.org/wiki/Group_(periodic_table)

https://en.m.wikipedia.org/wiki/Period_(periodic_table)

https://en.m.wikipedia.org/wiki/Block_(periodic_table)

https://en.m.wikipedia.org/wiki/Period_7_element

https://ptable.com/?lang=de#Eigenschaften

https://de.wikihow.com/Das-Periodensystem-lesen

https://studyflix.de/chemie/periodensystem-270

![ggg](https://upload.wikimedia.org/wikipedia/commons/thumb/0/00/Periodensystem_Einfach.svg/1024px-Periodensystem_Einfach.svg.png)

*Source: https://de.m.wikipedia.org/wiki/Periodensystem*

![ggg](https://raw.githubusercontent.com/deltorobarba/repo/master/chem_033.jpeg)

![ggg](https://raw.githubusercontent.com/deltorobarba/repo/master/chem_010.JPG)

![ggg](https://raw.githubusercontent.com/deltorobarba/repo/master/chem_012.jpg)

*Tendenzen im Periodensystem*

![gg](https://upload.wikimedia.org/wikipedia/commons/thumb/a/aa/Periodic_trends_%282%29.svg/langde-640px-Periodic_trends_%282%29.svg.png)

https://de.m.wikipedia.org/wiki/Periodensystem

Video 1: https://youtu.be/J2KJRRH0E3Y

Video 2: https://youtu.be/f5-W87IGLFY

###### *Phasendiagramm & Phasengrenzlinie*

**Wasser siedet auf dem Mount Everest bei bereits 71 Grad aufgrund des geringeren Drucks**

* Ob Wasser fest, flüssig oder gasförmig ist, hängt nicht nur von der Temperatur ab, sondern auch vom Druck. Die einzelnen Moleküle im Wasser (wie auch in jeder anderen Flüssigkeit) sind ständig in Bewegung. Wenn die Temperatur erhöht wird, bewegen sie sich stärker und schneller. Bei Temperaturen um den Siedepunkt herum (also wenn das Wasser kocht) gelingt es immer mehr Molekülen an der Wasseroberfläche, in die Luft zu entweichen. Die Anziehungskraft zwischen den Teilchen ist dann nämlich nicht mehr ausreichend, um alle Teilchen in der Flüssigkeit zusammenzuhalten. Das Wasser wird gasförmig: Es wird zu Wasserdampf. Dies passiert nicht nur an der Wasseroberfläche, sondern überall in der Flüssigkeit, wo sich genügend schnelle Moleküle zusammentun. Auch am Boden des Kochgefässes siehst du zum Beispiel, wie sich kleine Gasblasen bilden und aufsteigen.

* Teilchen in Bewegung kämpfen gegen den Druck

* Der Druck, der aus der Bewegung (also der Temperatur der Moleküle) entsteht, hat jedoch einen Gegenspieler: den Druck der Luft, die sie umgibt. Die Luftmoleküle möchten ja ebenfalls genügend Platz haben und verteidigen ihr Areal. Die gasförmigen Wassermoleküle werden somit zu einem gewissen Teil wieder in die Flüssigkeit gedrückt. Dies führt zu einem Gleichgewicht, das sich sehr einfach in die eine oder andere Richtung verschieben lässt.

* Bei einem Luftdruck von 1.013 bar, was als Normaldruck bezeichnet wird, verdampft Wasser bei 100°C. Der Luftdruck in den Bergen ist kleiner als am Meer, und das Wasser siedet darum bereits bei tieferer Temperatur. Auf dem Mount Everest beträgt der Luftdruck aufgrund der grossen Höhe mit 0.326 bar nur knapp ein Drittel des Normaldrucks, und Wasser siedet dort bereits bei 71°C. Auch in der Schweiz kann man Wasser bereits bei 85°C kochen. Dazu reicht es, auf einen der höchsten Schweizer Berge zu klettern und auf 4500 Meter bei ca. 0.577 bar den Gaskocher zu entfachen.



**Phasendiagramm**

Ein [Phasendiagramm](https://de.m.wikipedia.org/wiki/Phasendiagramm) (auch Zustandsdiagramm, Zustandsschaubild oder Gleichgewichtsschaubild) ist eine Projektion von Phasengrenzlinien aus dem Zustandsraum eines thermodynamischen Systems in ein zweidimensionales kartesisches Koordinatensystem oder in ein Dreiecksdiagramm.

* Neben Phasengrenzlinien können Phasendiagramme auch Spinodalen enthalten.

* Phasendiagramme sind ein häufig genutztes Hilfsmittel in der Chemie, den Materialwissenschaften, der Physik und der Verfahrenstechnik, **um das Phasenverhalten von Reinstoffen und Stoffgemischen graphisch darzustellen**.

![gg](https://upload.wikimedia.org/wikipedia/commons/thumb/0/00/Phasendiagramme.svg/505px-Phasendiagramme.svg.png)

**Phasendiagramm von Wasser**

![ggg](https://upload.wikimedia.org/wikipedia/commons/thumb/0/08/Phase_diagram_of_water.svg/1214px-Phase_diagram_of_water.svg.png)

https://de.m.wikipedia.org/wiki/Eis_VI

**Phasengrenzlinie**

*Druck-Volumen-Temperatur-Diagramm eines Reinstoffes. Die Koexistenzkurve umschließt das blau kolorierte Koexistenzgebiet der flüssigen Phase und der Gasphase:*

![ggg](https://upload.wikimedia.org/wikipedia/commons/thumb/b/b5/P-v-T-Diagramm.jpg/796px-P-v-T-Diagramm.jpg)

https://de.m.wikipedia.org/wiki/Phasengrenzlinie

###### *Reinstoffe & Stoffgemische*

![ggg](https://raw.githubusercontent.com/deltorobarba/repo/master/chem_030.jpg)

*Schematische Einteilung der Stoffe* - [Source](https://de.m.wikipedia.org/wiki/Chemische_Verbindung#Organische_Verbindungen)

https://de.m.wikipedia.org/wiki/Stoffgruppe

**Reinstoffe**

> **Reinstoffe**: Stoffe, die nur aus einer Sorte Stoff bestehen (zB Wasser)

* Reinstoffe können aus einer Sorte **Atome (Element)** bestehen, zB Aluminiumwürfel

* oder aus einer Sorte **Verbindungen** bestehen: ein Zusammenschluss aus zwei oder mehreren Atomen (müssen nicht von derselben Sorte sein),

  * **Molekül**: zB H2O, weil es Wasser nur als Verbindungen gibt, nicht als Wasseratome. Noch spezifischer ist es ein Molekül.

  * **Ionenverbindung / Salze**: anderes Beispiel: Kochsalz bzw. Natriumchlorid NaCl. Elemente sind hierbei aber nicht als Molekül verbunden, sondern bilden ein Salz

**Stoffgemische**

> **Stoffgemische**: Mischungen aus zwei oder mehreren Reinstoffen (zB Apfelsaft aus Wasser, Vitamin C, Zucker, Aromen, Kohlenhydrate)

![ggg](https://raw.githubusercontent.com/deltorobarba/repo/master/chem_032.png)

**Homogenes Gemisch (fest-fluessig)**

* Wasser und Salz = homogenes Gemisch (1 Phase)

* Homogenes fest-fluessig Gemisch nennt man **Loesung** (zB Lebensmittelfarbe vermischt mit Wasser ergibt eine Phase und Gemisch ist durchsichtig)

**Heterogenes Gemisch (fest-fluessig)**

* Wasser und Sand = heterogene Stoffgemische mit 2 Phasen (Wasserphase und Sandphase)

* Wasserfarbe mit Wasser mischen - nicht durchsichtig. Es ist ein heterogenes Gemisch. Wir haben hier wie beim Sand immer noch eine feste und eine fluessige Phase (undurchsichtig weil farbpartikel schwimmen).

* auch Orangensaft

* So ein heterogenes fest-fluessig Gemisch nennt man **Suspension**.

**Homogenes Gemisch (fluessig-fluessig)**

* Alkohol + Wasser (Schnaps)

**Heterogenes Gemisch (fluessig-fluessig)**

* Oele und Fette

* Nagellackentferner mischen, es entsteht **Emulsion**: kleine Oletroepfchen, die im Wasser schwimmen, nicht stabil, die beiden Phasen trennen sich wieder

**Homogenes Gemisch (fest-fest)**

* ein oder mehrere Metalle einschmelzen und wieder erstarren lassen

* Homogene fest-fest Gemische nennt man **Liegerung**

* Kupfer-Zink-Legierung = Messing

**Heterogenes Gemisch (fest-fest)**

* 1 Euro Muenzen


Ein chemisches System wird als homogen bezeichnet, wenn es nur aus einer einzigen Phase (fest, flüssig oder gasförmig) besteht, deren Zusammensetzung an jedem Ort des Systems gleich ist. Homogene Systeme können aus einem Reinstoff oder aus vielen Bestandteilen bestehen. Beispiele für homogene Stoffgemische sind: Mehl, Messing, Speisefett, Wein, Tee, Salzlösungen, Luft oder Erdgas.

* Heterogene Stoffgemische bestehen aus mindestens zwei nicht miteinander mischbaren Phasen. Die Zusammensetzung solcher Gemische wie Schlamm oder Metallerzen ist nicht an jedem Punkt des Systems gleich. Oftmals sind die einzelnen Phasen mit bloßem Auge nicht mehr so einfach zu unterscheiden, sondern werden erst bei mikroskopischer Betrachtung deutlich. In solchen Fällen (Milch, Flüssigseifen) spricht man von mikroheterogenen oder kolloiden Systemen.

Video: https://www.youtube.com/watch?v=xqUI2G4O2WE&list=PLMRuqWRlK1n1ILFs-np0p6A-gUb9jbcYz&index=4

**Trennverfahren**

![ggg](https://raw.githubusercontent.com/deltorobarba/repo/master/chem_031.jpg)

* einzelne Stoffe abtrennen oder Stoffgemisch in Reinstoffe  trennen

* Trennverfahren für (heterogene) fest-flüssig Gemische

  * Dekantieren (=Abgiessen, weil Feststoffe am Boden bleiben)

  * Filtrieren: viele kleine Löcher, durch die Flüssigkeit fliesst

  * Zentrifugieren: wenn Feststoff zu klein ist für Filter, oder klebrig. In Zentrifuge in jeweils kleinen Gefässen geschleudert. Der Feststoff wird nach aussen in den Boden der Zentrifugenröhrchen gedrückt.

* Trennverfahren für (homogene) fest-flüssig Gemische, zB Salz aus einer Salzlösung

  * Trocknen, braucht aber Zeit

  * Verdampfen, zB durch Erhitzen zum Beschleunigen

* Trennverfahren für (homogene) flüssig-flüssig Gemische

  * Destillieren: zunutze machen, dass verschiedene Flüssigkeiten verschiedene Siedetemperaturen haben. zB Ethanol-Wasser-Gemisch: 70 Grad, da wird Ethanol zu Gas, Wasser aber noch nicht (Ethanol verdampft und wieder Auffangen durch Kondensieren)


*Löslichkeit*

* Loesung: homogenes Gemisch aus mindestens 2 Stoffen

* zB Salz in Wasser loest sich, aber irgendwann kann sich kein Salz mehr loesen - das Wasser ist gesaettigt

* Hitze spielt auch eine Rolle: Zucker und Wasser bei 100 Grad loest sich doppelt so viel Zucker als bei Raumtemperatur

* bei Gasen andersherum: loesen sich besser bei kalten Loesungsmitteln

* Stoffe kann man in hydrohile (Salz, Zucker) und hydrophobe Stoffe (Fette, Oele, Kerzenwachs) einteilen, was die Loeslichkeit betrifft

* hydrohobe Loesungsmittel: Benzin oder Ether (haeufig giftig beim Einatmen)

* Aceton (Basis von Nagellackentferner) zwischen hydrohil und hydrophob: kann sich in Wasser loesen, aber auch Styropor (was mit hydrophilen Stoffen nicht geht)

Video: https://www.youtube.com/watch?v=EvqDernIcJA&list=PLMRuqWRlK1n1ILFs-np0p6A-gUb9jbcYz&index=3

###### *Moleküle (Chemische Bindungen & Chemische Verbindung)*

**Chemische Bindungen**

> Dass es überhaupt zur Ausbildung von [chemischen Bindungen](https://de.m.wikipedia.org/wiki/Chemische_Bindung) kommt, beruht darauf, **dass es für die betroffenen Atome oder Ionen energetisch günstiger ist, mit geeigneten Bindungspartnern verbunden zu sein, als separat in Form von einzelnen, ungebundenen Teilchen vorzuliegen.**

* Grundlage der Ausbildung von Bindungen sind elektrostatische Wechselwirkungen oder Wechselwirkungen der Elektronen zwischen zwei oder mehreren Atomen.

* In vielen Fällen spielen beide Arten von Wechselwirkungen eine Rolle.

**Chemische Verbindung**

* [Chemische Verbindung](https://de.m.wikipedia.org/wiki/Chemische_Verbindung) besteht aus zwei oder mehr verschiedenen chemischen Elementen bestehender Reinstoff

* Als chemische Verbindung bezeichnet man einen **Reinstoff**, der aus Atomen von zwei oder mehreren chemischen Elementen besteht, wobei – im Gegensatz zu Gemischen – die Atomarten **zueinander in einem festen stöchiometrischen Verhältnis stehen**.

* Das zahlenmäßige Verhältnis der Atome zueinander wird durch chemische Bindungen zwischen den Atomen bestimmt, und das Verhältnis lässt sich in einer Summenformel darstellen.

* Eine genauere Unterscheidung von Verbindungen und deren Zuordnung zu einem dieser vier Typen (Molekul also kovalent, ionisch, metallisch oder komplex) kann mit Hilfe der Elektronegativitätsdifferenz der an der Bindung beteiligten Elemente vorgenommen werden. Zudem gibt es Übergangsformen zwischen den vier o. g. Idealtypen.



**Molekül**

* [Moleküle](https://de.m.wikipedia.org/wiki/Molekül) sind Teilchen aus zwei oder mehreren Atomen, die durch [chemische Bindungen](https://de.m.wikipedia.org/wiki/Chemische_Bindung) zusammengehalten werden

* Im engen Sinn und im allgemeinen Sprachgebrauch der Chemie sind Moleküle elektrisch neutrale Teilchen, die aus zwei oder mehreren Atomen aufgebaut sind. Die Atome sind kovalent miteinander verknüpft und bilden einen in sich abgeschlossenen, chemisch abgesättigten Verband.

* Ein so definiertes Molekül ist das kleinste Teilchen eines bestimmten Reinstoffes und hat eine bestimmbare Molekülmasse. Ein Molekül ist kein starres Gebilde, bei Energiezufuhr treten unterschiedliche Molekülschwingungen auf; hierzu reicht schon die Normaltemperatur.

> Die Bindungsverhältnisse in Molekülen beruhen auf quantenmechanischen Effekten und werden beispielsweise mit dem VSEPR-Modell, der Valenzbindungstheorie oder der MO-Theorie erklärt und beschrieben.

* [Elementmolekül](https://de.m.wikipedia.org/wiki/Elementmolekül): Moleküle können aus Atomen eines einzigen chemischen Elements aufgebaut sein, wie z. B. Sauerstoff (O2) und Stickstoff (N2) (sogenannte Elementmoleküle).

* Die meisten Moleküle sind jedoch Verbände von Atomen verschiedener Elemente, wie beispielsweise Wasser (H2O) und Methan (CH4). Die Anordnung der Atome (ihre Konstitution) in einem Molekül ist durch die chemischen Bindungen fixiert. So unterscheiden sich trotz gleicher Anzahl der beteiligten Atome Ethanol (H3C–CH2–OH) von Dimethylether (H3C–O–CH3); sie werden durch unterschiedliche chemische Formeln dargestellt. In bestimmten Fällen können Moleküle wie z. B. die Moleküle der Milchsäure Formen mit gleicher Konstitution, aber unterschiedlicher räumlicher Anordnung (der Konfiguration) vorliegen. Dass gleiche Summenformeln unterschiedliche Moleküle zulassen, wird allgemein Isomerie genannt.

**Makromoleküle**

* [Makromoleküle](https://de.m.wikipedia.org/wiki/Makromolekül) (Riesenmoleküle) sind sehr große Moleküle, die aus **sich wiederholenden, gleichen oder unterschiedlichen Struktureinheiten** (formale Grundbausteine) bestehen und eine **hohe Molekülmasse haben (über 10.000 g·mol−1 liegt)**.

* Die Grundbausteine sind **meist Atomgruppen, können aber auch Atome sein**. Die Massen der Grundbausteine sind im Vergleich zur Molekülmasse sehr klein.

* Makromolekulare Stoffe werden unterteilt in:

  * **natürliche** Stoffe: Kohlenwasserstoffe, wie cis-1,4-Polyisopren (Hauptbestandteil von Naturkautschuk), Lignin, Polynukleotide, wie Nukleinsäuren, Polysaccharide, wie Stärke und Zellulose, Proteine, wie Seide, Collagen, Enzyme, Antikörper. zB **Weiße Naturseide** – ein natürlicher makromolekularer Stoff

  * **halbsynthetische** Stoffe: Acetatseide Viskose, Cellulosenitrat (Zelluloid), Vulkanisierter Kautschuk, zB **Tischtennisbälle** aus Zelluloid – einem halbsynthetischen makromolekularen Stoff

  * **synthetische** Stoffe: Polystyrol (PS), Polyethylen (PE), Polypropylen (PP), Polyamide (PA), Polyvinylchlorid (PVC), Polyethylenterephthalat (PET), Silikone, Synthesekautschuk, zB **PET-Flasche** aus einem synthetischen makromolekularen Stoff



**Polymer**

* [Polymere](https://de.m.wikipedia.org/wiki/Polymer) sind meist organischer Stoff, der aus sich wiederholenden gleichen Einheiten aufgebaut ist

* Polymere können in natürliche und synthetische Polymere unterteilt werden.

* **Natürliche Polymere (Biopolymere)** werden in Lebewesen synthetisiert und bilden die Grundbausteine der Organismen.

  * Zu diesen Polymeren zählen die Proteine, sie bilden beispielsweise Haare und die Seide. Zu den Polysacchariden gehören Zellulose, Stärke und Chitin.

  * Nukleinsäuren erfüllen viele biologische Funktionen, wie Desoxyribonukleinsäure als Erbsubstanz.

* **Synthetische Polymere** sind durch Polyreaktionen industriell oder im Labormaßstab hergestellte Stoffe, dazu gehören Polyethylen, Polystyrol und Polyvinylchlorid.

  * Chemisch modifizierte Polymere entstehen durch die Weiterverarbeitung von Biopolymeren, beispielsweise Nitrocellulose, Celluloid oder Stärkederivate.



**Supramolekulare Chemie**

https://de.m.wikipedia.org/wiki/Supramolekulare_Chemie

##### <font color="blue">*Hauptgruppenelemente*

###### *Alkalimetalle (I)*

* Lithium, Natrium, Kalium, Rubidium, Caesium, Francium (ohne Wasserstoff)

* Normale Metalle: elektrische Leitfähigkeit, Glanz, guter Verformbarkeit, gute Wärmeleitung

* Eigenschaften: Lithium, Natrium, Kalium und Rubidium sind silbrig-glaenzend. Caesium ist schwach goldfarben. Francium ist stark radioaktiv (ignorieren wir).

* Alkalimetalle sind weiche und leichte Feststoffe  (geringe Dichte 0,5 - 2 g pro cm3, Eisen hat zB 8 g/cm3). Von oben nach unten (Lithium zu Kalium) sie immer weicher.

* Wie kann man Alkalimetalle nachweisen? - Man hält Alkalimetallsalze in eine Flamme und schauen welche Farbe die Flamme bekommt: Natrium gelb, Lithium purpurrot, Kalium hellviolett, Rubidium rotviolett, Caesium blauviolett.

* Aufgrund dieser Flammenfärbung werden Alkalimetallverbindungen für Feuerwerke benutzt.

* In der Atomphysik werden Alkalimetalle eingesetzt, da sie sich aufgrund ihrer besonders einfachen elektronischen Struktur besonders leicht mit [Lasern kühlen](https://de.m.wikipedia.org/wiki/Laserkühlung) lassen.

* Alle Alkalimetalle kristallisieren in der [kubisch-raumzentrierten](https://de.m.wikipedia.org/wiki/Kubisches_Kristallsystem) Struktur. Lediglich Lithium und Natrium kristallisieren in der [hexagonal-dichtesten](https://de.m.wikipedia.org/wiki/Hexagonales_Kristallsystem) Packung, wenn tiefe Temperaturen vorherrschen.

**Lithium** hat eine Dichte von 0,53 g/cm3 und damit leichter als Wasser. Es wuerde also auf Wasser schwimmen (wenn es nicht sofort damit reagieren wuerde). Weil es so leicht ist, wird es gerne bei Legierungen verwendet, zB an Flugzeugen. Legierung LA141: Lithium, Magnesium und Aluminium. Nur halb so schwer wie das Leichtmetall Aluminium (Alufolie).

**Natrium**: grösste Bedeutung fur die Industrie. Bekannte Verbindungen: Kochsalz (Natriumchlorid), Natriumhydroxid (nutzt man in Haushaltsreinigern) und Natriumkarbonat.

**Kalium**: ist in Bananen und Kaffee. Kaliumverbindungen, wie Kaliumchlorid, spielt in der Landwirtschaft eine wichtige Rolle (fur Düngemittel).

![ggg](https://raw.githubusercontent.com/deltorobarba/repo/master/sciences_0783.jpeg)

In der Natur kommen Alkalimetalle als Reinstopfe nicht vor, sie sind sehr reaktiv (in gediegener Form nicht stabil).

* Sie kommen in der Natur daher idR in Form von Salzen (Alkalimetalle + Halogenen) vor, einfach geladene Ionen.

* Grund dafur ist die Oktettregel: sie haben nur ein Elektron in der Valenzschale und wollen es abgeben.

* Zusaetzlich sind die Elemente immer reaktiver nach unten: mit jeder Periode (Reihe) kommt eine zusatzliche Elektronenschale hinzu. Das Valenzelektron ist bei Caesoium weiter entfernt vom Atomkerne, als in Natrium oder Lithium, und kann es daher weniger stark binden = staerkere Reaktivitaet.



Wichtige Alkalimetallverbindungen:
* Salze (Alkalimetalle + Halogene), zB Kochsalz (Natriumchlorid)

* Oxide (Alkalimetalle + Sauerstoff), zB

* Hydroxide (Alkalimetalle + Wasser), zB [Natriumhydroxid](https://de.m.wikipedia.org/wiki/Natriumhydroxid)

  * Lithium 2Li + Wasser 2 H2O $\rightarrow$ Lithiumhydroxid 2 LiOH + Wasserstoffgas H2

  * Natrium 2Na + Wasser 2 H2O $\rightarrow$ Natriumhydroxid 2 NaOH + Wasserstoffgas H2

* starke Reaktionen, H2 kann sich selbst entzuenden (daher sollte man reines Lithium oder Natrium nicht mit feuchten Haenden anfassen).

* Bei Rubidium oder Caesium kann es zu Explosionen kommen bei Kontakt mit Wasser.

* Damit sie nicht mit Wasserstoff oder Sauerstoff reagieren: Lithium, Natrium, Kalium bewahrt man in [Paraffinöl](https://de.m.wikipedia.org/wiki/Paraffin) (acyclischen Alkanen, also gesättigten Kohlenwasserstoffen) auf. Und Rubidium und Caesium bewahrt man in Ampullen auf, wo sie eingeschmolzen sind


Video: [Mai Thi Nguyen-Kim - Alkalimetalle I Hauptgruppe Periodensystem](https://www.youtube.com/watch?v=NpgVk9leUbs)

https://de.m.wikipedia.org/wiki/Alkalimetalle

###### *Erdalkalimetalle (II)*

* Calcium, Magnesium und Beryllium, dazu Strontium, Barium, Radium

* Unterschied zu Alkalimetalle: sind haerter, haben hoehrere Schmelz- und Siedepunkte und sind nicht alle so reaktionsfreudig.

  * Im Vergleich zu den Alkalimetallen sind sie jedoch weniger reaktiv, weil es eine höhere [Ionisierungsenergie](https://de.m.wikipedia.org/wiki/Ionisierungsenergie) (In der Atomphysik wird die Ionisierungsenergie auch als Bindungsenergie bezeichnet) erfordert, zwei Außenelektronen abzuspalten als eins wie bei den Alkalimetallen.

  * Dies lässt sich damit begründen, dass die Erdalkalimetalle eine höhere [Kernladung](https://de.m.wikipedia.org/wiki/Kernladung) und somit entsprechend kleinere [Atomradien](https://de.m.wikipedia.org/wiki/Atomradius) aufweisen als die Alkalimetalle.

* Werden (wie Alkalimetalle) auch als unedle Metalle bezeichnet. Liegen in der Natur nicht in Reinform vor, es gibt sie nur als Verbindungen. Meistens sind das Salze, in denen das Metallion ein zweifach positiv geladenes Kation ist (wegen Oktettregel), wie CaCO3 (Calciumcarbonat) oder MgO (Magnesiumoxid).

* Die Bezeichnung leitet sich von den beiden benachbarten Hauptgruppen ab, den Alkalimetallen, mit denen sie die Bildung [starker Basen](https://de.m.wikipedia.org/wiki/Starke_Basen) gemeinsam haben, und den Erdmetallen, mit denen sie gemeinsam haben, schlecht wasserlöslich zu sein.

* Aufgrund der Flammenfärbung werden Erdalkalimetallverbindungen für Feuerwerke benutzt.

* Die Erdalkalimetalle geben leicht ihre beiden Außenelektronen ab, wobei zweifach positiv geladene Ionen entstehen, und sind daher unedle Metalle, die an der Luft oxidiert werden.

  * Beryllium und Magnesium bilden aber stabile Oxidschichten aus und werden dadurch passiviert, d. h. nur ihre Oberfläche wird oxidiert.

  * Diese **Passivierung** bewirkt auch, dass Wasser Beryllium und Magnesium nur langsam angreift.

  * Calcium, Strontium und Barium hingegen reagieren mit Wasser zu den [Hydroxiden](https://de.m.wikipedia.org/wiki/Hydroxide) (Hydroxide sind salzähnliche Stoffe, die Hydroxid-Ionen ([OH]−) als negative Gitterbausteine (Anionen) enthalten), wobei Wasserstoff entsteht. Wie die Alkalimetalle, so sind also auch die Erdalkalimetalle Basenbildner.

  * Ansonsten reagieren die Erdalkalimetalle gut mit Nichtmetallen, z. B. mit Sauerstoff oder mit den Halogenen.







![ggg](https://raw.githubusercontent.com/deltorobarba/repo/master/sciences_0784.jpeg)

* Beryllium: seine Eigenschaften eher dem Aluminum ähnlich (aus der dritten Haugruppe), das nennt man "Schraegbeziehung". Ist sehr leicht und wird in Legierungen im Flugzeugbau oder Raumfahrt eingesetzt. Bestandteil von Smaragden. Das Element oder viele seiner Verbindungen sind giftig oder gesundheitsschaedlich.


* Magnesium: 4% der Erdkruste sind aus Magnesium. Nimmt man alles Mg aus den Meeren, hätte man einen Würfel mit Kantenlänge von 1000 km.

	* Verbrennt in Feuer zu Magnesiumoxid in hell-weisser Flamme. Daher war es früher Bestandteil von Blitzpulver (zum Fotografieren). Heute ist es in Wunderkerzen oder anderer Pyrotechnik.

	* Mg ist auch lebensnotwendig: Pflanzen haben es in Chlorophyll fur Photosynthese (Pflanzen bauen Kohlendioxid zu Sauerstoff um).

	* Menschen brauche pro Tag ca 250 Milligramm Mg.


* Calcium: reines Ca hat nur wenige Anwendungen. Calciumverbindungen hingegen gibt es viele: Kalk (Calcumcarbonat), Marmor, Gips, Kreide. Menschen brauchen Calcium fur den Bau der Knochen und Zähne sowie Funktionsfähigkeit der Muskeln. Calcium färbt Flamme ziegelrot.


* Strontium und Barium: reagieren sehr heftig mit Wasser, kaum Anwendung in Reinform. Werden gerne in der Pyrotechnik verwendet. Strontium färbt Flamme richtig rot. Barium in Silvesterraketen und faerbt Flammen grün. Barium selbst in giftig, aber als schwer loesliches Bariumsulfat wird es in der Radiologie verwendet (Roentgen).

  * Calcium, Strontium und Barium absorbieren wegen ihrer hohen Atommassen Röntgenstrahlen recht gut: deshalb heben sich Knochen, die Calcium enthalten, deutlich von den Organen ab.

  * Wenn man Magen untersuchen will, bekommt Patient eine Bariumsulfatloesung, dann wird auch der Magen sichtbar.

* Radium: radioaktiv. Entdeckt durch Marie Curie. Anfangs schmierten sich Taenzer und Künstler damit ein, damit die Haut im Dunkeln leuchtet. Radium wurde daher sogar noch bis in die 1960er Jahre in der Uhrenindustrie verwendet.


Video: [Mai Thi Nguyen-Kim - Erdalkalimetalle](https://www.youtube.com/watch?v=4XpfUyMV6Bo)

https://de.m.wikipedia.org/wiki/Erdalkalimetalle

###### *Borgruppe (Erdmetalle) (III)*

https://de.m.wikipedia.org/wiki/Borgruppe

![ggg](https://raw.githubusercontent.com/deltorobarba/repo/master/sciences_0784.jpeg)

###### *Kohlenstoff(-Silicium)-Gruppe (IV)*

https://de.m.wikipedia.org/wiki/Kohlenstoffgruppe

![ggg](https://raw.githubusercontent.com/deltorobarba/repo/master/sciences_0787.jpeg)

###### *Stickstoff(-Phosphor)-Gruppe (V)*

https://de.m.wikipedia.org/wiki/Stickstoffkreislauf

https://en.m.wikipedia.org/wiki/FeMoco

Phosphor Mai Thi : https://youtu.be/5qVksiH92ZQ

Phosphor und stickstoff: düngemittel, justus von liebig, anorganische meersalze

https://de.m.wikipedia.org/wiki/Stickstoffgruppe

![ggg](https://raw.githubusercontent.com/deltorobarba/repo/master/sciences_0788.jpeg)

###### *Chalkogene (VI) (Sauerstoffgruppe)*

https://de.m.wikipedia.org/wiki/Chalkogene

![ggg](https://raw.githubusercontent.com/deltorobarba/repo/master/sciences_0788.jpeg)

###### *Halogene (VII)*

* Fluor, Chlor, Brom, Iod, dem äußerst seltenen radioaktiven Astat und dem 2010 erstmals künstlich erzeugten, sehr instabilen Tenness

https://de.m.wikipedia.org/wiki/Halogene

https://youtu.be/CC1IIka6Khg

![ggg](https://raw.githubusercontent.com/deltorobarba/repo/master/sciences_0789.jpeg)

###### *Edelgase (VIII)*

https://de.m.wikipedia.org/wiki/Edelgase

https://youtu.be/ITi6FA67nPk

https://youtu.be/4LAKxGIC8UQ

![ggg](https://raw.githubusercontent.com/deltorobarba/repo/master/sciences_0789.jpeg)

###### *Synthetic Elements*

* A synthetic element is one of 24 known chemical elements that do not occur naturally on Earth: they have been created by human manipulation of fundamental particles in a nuclear reactor, a particle accelerator, or the explosion of an atomic bomb; thus, they are called "synthetic", "artificial", or "man-made".

* The synthetic elements are those with atomic numbers 95–118

* The mechanism for the creation of a synthetic element is to force additional protons onto the nucleus of an element with an atomic number lower than 95.

* All synthetic elements are unstable, but they decay at widely varying rates: their half-lives range from a few hundred microseconds to millions of years.

https://www.sciencedaily.com/releases/2020/02/200211091815.htm

https://en.m.wikipedia.org/wiki/Synthetic_element

##### <font color="blue">*Nebengruppenelemente*

###### *Nebengruppe*

https://de.m.wikipedia.org/wiki/Nebengruppe

###### *Übergangsmetalle*

[Übergangsmetalle](https://de.m.wikipedia.org/wiki/Übergangsmetalle):

* Die chemischen Elemente mit den Ordnungszahlen von 21 bis 30, 39 bis 48, 57 bis 80 und 89 bis 112 werden üblicherweise als Übergangselemente bezeichnet.

* Da diese Elemente alle Metalle sind, wird auch der Ausdruck Übergangsmetalle benutzt.

* Bisher 200.000 Tonnen Gold geschürft

![ggg](https://raw.githubusercontent.com/deltorobarba/repo/master/sciences_0785.jpeg)

###### *Lanthanoide (Seltene Erden)*

https://de.m.wikipedia.org/wiki/Metalle_der_Seltenen_Erden

https://de.m.wikipedia.org/wiki/Lanthanoide

![ggg](https://raw.githubusercontent.com/deltorobarba/repo/master/sciences_0786.jpeg)

###### *Actinoide*

wie thorium zur altersbestimmung

https://www.chemie.de/lexikon/Actinoide.html

![ggg](https://raw.githubusercontent.com/deltorobarba/repo/master/sciences_0786.jpeg)

##### <font color="blue">*Radikale*

###### *Radikale*

**<u>Radikale</u>**

> Als Radikale bezeichnet man in der Chemie Atome oder Moleküle, die ein ungepaartes Elektron besitzen und damit sehr reaktiv sind. Sie sind üblicherweise instabil und kurzlebig.

**Radikale bilden sich durch**:

* Hitze (**thermolytische** Bindungsspaltung)
* UV-Strahlung, die die **homolytische Spaltung** (siehe obe Reaktionsmechanismen) einer kovalenten Bindung bewirkt (Photolyse)
* Röntgen- und andere **ionisierende** Strahlung
* Elektrochemisch durch **Oxidation** bzw. Reduktion

* Als [Radikale](https://de.m.wikipedia.org/wiki/Radikal_(Chemie)) bezeichnet man in der Chemie Atome oder Moleküle mit mindestens einem ungepaarten Valenzelektron. Die meisten Radikale sind besonders reaktionsfreudig.

* Radikale werden mit einem „Punkt“ dargestellt, der das ungepaarte Elektron symbolisiert, zum Beispiel Stickstoffmonoxid (NO•).

* Eine Verbindung mit zwei ungepaarten Elektronen heißt Diradikal (auch Biradikal), bei drei ungepaarten Elektronen spricht man von einem Triradikal usw.

* Radikale spielen eine wichtige Rolle bei bestimmten [Oxidationsprozessen](https://de.m.wikipedia.org/wiki/Oxidation), bei [Kettenpolymerisationen](https://de.m.wikipedia.org/wiki/Kettenpolymerisation) und bei manchen [Substitutionsreaktionen](https://de.m.wikipedia.org/wiki/Substitutionsreaktion).

* In der Nomenklatur werden Radikale mit der Endung -yl bezeichnet, für Biradikale wird -yliden verwendet. Ausnahmen sind hierbei die Diradikale Methylen, Silylen und Aminylen. Beispiele sind -oxyl für Verbindungen mit radikalischem Sauerstoff, z. B. [2,2,6,6-Tetramethylpiperidinyloxyl](https://de.m.wikipedia.org/wiki/2,2,6,6-Tetramethylpiperidinyloxyl), oder [Thiyl-Radikale](https://de.m.wikipedia.org/wiki/Thiyl-Radikale) mit radikalischen Schwefelatom.



![ggg](https://raw.githubusercontent.com/deltorobarba/repo/master/sciences_0853.png)


###### *Radikale bei der Polymerisation bei Kunststoffen*

**Radikale in der Polymerisation bei Kunststoffen**

Zur Initiierung radikalischer Reaktionen in der chemischen Synthese werden dem Reaktionsgemisch häufig sogenannte **Radikalstarter** zugesetzt. Radikalstarter sind Moleküle, die sich besonders leicht – beispielsweise durch Bestrahlung mit ultraviolettem Licht – in Radikale spalten lassen. Beispiele für Radikalstarter sind:

* [Azobis(isobutyronitril)](https://de.m.wikipedia.org/wiki/Azobis(isobutyronitril)): zu den Stoffklassen der Azoverbindungen und Nitrile zählt. Es wird in der Organischen Chemie als Initiator für die Radikal-Kettenreaktion bei der Herstellung von Kunststoffen eingesetzt.
* [Dibenzoylperoxid](https://de.m.wikipedia.org/wiki/Dibenzoylperoxid) (Benzoyloxyradikal - ein Radikal, das vor allem bei der Synthese von Kunststoffen bedeutend ist). Strukturformel:

> ![gg](https://upload.wikimedia.org/wikipedia/commons/thumb/c/cb/Dibenzoylperoxid.svg/320px-Dibenzoylperoxid.svg.png)

**Die Bindung zwischen den beiden Sauerstoffatomen ist labil und spaltet sich homolytisch unter geringer Energieaufwendung, z. B. durch Lichteinstrahlung (photolytisch) oder Zuführung von Wärme (thermolytisch)**.

> Bei der radikalischen Polymerisation von Alkenen spielt die Bildung der Benzoyloxy-Radikale eine wichtige Rolle:

> ![ggg](https://upload.wikimedia.org/wikipedia/commons/thumb/c/c8/Dibenzoylperoxide_Reaction_V.1.svg/640px-Dibenzoylperoxide_Reaction_V.1.svg.png)

Dabei zerfällt Benzoylperoxid leicht in zwei Benzoyloxyradikale, die weiter unter CO2-Abspaltung in Phenylradikale zerfallen. Die Phenylradikale können anschließend zu Biphenyl dimerisieren:

> ![gg](https://upload.wikimedia.org/wikipedia/commons/thumb/9/99/Decarboxylation-Benzoyloxy_Radical_V.1.svg/640px-Decarboxylation-Benzoyloxy_Radical_V.1.svg.png)

* Dilauroylperoxid,
* Di-tert-butylperoxid,
* Diisopropylperoxydicarbonat
* Kaliumperoxodisulfat.

> Radikalstarter koennen auch bei anderen Prozessen als der Kunststoffproduktion verwendet werden!

###### *Radikale in der Biochemie*

> **Siehe auch: Chemische Elemente $\rightarrow$ Radioaktivität: Wirkung auf den Körper**

* Radikale, etwa [reaktive Sauerstoffspezies](https://de.m.wikipedia.org/wiki/Reaktive_Sauerstoffspezies) (ROS) (Sauerstoffradikale), spielen bei einer Vielzahl biologischer Prozesse eine wichtige Rolle, können aber auch Zellschäden hervorrufen, die unter anderem zur Entstehung von Krebserkrankungen beitragen können.

* Auch für die Entstehung der Arteriosklerose, der Alzheimerschen Krankheit, der Leberschädigung durch Alkohol und des Lungenemphysems durch Tabakrauch **wird der durch freie Radikale vermittelten [Oxidation](https://de.m.wikipedia.org/wiki/Oxidation) verschiedener Stoffe eine bedeutsame Rolle zugeschrieben**.

> Oxidation: Später erfolgte eine Erweiterung des Begriffs, indem Reaktionen einbezogen wurden, bei denen einer Verbindung Wasserstoffatome entzogen wurden ([Dehydrierung](https://de.m.wikipedia.org/wiki/Dehydrierung)). Z. B. „entreißen“ bei vielen biochemischen Vorgängen bestimmte Coenzyme (NAD, NADP, FAD) organischen Verbindungen Wasserstoffatome.

* Unter den intrazellulären Signalwegen, die durch freie Radikale aktiviert werden, ist der [NF-κB-Signalweg](https://de.m.wikipedia.org/wiki/NF-%CE%BAB) (Proteinkomplex) einer der wichtigsten.



**Der Schutz vor der Wirkung der Radikale ist lebensnotwendig**; der Körper besitzt deshalb wirksame Abwehr- und Reparaturmechanismen in Form von Enzymen, Hormonen oder anderen Substanzklassen, die die schädliche Wirkung minimieren.

* An diesen Abwehrmechanismen sind [Antioxidantien](https://de.m.wikipedia.org/wiki/Antioxidans) beteiligt, wie
  * die chemische Verbindung [Epigallocatechingallat](https://de.m.wikipedia.org/wiki/Epigallocatechingallat)
  * die Enzymfamilien [Superoxiddismutase](https://de.m.wikipedia.org/wiki/Superoxiddismutase) und [Glutathionperoxidase](https://de.m.wikipedia.org/wiki/Glutathionperoxidase)
  * [Vitamin A](https://de.m.wikipedia.org/wiki/Vitamin_A), [Vitamin C](https://de.m.wikipedia.org/wiki/Ascorbins%C3%A4ure) und [Vitamin E](https://de.m.wikipedia.org/wiki/Vitamin_E)
  * [Coenzym Q10](https://de.m.wikipedia.org/wiki/Ubichinon-10)
  * [Flavonoide](https://de.m.wikipedia.org/wiki/Flavonoide) (Gruppe von Naturstoffen der Polyphenole) wie [Taxifolin](https://de.m.wikipedia.org/wiki/Taxifolin) und [Anthocyane](https://de.m.wikipedia.org/wiki/Anthocyane)

* Auch [Bilirubin](https://de.m.wikipedia.org/wiki/Bilirubin) und [Harnsäure](https://de.m.wikipedia.org/wiki/Harns%C3%A4ure) sollen bestimmte freie Radikale neutralisieren können.

* Das Hormon [Melatonin](https://de.m.wikipedia.org/wiki/Melatonin) gilt ebenfalls als Radikaleinfänger gegen den [oxidativen Stress](https://de.m.wikipedia.org/wiki/Oxidativer_Stress).

* Das stärkste bekannte Antioxidans, das Hydridion H−, spielt zum Beispiel im [Citratzyklus](https://de.m.wikipedia.org/wiki/Citratzyklus) und bei vielen [Redoxreaktionen des Stoffwechsels](https://de.m.wikipedia.org/wiki/Stoffwechsel#Reaktionstypen) eine wichtige Rolle.

Radikale spielen im Rahmen der sogenannten „Abnutzungstheorien“ der Alterungsprozesse im Körper eine Rolle, so dass Wirksubstanzen gegen oxidativen Stress als Mittel gegen das Altern im Gespräch sind → [Theorie der freien Radikale](https://de.m.wikipedia.org/wiki/Theorie_der_freien_Radikale).

* So ist bekannt, dass Zellen von Vögeln weitaus besser freien Radikalen widerstehen können. **Da aber nur der Einfluss synthetischer Antioxidantien untersucht wurde, sollten keine Schlüsse zu möglichen Effekten von Obst und Gemüse gezogen werden**. 2007 forderte ein JAMA-Editorial weitere randomisierte Studien, um die Wirkung von Vitamin C und Selen zu etablieren.

* Eine seither mehrfach aktualisierte, sehr große Metaanalyse der Cochrane Collaboration (2007–2012) konnte abschließend keinerlei positive Effekte vitaminhaltiger Nahrungsergänzungspräparate nachweisen; im Gegenteil stieg die Sterblichkeit in der Verumgruppe sogar leicht an.

Siehe hier unter Biologie: https://de.m.wikipedia.org/wiki/Reaktive_Sauerstoffspezies

###### *Kohlenstoff-Radikale und Stabilität von Radikalen*

**Kohlenstoff-Radikale**

* C-zentrierte Radikale zeigen eine zunehmende Stabilität in der Reihe Methylradikal **< primäres C-Atom < sekundäres C-Atom < tertiäres C-Atom**, was durch induktive Effekte und Hyperkonjugation bedingt ist.

* Außerdem sind [sp³-hybridisierte](https://de.m.wikipedia.org/wiki/Hybridorbital) Kohlenstoff-Radikale stabiler als Radikalzentren, in denen der Kohlenstoff eine sp²- oder sp-Hybridisierung aufweist. Auch Aryl- oder Allylgruppen stabilisieren das Radikal.

**Stabilität von Radikalen**

* Je mehr Kohlenstoffatome an das C-Atom mit dem ungepaarten Elektron (also der Radikalfunktion) gebunden sind, desto stabiler ist dieses Radikal. Ein tertiäres Radikal ist daher stabiler als ein sekundäres und dieses wiederum stabiler als ein primäres Radikal.

*Im Energiediagramm zeigt sich dieser Unterschied in der geringeren Energie des Radikals und der reduzierten Aktivierungsenergie bei der Bildung:*

![ggg](https://raw.githubusercontent.com/deltorobarba/repo/master/sciences_0854.png)


##### <font color="blue">*Sterilisation & Desinfektion*

Technisch wird zwischen [Desinfektion](https://de.m.wikipedia.org/wiki/Desinfektion) und [Sterilisation](https://de.m.wikipedia.org/wiki/Sterilisation) unterschieden.

  * **Desinfektion: $10^{-5}$ Dezimalreduktionszeit.** Von Desinfektion spricht man bei einer Keimreduktion in einem festgelegten Testverfahren mit bestimmten Prüfkörpern um einen Faktor von mindestens 10^-5, das heißt, dass von ursprünglich 1.000.000 vermehrungsfähigen Keimen (sogenannten koloniebildende Einheiten (KbE)) nicht mehr als zehn überleben (Ausnahme: Wäschedesinfektionsverfahren: Keimreduktion um einen Faktor von mindestens 10−7). Auch zur Konservierung werden desinfizierende Verfahren angewandt.

  * **Sterilisation: $10^{-6}$ Dezimalreduktionszeit.** Typischerweise wird für die Sterilisation gefordert, dass die Anzahl der lebenden Individuen um sechs Zehnerpotenzen (also auf ein Millionstel, 10^-6) vermindert wird, daher muss die Dauer der Sterilisation mindestens das Sechsfache der Dezimalreduktionszeit betragen (In der technischen Abgrenzung zur Desinfektion wird bei der Sterilisation in der Regel eine um eine Zehnerpotenz höhere Wahrscheinlichkeit der vollständigen Sterilisation gefordert)

[**Dezimalreduktionszeit**](https://de.m.wikipedia.org/wiki/Sterilisation#Dezimalreduktionszeit): Man bezeichnet die Zeit, in der neun Zehntel der Population absterben, die Population also auf ein Zehntel reduziert wird, als Dezimalreduktionszeit D (D-Wert). Diese Zeit ist stark von der Art oder dem Stamm des Mikroorganismus, der Temperatur und weiteren Bedingungen abhängig, vor allem der Wasseraktivität, dem pH-Wert sowie der Ionenstärke.

![gg](https://upload.wikimedia.org/wikipedia/commons/thumb/3/31/Dezimalreduktionszeit1.svg/320px-Dezimalreduktionszeit1.svg.png)

  * Eine log10-Stufe ist eine Maßeinheit für Keimreduktion um jeweils eine Zehnerpotenz: Eine log10-Stufe reduziert die vorhandene Keimmenge um 90 %; von der Ursprungspopulation 100 haben lediglich zehn überlebt. Das Händewaschen mit Seife entspricht zwei log10-Stufen und reduziert damit die Keimzahl um 99 %. Eine Desinfektion wird erst mit etwa vier bis sieben log10-Stufen erreicht.

**Physikalische Desinfektionsmittel**

* **Hitze**: Thermische Desinfektion

  * [Pasteurisierung](https://de.m.wikipedia.org/wiki/Pasteurisierung), [Thermische Sterilisation](https://de.m.wikipedia.org/wiki/Sterilisation#Thermische_Sterilisation) und [Ultrahocherhitzung](https://de.m.wikipedia.org/wiki/Ultrahocherhitzung)

  *  Die meisten lebenden Krankheitserreger (Mikroorganismen) lassen sich durch ein Erhitzen mit ausreichend hohen Temperaturen abtöten. Dies ist z. B. ein übliches Verfahren bei der Haltbarmachung von Milch und anderen Lebensmitteln (Einkochen).

* **Ionisierende Strahlung**: Desinfektion durch Bestrahlung

  * [Ultraviolettstrahlung: Desinfektion und Virusinaktivierung](https://de.m.wikipedia.org/wiki/Ultraviolettstrahlung#Desinfektion_und_Virusinaktivierung). Ultraviolette Lampen (LEDs) koennen Krankheitserreger (Bakterien) und Viren abtöten, indem es die DNA schädigen. Fern-Ultraviolett-LED entwickelt, die Bakterien töten kann, dabei aber für Menschen unschädlich ist [(Artikel)](https://t3n.de/news/fern-ultraviolett-led-gegen-viren-bakterien-1506450/amp/)

    * Die japanischen Forscher von RIKEN haben ihre LEDs aus verschiedenen Schichten mit unterschiedlichen Anteilen von Aluminium konzipiert. Zusätzlich fügten sie Silizium und Magnesium hinzu.

    * So haben sie eine Art Hinderniskurs für Elektronen erschaffen. Dieser bewirkt, dass sie in ihrer Bewegungsfreiheit eingeschränkt sind und länger in bestimmten Bereichen verharren. Das wiederum führt zu mehr Licht bei weniger Energieverbrauch.

  * [Gammastrahlung: Sterilisation, Keimverminderung, strahlenchemische Vernetzung](https://de.m.wikipedia.org/wiki/Gammastrahlung#Sterilisation,_Keimverminderung,_strahlenchemische_Vernetzung)

  * Siehe auch [Lebensmittelbestrahlung](https://de.m.wikipedia.org/wiki/Lebensmittelbestrahlung) und [Strahlensterilisation](https://www.chemie-schule.de/KnowHow/Sterilisation#Strahlensterilisation)

* **Metalle (Kationen)**: [Oligodynamie](https://de.m.wikipedia.org/wiki/Oligodynamie): Metall-Kationen wie Quecksilber, Silber und Kupfer zeigen ebenfalls eine desinfizierende Wirkung auf viele Krankheitserreger. Beispiel Kupfer: ist für viele Mikroorganismen (Viren, Keime) bereits in geringen Konzentrationen toxisch.

  * Aufgrund der antimikrobiellen Eigenschaft von Kupfer wird das Material z. B. in Krankenhäusern in Großversuchen eingesetzt.

  * Experimente legen nahe, dass die Kontaktabtötung durch einen Mechanismus abläuft, bei dem der Metall-Bakterien-Kontakt die Zellhülle schädigt, was wiederum die Zellen anfällig für weitere Schäden durch Kupfer macht.

  * **Die keimreduzierende Wirkung entsteht genauer gesagt dadurch, dass Kupferionen sich an Thiolgruppen von Proteinen binden und Lipide der Zellmembran peroxidieren, was zur Bildung von freien Radikalen führt, welche die DNA und Zellmembranen schädigen.**

* **Mechanische Desinfektion**:

  * Lyse (Biologie), Zellaufschluss und Membrantechnik

  * Auch durch die mechanische Zerstörung oder Filterung von Krankheitserregern kann ein Medium desinfiziert werden. Dies findet etwa durch [Homogenisierung (Zellaufschluss)](https://de.m.wikipedia.org/wiki/Zellaufschluss), [Ultraschallreinigung (zB Juweliere)](https://de.m.wikipedia.org/wiki/Ultraschallreinigungsgerät), [Mikrofiltration](https://de.m.wikipedia.org/wiki/Mikrofiltration) und [Ultrafiltration](https://de.m.wikipedia.org/wiki/Ultrafiltration) statt.

* **Plasmadesinfektion**: Die Anwendung von [Niedertemperaturplasma](https://de.m.wikipedia.org/wiki/Nichtthermisches_Plasma) (TTP) ist eine Technologie zur Desinfektion mit kaltem [Plasma](https://de.m.wikipedia.org/wiki/Plasma_(Physik)), die bei einer Temperatur von unter 100 °C zeitsparend auch antibiotikaresistente Erreger sogar durch die Kleidung abtöten kann. Sie eignet sich zum Beispiel zur Desinfektion von Luft, Oberflächen, Gegenständen, zur Handdesinfektion und zur Behandlung von schlecht heilenden chronischen Wunden.



**Chemische Desinfektionsmittel**

* [**Oxidationsmittel als Wirkstoffe**](https://de.m.wikipedia.org/wiki/Desinfektion#Oxidationsmittel_als_Wirkstoffe): Alle Stoffe, die als Oxidationsmittel Sauerstoff abspalten, sind bakterizid und wirken sowohl gegen behüllte wie auch unbehüllte Viren. Gegen Pilze, Sporen und Tuberkuloseerreger sind sie nur teilweise und in begrenztem Umfang effektiv, siehe folgende Wirkstoff-Tabelle:

  * **Peressigsäure**	sporizid	Oberflächen, Instrumente
  * **Chlordioxid**	schnell sporizid	Oberflächen, Instrumente, Wasser
  * **Wasserstoffperoxid**	langsam sporizid	Oberflächen, Instrumente, Wasser, Haut, Schleimhaut
  * **Natriumhypochlorit**	sporizid	Oberflächen, Instrumente, Wasser, Haut
  * **Chlor**	langsam sporizid	Wasser, Instrumente
  * **Ozon**	langsam sporizid	Wasser, Instrumente; Ozon für Fahrzeuge
  * **Chloramin T**	sporizid	Oberflächen, Wasser, Instrumente, Haut, Schleimhaut
  * **Iod**	langsam sporizid	Haut, Schleimhaut

* Ein Wirkstoff, der Sporen keimunfähig machen kann, ist ein sporizider Wirkstoff bzw. ein Sporizid. Zu den Sporiziden gehören die in der Tabelle aufgeführten Wirkstoffe wie Peressigsäure, Wasserstoffperoxid, Ozon und Natriumhypochlorit. Es sind stark reagierende bzw. schnell zerfallende Oxidationsmittel.

![ggg](https://raw.githubusercontent.com/deltorobarba/repo/master/sciences_1198.png)

* Auch viele Tenside, insbesondere kationische Tenside wie Benzalkoniumchlorid haben eine biozide Wirkung.

[**Arzneimitteln**](https://de.m.wikipedia.org/wiki/Arzneimittel): [Antibiotika](https://de.m.wikipedia.org/wiki/Antibiotikum), [Virustatika](https://de.m.wikipedia.org/wiki/Virostatikum) und [Antimykotika](https://de.m.wikipedia.org/wiki/Antimykotikum)

* [Metronidazol](https://de.m.wikipedia.org/wiki/Metronidazol)  ist der Hauptvertreter der Nitroimidazole. Diese Gruppe von Antibiotika eignet sich zur Therapie von bakteriellen Infektionen durch Anaerobier oder von Protozoeninfektionen.

  * Metronidazol kann nach Elektronenübertragung auf seine Nitrogruppe die Nukleinsäuresynthese durch Strangbrüche hemmen und wirkt mutagen.

  * Die hierfür nötige reduzierende Reaktion läuft nur unter sauerstoffarmen Verhältnissen ab und entfaltet so einen antimikrobiellen Effekt auf anaerobe Mikroben.

  * Metronidazol wirkt bakterizid gegen (obligate) Anaerobier wie Helicobacter, Gardnerella und Campylobacter sowie schädigend auf einige Protozoen wie Trichomonas, Entamoeba histolytica und Giardia.

  * **Es wird in deren Zellmilieu unter bestimmten Bedingungen (anaerobes Wachstum) durch Reduktion in hochreaktive Zwischenprodukte wie [Acetamid](https://de.m.wikipedia.org/wiki/Acetamid) und N-(2-Hydroxyethyl)-oxamidsäure umgewandelt, die durch Komplexbildung bzw. Verursachen von Strangbrüchen die DNA der Erreger schädigen**.

* Siehe auch [Mutagen](https://de.m.wikipedia.org/wiki/Mutagen) (Antibiotika fur Bakterien)



**Tenside**

1. SOAP DESTROYS THE VIRUS when the water-shunning tails of the soap molecules wedge themselves into the lipid membrane and pry it apart (bindet sich in Fette der Zellmembrane und zerstört sie - Membranbildende Lipide)

2. SOAP TRAPS DIRT and fragments of the destroyed virus in tiny bubbles called micelles, which wash away in water.

![ggg](https://raw.githubusercontent.com/deltorobarba/repo/master/sciences_0751.JPG)

**Sterilisation von Kunststoffen – Autoklavieren**

https://de.m.wikipedia.org/wiki/Autoklav

**Fluorid (antibakteriell)**

* Antibakterielle Wirkung: Fluorid wirkt antibakteriell, da es die **Stoffwechselproduktion der Plaque-Bakterien hemmt**. Auf diese Weise wird auch die Entstehung von Säuren im Stoffwechselprozess gemindert.

* Stärkung des Zahnschmelzes: Fluorid wird auch selbst in den Zahnschmelz eingebaut und trägt so zur Härtung bei. Aus chemischer Sicht funktioniert die Karies-Prophylaxe durch Fluorid so: Fluoride sind Salze des Fluors. Zahnbein und Zahnschmelz bestehen zu einem großen Teil aus dem Mineral Hydroxylapatit, einer Form des Calciumphosphats.Putzen wir uns die Zähne mit fluoridhaltiger Zahnpasta, wird die Verbindung von Wasserstoff und Sauerstoff, also die Hydroxylgruppe, ausgetauscht gegen Fluorid. So entsteht Fluorapatit anstelle des Hydroxylapatits. Fluorapatit ist härter und säurefester als Hydroxylapatit. So sind unsere Zähne besser geschützt vor Karies. Fluoride sollen außerdem der Säureproduktion von Kariesbakterien entgegenwirken.

##### <font color="blue">*Radioaktive Strahlung*

https://www.welt.de/wissenschaft/plus226444035/Einsteinium-Das-hochradioaktive-Element-mit-99-Elektronen.html

video: [types of radiation explained](https://youtu.be/Nj4pxrdAHzc)

###### *Radioaktivität: Alpha-, Beta-, Gamma-Radiation*

https://www.weltderphysik.de/gebiet/leben/einfluesse-auf-den-menschen/strahlungsauswirkung/

https://www.kernenergie.ch/de/radioaktivitaet-_content---1--1086.html

https://de.m.wikipedia.org/wiki/Radioaktivität

https://de.m.wikipedia.org/wiki/Strahlung#Teilchenstrahlung

https://um.baden-wuerttemberg.de/de/umwelt-natur/kernenergie-und-strahlenschutz/strahlenschutz/informationen-zum-strahlenschutz/radioaktivitaet-und-ionisierende-strahlung/natuerliche-zerfallsreihen/

> **Radioaktivität: Eigenschaft von Atomkernen, sich in
andere Atomkerne umzuwandeln**

**Radioaktive Strahlung wirkt auf Zellteilfähigkeit, nicht auf kinetische Energie der Teilchen (wie zB Infrarotstrahlung als Wärme).** Daher spürt man radioaktive Strahlung auch nicht.

Chemotherapie wendet Gammastrahlung an, weil Alpha- und Betastrahlung kaum in die Haut eindringen.

![ggg](https://raw.githubusercontent.com/deltorobarba/repo/master/sciences_0891.png)

![ggg](https://raw.githubusercontent.com/deltorobarba/repo/master/sciences_0893.jpg)

*Vereinfachende Darstellung zu der relativen Fähigkeit von drei verschiedenen Arten ionisierender Strahlung, Feststoffe zu durchdringen. Typische Alpha-Partikel (α) werden durch ein Blatt Papier gestoppt, während Beta-Partikel (β) haben mehrere Meter Reichweite und werden durch eine Aluminiumplatte gestoppt werden. Gammastrahlung (γ) wird gedämpft, wenn sie in Blei eindringt:*

![gg](https://upload.wikimedia.org/wikipedia/commons/thumb/6/61/Alfa_beta_gamma_radiation_penetration.svg/640px-Alfa_beta_gamma_radiation_penetration.svg.png)

*Bei der Radioaktivität wird ein Atomkern in einen anderen umgewandelt. Dafur kann der Alpha- oder der Beta-Zerfall verantwortlich sein. Infolge dieser Zerfaelle kann zusaetzlich noch Gamma-Strahlung auftreten. Und weil sich diese drei Faelle so stark unterscheiden, haben auch die daraus resultierenden Strahlungen ganz. unterschiedliche Eigenschaften.*

![ggg](https://raw.githubusercontent.com/deltorobarba/repo/master/sciences_0892.png)

**Exkurs: Alpha-, Beta-, Gamma- und Neutronenstrahlung als Arten der Ionisierende Strahlung**

> Freie Protonen, Elektronen oder andere geladene Teilchen werden ab einer kinetischen Energie von etwa 5 eV zur ionisierenden Strahlung gezählt. **Demnach sind [Alphastrahlung](https://de.m.wikipedia.org/wiki/Alphastrahlung) (positiv geladene Heliumkerne) und [Betastrahlung](https://de.m.wikipedia.org/wiki/Betastrahlung) (negativ geladene Elektronen oder positiv geladene Positronen) stets ionisierende Strahlung**.

* [Teilchenstrahlung](https://de.m.wikipedia.org/wiki/Strahlung#Teilchenstrahlung):

  * Die Ausdrücke Teilchenstrahlung und Korpuskularstrahlung werden manchmal als Oberbegriffe für Strahlung verwendet, deren Bestandteile eine von Null verschiedene Masse haben.

  * Teilchenstrahlung unterscheidet man nach der Sorte der Teilchen, aus denen sie besteht, beispielsweise **Alphastrahlung** (α-Teilchen), **Betastrahlung** (Elektronen oder Positronen) oder [Neutronenstrahlung](https://de.m.wikipedia.org/wiki/Neutronenstrahlung). Handelt es sich um Ionen, ist auch (gelegentlich) von Ionenstrahlung die Rede.

* Zusammenhang mit [Compton-Effekt](https://de.m.wikipedia.org/wiki/Compton-Effekt): Die sehr geringen Wellenlängenänderungen sind der Grund dafür, dass der Compton-Effekt nur bei sehr kurzwelliger Strahlung, im Bereich der Röntgen- und Gammastrahlung, beobachtet werden kann. Bei großer Wellenlänge ist deren relative Zunahme zu gering, die Streuung scheint ohne Energieverlust stattzufinden, man spricht dann von [Thomson-Streuung](https://de.m.wikipedia.org/wiki/Thomson-Streuung).

![ggg](https://raw.githubusercontent.com/deltorobarba/repo/master/sciences_0784.jpeg)

https://www.youtube.com/watch?v=N_UbowOhN6s

###### *Radioaktive Quellen*

Uran, Radon, Cösium, Kalium oder Polonium zerfallen

250 mSv milli Sievert
Tödlich ab 5000 mSv
Natürlich: 4,5 mSv pro Jahr,
* davon 7% Bodenstrahlung,
* 7% aus Nahrung (pro Banane 12 Bequerel an Kalium40), Beqceurel gibt an, wieiviel Kalium in 1 sek zerfällt, in DE 600 B pro Kg Grenzwert, bei Babynahrung 370. getreide etwas höherer Anteil. Pilze und wildfleisch relativ hoch (wegen Cäsium von Tschernobyl)
* 9% Höhenstrahlung (kosmische Strahlung). Von fremden Galaxien und von Sonnenwind. Beim Fliegen 100x stärker. In Hamburg bei 0,3mSv pro Jahr, Zugspitze 1 mSv pro Jahr,
* 31% Radon in der Luft, in Süddeutschland, baumaterialien: beton, ziegel, lehm-stroh-gemische
* 45% medizinische Verfahren

###### *Radioaktivität: $\alpha$-Zerfall (Helium-Atomkerne, Protonen und 2 Neutronen, entweichen dem Atomkern)*

> Helium-Atomkerne (2 Protonen und 2 Neutronen) entweichen dem Atomkern

*4 = Massenzahl, Nukleonenzahl, Ordnungszahl = Anzahl Protonen und Neutronen im Atomkern:*

![ggg](https://raw.githubusercontent.com/deltorobarba/repo/master/sciences_0863.png)

*Dieses Helium-Atom ist zweifach positiv geladen, es hat also beide Elektronen verloren*:

![ggg](https://raw.githubusercontent.com/deltorobarba/repo/master/sciences_0864.png)

*Beim Alpha-Zerfall wird ein einfacher Helium-Atomkern aus dem Atomkern geschossen. Diesen Helium-Atomkern nennt man dann Alpha-Teilchen:*

![ggg](https://raw.githubusercontent.com/deltorobarba/repo/master/sciences_0865.png)

*Das Actinium-Atom hat 89 Protonen:*

![ggg](https://raw.githubusercontent.com/deltorobarba/repo/master/sciences_0866.png)

*Damit der Kern von Actinium auch radioaktiv zerfaellt brauchen wir ein Isotop von Actinium. Das heisst, dass das Actinium zwar die gleiche Protonenanzahl behaelt, aber mehr oder weniger Neutronen bekommt. Statt einer Massenzahl von 178 kann ein Isotop von Actinium zum Beispiel eine Massenzahl von 226 haben:*

![ggg](https://raw.githubusercontent.com/deltorobarba/repo/master/sciences_0867.png)

*Schiessen wir jetzt das Alpha-Teilchen aus dem Atomkern, dann fehlen dem Actinium 2 Protonen und 2 Neutronen:*

![ggg](https://raw.githubusercontent.com/deltorobarba/repo/master/sciences_0868.png)

*Wir haben jetzt also eine Ordnungszahl von 87 und eine Massenzahl von 222:*

![ggg](https://raw.githubusercontent.com/deltorobarba/repo/master/sciences_0869.png)

*Damit veraendert sich das Element selbst. Es handelt sich nicht laenger um das Element Actinium, sondern ist jetzt das Element Francium. **Beim Alpha-Zerfall entsteht aus einem Usprungselelement also ein neues Element**:*

![ggg](https://raw.githubusercontent.com/deltorobarba/repo/master/sciences_0870.png)

###### *Radioaktivität: $\beta^-$-Zerfall (Neutron $\rightarrow$ Proton, -Elektron)*

> **Beim Beta-Plus-Zerfall wandelt sich ein Neutron in ein Proton um, unter Abgabe eines Elektrons**

https://de.m.wikipedia.org/wiki/Betastrahlung

Beispiel: [Jod 131](https://de.m.wikipedia.org/wiki/Iod#Isotope)

![ggg](https://raw.githubusercontent.com/deltorobarba/repo/master/sciences_0871.png)

*Das Actinium-Atom besitzt deutlich mehr Neutronen als Protonen:*

![ggg](https://raw.githubusercontent.com/deltorobarba/repo/master/sciences_0872.png)

*Um das auszugleichen koennen sich einige der Neutronen in Protonen umwandeln. Dazu geben sie einfach ein Elektron ab. Dieses Elektron wird Beta-Teilchen genannt. Und weil ein Elektron immer mit einem Minus dargestellt wird, heisst dieser Zerfall auch Beta-Minus-Zerfall:*

![ggg](https://raw.githubusercontent.com/deltorobarba/repo/master/sciences_0873.png)

*Das Actinium hat jetzt 1 Proton mehr. Es besitzt damit eine neue Ordnungszahl von 90:*

![ggg](https://raw.githubusercontent.com/deltorobarba/repo/master/sciences_0874.png)

*Mit einer Ordnungszahl von 90 wird das Element zu Thorium:*

![ggg](https://raw.githubusercontent.com/deltorobarba/repo/master/sciences_0875.png)

###### *Radioaktivität: $\beta^+$-Zerfall (Proton $\rightarrow$ Neutron, -Positron)*

> **Beim Beta-Plus-Zerfall wandelt sich ein Proton in ein Neutron um, unter Abgabe eines Positron**

https://de.m.wikipedia.org/wiki/Betastrahlung

*Ein Proton wandelt sich in ein Neutron um:*

![ggg](https://raw.githubusercontent.com/deltorobarba/repo/master/sciences_0876.png)

*Dazu gibt das Proton ein Positron ab, also ein positiv geladenes Teilchen. Die Ordnungszahl wird um 1 reduziert von 89 auf 88*

![ggg](https://raw.githubusercontent.com/deltorobarba/repo/master/sciences_0877.png)

*Aus dem Actinium wird auf diese Weise Radium:*

![ggg](https://raw.githubusercontent.com/deltorobarba/repo/master/sciences_0878.png)

**Und da das Positron mit einem Plus dargestellt wird, heisst dieser Prozess auch Beta-Plus-Zerfall**.

> Und anders als beim Beta-Minus-Zerfall kann das sowohl bei mehr Protonen als auch bei mehr Neutronen im Kern passieren.

https://www.youtube.com/watch?v=N_UbowOhN6s&list=WL&index=18&t=2s

###### *Radioaktivität: $\gamma$-Zerfall (elektromagnetische Strahlung, kein neues Element entsteht)*

*Bei der Gamma-Strahlung findet kein Kernzerfall statt:*

![ggg](https://raw.githubusercontent.com/deltorobarba/repo/master/sciences_0879.png)

*Stattdessen wird eine energiereiche elektromagnetische Strahlung ausgesendet. Das passiert, wenn ein Kern nach eine Alpha- oder Beta-Zerfall zuviel Energie hat. Bei der Gamma-Strahlung entsteht also **kein** neues Element:*

![ggg](https://raw.githubusercontent.com/deltorobarba/repo/master/sciences_0890.png)

###### *Radioaktivität: Neutronenstrahlung*

Video: https://youtu.be/Nj4pxrdAHzc

neutron radiation: nuclear fission, nuclear fusion, neutrom emission

Neutronenstrahlung ist eine ionisierende Teilchenstrahlung, die aus freien Neutronen (mit u. U. verschiedenen kinetischen Energien) besteht.



https://de.m.wikipedia.org/wiki/Neutronenstrahlung

http://www.umweltinstitut.org/themen/radioaktivitaet/hintergrundinfos-radioaktivitaet/was-ist-radioaktivitaet.html

###### *Radioaktivität: Wirkung auf den Körper*

https://youtu.be/5nPSgZwfVb4

![ggg](https://raw.githubusercontent.com/deltorobarba/repo/master/sciences_0893.jpg)

Beispiel: radioaktives [Jod 131](https://de.m.wikipedia.org/wiki/Iod#Isotope) ist ein $\beta^-$-Strahler (=schiesst Elektronen in die Umgebung).
* Der Koerper lagert nicht-radioaktives Iod 127 aber auch radioaktives Jod 131 in der Schilddruese an. Es macht daraus die wichtigen Schilddruesenhormone.
* Gelangt radioaktives Jod in das Gewebe, richtet es dort grossen Schaden an.
* Die Elektronen, die vom Iod abgegeben werden, zerstoeren die Atomhuellen von den Molekulen in den Zellen. Aus den Zellmolekulen werden Elektronen rausgeschlagen. Die Ladung der Molekule aendert sich damit.
* Deshalb nennt man diese Strahlung auch [ionisierende Strahlung](https://de.m.wikipedia.org/wiki/Ionisierende_Strahlung).

![ggg](https://raw.githubusercontent.com/deltorobarba/repo/master/sciences_0894.png)

Wenn jetzt aus den Molekulen Bindungselektronen entfernt werden, dann bleiben chemisch agressive Molekulreste, die [freien Radikale](https://de.m.wikipedia.org/wiki/Radikal_(Chemie)), zurueck.
* Sie haben ein ungepaartes Elektron, das andere Molekule angreift, weil es nach einem zweiten Elektron sucht.
* Diese Radikale koennen sich also mit anderen Molekulen in der Zelle verbinden.
* Das fuehrt dazu, dass alle moeglichen Zellbausteine beschaedigt oder sogar zerstoert werden koennen. Sind die Schaeden zu gross, dann stirbt die Zelle. Verstorbe Zellen in der Schilddruese koennen schwere gesundheitliche Folgen haben.

![ggg](https://raw.githubusercontent.com/deltorobarba/repo/master/sciences_0895.png)

Am schlimmsten ist es, wenn das Ergbut betroffen ist.
* Es kann direkt durch die ionisierende Strahlung oder haupstaechlich auch durch die entstandenen Radikale veraendert werden.
* Es entstehen Mutationen, oder Zerstoerungen die DNA-Strangbrueche.

![ggg](https://raw.githubusercontent.com/deltorobarba/repo/master/sciences_0896.png)

Sind normale Koerperzellen betroffen, dann erhoehen diese Veraenderungen und Beschaedigungen das Risiko fur Krebs.
* Wenn Gene betroffen sind, die die Zellteilung regulieren, wovon es sehr viele gibt, kann es zu unkontrolliertem Wachstum kommen und so zu Krebsgeschwueren (zb Blutkrebs / Leukamie).
* Siehe [Zellzyklus](https://de.m.wikipedia.org/wiki/Zellzyklus): Mitose $\rightarrow$ Zytokinese $\rightarrow$ Interphase $\rightarrow$ Mitose ..
* Sind Keimzellen betroffen, gibt es ein Risiko fur Fehlbildungen der nachsten Generation. Die koennen je nach betroffenen Genen schlimme Auswirkungen haben.

Strontium 90 und Casium 137, beide sind radioaktiv, werden vom Koerper im Knochen und im Muskelgewebe eingelagert, weil er sie mit Calcium (40) verwechselt.
* Im Knochenmark finden sich aber auch die Zellen, die Blutkoerperchen bilden.
* Werden diese beschaedigt, dann geraet die Blutkoerperchenbildung ausser Kontrolle und es kann zu **Leukamie** kommen.
* Strontium 90 und Casium 137 sorgen auch sehr hauefig fur **Knochenkrebs**.

![ggg](https://raw.githubusercontent.com/deltorobarba/repo/master/sciences_0897.png)

> Es gibt daruber hinaus noch die [akute Strahlenkrankheit](https://de.m.wikipedia.org/wiki/Strahlenkrankheit) bei einer sehr hohen Dosis an radioaktiver Strahlung.

###### *Ionisierende und nichtionisierende Strahlung*

> **Siehe auch: Chemische Elemente $\rightarrow$ Radioaktivität: Wirkung auf den Körper**

> **Extreme ultraviolet, soft X-rays, hard X-rays and gamma rays are classified as ionizing radiation** as their photons have enough energy to ionize atoms, causing chemical reactions. Exposure to these rays can be a health hazard, causing radiation sickness, DNA damage and cancer. Radiation of visible light wavelengths and higher are called nonionizing radiation as they cannot cause these effects.

> For details about **Radioactivity (Alpha-, Beta-, Gamma-Radiation)** please check under Chemie > Elemente and after Isotopes!

[Nichtionisierende Strahlung](https://de.m.wikipedia.org/wiki/Nichtionisierende_Strahlung) sind diejenigen elektromagnetischen Wellen, deren Energie nicht ausreicht, um andere Atome zu [ionisieren](https://de.m.wikipedia.org/wiki/Ionisation), da die Energiemenge der Photonen unter den meisten [Bindungsenergien](https://de.m.wikipedia.org/wiki/Bindungsenergie) liegt. Dazu zählen insbesondere technisch genutzte Frequenzen im Bereich der Radiowellen und Mikrowellen sowie der größte Teil des sichtbaren Lichtes.

* <font color="blue">**Photonen mit einer Energie unter rund 3 eV gelten als nichtionisierend**, da diese Energie kleiner als die **typischen Bindungsenergien sind, welche im Bereich von 3 eV bis 7 eV liegen**. Moleküle, die durch Strahlung unter 3 eV zerstört werden, können bei Zimmertemperatur nicht existieren. Sie werden durch die [thermische Anregung](https://de.m.wikipedia.org/wiki/Angeregter_Zustand) zerstört.

* Zu der nichtionisierenden Strahlung werden elektromagnetische Wellen im Frequenzbereich unter 750 THz oder einer Wellenlänge von mehr als 400 nm gezählt. Der Zusammenhang zwischen Energie und der Frequenz ergibt sich mit dem planckschen Wirkungsquantum.

* Zu der nichtionisierenden Strahlung zählt im oberen Bereich Infrarotstrahlung und im Grenzbereich zur ionisierenden Strahlung sichtbares Licht mit Wellenlängen von 400 nm bis 780 nm. Wellenlängen unter 400 nm, welche als UV-Strahlung bezeichnet werden, werden im Regelfall zu der ionisierenden Strahlung gezählt.

* Bindungsenergien zwischen Atomen liegen bei Molekülen zwischen 200 und 700 kJ·mol−1 (2 bis 7 eV pro Bindung).

* Besonders geringe Bindungsenergien beobachtet man bei Wasserstoffbrückenbindungen. Sie sind mit nur 17 bis 167 kJ/mol (0,18 bis 1,7 eV pro Bindung) deutlich schwächer als die Bindungskraft innerhalb eines Moleküls.

* siehe auch [Ionisierende Strahlung](https://de.m.wikipedia.org/wiki/Ionisierende_Strahlung): Zur ionisierenden Strahlung rechnet man jede Strahlung, deren kinetische Energie (bei Teilchen) bzw. Quantenenergie (bei Wellen) ausreicht, um Elektronen – auch über Zwischenreaktionen – aus einem Atom oder Molekül herauszulösen.

  * Um die dazu nötige Ionisationsenergie aufzubringen, muss die Teilchen- oder **Quantenenergie meist mehr als etwa 5 Elektronenvolt (eV)** betragen.

  * **Im elektromagnetischen Spektrum entspricht das Wellenlängen von weniger als etwa 250 nm**; daher haben nur Höhenstrahlung (kosmische Strahlung), Gammastrahlung, Röntgenstrahlung und kurzwelligere Ultraviolettstrahlung genügend Quantenenergie, um Elektronen aus den Atomhüllen zu lösen und so auch Kovalente Bindungen aufzutrennen.

  * Dagegen sind Radiowellen, Radarwellen, Mikrowellen, Infrarotstrahlung oder sichtbares Licht keine ionisierende Strahlung, denn sie können keine Moleküle (außer von speziellen, lichtempfindlichen Substanzen) dauerhaft verändern oder gar zerlegen. Moleküle, die durch so energiearme Photonen zerlegt würden, könnten unter Normalbedingungen nicht existieren.


*Article:* [*Non-ionizing radiation*](https://en.m.wikipedia.org/wiki/Non-ionizing_radiation)

![ggg](https://upload.wikimedia.org/wikipedia/commons/thumb/4/48/EM-spectrum.svg/640px-EM-spectrum.svg.png)

*Ionisationsenergie in Abhängigkeit von der Ordnungszahl*

![ggg](https://upload.wikimedia.org/wikipedia/commons/thumb/1/1d/First_Ionization_Energy.svg/1024px-First_Ionization_Energy.svg.png)

##### <font color="blue">*Isotopenuntersuchung*

###### *Isotope*

> Normalerweise ist **Anzahl der Protonen = Anzahl der Neutronen** im Atomkern (und dazu noch Elektronen aussen herum)

* **Gibt es mehr Neutronen als Protonen, spricht man von Isotopen**.

* Die verschiedenen Atomsorten (Elemente) unterscheiden sich in der Anzahl der Protonen, die im Atomkern stecken.

* Die Ordnungszahl im Periodensystem sagt uns, wie viele Protonen im Kern vorhanden sind.

![ggg](https://raw.githubusercontent.com/deltorobarba/repo/master/sciences_1248.png)

![ggg](https://raw.githubusercontent.com/deltorobarba/repo/master/sciences_1247.png)

**Isotope: unterschiedliche Neutronenanzahl**

* **zB Kohlenstoff kommt in drei Isotopen vor (98,8% haben 6 Neutronen (=Anzahl der Protonen), aber es gibt auch einige mit 7 und 8 Neutronen. C12, C13 und C14**.

* Von [Iod sind bisher 36 Isotope](https://de.m.wikipedia.org/wiki/Iod#Isotope) und 10 Kernisomere bekannt. Von diesen ist nur ein Isotop stabil, so dass natürlich vorkommendes Iod zu 100 % aus dem einzigen stabilen Isotop 127I besteht. Iod ist daher ein Reinelement (anisotop). Von den instabilen Isotopen besitzt der Betastrahler 129I mit 15.700.000 Jahren eine sehr lange Halbwertszeit. Daneben gibt es vier Isotope mit mehr als einem Tag Halbwertszeit: 124I (4,2 Tage), 125I (59 Tage), 126I (13 Tage) und 131I (8,0 Tage).

* Masse eines Atoms bestimmt durch Anzahl an Protonen und Neutronen im Kern (Elektronen wiegen fast nichts).

* Wasserstoff: H1 (ohne Neutron). Mit einem Neutron: H2 = Deuterium. Letzteres ist doppelt so schwer wie H1. Daher nennt man D2O auch schweres Wasser (Wasser das aus Deuterium besteht, anstatt Wasserstoffof mit H2O).

* Zuviele oder zuwenig Neutronen = Atomkern wird instabil und zerfällt (Radioaktivität).

* C14 durch Reaktion zw kosmischer Strajlung und Stickstoffatomen in der Atmosphäre. Wir alle haben einen kleine Anteil davon in uns.

###### *Isotopes of Chromium*

https://en.m.wikipedia.org/wiki/Isotopes_of_chromium

###### *Radiometrische Datierungsmethoden (Isotopenuntersuchung)*

![ggg](https://raw.githubusercontent.com/deltorobarba/repo/master/sciences_0737.jpg)

**In der Biologie:** So kann beispielsweise aus dem mikroskopisch sichtbar zu machenden Zahnschmelz-Abrieb auf die Härte der regelmäßig verzehrten Nahrung geschlossen werden, auch werden gelegentlich im Zahnbelag eingebettete, versteinerte Samen oder Stärkekörner (Phytolithen) entdeckt.

- Zu den physikalischen Methoden gehören ferner – neben der seit 1946 bekannten C14-Datierung und anderen radiometrischen Datierungsmethoden – die Bestimmung der Isotopenverhältnisse von 13C und 12C, von 15N und 14N sowie von 18O und 16O:
- Anhand von δ13C kann die Bevorzugung von C3-Pflanzen von der Bevorzugung von C4-Pflanzen unterschieden werden;
- anhand von Δ15N können Fleischfresser und Pflanzenfresser unterschieden werden;
- anhand von Δ18O kann unterschieden werden, ob ein Tier primär Wasser aus offenem Gewässer getrunken oder ob es seinen Flüssigkeitsbedarf primär durch das Verzehren von Laub gedeckt hat.

Isotope geochemistry

https://en.m.wikipedia.org/wiki/Isotope_geochemistry

https://www.chemie.de/lexikon/Isotopenuntersuchung.html

https://pubs.usgs.gov/gip/geotime/radiometric.html

[Methoden der Geochronologie](https://de.m.wikipedia.org/wiki/Geochronologie)

[Isotopenverhältnis](https://de.m.wikipedia.org/wiki/Isotopenverhältnis)

[Isotopengeochemie](https://de.m.wikipedia.org/wiki/Isotopengeochemie)

[Isotopeneffekt](https://de.m.wikipedia.org/wiki/Isotopeneffekt)

[Isotopenuntersuchung](https://de.m.wikipedia.org/wiki/Isotopenuntersuchung)

[Radiometrische Datierung](https://de.m.wikipedia.org/wiki/Radiometrische_Datierung)

###### *Isotope Wasserstoff vs Deuterium*

Genutzt in der Astronomie und Geologie

Warum gibt es Wasser auf der Erde?

  * Isotope Wasserstoff Deuterium: 150 auf Erde. Protosolarem Nebel war es 25.

  * Ähnliche Zusammensetzung haben nur die [kohligen Chondrite](https://de.m.wikipedia.org/wiki/Kohliger_Chondrit).

  * Kometen haben zu hohes Wasserstoff zu Deuterium Verhältnis.

  * Im frühen sonnensystem: wo könnten objekte sein mit wasser? (Am temperaturverlauf, wo temperatur so niedrig, dass objekte gefrorenes wasser tragen könnten?). Es stellt sich heraus: Abstand zw Mars und Jupiter. Von dort kamen Objekte mit Wasser auf die Erde.

  * Simulationen ergaben, dass 3-4 grosse Brocken reichten.

![ggg](https://raw.githubusercontent.com/deltorobarba/repo/master/sciences_0733.png)

###### *Uran-Blei-Datierung*

[Uran-Blei-Datierung](https://de.m.wikipedia.org/wiki/Uran-Blei-Datierung)

* Uran 238

* Uran 235

Beispiel: Sterkfontain-Höhle in Südafrika, https://www.wissenschaft.de/geschichte-archaeologie/australopithecus-fossilien-aelter-als-gedacht/

###### *Rubidium-Strontium-Datierung*

[Rubidium-Strontium-Datierung](https://de.m.wikipedia.org/wiki/Rubidium-Strontium-Datierung)


###### *Kalium-Argon-Datierung (Argon-Argon)*

[Kalium-Argon-Datierung](https://de.m.wikipedia.org/wiki/Kalium-Argon-Datierung)


###### *Uran-Thorium-Datierung*

[Uran-Thorium-Datierung](https://de.m.wikipedia.org/wiki/Uran-Thorium-Datierung) bzw. [chemie.de](https://www.chemie.de/lexikon/Uran-Thorium-Datierung.html) - wurde u.a. benutzt, um das Alter der Homo Naledi Fossilien zu bestimmen


###### *Kohlenstoff-Stickstoff-Datierung (C14 Radiokarbonmethode)*

[Radiokarbonmethode C14](https://de.m.wikipedia.org/wiki/Radiokarbonmethode)


###### *Aluminium-26 und Beryllium-10*

Sie analysierten die steinernen Anhaftungen der Fossilien auf ihren Gehalt an bestimmten seltenen Isotopen hin. Diese sogenannte **kosmogenen Nuklide Aluminium-26 und Beryllium-10 entstehen im gesteinsbildenden Mineral Quarz**, solange dieses an der Erdoberfläche der energiereichen kosmischen Strahlung ausgesetzt ist. „Als diese Gesteine zusammen mit den Fossilien in die Höhle fielen, begann ihr radioaktiver Zerfall“, erklärt Granger. Der Gehalt dieser Isotope verrät daher, wie lange dieses die Fossilien umhüllende Gestein schon von darüberliegenden Ablagerungen bedeckt ist.

https://www.wissenschaft.de/geschichte-archaeologie/australopithecus-fossilien-aelter-als-gedacht/

###### *Delta-Datierung: Kohlenstoff (δ13C)*

Biogenic Carbon: Kohlenstoff: https://de.m.wikipedia.org/wiki/Δ13C (C-12 to C-13 Ratio)

> Anhand von δ13C kann die Bevorzugung von [C3-Pflanzen](https://de.m.wikipedia.org/wiki/C3-Pflanze) (Pflanzengruppe mit üblicher Kohlehydrat-Synthese im Calvin-Zyklus) von der Bevorzugung von [C4-Pflanzen](https://de.m.wikipedia.org/wiki/C4-Pflanze) (Pflanzen mit zweistufiger Kohlenhydrat-Synthese über Dicarbonsäuren und im Calvin-Zyklus) unterschieden werden.

https://gml.noaa.gov/ccgg/isotopes/c13tellsus.html

* Looking for earliest life, like 4 billion years ago (beginning of Archaen eon): you are not finding fossiles, but looking for **biogenic carbon - chemical evidence of life**

* Important to paleontologists: all living things have a preference for Carbon 12 isotopes over Carbon 13. Carbon 12 makes up 99% of all the carbon on earth. And our cells use them with help of enzymes.

* Because carbon 12 hats one less neutron than carbon 13, it's lighter and easier for our cells to use

* If enzymes in some microbes bump into atoms of Carbon 12 - Those atoms will be easier for the enzymes to take up and incorporate. But the Carbon 13 tends to get left behind.

* As a result, the carbon in that microbe will have a much lower ratio of carbon-13 to carbo 12 than something that was never alive.

* Specifically that ratio will be 25% lower. So minus 25% ratio means that you are looking at **biogenic carbon**.

* That's what scientists are looking for when they are searching for the earliest signs of life on earth

> It looks more and more that life goes back to the Hadean eon more than 4 bio years ago - almost the entire time of the planet.

![ggg](https://raw.githubusercontent.com/deltorobarba/repo/master/sciences_374.png)

*Some findings of biogenic carbon date back to 3.7 to 3.95 Bio years (still under debate):*

![ggg](https://raw.githubusercontent.com/deltorobarba/repo/master/sciences_375.png)

*The potentially oldest biogenic carbon ever found: trapped inside of a single crystal of zircon from the Jack Hills in Western Australia: they are practically indestructible: While most rocks haven't been able to last 4 bio years, sometimes zircon crystals do. Researchers found in 2015 inside of Carbon a bit of carbon that was 4.1 bio years old. The sample had the same ratio of carbon-13 to carbon-12 as biogenic carbon: https://www.pnas.org/doi/abs/10.1073/pnas.1517557112*

![ggg](https://raw.githubusercontent.com/deltorobarba/repo/master/sciences_376.png)

Video: [The Search for the Earliest Life](https://www.youtube.com/watch?v=uCVnRIP3pIk&list=WL&index=14)

###### *Delta-Datierung: Stickstoff (δ15N)*

Stickstoff: https://de.m.wikipedia.org/wiki/Δ15N

> Anhand von Δ15N können Fleischfresser und Pflanzenfresser unterschieden werden

###### *Delta-Datierung: Sauerstoff (δ18O)*

Sauerstoff: https://de.m.wikipedia.org/wiki/Δ18O

> Anhand von Δ18O kann unterschieden werden, ob ein Tier primär Wasser aus offenem Gewässer getrunken oder ob es seinen Flüssigkeitsbedarf primär durch das Verzehren von Laub gedeckt hat.

*Klimadiagramm anhand von 57 weltweiten Tiefseesediment-Bohrkernen. Gemessen wurde das Isotop δ18O in bentischen Foraminiferen, das als Proxy für die Gesamtmasse von Gletschereis gilt:*

![ggg](https://upload.wikimedia.org/wikipedia/commons/thumb/a/ad/Five_Myr_Climate_Change_%28de%29.svg/800px-Five_Myr_Climate_Change_%28de%29.svg.png)

δ18O (in Promille ‰) im Tiefseesediment; Karbonate der Foraminifera über einen Zeitraum von 0 bis 600.000 Jahren, gemittelt über eine große Zahl von Bohrkernen, um ein globales Signal isolieren zu können [Source](https://de.m.wikipedia.org/wiki/Δ18O):

![ggg](https://upload.wikimedia.org/wikipedia/commons/thumb/f/f0/Delta-O-18_in_deep_sea_sediment-svg.svg/746px-Delta-O-18_in_deep_sea_sediment-svg.svg.png)

###### *Sauerstoff-Isotopenstufe (MIS)*

* [Sauerstoff-Isotopenstufe](https://de.m.wikipedia.org/wiki/Sauerstoff-Isotopenstufe): Datierung von Sedimentschichten anhand der in ihnen enthaltenen stabilen Isotope des Sauerstoffs

* Beispiel: Genetischer Flaschenhals von Menschen: Sie wird als geologisches Stadium **Sauerstoff-Isotopenstufe 6 (MIS 6)** bezeichnet. „Lebten vorher wohl stets über 10.000 Erwachsene im fortpflanzungsfähigen Alter, so dürften es nun kaum noch einige hundert gewesen sein. […] Der Kontinent wies zu jener Zeit nur wenige Gegenden auf, die für Jäger und Sammler genügend Ressourcen bereithielten.“ Diese Kaltzeit entspricht in Nordeuropa einem Teil des Saale-Komplexes, im Alpenraum der Riß-Kaltzeit.

###### *Thermolumineszenz-Datierung*

[Thermolumineszenzdatierung](https://de.m.wikipedia.org/wiki/Thermolumineszenzdatierung)

### <font color="blue">**Chemische Verbindungen**

##### <font color="blue">*Atombindungen (kovalent)*

###### *Atombindung*

**Atombindung (kovalente Bindung oder Elektronenpaarbindung)**

> Bei [Atombindungen](https://de.m.wikipedia.org/wiki/Kovalente_Bindung) werden Elektronen untereinander geteilt  (Elektronenpaare) und nicht vergeben wie bei Ionenverbindungen

* Atombindungen sind Bindungen Nichtmetall-Nichtmetall: [Nichtmetalle](https://de.m.wikipedia.org/wiki/Nichtmetalle) bzw. [Nonmetal](https://en.m.wikipedia.org/wiki/Nonmetal) entsprechend der vorstehenden Definition sind:

  * [Wasserstoff](https://de.m.wikipedia.org/wiki/Wasserstoff), [Kohlenstoff](https://de.m.wikipedia.org/wiki/Kohlenstoff), [Stickstoff](https://de.m.wikipedia.org/wiki/Stickstoff), [Phosphor](https://de.m.wikipedia.org/wiki/Phosphor), [Sauerstoff](https://de.m.wikipedia.org/wiki/Sauerstoff), [Schwefel](https://de.m.wikipedia.org/wiki/Schwefel), [Selen](https://de.m.wikipedia.org/wiki/Selen),

  * die [Halogene](https://de.m.wikipedia.org/wiki/Halogene) ([Fluor](https://de.m.wikipedia.org/wiki/Fluor), [Chlor](https://de.m.wikipedia.org/wiki/Chlor), [Brom](https://de.m.wikipedia.org/wiki/Brom), [Iod](https://de.m.wikipedia.org/wiki/Iod), [Astat](https://de.m.wikipedia.org/wiki/Astat) und [Tenness](https://de.m.wikipedia.org/wiki/Tenness))

  * die [Edelgase](https://de.m.wikipedia.org/wiki/Edelgase) ([Helium](https://de.m.wikipedia.org/wiki/Helium), [Neon](https://de.m.wikipedia.org/wiki/Neon), [Argon](https://de.m.wikipedia.org/wiki/Argon), [Krypton](https://de.m.wikipedia.org/wiki/Krypton), [Xenon](https://de.m.wikipedia.org/wiki/Xenon), [Radon](https://de.m.wikipedia.org/wiki/Radon) und [Oganesson](https://de.m.wikipedia.org/wiki/Oganesson)).

  * Siehe auch [Dividing line between metals and nonmetals](https://en.m.wikipedia.org/wiki/Dividing_line_between_metals_and_nonmetals)

* Elektronen auf der **Aussenschale (= Valenzschale)** sind wichtig für Bindungen

* wieviele Elektronen in der Aussenschale sind sieht man an der Hauptgruppe im Periodensystem

* Oktettregel: alle Elemente wären gerne so wie die Edelgase (8 Aussenelektronen

  * Die [Oktettregel](https://www.chemie.de/lexikon/Oktettregel.html) oder Acht-Elektronen-Regel besagt, dass viele Atome Moleküle oder Ionen bilden, bei denen die Zahl der äußeren Elektronen (Valencelektronen) acht beträgt, d. h., dass die Elemente versuchen ihre Edelgasschale voll zu bekommen. Sie ist ein Spezialfall der umfassenderen Edelgasregel.

  * Die Edelgasregel besagt, dass Ionen beziehungsweise Atome in Molekülen besonders stabil sind, wenn sie die Edelgaskonfiguration haben, also die gleiche Elektronenanordnung wie Edelgase besitzen.

* es gibt unterschiedliche Strukturen: C02 ist gestreckt (mit Doppelbindung) während H2O gewinkelt ist. Das hängt davon ab, wo die freien Elektronenpaare lokalisiert sind am Atom

* **Kovalente Bindung: Zur Beschreibung dieser Bindungsart benutzt der Chemiker im wesentlichen zwei Theorien. Diese sind als Molekülorbitaltheorie (MO-Theorie ) und Valenzbindungstheorie (VB-Theorie ) bekannt. Beide Theorien sind Näherungsverfahren zur Lösung der Schrödinger-Gleichung für Moleküle**.

* Video: https://youtu.be/PsR-HRiGAzA

* https://de.m.wikipedia.org/wiki/Kovalente_Bindung

![ggg](https://raw.githubusercontent.com/deltorobarba/repo/master/sciences_0791.jpeg)

![ggg](https://raw.githubusercontent.com/deltorobarba/repo/master/sciences_0792.jpeg)

###### *Elektronegativität*

**Elektronegativität**

* [Elektronegativität](https://de.m.wikipedia.org/wiki/Elektronegativität) ist ein  Maß für die Fähigkeit eines Atoms, bei einer Atombindung Elektronenpaare an sich zu ziehen

> **Elektronegativitätsgrenze bei 1,7:**
* Grösser > 1,7 = Ionenverbindung (Salze)
* Kleiner > 1,7 = Kovalente Bindung (Atombindung)

> **Elektronegativitätsgrenze bei 0,5:**
* Grösser > 0,5 = Unpolare (kovalente) Bindung (Elementmoleküle wie O2)
* Kleiner > 0,5 = Polare (kovalente) Bindung

* Manche Atome gehen eine Ionenverbindung ein (wie NaCL - Natriumchlorid), andere gehen eine Atombindung / kovalente Bindung ein (wie Wasserstoff + Sauerstoff = Wasser H2O), oder Metallbindung.

* Bei zunächst unbekannten Verbindungen weiss man nicht immer sofort, welche Bindungsart vorliegt. zB Schwefelhexaflurid SF6. Da hilft es die Elektronegativität heranzuziehen.

> **Elektronegativität sagt aus, wie stark ein Element die Bindungselektronen an sich ziehen kann** (Daher bei starker Elektronegativität findet ein Wechsel des Elektrons von einem Atom zum anderen statt, bei schwacher Elektronegativität nutzen zwei Atome ein Elektronenpaar gemeinsam. Diese sind einseitig polar, wenn eines der Atome stärker elektronegativ ist, und unpolar wenn beide gleich stark elektronegativ sind.

* Jedes Element hat eine bestimmte Elektronegativität (EN oder $\chi$ ). Steht im Periodensystem.

* EN ist zwischen 0,7 und 4 (mit Variierung je nach Tabelle).

  * Halogen Fluor hat immer grösste EN mit 3,98 oder 4,10

  * Alkalimetall Francium hat kleinste EN mit 0,7 oder 0.9.

* Jetzt die Differenz der EN's berechnen: Natriumchlorid

  * Natrium: EN von 0,9

  * Chlor: EN von 3,2

  * Differenzen: immer den kleineren Wert vom grösseren Wert abziehen, denn Differenz muss immer positiv sein

  * $\chi_{\text{Cl}}$ - $\chi_{\text{Na}}$ = 3,2 - 0,9 = 2,3 (=Ionenbindung)

* **Elektronegativität von 1,7 ist die Grenze:**

  * Grösser > 1,7 = Ionenverbindung

  * Kleiner > 1,7 = Kovalente Bindung (Atombindung)


* Grenzen der EN-Methode: Natriumsulfid Na2S: 2,6 - 0,9 = 1,7 - Hier kann man nur mit Hilfe der Elektronegativitäten nicht sagen, welche Bindung vorliegt. Andere Untersuchungen sagen, dass eine Ionenbindung vorliegt.

###### *Polare vs unpolare Atombindungen*

**Polare vs unpolare Atombindungen**

> **Grenze liegt bei Differenz der Elektronegativität von 0,5. Darunter: unpolare Bindung, darüber: polare Bindung.**

* **Polare Atombindungen**

  * Polare Atombindungen sind chemische Bindungen, bei denen die beteiligten Atome infolge ihrer unterschiedlichen Elektronegativität Teilladungen tragen. Die Differenz ist jedoch nicht groß genug, dass eine reine Ionenbindung entsteht.

  * [Molekulare Verbindungen](https://de.m.wikipedia.org/wiki/Molekül) (in der Regel aus einem nichtmetallischen Element und einem oder mehreren weiteren Nichtmetallen). Die Atome in den Molekülen sind über kovalente Bindungen verknüpft, das heißt, dass die miteinander verbundenen Atome Paare von Außenelektronen gemeinsam nutzen (Elektronenpaarbindung). Molekulare Verbindungen entstehen typischerweise aus Nichtmetall und Nichtmetall – sie sind Nichtleiter (Isolatoren) mit zumeist relativ niedrigem Siedepunkt (diamantartige oder kunststoffartige Verbindungen mit Riesenmolekülen ausgenommen). Beispiele für molekulare Verbindungen sind neben Wasser auch Methangas, Zucker, Kohlendioxid, Polyethylen usw.

    * Diese bestehen aus **zwei Atomen** (zum Beispiel im Kohlenmonoxid, CO),

    * aus **drei Atomen** (zum Beispiel im Kohlendioxid)

    * oder auch aus **einigen Tausend oder Zehntausend Atomen** (Riesenmoleküle, Polymere, zum Beispiel im Kunststoff Polyethylen oder im Erbgut-Molekül Desoxyribonukleinsäure DNA).

  * Chlorwasserstoff HCl (Salzsäure): Bindungspartner sind nicht gleich. **Chlor Cl 3,2 - Wasserstoff H 2,2 = 1 (Chlor ist stärker, also elektronegativer). Chlor zieht Bindungselektronen mehr zu sich**, und Wasserstoff kann weniger Bindungselektronen an sich ziehen. Solche unausgeglichenen Bindungen nennt man polare Atombindungen.

    * Macht man in der Strukturformel sichtbar durch
$\delta^+$ und $\delta^-$ (an das elektronegativere Atom)

* **Unpolare Atombindung**

  * **Unpolare Atombindung** - zB Sauerstoffatome: Elektronegativitäten jeweils gleich uns Differenz ist 3,4 - 3,4 = 0. Wenn gleich starke Partner an den Bindungselektronen ziehen, dann passiert nichts. beide Sauerstoffatome haben gleich viel von den Bindungselektronen. Das nennt man eine Unpolare Atombindung.

  * In der Regel sind [Elementmoleküle](https://de.m.wikipedia.org/wiki/Elementmolekül) unpolar. In Elementmolekülen herrscht aufgrund der absoluten Gleichheit der Verbindungspartner eine reine Kovalenzbindung (Atombindung) vor. Dabei kann es sich um Einfachbindungen wie in Wasserstoff (H2) oder Mehrfachbindungen (Doppelbindung in O2, Dreifachbindung in N2) handeln.

###### *Bindungswinkel*

**Bindungswinkel**

> *Bindungswinkel*: Als [Bindungswinkel](https://de.m.wikipedia.org/wiki/Bindungswinkel) bezeichnet man in der Chemie und Molekülphysik den Winkel zwischen kovalenten Bindungen eines Atoms zu zwei Nachbaratomen

* Bindungswinkel sind zB wichtig bei polaren Bindungen: "Es kann vorkommen, dass ein Molekül mit polaren Atombindungen nach außen kein Dipolmoment aufweist. Dies tritt immer dann auf, wenn wegen des symmetrischen Aufbaus des Moleküls der positive und der negative Ladungsschwerpunkt identisch sind.

* Dies ist zum Beispiel bei Molekülen wie Kohlenstoffdioxid (CO2) oder Kohlenstofftetrafluorid (CF4) der Fall. Hier beträgt die delta-EN 1,0 bzw. 1,5. Weil die Moleküle linear bzw. tetraedrisch aufgebaut sind, fallen in beiden Fällen die Ladungschwerpunkte zusammen und beide haben kein Dipolmoment.

* Erkennbar ist das zum Beispiel an den relativ niedrigen Siedepunkten. Die zwischenmolekulare Anziehung beruht hier nur auf Van-der-Waals-Kräften." (https://de.m.wikipedia.org/wiki/Polare_Atombindung)

*Der Bindungswinkel zwischen den Wasserstoffatomen im Wasser beträgt 104,45° (Realstruktur)*

![xxx](https://upload.wikimedia.org/wikipedia/commons/thumb/4/4e/Watermolecule.svg/516px-Watermolecule.svg.png)

###### *Funktionelle Gruppe*

**Funktionelle Gruppe**

* **In der Organischen Chemie** versteht man unter einer [funktionellen Gruppe](https://de.m.wikipedia.org/wiki/Funktionelle_Gruppe) (auch charakteristische Gruppe) die **Atomgruppe** (kovalent) in einer Verbindung, die die Stoffeigenschaften und das Reaktionsverhalten der Verbindung maßgeblich bestimmt. Eine Verbindung kann auch mehrere funktionelle Gruppen mit unterschiedlichen Eigenschaften tragen.[1] Verbindungen, die gleiche funktionelle Gruppen, aber unterschiedliche Alkyl- oder Arylreste haben, werden auf Grund ihrer ähnlichen Eigenschaften zu Stoffklassen zusammengefasst.

* **Im Fall von anorganischen oder metallorganischen Verbindungen** werden auch einfache Alkyl- oder Arylgruppen als funktionelle Gruppen angesehen. Alle organischen Seitenketten und funktionellen Gruppen werden dann unter der allgemeinen Bezeichnung Organylgruppe zusammengefasst.

* Siehe auch [Table of common functional groups](https://en.wikipedia.org/wiki/Functional_group#Table_of_common_functional_groups)

Video: https://youtu.be/DxE1xEc44iU

Video: https://youtu.be/DxE1xEc44iU

https://de.m.wikipedia.org/wiki/Hydroxygruppe

https://de.m.wikipedia.org/wiki/Alkohole#Nomenklatur_und_Einteilung

https://de.m.wikipedia.org/wiki/Alkanole

##### <font color="blue">*Ionische Verbindungen (Salze)*

https://de.m.wikipedia.org/wiki/Anorganische_Chemie#Salze,_Mineralien

https://de.m.wikipedia.org/wiki/Ionisation

**[Ionische Verbindungen (Salze)](https://de.m.wikipedia.org/wiki/Salze) sind in der Regel aus einem Metall und einem oder mehreren Nichtmetallen**.

*Nichtmetalle: Wasserstoff, Kohlenstoff, Stickstoff, Phosphor, Sauerstoff, Schwefel, Selen. Die Halogene (Fluor, Chlor, Brom, Iod, Astat und Tenness). Die Edelgase (Helium, Neon, Argon, Krypton, Xenon, Radon und Oganesson).*

Abgrenzung zu kovalenten Atombindungen: Bei Ionenbindungen werden Elektronen abgegeben oder aufgenommen, nicht geteilt (entgegengesetzte Ionen führen zu Ionenbindung).

  * Zum Beispiel gibt Na ein Elektron ab, um die Oktettregel zu erfuellen und wird damit zu einem positiv geladenen Kation,

  * wahrend S zwei Elektronen aufnimmt, um die Oktettregel zu erfuellen und wird damit zu einem zweifach negativ gelandenen Anion.

  * Beispiel Kochsalz: Natriumchlorid. Natrium (Na+) hat 1 Elektron in der Aussenschale und gibt es an Chlorid ab (Cl-). Es entsteht uber eine elektrostatische Anziehung NaCl, Natriumchlorid oder Kochsalz.

**Elektronegativität ist grösser als 1,7 = Ionenverbindung (Salze)**. Kleiner > 1,7 = Kovalente Bindung (Atombindung).

![ggg](https://raw.githubusercontent.com/deltorobarba/repo/master/sciences_0790.jpeg)

*Beispiel Kation: Natrium - als Alkalimetall in der ersten Hauptgruppe, will es 1 Elektron abgeben und wird positiv geladenes Kation*

*Beispiel Anion: Chlor - als Halogen (Nichtmetall!) in der siebten Hauptgruppe, will es 1 Elektron aufnehmen und wird ein negativ geladenes Anion, dem Chlorid*

*Metall + Nichtmetall = Salz*

> *Siehe [Liste der Anionen](https://de.m.wikipedia.org/wiki/Anion) und siehe [Liste der Kationen](https://de.m.wikipedia.org/wiki/Kation)*

![ggg](https://raw.githubusercontent.com/deltorobarba/repo/master/sciences_082.png)


Ionen (geladene Teilchen) = Bausteine von Salzen. Eine Ionenverbindung nennt man Salz. **Wenn wir Elektronen entfernen, dann werden aus neutralen Atomen geladene Ionen.**

  * [Ion](https://de.m.wikipedia.org/wiki/Ion): Atom oder Molekül, bei dem die Gesamtzahl der Elektronen größer oder kleiner ist als die Gesamtzahl der Protonen. Atome und Moleküle haben im gewöhnlichen, neutralen Zustand genauso viele Elektronen wie Protonen. Besitzt ein Atom oder Molekül jedoch ein oder mehrere Elektronen weniger oder mehr als im Neutralzustand, hat es dadurch eine elektrische Ladung und wird als Ion bezeichnet.

  * Positiv geladene Ionen heissen Kationen, negativ geladene Ionen heissen Anionen, und die ziehen sich gegenseitig [elektrostatisch](https://de.m.wikipedia.org/wiki/Elektrostatik) an. Diese Anziehung heißt ionische Bindung.

* Beim Aluminiumatom sind 3 Elektronen zu viel (Al3+), und es gibt an 3 Chloridatomem jeweils 1 Elektron ab. Es entsteht (AlCl3)

* Etwa 90% aller Mineralien weisen im wesentlichen Ionenbindungen auf. Die Bindung entsteht auf Grund elektrostatischer Kräfte zwischen Ionen unterschiedlicher Ladung. Die Kräfte zwischen den Ionen sind um so stärker, je größer die Ladung und je kleiner die Ionenradien sind.

**Unterscheidung von ionischen Verbindungen**:

  * Nach den Nichtmetallen unterscheidet man bei ionischen Verbindungen zum Beispiel Oxide (Sauerstoff als Anion), Sulfide (mit Schwefel), Fluoride, Chloride, Bromide, Iodide, Nitride (mit Stickstoff), Carbide (mit Kohlenstoff), Hydride (mit Wasserstoff) usw.

  * Oft kommt Sauerstoff als drittes Element hinzu; man spricht dann von Sulfaten, Chloraten, Nitraten, Carbonaten usw.

**Wie sehen Ionenverbindungen aus?**

* Sie bilden [Kristallgitter](https://de.m.wikipedia.org/wiki/Kristallstruktur). Siehe hier auch Gruppentheorie: Punktgruppen und Raumgruppen.

* Zusammengehalten wird das Kristallgitter durch die [elektrostatische Anziehung](https://de.m.wikipedia.org/wiki/Elektrostatik) (=Anziehung zwischen positiven und negativen Ionen)

* Kristallgitter in Ionenbindungen bestimmen sehr starke Eigenschaften: hoher Schmelzpunkt bei Salzen, nicht verformbar (spröde), leite elektrischen Strom

* Unter einem [Ionengitter](https://de.m.wikipedia.org/wiki/Ionengitter) oder Ionenkristall versteht man in der Chemie und der Festkörperphysik die regelmäßige räumliche Anordnung von Anionen und Kationen in einem homogenen Stoff im festen Zustand. Der Zusammenhalt des Gitterverbandes erfolgt durch Ionenbindungen.

* Siehe auch [Kristall](https://de.m.wikipedia.org/wiki/Kristall) und [Kriallographie](https://de.m.wikipedia.org/wiki/Kristallographie)

* Video: https://youtu.be/n6Dr3qY7c6M

**Beispiele von Ionenverbindungen:**

* [Oxide](https://de.m.wikipedia.org/wiki/Oxide): sind Sauerstoff-Verbindungen eines Elements, in denen der Sauerstoff die Oxidationszahl −2 hat.

  * Oxide entstehen, wenn Elemente oder Verbindungen mit elememtarem Sauerstoff (Oxidationszahl 0) reagieren (Ursprung des Wortes Oxidation).

  * Bei der Oxidation geben geeignete Elemente in den Verbindungen Elektronen an das Oxidationsmittel Sauerstoff ab, so dass in den neu gebildeten Oxiden die Oxidationszahl des Sauerstoffatoms auf −2 erniedrigt ist und die Oxidationszahl des oxidierten Elements in der Verbindung entsprechend erhöht ist.

* [Hydroxide](https://de.m.wikipedia.org/wiki/Hydroxide): sind salzähnliche Stoffe, die Hydroxid-Ionen ([OH]−) als negative Gitterbausteine (Anionen) enthalten. Lösliche Hydroxide wie Natriumhydroxid und Kaliumhydroxid bilden mit Wasser stark alkalische Lösungen (Laugen), die unter der Bezeichnung Natronlauge und Kalilauge bekannt sind.

* [Fluoride](https://de.m.wikipedia.org/wiki/Fluoride): sind die Salze der Fluorwasserstoffsäure (HF), die auch als Flusssäure bekannt ist.

* [Chloride](https://de.m.wikipedia.org/wiki/Chloride): sind Verbindungen des chemischen Elementes Chlor. Dieses kann mit Metallen, Halb- oder Nichtmetallen verbunden vorliegen.

  * Metallchloride wie z. B. Natrium- und Cobalt(II)-chlorid sind Salze der Chlorwasserstoffsäure, besser bekannt als Salzsäure (chemische Formel: HCl).

  * Nichtmetallchloride wie Chlorwasserstoff, Schwefelchloride, Kohlenstofftetrachlorid (Tetrachlormethan) und Chlordioxid sind als molekulare Verbindungen wesentlich flüchtiger als salzartige Chloride.

  * Chlorhaltige Kohlenwasserstoffe werden in der Organik als Derivate der verschiedensten Verbindungen der Kohlenwasserstoffe betrachtet und benannt. So wird Methan, bei dem ein Wasserstoffatom gegen ein Chloratom ausgetauscht (substituiert) wurde, Chlormethan bzw. Methylchlorid genannt.

* [Sulfate](https://de.m.wikipedia.org/wiki/Sulfate): Anion, sind Salze oder Ester der Schwefelsäure.

* [Nitrate](https://de.m.wikipedia.org/wiki/Nitrate): Anion bestehend aus Stickstoff und Sauerstoff. Als Nitrate werden sowohl die Salze als auch die Ester der Salpetersäure (HNO3) bezeichnet.

* [Sulfide](https://de.m.wikipedia.org/wiki/Sulfide): Salze des Schwefelwasserstoffs. in der Anorganischen Chemie Verbindungen von Metallen mit Schwefel.

https://de.m.wikipedia.org/wiki/Erdalkalimetalle#Verbindungen

[Carbonate](https://de.m.wikipedia.org/wiki/Carbonate):

* Salze der Kohlensäure. sind die anorganischen Salze und organischen Ester der per definitionem anorganischen Kohlensäure ($H_2CO_3$).

* Carbonate kommen überall in der Natur vor, meist in Form verschiedener Minerale.

*Struktur der Kohlensäure-Ester. Die Carbonat-Gruppe ist blau markiert. R1 und R2 sind Alkyl- oder Aryl-Reste. Bei einem Halbester ist R2=H:*

![ggg](https://raw.githubusercontent.com/deltorobarba/repo/master/sciences_081.png)


##### <font color="blue">*Metallverbindungen*

###### *Metallische Verbindungen*

https://de.m.wikipedia.org/wiki/Anorganische_Chemie#Metalle

* [Metallische Bindung](https://de.m.wikipedia.org/wiki/Metallische_Bindung) sind Bindungen von Metallatomen in einem festen Gitter mit frei beweglichem Elektronengas

> Metallbindungen: Metallatome lagern sich in Gitter zusammen, während die Elektronen in einem **'Elektronengas'** vorliegen (können sich frei im Atomgitter bewegen), **weil die Elektronen der äusseren Hülle nur sehr schwach an ihre Atomkerne gebunden sind**. Im Grundgerust des Metalls, dem Metallgitter, schwirren die Aussenelektronen dann frei herum.

* Bindungen von Metallen mit Metallen

* die Mehrheit der Elemente sind Metalle: 95 von 118 bisher bekannten Elementen

* kommen in reiner Form in der Natur meist nicht vor

* Elektronengas fuehrt zu Biegsamkeit, guter Wärmeleitung und elektrischer Leitfähigkeit. Eigenschaften: Wärmeleiter, Stromleiter, gut verformbar, glänzen

* Warum fuehlt sich Metall so kuehl an, auch bei Raumtemperatur? - Man spuert eigene Koepertemperatur, da diese bei Metallen besonders schnell weggeleitet wird.

* [Intermetallische Verbindungen](https://de.m.wikipedia.org/wiki/Intermetallische_Verbindung) (aus Metallen): zeigen im Unterschied zu Legierungen Gitterstrukturen, die sich von denen der konstituierenden Metalle unterscheiden.


![ggg](https://raw.githubusercontent.com/deltorobarba/repo/master/sciences_0794.jpeg)

![ggg](https://raw.githubusercontent.com/deltorobarba/repo/master/sciences_0782.jpeg)

* Video: [Metallbindungen einfach erklärt](https://youtu.be/0bvldHVL_TU)

* Video: [Wie die Chemie der Metalle die Welt verändert hat](https://www.youtube.com/watch?v=ChGhM8eLoYs)

Metallografie https://de.m.wikipedia.org/wiki/Metallografie

###### *Arten an Metallen*

[Edelmetalle und Halbedelmetalle](https://de.m.wikipedia.org/wiki/Edelmetalle#Halbedelmetalle):

* sind korrosionsbeständige Metalle

* Edelmetalle: Gold, Platin, Silber etc

* Halbedelmetalle: Kupfer, Bismut etc

* Lurzlebiges radioaktives Edelmetall: Bohrium, Darmstadtium etc.

* Alle Edelmetalle und Halbedelmetalle zählen zu den Schwermetallen.

[Halbmetalle](https://de.m.wikipedia.org/wiki/Halbmetalle)

* sind Elemente und stehen im Periodensystem zwischen den Metallen und den Nichtmetallen. Sie können von der elektrischen Leitfähigkeit und vom Aussehen her weder den Metallen noch den Nichtmetallen zugeordnet werden.

* Alle Halbmetalle sind Feststoffe bei Normalbedingungen.

* Halbleiter ist ein Überbegriff und umfasst Halbmetalle und Verbindungshalbleiter gleichermaßen

[Übergangsmetalle](https://de.m.wikipedia.org/wiki/Übergangsmetalle):

* Die chemischen Elemente mit den Ordnungszahlen von 21 bis 30, 39 bis 48, 57 bis 80 und 89 bis 112 werden üblicherweise als Übergangselemente bezeichnet.

* Da diese Elemente alle Metalle sind, wird auch der Ausdruck Übergangsmetalle benutzt.

* Bisher 200.000 Tonnen Gold geschürft

[Leichtmetall](https://de.m.wikipedia.org/wiki/Leichtmetalle)

* Als Leichtmetalle werden allgemein Metalle und Legierungen bezeichnet, deren Dichte unter 5,0 g/cm³ liegt.

* Magnesium ist mit einer Dichte von 1,738 g/cm³ (20 °C) das leichteste Nutzmetall.

* Im technischen Bereich sind vor allem Aluminium, Magnesium, Titan sowie in geringem Umfang Beryllium und Lithium im Gebrauch – sowie weitere Elemente als Legierungselemente in geringer Konzentration.

[Schwermetalle](https://de.m.wikipedia.org/wiki/Schwermetalle): Metalle mit hoher Dichte

* Schwermetalle sind unter uneinheitlichen Definitionen zusammengefasste Metalle, deren Dichte oder Atommasse einen bestimmten Wert übersteigt. Teilweise werden in die Definition noch weitere Eigenschaften wie Ordnungszahl und Toxizität einbezogen (mindestens 38 Definitionen).

* Viele Quellen stufen als Schwermetall ein Metall ein, dessen Dichte größer als 5,0 g/cm³ oder – bei älteren Quellen – größer als 4,5 g/cm³ ist.

* Zu diesen werden üblicherweise unter anderem die Edelmetalle, die Basismetalle Eisen, Kupfer, Blei, Zink, Zinn und Nickel sowie Bismut, Cadmium, Chrom und Uran gerechnet.

* essentielle Schwermetalle oder Spurenelemente bezeichnet. Dazu gehören Chrom, Eisen, Cobalt, Kupfer, Mangan, Molybdän, Nickel, Vanadium, Zink und Zinn.

[Basismetalle](https://de.m.wikipedia.org/wiki/Basismetalle):

* Zu den Basismetallen werden Metalle gezählt, die in relativ hoher Tonnage produziert und häufig als Hauptbestandteil in Legierungen eingesetzt werden.

* Dazu gehören unter anderem Eisen, Aluminium, Kupfer, Zink, Blei und Nickel

[Legierung](https://de.m.wikipedia.org/wiki/Legierung):

* Legierung: verschiedene Metalle vermischt.

* [Legierungselemente](https://de.m.wikipedia.org/wiki/Legierungselement) sind chemische Elemente, die einem Metall hinzugefügt (legiert) werden, um dessen Werkstoffeigenschaften auf die gewünschte Weise zu verbessern.

 * Legierungselemente können selbst Metalle wie beispielsweise Chrom, Molybdän und Mangan, aber auch Halbmetalle wie Bor und Silicium oder Nichtmetalle wie z. B. Kohlenstoff und Stickstoff sein.

###### *Beispiele wichtiger Metalle*

[Kupfer](https://de.m.wikipedia.org/wiki/Kupfer):

* erstes Metall, das der mensch benutzt hat fur Waffen, Schmuck und Werkzeuge (Scheibe von Nebra vor 4000 Jahren).

* Gewinnung von Rohkupfer ist Spezialarbeit, Schmelzpunkt bei 1.085 Grad.

* Kupfer (lateinisch Cuprum) ist ein chemisches Element mit dem Elementsymbol Cu und der Ordnungszahl 29. Es ist ein Übergangsmetall, im Periodensystem steht es in der 4. Periode und der 1. Nebengruppe (nach neuer Zählung Gruppe 11) oder Kupfergruppe.

* Da Kupfer leicht zu verarbeiten ist, wurde es bereits von den ältesten bekannten Kulturen vor etwa 10.000 Jahren verwendet. Die Zeit seines weiträumigen Gebrauchs vom 5. Jahrtausend v. Chr. bis zum 3. Jahrtausend v. Chr.

* Die ersten Spiegel wurden aus Kupfer hergestellt.

* Kupfer verfügt über eine ausgezeichnete Korrosionsbeständigkeit und ist zu 100 Prozent ohne Qualitätsverluste recycelbar.

* Kupfer ist eines der wenigen metallischen Elemente mit einer anderen natürlichen Farbe als Grau oder Silber. Reine Kupferoberflächen sind lachsrot und verfärben sich an der Luft rotbraun. Die charakteristische Farbe von Kupfer ergibt sich aus [Interband-Übergängen](https://de.m.wikipedia.org/wiki/Elektronischer_Übergang) (Elektronischer Übergang) von im d-[Atomorbital](https://de.m.wikipedia.org/wiki/Atomorbital) vorhandenen Elektronen.

* Für den menschlichen Organismus ist Kupfer ein essentielles Spurenelement, d. h. der Mensch benötigt Kupfer, um zu überleben. Gewöhnlich wird der Tagesbedarf eines Erwachsenen von ca. 2 mg durch eine ausgewogene Ernährung mit einem reichlichen Anteil Getreide, Fleisch, Wurzelgemüse, Hülsenfrüchten, Nüssen oder auch Schokolade erreicht.

* Kupfer ist für viele Mikroorganismen (Viren, Keime) bereits in geringen Konzentrationen toxisch. Aufgrund der antimikrobiellen Eigenschaft von Kupfer wird das Material z. B. in Krankenhäusern in Großversuchen eingesetzt.

  * Experimente legen nahe, dass die Kontaktabtötung durch einen Mechanismus abläuft, bei dem der Metall-Bakterien-Kontakt die Zellhülle schädigt, was wiederum die Zellen anfällig für weitere Schäden durch Kupfer macht.

  * Die keimreduzierende Wirkung entsteht genauer gesagt dadurch, dass **Kupferionen sich an Thiolgruppen von Proteinen binden und Lipide der Zellmembran peroxidieren, was zur Bildung von freien Radikalen führt**, welche die DNA und Zellmembranen schädigen.

[Zinn](https://de.m.wikipedia.org/wiki/Zinn)

* Zinn ist ein chemisches Element mit dem Elementsymbol Sn

* silberweiß glänzende und sehr weiches Schwermetall, sehr niedrigen Schmelzpunkt.

[Bronze](https://de.m.wikipedia.org/wiki/Bronze)

* 10% Zinn und 90% Kupfer

* Mit dem Sammelbegriff Bronzen werden Legierungen mit mindestens 60 % Kupfer bezeichnet, soweit sie nicht durch den Hauptlegierungszusatz Zink den Messingen zuzuordnen sind.

* Auf der Verhüttung von Kupfererzen im 3. Jahrtausend vor Christus ([Bornit](https://de.m.wikipedia.org/wiki/Bornit) Cu5 FeS4 - Mineral aus der [Mineralklasse der „Sulfide und Sulfosalze“](https://de.m.wikipedia.org/wiki/Kategorie:Sulfide_und_Sulfosalze)) baute die Herstellung von Bronze auf, die die Menschen als Legierung benutzten (Zusammenschmelzen verschiedener Metalle). Ziel war es, die Materialeigenschaften des vergleichsweise weichen Kupfers zu verbessern.

* Der Trick bei der Bronze: Zusatz von 10 Prozent [Zinn](https://de.m.wikipedia.org/wiki/Zinn) erhärtet das Material (verändert Werkstoffeigenschaften des Materials). Legierung von Kupfer mit Zinn. Schmelzpunkt von Zinn bei 231,9 Grad.

* Das Verhalten der Elektronengase wird durch fremde Atome im Kristallgitter verändert

[Messing](https://de.m.wikipedia.org/wiki/Messing)

* Kupferlegierung mit Zink als Legierungsmetall

* Die goldgelbe Kupfer-Zink-Legierung „Messing“ war bereits im antiken Griechenland bekannt. Es wurde durch gemeinsames Verarbeiten der jeweiligen Erze erschmolzen, aber erst die Römer haben dieses Verfahren verstärkt verwendet. In Altkolumbien wurde die Gold-Kupfer-Legierung Tumbaga häufig verwendet.

[Eisen](https://de.m.wikipedia.org/wiki/Eisen):

* ist ein **chemisches Element** mit dem Symbol Fe (lateinisch ferrum ‚Eisen‘) und der Ordnungszahl 26. Es gehört zu den Übergangsmetallen, die im Periodensystem die 8. Nebengruppe (Eisen-Platin-Gruppe), nach der neuen Zählung die Gruppe 8 (Eisengruppe) bilden.

* Eisen ist, auf den Massenanteil (ppmw) bezogen, nach Sauerstoff, Silicium und Aluminium das **vierthäufigste Element in der Erdkruste und nach Aluminium das häufigste Metall**.

* Eisen in den ältesten Perioden der Geschichte nur in geringem Umfang genutzt, und neigt Eisen an feuchter Luft, im Wasser und in der nassen Erde zur Korrosion, wodurch viele Gegenstände nicht erhalten blieben

* [Eisenzeit](https://de.m.wikipedia.org/wiki/Eisenzeit):  dritte Periode nach Steinzeit und  Bronzezeit, im südlichen Mitteleuropa von etwa 800 v. Chr. bis um die Zeitenwende und im nördlichen Mitteleuropa von etwa 750 v. Chr. bis ins 5. Jahrhundert n. Chr.

* [Gusseisen](https://de.m.wikipedia.org/wiki/Gusseisen) ist ein Eisenwerkstoff mit hohem Kohlenstoffgehalt (Massenanteil über 2 %), der diesen Werkstoff von Stahl unterscheidet.

* [Schmiedeeisen](https://de.m.wikipedia.org/wiki/Schmiedeeisen): kohlenstoffarme nicht härtbare Eisenlegierung. Schmiedeeisen ist durch Schmieden bearbeiteter Stahl, der durch einen Schmied oder Kunstschmied in Form gebracht wurde. Handwerklich geschmiedete Erzeugnisse



[Stahl](https://de.m.wikipedia.org/wiki/Stahl):

* Stahl ist eine kohlenstoffarme Legierung aus Eisen und einem geringen Kohlenstoff-Anteil (Eisen-Kohlenstoff-Legierung mit einem Kohlenstoff-Massenanteil von maximal 2 %). Auch diese Bearbeitung verändert die Kristallstruktur und damit die Eigenschaften des Metalls.

* Eisen-Kohlenstoff-Legierungen mit höheren Kohlenstoffanteilen werden Gusseisen genannt und sind nicht plastisch umformbar, lassen sich also nicht schmieden oder walzen.

* Stahl ist einer der vielseitigsten Konstruktionswerkstoffe und ist nahezu unbegrenzt wiederverwertbar.

* Stahl ist in großen Mengen und zu geringen Kosten verfügbar. Seine Eigenschaften lassen sich durch Legieren und Wärmebehandeln in weiten Bereichen variieren. Es gibt etwa 3.500 Stahlsorten.

* Siehe auch [Stahlerzeugung](https://de.m.wikipedia.org/wiki/Stahlerzeugung): Prozess der Dekarbonisierung zur Herstellung von Stahl

  * Das Bessemer-Verfahren (ab 1864) und später das Siemens-Martin-Verfahren erlauben das gezielte frischen des Roheisens, um den Kohlenstoffgehalt zu reduzieren.

  * Beim Frischen werden auch unerwünschten Begleitelemente wie Silicium, Mangan, Schwefel und Phosphor durch Zugabe von Sauerstoff verbrannt. Diese erlaubten die Herstellung von qualitativ höherwertigen Stahl, der jedoch wegen des aufwendigeren Prozesses zunächst etwas teurer war.

  * Das Siemens-Martin-Verfahren wurde erst Mitte des 20. Jahrhunderts langsam vom sogenanntem Elektrostahl aus Lichtbogenöfen und dem Linz-Donawitz-Verfahren abgelöst.



**Seltene Erdmetalle**

* Heutzutage werden fur Smartphones oder Elektroautos andere Metalle benötigt, wie [Seltenerdmetalle](https://de.m.wikipedia.org/wiki/Metalle_der_Seltenen_Erden) oder Lithium, und die sind endlicher als Eisen, und der Abbau und nicht umweltschonend.

* Zu den Metallen der Seltenen Erden gehören die chemischen Elemente der [3. Nebengruppe (Scandiumgruppe)](https://de.m.wikipedia.org/wiki/Scandiumgruppe) des Periodensystems (mit Ausnahme des [Actiniums](https://de.m.wikipedia.org/wiki/Actinium)) und die [Lanthanoide](https://de.m.wikipedia.org/wiki/Lanthanoide) – insgesamt also 17 Elemente.

![ggg](https://upload.wikimedia.org/wikipedia/commons/thumb/e/eb/Periodensystem_der_Elemente_-_Metalle_der_Seltenen_Erden.svg/1280px-Periodensystem_der_Elemente_-_Metalle_der_Seltenen_Erden.svg.png)

[Metalle der Seltenen Erden](https://de.m.wikipedia.org/wiki/Metalle_der_Seltenen_Erden):

* Zu den Metallen der Seltenen Erden gehören die chemischen Elemente der 3. Nebengruppe des Periodensystems (mit Ausnahme des Actiniums) der [Scandiumgruppe](https://de.m.wikipedia.org/wiki/Scandiumgruppe) und die [Lanthanoide](https://de.m.wikipedia.org/wiki/Lanthanoide) – insgesamt also 17 Elemente.

* Nach den Definitionen der anorganischen Nomenklatur heißt diese Gruppe chemisch ähnlicher Elemente Seltenerdmetalle. Im Deutschen gibt es des Weiteren den Begriff Seltene Erdelemente und passend dazu das dem englischen REE (Rare Earth Elements).

* zB gehört dazu Ytterbium

*Metalle der Seltenen Erden im Periodensystem der Elemente (fett umrandet):*

![gg](https://upload.wikimedia.org/wikipedia/commons/thumb/e/eb/Periodensystem_der_Elemente_-_Metalle_der_Seltenen_Erden.svg/1024px-Periodensystem_der_Elemente_-_Metalle_der_Seltenen_Erden.svg.png)

###### *Elektrochemische Spannungsreihe*

[Elektrochemische Spannungsreihe](https://de.m.wikipedia.org/wiki/Elektrochemische_Spannungsreihe): materialtypische Eigenschaft verschiedener Metalle

##### <font color="blue">*Intermolekulare Wechselwirkungen*

* Wasserstoffbrücken untereinander ausbilden, wodurch Anziehungskräfte zwischen den Molekülen entstehen

* Van der Waals Kräfte (inkl London-Kräfte): Anziehungskräfte zwischen unpolaren Molekülen

https://de.wikipedia.org/wiki/Molek%C3%BCl

https://de.wikipedia.org/wiki/Makromolek%C3%BCl

###### *Voraussetzung: Intermolecular force (Zwischenmolekulare Kräfte)*

* Wechselwirkung zwischen valenzmäßig abgesättigten Molekülen

* Als [zwischenmolekulare oder intermolekulare Kräfte](https://de.m.wikipedia.org/wiki/Zwischenmolekulare_Kräfte) bezeichnet man Wechselwirkungen zwischen valenzmäßig abgesättigten Molekülen.

* Nicht zu diesen Kräften zählen die Wechselwirkungen, die die chemische Bindung von Atomen zu Molekülen bewirken. Zwischenmolekulare Kräfte beruhen auf Van-der-Waals-Kräften.

* Zwischenmolekulare Kräfte sind üblicherweise sehr viel schwächer als chemische Bindungen. Sie sind durch Effekte wie Oberflächenspannung, Kapillarität sowie Adhäsions- und Kohäsionskräfte makroskopisch beobachtbar. So verursachen zwischenmolekulare Kräfte die Existenz von Aggregatzuständen von molekularen Verbindungen. Ohne sie würden sich z. B. Flüssigkeiten wie Gase verhalten.

* Eine besondere Form von zwischenmolekularen Kräften sind Wasserstoffbrückenbindungen, deren Stärke relativ nah an die Stärke von chemischen Bindungen kommen kann.

https://en.m.wikipedia.org/wiki/Intermolecular_force

###### *Wasserstoffbrückenbindungen*

* Siehe auch unter <font color="blue">Atombindungen -> Elektronegativität sowie Polare und unpolare Atombindung</font>

* Molekülbindung ("zwischenmolekulare Wechselwirkung"), nicht Atombindung

* Wasserstoffbrückenbindungen entstehen zwischen Molekülen, wenn Wasserstoff an einen elektronegativeren Partner gebunden ist

* sehr leicht: 20% einer Atombindung

* dipolare Bindung

* Form der Nebenvalenzbindung aufgrund zwischenmolekularer Kraft

* DNA besteht aus langen Molekülen die durch Wasserstoffbrückenbindungen zusammengehalten

Bindung zwischen Wasserstoff und Sauerstoff ist polar (Sauerstoff ist elektronegativer als Wasserstoff). Wenn beide eine Bindung eingehen, dann kann Sauerstoff eine stärkere Anziehungskraft auf das Elektronenpaar ausüben, als die Wasserstoffatome.

* die elektronegativsten Elemente sind Fluor, Sauerstoff, Stickstoff und Chlor. Und alle vier sind in der Lage mit Wasserstoff sogenannten Wasserstoffbrückenbindungen auszubilden.

* **Wassermoleküle sind Dipole**

![ggg](https://raw.githubusercontent.com/deltorobarba/repo/master/chem_015.jpg)

Um Sauerstoff herum vergössert sich Elektronendichte, um Wasserstoffatome verkleinert. Von Ionen wissen wir das elektirsche Ladung entsteht, dass Atome Elektronen aufnehmen oder abgeben.

Partialladung entsteht, weil Sauerstoffatom mehr Elektronen an sich zieht und ist daher teilweise negativ geladen. Geschrieben mit delta minus.

Ddaurch, dass das Wassermolekül geknickt ist, haben wir auf einer Seite eine negative Partialladung am Sauerstoffatom und unten jeweils zwei positive Partialladungen an den Wasserstoffatomen. **Wassermoleküle sind daher Dipole.**

Betrachten wir mehr als ein Wassermolekül, sehen wir, dass sich die Moleküle so ausrichten, dass sich der positive Pol des Sauerstoffatoms und negative Pol der Wasserstoffatome jeweils binden. **Wir haben dann sogenannte Wasserstoffbrückenbindungen.**

Wasserstoffbrückenbindungen können auch Flusssäure HF, Ammonika NH3 und Salzsäure HCl bilden.

![ggg](https://raw.githubusercontent.com/deltorobarba/repo/master/chem_014.jpg)

Deshalb schwimmt Eis auf dem Wasser und sinkt nicht ab: beim Gefrieren von Wasser: die Wassermoleküle lagern sich zu einem Sechsring zusammen, die durch Wassserstoffbrückenbindungen entstanden sind. Dadurch entsteht Freiraum in der Mitte, wodurch geringere Dichte entsteht.

![ggg](https://raw.githubusercontent.com/deltorobarba/repo/master/chem_016.jpg)

Video: https://youtu.be/lO_NSF1PeVI

https://de.m.wikipedia.org/wiki/Wasserstoffbrückenbindung

###### *Dipol-Dipol-Kräfte*

<font color="blue">Dipol-Dipol-Kräfte (Wechselwirkungen)

* Siehe auch unter Atombindungen - Polare und unpolare Atombindung

* Kräfte, die zwischen Molekülen herrschen

* Diese zwischenmolekularen Kräfte sind schwächer als die Kräfte bei den Wasserstoffbrückenbindungen, jedoch stärker als die London-Kräfte (Van-der-Waals-Kräfte im engeren Sinne).

* Bei Atombindungen innerhalb von Molekülen unterscheidet man anhand der Differenz der Elektronegativität zwischen den kovalenten oder unpolaren Bindungen (Differenz der Elektronegativität ist 0,0–0,4) und den polaren Bindungen (Differenz der Elektronegativität ist 0,4–1,7).

*Dipol-Dipol-Kräfte zwischen zwei Chlorwasserstoffmolekülen (HCl)*

![ggg](https://upload.wikimedia.org/wikipedia/commons/thumb/5/59/Dipole-dipole-interaction-in-HCl-2D.png/320px-Dipole-dipole-interaction-in-HCl-2D.png)

https://de.m.wikipedia.org/wiki/Dipol-Dipol-Kräfte

###### *Van der Waals Kräfte (inkl London-Kräfte)*

**Van der Waals Kräfte (inkl London-Kräfte)**: Anziehungskräfte zwischen unpolaren Molekülen

<font color="blue">Van-der-Waals-Kräfte

* Klasse von Dipolkräften

https://de.m.wikipedia.org/wiki/Van-der-Waals-Kräfte

<font color="blue">London-Kraft

* gehören zu den van-der Waalschen Wechselwirkungen: London-Kräfte sind Van-der-Waals-Kräfte im engeren Sinne

* schwache Anziehungskräfte durch induzierte Dipole

https://de.m.wikipedia.org/wiki/London-Kraft

**Nichtkovalente Bindungen**

* schwächer als kovalente Bindungen

* sehr wichtig in biochemischen Prozessen, wie der Bildung der Doppelhelix

* Die vier grundlegenden Arten von nichtkovalenten Bindungen sind **ionische Wechselwirkungen, Wasserstoffbrücken, van-der-Waals-Wechselwirkungen und hydrophobe
Wechselwirkungen**.

* Sie unterscheiden sich in ihrer Geometrie, Stärke und Spezifität. Darüber hinaus werden diese Bindungen auf sehr unterschiedliche Weise durch das Vorhandensein von
Wasser beeinflusst.

##### <font color="blue">*Komplexchemie (Liganden)*

* **Komplexe Verbindungen / Liganden (aus Metall-Kationen und Ionen oder Molekülen)**.

* Komplexe Verbindungen (aus Metall-Kationen und Ionen oder Molekülen) oder auch [koordinative Bindung](https://de.m.wikipedia.org/wiki/Koordinative_Bindung) (Donator-Akzeptor-Bindung oder veraltet dative Bindung)

> Eine komplexe Bindung besteht, **wenn in einer Elektronenpaarbindung (Atombindung) die Bindungselektronen von nur einem der beiden Bindungspartner stammen**. Dabei bezeichnet man das Molekül oder Ion mit Elektronenmangel als Akzeptor (= Lewis-Säure), dasjenige mit den freien Elektronen als Donator (= Lewis-Base). Siehe hierfür [Lewis-Säure-Base-Konzept](https://de.m.wikipedia.org/wiki/Lewis-Säure-Base-Konzept).


*Komplexchemie*

Die [Komplexchemie](https://de.m.wikipedia.org/wiki/Komplexchemie) (lat. complexum, „umgeben“, „umarmt“, „umklammert“) ist ein **Bereich der Anorganischen Chemie**.

* Der Begriff Koordinationschemie wird im Allgemeinen synonym dazu verwendet. Die Komplexchemie befasst sich mit Komplexen bzw. **Koordinationsverbindungen, die aus einem oder mehreren Zentralteilchen und einem oder mehreren Liganden aufgebaut sind**.

* Die Zentralteilchen sind dabei meist Atome bzw. Ionen von Übergangsmetallen, die ungeladen oder geladen sein können.

* Im Gegensatz zu herkömmlichen kovalenten Bindungen steuern in Komplexen gewöhnlicherweise die Liganden alle Elektronen zur Bindung bei, anstatt dass jeder Reaktionspartner mit jeweils einem Elektron zu einer Elektronenpaarbindung beiträgt (koordinative Bindung); trotzdem gibt es auch Komplexe eher kovalenter Natur. Die Bildung von Komplexen lässt sich somit als Säure-Base-Reaktion im Sinne der Lewis-Definition verstehen, in der die Liganden (Basen) als Elektronenpaardonatoren auftreten und das Zentralteilchen (Säure) durch Lücken in seiner Elektronenkonfiguration als Akzeptor.

* Komplexverbindungen spielen in verschiedenen Bereichen eine wichtige Rolle: In der Technik (zum Beispiel als Katalysator, siehe Abbildung des Grubbs-Katalysators rechts), in der Biologie (Hämoglobin und Chlorophyll), in der Medizin (Chelat-Therapie) oder in der Analytischen Chemie. Verbindungen mit organischen Liganden sind zudem Gegenstand der Metallorganischen Chemie. Da Komplexe von Übergangsmetallen mitunter sehr farbig sind, werden diese außerdem auch als Farbstoffe bzw. Pigmente eingesetzt (Berliner Blau). Besonders intensive Färbungen zeigen die Charge-Transfer-Komplexe, wie z.  B. Permanganate.

*Beispiel: [Grubbs-Katalysatoren](https://de.m.wikipedia.org/wiki/Grubbs-Katalysatoren):*

![gg](https://upload.wikimedia.org/wikipedia/commons/thumb/a/aa/Grubbs-1G-3D-balls.png/410px-Grubbs-1G-3D-balls.png)

*Ligand*

* Ein [Ligand](https://de.m.wikipedia.org/wiki/Ligand) (lat. ligare = „binden“) ist in der [Komplexchemie](https://de.m.wikipedia.org/wiki/Komplexchemie) (sowie in Organometallchemie und Bioanorganik) **ein Atom oder Molekül, welches sich über eine koordinative Bindung (veraltet auch „dative Bindung“) an ein zentrales Metall-Ion binden („koordinieren“) kann**.

> **Ein Ligand kann ein Atom oder Molekül, welches ein Metallion koordinieren**.

* Die koordinative Bindung kommt durch den Lewis-Charakter der beteiligten Bindungspartner zustande: In vielen Komplexen sind die Liganden Lewis-Basen (Elektronenpaar-Donatoren) und die Metallionen Lewis-Säuren (Elektronenpaar-Akzeptoren).

  * Komplexverbindungen sind zum Beispiel das rote Blutlaugensalz (Kaliumhexacyanidoferrat(III)), bei dem das zentrale Eisen(III)-Ion von sechs Liganden, in diesem Fall Cyanid-Ionen, umgeben ist.

  * Beispiele aus der Bioanorganik sind das Häm und das Chlorophyll: hier agieren die vier Stickstoffatome im Porphyringerüst als mehrzähniger Ligand für das zentrale Eisen- bzw. Magnesium-Ion.

  *  Vitamin B12: das aktive Zentrum ist um das Cobalt-Ion herum (nicht 100%ig sicher, ob Elemente herum Liganden sind). siehe mehr zu [Vitamin B12](https://www.spektrum.de/lexikon/chemie/vitamin-b12/9807)

*Beispiel: Koordinative Bindung aus Ammoniak und [Bortrifluorid](https://de.m.wikipedia.org/wiki/Bortrifluorid)*

* Ein typisches Beispiel ist [Ammoniak](https://de.m.wikipedia.org/wiki/Ammoniak) (NH3), das sein freies Elektronenpaar für eine koordinative Bindung zur Verfügung stellt, siehe H3N-BF3 in untenstehender Skizze.

* **Formal gibt dadurch der Stickstoff ein Elektron dem Bor, wodurch ersterer (allgemein: der Donator) eine formale positive, letzteres (allgemein: der Akzeptor) eine formal negative Ladung erhält.**

* Man beachte, dass diese formalen Ladungen nichts mit der tatsächlichen Ladungsverteilung zu tun haben: Da Stickstoff eine wesentlich höhere Elektronegativität als Bor besitzt (3,07 gegenüber 2,01), halten sich die Elektronen eher beim Stickstoff auf.

![gg](https://upload.wikimedia.org/wikipedia/commons/thumb/6/6c/NH3-BF3-adduct-bond-lengthening-2D.png/640px-NH3-BF3-adduct-bond-lengthening-2D.png)

https://de.m.wikipedia.org/wiki/Chemische_Verbindung

### <font color="blue">**Chemische Reaktionen**

https://de.m.wikipedia.org/wiki/Hydrolyse

###### *Beispiel: CO2-Umwandlung*

Diese porösen, verästelten Strukturen umfassenden Kugeln bestehen aus Molybdänsulfid und sind Teil eines neuen Katalysators, durch den Kohlendioxid in Methanol umgewandelt werden kann.

Um den Klimaschutz voranzubringen, darf möglichst kein klimaschädliches Kohlendioxid (CO2) mehr in die Atmosphäre. Eine Methode, die CO2-Emissionen einzudämmen, ist das Abfangen des Treibhausgases aus dem Abgasstrom von Industrieanlagen oder Kraftwerken. Idealerweise wird das abgeschiedene CO2 dann in anderen, nicht flüchtigen Materialien gebunden oder weiterverwendet. Eine Möglichkeit dafür ist die chemische Umwandlung des Gases zu Methanol, einer als Kraftstoff und Chemierohstoff nutzbaren Flüssigkeit.

„Um Kohlendioxid umzuwandeln, verwendete man bisher oft Katalysatoren, die auf Kupfer basieren“, erklärt Karin Föttinger vom Institut für Materialchemie der TU Wien. „Sie haben allerdings den großen Nachteil, dass sie nicht robust sind. Wenn im Abgasstrom neben Kohlendioxid auch noch bestimmte andere Substanzen vorkommen, zum Beispiel Schwefel, dann verliert der Katalysator rasch seine Wirkung. Man sagt, der Katalysator wird vergiftet.“ Föttinger und ihr Team haben daher nach Katalysator-Alternativen gesucht, die weniger anfällig für diese chemische Degradation sind.

„Wenn man solche Methoden nicht nur im Labor, sondern auch im großen Maßstab in der Industrie einsetzen will, dann braucht man einen Katalysator, der vielleicht ein bisschen weniger aktiv ist, aber dafür robust, haltbar und zuverlässig“, sagt Föttinger. „Man möchte ganz gewöhnliche Industrieabgase ohne Vorbehandlung verarbeiten können.“ Durch ihre Forschungen fanden die Wissenschaftler heraus, dass sich Verbindungen aus Molybdän und Schwefel für die Katalyse von CO2 zu Methanol eignen.

Das in unserem Bild in einer elektronenmikroskopischen Aufnahme abgebildete Molybdänsulfid bildet poröse kugelige Mikrostrukturen, die das Gas einfangen und durch ihre große Oberfläche viel Raum für chemische Reaktionen geben. Spezielle Zusätze wie Mangan erleichtern dabei die Aufspaltung und Umwandlung des eigentlich eher unreaktiven Kohlendioxids. „Wenn man möchte, kann man mit unseren Katalysatoren aber auch andere Moleküle herstellen, etwa höhere Alkohole“, erklärt Föttinger. „Wir arbeiten derzeit noch daran, genau herauszufinden, wie man Parameter wie Druck und Temperatur am besten wählt, um unterschiedliche Produkte zu erzeugen.“ Die neue Technologie wurde bereits patentiert, gemeinsam mit Industriepartnern soll das Verfahren nun auf industriellen Maßstab skaliert werden.

https://www.wissenschaft.de/bildervideos/bild-der-woche/filigrane-strukturen-fuer-die-co2-umwandlung/

###### *Homolytische und Heterolytische (Atom)Spaltung*

> **Häufig muss bevor ein Stoff mit einem anderen Stoff reagieren kann eine Atombindung gespalten werden. Dieser Vorgang benötigt immer Energiezufuhr. Man unterscheidet zwei Arten der Spaltung von Atombindungen**.

**Homolytische Spaltung**

* Andere Namen für diese Art der Spaltung sind: symmetrische oder radikalische Spaltung.

* Zum Beispiel können sich H2 oder Cl2 in Radikale spalten.

* **Radikale** sind Teilchen mit Einzelelektronen = ungepaarte Elektronen, einfach besetztes Orbital.

![ggg](https://raw.githubusercontent.com/deltorobarba/repo/master/sciences_0847.png)

**Heterolytische Spaltung**

* Andere Namen für diese Art der Spaltung sind: unsymmetrische oder ionische Spaltung.

* Zum Beispiel können sich HCl und HF in Ionen spalten.

* Ionen sind geladene Teilchen.

![ggg](https://raw.githubusercontent.com/deltorobarba/repo/master/sciences_0848.png)

http://basischemie.w-hoelzel.de/#spalten_atombindung.html

###### *Reaktionsmechanismen*

**Zusammenfassung:**

* Reaktionen werden nach der Funktion der angreifenden Teilchen benannnt.
* Als Angreifer wird i. a. das kleinere Teilchen bezeichnet.

![ggg](https://raw.githubusercontent.com/deltorobarba/repo/master/sciences_0851.png)

**Teilchen mit Elektronpaarlücke: <u>Elektrophile</u> Teilchen**

Elektrophile Teilchen sind sehr reaktionsfreudige Teilchen; sie wollen sich an ein freies Elektronenpaar anlagern: (= Lewis-Säuren).

![ggg](https://raw.githubusercontent.com/deltorobarba/repo/master/sciences_0849.png)

**Teilchen mit freien Elektronenpaaren: <u>Nucleophile</u> Teilchen oder Basen**

a) wollen positiv geladene oder positivierte Teilchen anlagern:

* als Basen Protonen
* als nucleophile Teilchen (Lewis-Basen) andere Teilchen mit Elektronenpaarlücke (elektrophile Teilchen)


b) Basizität und Nucleophilie verlaufen annähernd parallel

* so ist z.B. das Hydroxid-Ion stärker basisch und stärker nucleophil als Wasser
* Basizität und Nucleophilie sind von der Zahl der freien Elektronenpaare unabhängig!

![ggg](https://raw.githubusercontent.com/deltorobarba/repo/master/sciences_0850.png)

https://w-hoelzel.de/chemie/themen/118-chemie/chemie-extern/chemie-themen/10-klasse/01-alkane/158-018-angriffsfreudige-teilchen

**<u>Radikale</u>**

> Als Radikale bezeichnet man in der Chemie Atome oder Moleküle, die ein ungepaartes Elektron besitzen und damit sehr reaktiv sind. Sie sind üblicherweise instabil und kurzlebig.

* Radikale sind meist unbeständige Teilchen. Sie entstehen durch homolytische Spaltung (meist) unpolarer Bindungen unter Energiezufuhr (z.B. Wärme, Licht).

* Radikale sind sehr reaktionsfreudig und reagieren mit ihresgleichen oder mit Molekülen unter Neubildung von Radikalen.

![ggg](https://raw.githubusercontent.com/deltorobarba/repo/master/sciences_0852.png)


###### *Säure-Base-Reaktionen*

Supersäuren: https://www.scinexx.de/news/technik/chemiker-erzeugen-supersaeuren/

https://de.m.wikipedia.org/wiki/Anorganische_Chemie#Säuren_und_Basen

**Sehr wichtige anorganische Säuren sind:**

* Schwefelsäure

* Salzsäure

* Salpetersäure

* [Phosphorsäure](https://de.m.wikipedia.org/wiki/Phosphorsäure) oder Orthophosphorsäure ist die wichtigste Sauerstoffsäure des Phosphors und eine der wichtigsten anorganischen Säuren. Sie ist eine dreiprotonige Säure und reagiert bezüglich der ersten Deprotonierung als mittelstarke Säure.

  * Ihre Salze und Ester heißen [Phosphate](https://de.m.wikipedia.org/wiki/Phosphate) = Salze und Ester der Orthophosphorsäure (H3PO4). Fur Ester siehe auch: Organische Chemie -> 'Kohlenstoff mit Sauerstoff- und Hydroxyverbindungen'.

**Wichtige anorganische Basen sind:**

* Natronlauge

* Ammoniak

**Starke Säuren:**

* HCI - hydrochloric acid
* HNO3 - nitric acid
* H2S04 - sulfuric acid
* HBr - hydrobromic acid
* HI - hydroiodic acid
* HCIO4 - perchloric acid

**Starken Basen:**

* LiOH - lithium hydroxide
* NaOH - sodium hydroxide
* KOH - potassium hydroxide
* Ca(OH)2 - calcium hydroxide
* Sr(OH)2 - strontium hydroxide
* Ba(OH)2 - barium hydroxide

**Säure-Base-Reaktionen**

https://de.m.wikipedia.org/wiki/Lewis-Säure-Base-Konzept

**Säuren:**

* PH Wert zw 0 und 14, bei 7 neutral, darunter Säure. PH Papier wird rot

* Lebensmittel: Weinsäure, Apfelsäure, Ascorbinsäure (Vitamin C), Citronensäure, Kohlensäure, Phosphorsäure, Milchsäure, Essigsäure

* Man kann Haltbarmachen durch Säure: Yogurt oder Quark mit Milchsäure, auch saure Gurken.

* Essigessenz gegen Kalk (Calciumcarbonat, bzw Calziumsalz CaCO3). Bei PH unter 7 löst es sich auf in CO2.

* Technik: Schwefelsäure, Salzsäure HCl, Salpetersäure sins starke Säuren

* Säuren (genauer: saure Lösungen) leiten zB Strom (Schwefelsäure in Autobatterien), da Ionen vorliegen

* Säuren in Wasser gelost sind Säurelösungen. zB ist Citronensäure ein Pulver, und damit ein Feststoff (ähnlich wie bei Basen und Laugen als wässrige Lösung)

> Starke Säure bedeutet, dass alle Säuremoleküle zerfallen, wenn sie mit Wasser reagieren. D.h. jedes HCL-Molekül produziert ein H+ Ion. Und Säuren erhalten nur ihre Eigenschaften bei Reaktion mit Wasser.

* Essig- oder Zitronensäure sind schwache Säuren (es reagiert nur ein Teil der Säuremoleküle mit Wasser), und daher können wir sie auch verzehren.

![ggg](https://raw.githubusercontent.com/deltorobarba/repo/master/chem_003.png)

> Bei Kontakt mit Wasser zerfallen alle Säure-Moleküle (zB Salzsäure HCl) in Protonen (H+ Ionen) und negativ gelandene Säurereste

* hier 3 Beispiele, wo man sieht wie Säure mit Wasser reagiert, und es entsteht ein positiv geladenes Proton und negativ gelandener Säurerest. Was genau mit dem Wassermolekul passiert (weil es dipolar ist) siehe unten bei Basen - es ist der gleiche Prozess

* (Beispiele: Citronensäure, Essigsäure, und Schwefelsäure)

* Video: https://youtu.be/x93KKVDahKY

![ggg](https://raw.githubusercontent.com/deltorobarba/repo/master/chem_002.png)

> Bei Kontakt mit Wasser zerfallen alle Säure-Moleküle (zB Salzsäure HCl) in **Protonen (H+ Ionen)** und negativ gelandene Säurereste, und alle Basenmoleküle (zB Natriumhydroxid NaOH)in **negativ geladene (OH-) Ionen (Hydroxid-Ionen)** und positiv geladene Basenreste.

![ggg](https://raw.githubusercontent.com/deltorobarba/repo/master/chem_004.png)

**Basen (Laugen)**

* Basen sind Alkalimetallverbindungen

* siehe auch [starke Basen](https://de.m.wikipedia.org/wiki/Starke_Basen)

* PH Wert grösser als 7, PH Papier oder PH Lösung werden blau

* Base ist reiner Stoff, Lauge ist eine wässrige Lösung dieses Stoffes

> Bei Kontakt mit Wasser zerfallen alle Basenmoleküle (zB Natriumhydroxid NaOH)in negativ geladene (OH-) Ionen (Hydroxid-Ionen) und positiv geladene Basenreste

* Natriumhydroxid (NaOH)

  * = Base

  * ist ein feststoff.

  * Löse ich es in Wasser, dann ist es eine Lauge (auch: alkalische Lösung).

  * Grösste technische Verwendung. Mit Wasser als Lauge: Natronlauge (zB für Abfluss-/ Rohrreiniger). NaOH ist starke Base, daher sehr ätzend.

* Basen sind ätzend: Seife im Auge = brennt. Laugen zersetzen Fette und Eiweisse.

* Im Alltag: Seifen, Spülmittel, Waschmittel

* Menschliche Haut ist aber leicht sauer (dünner Säureschutzmantel) mit PH Wert von 5,5. Daher gibt es PH-neutrale Seifen.

Video:  https://youtu.be/0odQLq6EJBc


*Hydrathülle bei Reaktion von Wasser mit Basen (und Säuren):*

* Natriumhydroxid ist einen Ionenverbindung, die aus positiv geladenen Natriumionen Na+ und negativ geladenen Hydroxidionen OH- aufgebaut ist. Festes Natriumhydroxid ist also als einem Ionengitter, auch genannt Kristallgitter (siehe Ionenbindung).

* Sobald dieses Ionengitter in Kontakt mit Wasser kommt, wird der Kristall durch die Wassermoleküle aufgebrochen. Die Ionenbindung  wird getrennt!

* Genau wie bei den Säuren werden Natriumionen und Hydroxidionen durch Wassermoleküle hydratisiert. **Wassermoleküle sind Dipolmoleküle: Wasser hat am Sauerstoffatom eine negative Partialladung und an den Wasserstoffatomen eine positive Partialladung**.

> **Dh die Sauerstoffseite sucht nach positiven Ladungen und die Wasserstoffseite nach negativen Ladungen.**

* Da sich positive und negative Ladungen anziehen passiert folgenden: Wassermoleküle lagern sich um positive Natriumionen an. **Sauerstoffatome "gucken" alle nach innen zum positiv geladenen Natriumion (Bild unten rechts).**

* **Beim Hydroxidion (OH-):**
  * **Auf der Sauerstoffseite des Hydroxidions (negative Partialladung) schauen die Wassermoleküle (positive Partialladung) mit den H-Atomen nach innen.**

  * **Und auf der Wasserstoffseite (positive Partialladung) des Hydroxidions schauen die Wasserstoffmoleküle mit dem O-Atom (negative Partialladung) nach innen (Bild unten links)**.

* Ergo: die Wassermoleküle sind so angeordnet dass Wasseratome jeweils zu Sauerstoffatomen gucken und andersrum. **Man nennt diese angeordneten Wassermoleküle Hydrathüllen: Das Wasser bildet eine Hülle und packt die Ionen dadurch regelrecht ein.**

> Die Reaktion beschreibt man dann so: NaOH(s) → Na+(aq) + OH-(aq)

* das ist eine Lauge / alkalische Loesung, hier: Natronlauge

* auch andere Hydroxide bilden mit Wasser alkalische Losungen: Kaliumhydroxid (KOH) oder Calciumhydroxid Ca(OH)2

* Video: https://youtu.be/J0-LFHwcgZs


![ggg](https://raw.githubusercontent.com/deltorobarba/repo/master/chem_001.png)

*Neutralisation*

* wenn man Sauren und Basen vermischt, dann neutralisieren sie sich (aber es entsteht viel Hitze) - exotherm!

> Chemische Reaktion zwischen NaOH (starke Base) und HCL (starke Saure): NaOH + HCL → NaCL (Kochsalz) + H2) (Wasser)

* Video: https://youtu.be/5c5SrQb2SM4

Beispiel: Reaktion von Salzsaure HCL mit Natriumhydroxid NaOH:

![ggg](https://raw.githubusercontent.com/deltorobarba/repo/master/chem_005.png)

H+ und OH- werden zu Wasser H2O:

![ggg](https://raw.githubusercontent.com/deltorobarba/repo/master/chem_006.png)

CL- und Na+ bleiben hydratisiert ubrig. Wir haben also eine wassrige NaCL-Losung:

![ggg](https://raw.githubusercontent.com/deltorobarba/repo/master/chem_007.png)

Wenn wir das Wasser verdampfen wuerden, wuerde festes Kochsalz ubrig bleiben:

![ggg](https://raw.githubusercontent.com/deltorobarba/repo/master/chem_008.png)

Beim backen: Natriumhydrogencarbonat NaHCO3 (zum aufbacken) vermischt mit Saure im Backpulver (festform)

* Kuchen entsteht durch CO2 in der Reaktion von Saure mit Natriumhydrogencarbonat (neutralisiert die Saure). CO2 laesst den Kucne aufbacken.

* Unterschied Natron zu Backpulver? - Natron ist reines Natriumhydrogencarbonat (ohne Saure hinzugefugt wie beim Backpulver). Natron benutzt man dann, wenn in den Zutaten bereits Sauren enthalten sind (zB bei Citronenkuchen, Joghurt, Quark, Honig und Schokolade haben Sauren. H+ kommt schon aus den Zutaten).

* Im Backpulver ist in der Regel 1/3 Natron enthalten

![ggg](https://raw.githubusercontent.com/deltorobarba/repo/master/chem_009.png)

###### *Redoxreaktionen*

**Redoxreaktionen (Speziell: Sauerstoff)**

Welcher der beteiligten Reaktionspartner Reduktions- beziehungsweise Oxidationsmittel ist, lässt sich anhand der Elektronegativitäten der beteiligten Elemente vorhersagen. Elemente mit niedrigen Elektronegativitäten, wie die meisten Metalle, geben leicht Elektronen ab und werden dementsprechend oxidiert, Nichtmetalle mit hohen Elektronegativitäten werden dagegen leicht reduziert. Sind Ionen an einer Redoxreaktion beteiligt, ist auch die Oxidationsstufe des Ions zu beachten. So sind Chromate oder Permanganate, bei denen die Elemente in hohen Oxidationsstufen vorliegen, starke Oxidationsmittel.

siehe artrn von reaktionen in https://de.m.wikipedia.org/wiki/Chemische_Reaktion

https://de.m.wikipedia.org/wiki/Chemische_Reaktion#Oxidation_und_Reduktion

**Redoxreaktionen**

* Redoxreaktionen sind alle energieliefernden und energiespeichernden Prozesse in Natur und Technik (ohne Redoxreaktionen kein Leben!)

* Redoxreaktionen: Oxidation und Reduktion laufen immer zusammen ab

* Redoxreaktionen laufen wie Säure-Base-Reaktionen nach dem **Donor-Akzeptor-Prinzip**

  * Säure: gibt Proton ab, Base: nimmt Proton auf

  * Oxidationsmittel: gibt Sauerstoff ab, Reaktionsmittel nimmt Sauerstoff auf (Wasserstoff zB)

**Oxidationen**

* Bei Oxidationen sind die Reaktionsprodukte schwere als vorher.

	* Grundlage: Sauerstoff. Es reagiert sehr heftig mit fast allen Stoffen. Sauerstoff enstand zum ersten Mal vor 2,3 Mrd Jahren als Fotosynthese-Produkt von Cyanobakterien und Blaualgen.

	* Rosten ist eine Oxidationsreaktion von Eisen: Eisen + Sauerstoff + Wasser = Eisenoxid (enthält Sauerstoff in chemisch gebundener Form, daher ist es schwerer am Ende als vorher)

Siehe auch: [Oxidationszahl](https://de.m.wikipedia.org/wiki/Oxidationszahl)

> **Oxidation: chemischer Vorgang, der mit der Aufnahme von Sauerstoff verbunden ist. Alle Verbrennungsvorgänge zählen zu Oxidationen.**

* Metall + Sauerstoff = Metalloxid

* Nichtmetall + Sauerstoff = Nichtmetalloxid

* Beispiel:

	* Eisen + Sauerstoff ➔ Metalloxid

	* Magnesium  + Sauerstoff ➔ Magnesiumoxid

	* Kohlenstoff + Sauerstoff ➔ Kohlenstoffoxid

	* Schwefel + Sauerstoff ➔ Schwefeldioxid


* Feuer, Atmung oder Rost basieren auf dem gleichen Prinzip: Oxidation

* Oxidation schließlich als Elektronenabgabe eines chemischen Stoffes aufgefasst werden

* Die Oxidation ist nicht zwangsläufig mit einer vollständigen Abgabe von Elektronen und damit der Ionisation der beteiligten Stoffe verbunden – insbesonders in der Organischen Chemie und Biochemie.

https://www.chemie.de/lexikon/Redoxreaktion.html

![ggg](https://raw.githubusercontent.com/deltorobarba/repo/master/chem_029.jpg)

* Menschen atmen CO2 aus, weil im Körper die Nahrung zwar nicht durch Verbrennungen, aber durch den Stoffwechsel biologisch oxidiert.

* braune Äpfel: Polyphenole (Moleküle) auf dem Apfel reagieren mit Sauerstoff in der Luft und bilden braune Farbe

* anderes Beispiel: Rost. Nagel aus Eisen draussen rumliegen lassen - dann entstehen durch die Reaktion von Eisen mit Sauerstoff sogenannte Eisenoxide (Rost). Und alte Cent-Münze aus Kupfer: Kupferoxide bei Reaktion mit Sauerstoff.

* Mit Küchenessig kann man das reinigen: Kupferoxid-Schicht löst sich in Essigsäure auf und Münzen glänzen wieder.

[Oxidation im Alltag](https://youtu.be/5xOr1ri-XK8)

**Reduktion**

* kann man auch von einem Oxid zuruck zum Element kommen, also Kann man aus einem Metalloxid das Metall und den Sauerstoff wieder zurückgewinnen?

* Beispiel: Silberoxid unter Feuer: $2Ag_2O$ ➔ 4 Ag + O2 (Sauerstoffgas kann man mit der Glimmspanprobe nachweisen)

> Reduktion: Die Abgabe von Sauerstoff aus einer Verbindung.

* durch einfaches Erhitzen ist soft schwer zu reduzieren, man braucht einen Partner, der Sauerstoff haben will. So ein Reaktionspartner ist zB Wasserstoff!

**Oxidationsmittel**: Reaktionspartner, die **Sauerstoff abgeben** (Kupferoxid unten)

**Reaktionsmittel**: Reaktionspartner, die **Sauerstoff aufnehmen** (Wasserstoff unten)

**Beispiel: Leitet man Wasserstoff über schwarzes Kupferoxid in einem Glasrohr, dann bildet sich rötliches elementares Kupfer.**

CuO (Kupferoxid) + H2 (Wasserstoff) ➔ Cu + H2O

* dem Kupferoxid wird der Sauerstoff entzogen (Reduktion)

* der Wasserstoff nimmt den Sauerstoff auf (Oxidation), er wird oxidiert

![ggg](https://raw.githubusercontent.com/deltorobarba/repo/master/chem_017.png)

Küchenbeispiel fur Oxidation: Apfel zu muss zerreiben. Dann werden aus den zerstörten Pflanzenzellen Enzyme freigesetzt, die die Reaktion mit dem Sauerstoff in der Luft zu eine braunen Farbstoff beschleunigen (Farbstoff: Melanin). Stoffe wie Vitamin C können die Braunfärbung von Apfelmus verhindern - daher nennt man Vitamin C ein Antioxidant (zB in Zitronensaft).

**Redoxreaktionen (Generell: Elektron)**

* Metalle reagieren mit Sauerstoff zu Metalloxiden, oft unter Feuer

* (reminder: Metalle und Nichtmetalle reagieren zu Ionenverbindungen bzw. Salzen)

* Verbrennungsvorgänge sind aber nicht nur auf Beteiligung von Sauerstoff beschrankt: Man kann Wasserstoff auch in Chloratmosphäre verbrennen. Auch eine Kerze brennt in Chloratmosphäre fats genauso gut wie in der Luft mit Sauerstoff.

* Bei Metallen, einmal mit Sauerstoff, dann mit Chlor:

	* 2 Mg (Magnesium) + O2 (Sauerstoff) ➔ 2MgO (Magnesiumoxid)

	* 2 Mg (Magnesium) + Cl2 (Chlor) ➔ 2MgCl2 (Magnesiumchlorid)

> **Gemeinsamkeit bei Oxidbildung und Chloridbildung: sogenannte "Elektronenubergange" liegen zugrunde**



* ein Magnesiumatom gibt 2 Elektronen ab, die sich ein Sauerstoffatom nimmt: Mg2+ Ion und O2- Ion (Sauerstoff wird elektronegativer)

* Es können aber auch zwei Chloratome sein, die sich die Elektronen nehmen: Metall wird zu einem positiv geladenen Ion Mg2+ Ion), und Nichtmetall-Atom bilden sich negativ geladene Ionen, die Anionen (2 x Cl- Ion)

* Es müssen dabei immer soviel Elektronen aufgenommen wie abgeben werden (es darf kein Mangel oder Überschuss geben)

* Reaktionsgleichungen:

	* Elektronenabgabe eine Magnesiumatoms: Mg ➔ Mg2+ + 2e- (es entstehen zweifach positiv geladene Magnesium-Kationen)

	* Sauerstoff nimmt zwei Elektronen auf: O + 2e- ➔ O2- (es bildet sich ein zweifach negativ geladenes Oxid-Anion)

> Moderne Definition der Redoxreduktion: **Eine Redoxreduktion ist eine Reaktion, bei der ein Reaktionspartner Elektronen auf einen anderen überträgt.**

* die Teilreaktion Elektronenabgabe: Oxidation

* die Teilreaktion Elektronenaufgabe: Reduktion


**Magnesium wird oxidiert und Chlor oder Sauerstoff werden reduziert.**

* **Donor**: Magnesium ist dann Reduktionsmittel (vermittelt den Vorgang der Reduktion durch Abgabe von Elektronen) und wird selbst oxidiert.

* **Akzeptor**: Chlor oder Sauerstoff sind Oxidationsmittel, weil sie die Oxidation durch Aufnahme der abgegebenen Elektronen vermitteln, und wird selbst reduziert.

Video: https://www.youtube.com/watch?v=Y3BmwGyVjt8&list=PLMRuqWRlK1n0t0YIGxaKNA4AbPA9ubnjD&index=5

**Redoxreaktionen (Redoxreihe der Metalle)**

Siehe auch: [Elektrochemische Spannungsreihe](https://de.m.wikipedia.org/wiki/Elektrochemische_Spannungsreihe): materialtypische Eigenschaft verschiedener Metalle

Redoxreihe der Metalle mit den positiven Metallkationen, um zu schauen, wer ist Donor und wer ist Akzeptor (wie Metallatome aussehen wuerden wenn sie Elektronen abgeben wuerden. Entsprechend einfach-, zweifach- oder dreifach geladen)

* grunsatzlich koennen alle Metalle oxidiert werden. Sie koennen Elektronen abgeben, um positive Kationen zu bilden. Und alle Metallkationen koennen prinzipiell reduziert werden (= Elektronen aufnehmen, um zu festem Metall zu werden).

* links: unedel (wollen Elektronen abgeben), rechts: edel (wollen Elektronen aufnehmen)

* ps: 80% aller Elemente sind Metalle

![ggg](https://raw.githubusercontent.com/deltorobarba/repo/master/chem_018.png)

Links: unedel (wollen Elektronen abgeben), rechts: edel (wollen Elektronen aufnehmen)

* Wenn die Metalle aber als Ionen vorliegen, also in ihrer oxidierten Form, dann nehmen sie sehr gerne Elektronen auf.

* Je edler ein Metall, desto eher neigen Metallionen dazu, Elektronen aufzunehmen.

![ggg](https://raw.githubusercontent.com/deltorobarba/repo/master/chem_019.png)

Beispiel: Nagel (Fe) in Kupfersulfatlösung (Cu2+SO4 2-). Eisen neigt eher dazu Elektronen abzugeben und wird daher oxidiert. Es bildet sich Rost. An der Nageloberflache lagert sich elementares Kupfer ab.

Video: https://www.youtube.com/watch?v=dLLxuL-AlOE&list=PLMRuqWRlK1n0t0YIGxaKNA4AbPA9ubnjD&index=2

**Redoxreaktionen (Korrosion von Metallen)**

* Korrosion: Reaktion von Stoffen mit Luft, Wasser, Laugen, Säuren oder aggressiven Gasen.

* Korrosionsreaktion: Rosten, anderes BeispielL Glas oder Zinkoberflaeche verliert Glanz

* Korrosionsreaktion bei Metallen liegen Redoxreaktionen zugrunde. Man nennt sie deshalb elektrochemische Korrosion.

**Zwei Oxidationsmittel fur Korrosion bei Metallen: Sauerstoff und Säuren**

* Korrosion mit Säuren: trifft Säure auf unedles Metall wie Zink, dann wirken H+ Ionen, also Säureprotonen als Oxidationsmittel. Sie selbst werden zu Wasserstoff reduziert.

* Sauerstoffkorrosion (hier zB mehrere Oxidationstufen): Rost. Eisen ist unedles Metall und gibt Elektronen ab und wird zu Eisen 2+ oxidiert. Es entsteht Eisenhydroxid, ein schwer löslicher Stoff, der sich ablagert. Dieser reagiert mit Sauerstoff in separater Redoxreaktion und wird zu Eisenoxidhydroxid (der Rost).

**Wie kann man Korrosionen vermeiden?**

* Metalle können sich selbst schützen (zB u.a. Kupfersalze bei Freiheitsstatue). Nennt man Patina und schützt darunter liegendes Metall vor Wasser, Sauerstoff, Sauren etc. Nennt man Passivierung.

* Nichtmetallbeschichtung, zB Lackierung,

* Mit anderen (unedleren) Metallen beschichten (zB mit Chrom oder Zinn, um Eisen zu schützen), und wird zu stabiler Oxidschicht (zB Regenrinnen aus Eisen werden mit Zinn beschichtet).

* Mit anderen (edleren) Metallen beschichten, zB Kupfer, weil es weniger korrosionsanfällig ist. Reminder: Redoxreihe, je edler ein Metall, desto geringer Neigung zur Oxidation.

Video: https://www.youtube.com/watch?v=ld83QYrMDrI&list=PLMRuqWRlK1n0t0YIGxaKNA4AbPA9ubnjD&index=3&t=224s

###### *Redoxreaktionen (Galvanische Zelle)*

![ggg](https://upload.wikimedia.org/wikipedia/commons/thumb/a/a5/Galvanic_cell_labeled.svg/853px-Galvanic_cell_labeled.svg.png)

Source: https://de.m.wikipedia.org/wiki/Elektrochemische_Spannungsreihe

* Galvanische Zelle (= Redoxreaktion, die räumlich in zwei Halbzellen getrennt ist): Batterie 1835

* Alle Batterien und Akkus sind galvanische Zellen. Akkus können im Gegensatz zu Batterien wieder aufgeladen werden, indem man eine äußere Spannung anlegt, um die Redoxreaktion wieder umzukehren.

* Redoxreaktion gezielt nutzen um Strom zu erzeugen: bei Redoxreaktion werden Elektronen von einem Reaktionspartner (unedel) yu einem anderen (edel) übertragen. Wenn wir diese abfangen können, produzieren wir Strom als Batterie.

> um sich eine Batterie zu erstellen, braucht man nur eine Redoxreaktion

* bei der Batterie trennt man Oxidation und Reduktion räumlich

Das unedle Metall (hier Zink) gibt Elektronen ab und wird zu Zink2+ oxidiert. Es verwandelt sich also vom festen Metall Zink Zn in ein löslichen Zink-Ion um, welches sich dann in der Wasser- bzw. Salzlösung löst.

![ggg](https://raw.githubusercontent.com/deltorobarba/repo/master/chem_021.jpg)

Das edle Metall (hier Kupfer), tendiert eher dazu Elektronen aufzunehmen. Deswegen passiert mit einem Kupferstab in einer Salzlösung meistens nicht so viel.

Wenn wir allerdings Kupfer-Ionen vorliegen haben, also Kupfer 2+ Ionen, dann nehmen die sich gerne die Elektronen, welche das Zink abzugeben hat und werden reduziert.

Die wasserlöslichen Ionen wandeln sich also in festes Kupfermetall Cu um = Das Kupfer scheidet sich ab (genau das passiert mit dem Eisennagel in der Kupfersulfatlösung).

![ggg](https://raw.githubusercontent.com/deltorobarba/repo/master/chem_022.jpg)

Jetzt nehmen wir einen Zinkstab, der in eine Zinksulfatlösung eingetaucht ist. Wir haben hier also sowohl elementares Zink Zn als auch Zinkionen Zn2+ vorliegen. Man auch sageb: wir haben Zink sowohl in seiner reduzierten Form (fest) als auch in seiner oxidierten Form (als Ionen).

![ggg](https://raw.githubusercontent.com/deltorobarba/repo/master/chem_023.jpg)

Das gleich machen wir nochmal, aber mit Kupfer: ein Kupferstab, der in einer Kupfersulfatlösung eingetaucht ist. Kupfer liegt dann sowohl in seiner reduzierten, festen Form vor, als auch in seiner oxidierten als Kupfer 2+ Ionen.

![ggg](https://raw.githubusercontent.com/deltorobarba/repo/master/chem_024.jpg)

Noch passiert aber nichts. Wir müssen beide Seiten mit einem Verbindungskabel verbinden:

![ggg](https://raw.githubusercontent.com/deltorobarba/repo/master/chem_020.jpg)

Zink gibt Elektronen ab und wird dadurch zum positiv geladenen Zink-Ion (links), welches dadurch wieder in Lösung geht. Für jedes Zink-Ion, das entsteht, und das zweifach positiv geladne ist, bleiben im Metallstab 2 Elektronen zurück. Dadurch entsteht ein Elektronenüberschuss. Zinkstab wird Minuspol der Batterie.

Beim Kupferstab (rechts) passiert das Gegenteil: Die Kupferionen in der Lösung nehmen 2 Elektronen aus dem Stab auf und werden dadruch zu festem Kupfer Cu. Das Kupfer setzt sich aus der Lösung an dem Stab ab (Reduktionsreaktion). Es entsteht an dieser Seite damit eine positive Ladung.

![ggg](https://raw.githubusercontent.com/deltorobarba/repo/master/chem_025.jpg)

Bisher haben wir aber noch keine Batterie: wir haben lediglich eine Spannung (zw. Plus- und Minuspol). Aber es fliesst noch kein Strom. Der Grund ist, dass am Zinkstab ein Überschuss an Zink 2+ Ionen entsteht, und die Lösung sich dadurch stark positiv auflädt.

Denn für jedes Zinkatom, das 2 Elektronen abgibt, geht ja ein zweifach geladenes Zink-Ion in Lösung. Also gehen nur noch so viele Zinkionen in Lösung, wie sich gleichzeitig an der Eletrode wieder abscheiden.

Das gleiche Problem haben wir am Kupferstab: wenn hier aus Kupfersulfat Kupfer wird, bleiben negativ geladene Sulfationen in der Lösung zurück. Diese lädt sich dann stark negativ auf.

![ggg](https://raw.githubusercontent.com/deltorobarba/repo/master/chem_026.jpg)

Deswegen brauchen wir noch eine letzte Sache: einen Ladungsausgleich. Die starke positive bzw. negative Aufladung der Lösungen in den beiden Halbzellen müssen wir ausgleichen.  Dazu packen wir den Kupferstab in der Kupfersalzlösung mit dem Zinkstab in der Zinklösung zusammen. Wir nennen sie jetzt 2 Halbzellen.

Diese Halbzellen werden verbunden durch eine poröse Tonwand: Ein sogenanntes Membran. Porös heisst, dass in der Wand ganz kleine Löcher vorhanden sind, so klein, dass da Ionen durchpassen (Zn2+ und SO4 2-. Also durch diese poröse Trennwand hin- und her fliessen können.


Und das ist der Knackpunkt: jetzt können die überschüssigen Sulfat-Ionen aus der Kupfer-Halbzelle in die Zink-Halbzelle wandern, und andersrum können die positiven Zink-Ionen in die Kupfer-Halbzelle wandern.

![ggg](https://raw.githubusercontent.com/deltorobarba/repo/master/chem_028.jpg)

Und damit ist der Stromkreis geschlossen. Jetzt fliesst durch das Kabel Strom, also Elektronen.

![ggg](https://raw.githubusercontent.com/deltorobarba/repo/master/chem_027.jpg)

https://www.youtube.com/watch?v=gifwGurgfVk&list=PLMRuqWRlK1n0t0YIGxaKNA4AbPA9ubnjD&index=4

###### *Komplexbildungsreaktion*

Bei der [Komplexbildungsreaktion](https://de.m.wikipedia.org/wiki/Chemische_Reaktion#Komplexbildungsreaktion) reagieren mehrere Liganden mit einem Metallatom zu einem Komplex. Dies erfolgt dadurch, dass freie Elektronenpaare der Liganden in leere Orbitale des Metallatoms eindringen und eine koordinative Bindung bilden. Bei den Liganden handelt es sich um Lewis-Basen, die freie Elektronenpaare besitzen. Dies können sowohl Ionen als auch neutrale Moleküle (etwa Kohlenstoffmonoxid, Ammoniak oder Wasser) sein.

https://de.m.wikipedia.org/wiki/Komplexbildungsreaktion

###### *Fällung*

Die [Fällung](https://de.m.wikipedia.org/wiki/Chemische_Reaktion#Fällung) ist eine Reaktion, bei der sich vorher gelöste Teilchen verbinden und zu einem neuen, wasserunlöslichen Stoff werden, dem Niederschlag. Dies findet vor allem bei gelösten Ionen statt, die sich bei Überschreitung des Löslichkeitsproduktes zusammenfinden und ein unlösliches Salz bilden. Dies kann beispielsweise durch Zugabe eines Fällungsmittels mit geringem Löslichkeitsprodukt zu einem schon gelösten Salz oder durch Entfernen des Lösungsmittels erfolgen.

https://de.m.wikipedia.org/wiki/Fällung

###### *Festkörperreaktionen*

Reaktionen können auch zwischen zwei festen Stoffen stattfinden. Jedoch ist die Diffusion, die die Reaktionsgeschwindigkeit hierbei maßgeblich bestimmt, sehr klein, [Festkörperreaktionen](https://de.m.wikipedia.org/wiki/Chemische_Reaktion#Festkörperreaktionen) sind dementsprechend langsame Reaktionen. Dies bewirkt, dass Festkörperreaktionen in der Regel bei hohen Temperaturen durchgeführt werden müssen. Gleichzeitig sollten die Reaktanten möglichst fein verteilt vorliegen, da so eine möglichst große Oberfläche, an der die beiden Stoffe reagieren können, geschaffen wird.

https://de.m.wikipedia.org/wiki/Festkörper

###### *Photochemische Reaktionen*

* In [photochemischen Reaktionen](https://de.m.wikipedia.org/wiki/Chemische_Reaktion#Photochemische_Reaktionen) spielt elektromagnetische Strahlung eine entscheidende Rolle. Von besonderer Bedeutung sind hierbei Licht und UV-Strahlung von etwa 200 bis 800 nm Wellenlänge. Durch diese Strahlung werden Elektronen in Atomen und Molekülen angeregt, es bilden sich angeregte Zustände. Diese sind durch die absorbierten Photonen sehr energiereich und können die Energie über verschiedene Prozesse abgeben. Neben physikalischen Prozessen wie Fluoreszenz und Phosphoreszenz sind hier auch Reaktionen möglich. Häufig erfolgen homolytische Bindungsbrüche, so dass Radikale entstehen. So können durch photochemische Reaktionen beispielsweise Kettenreaktionen wie die Knallgasreaktion von Wasserstoff und Sauerstoff ausgelöst werden. Aber auch Ionisierungen, Elektronentransferreaktionen, Isomerisierungen oder Umlagerungen können durch photochemische Reaktionen verursacht werden.

* Eine biologisch sehr wichtige photochemische Reaktion ist die [Photosynthese](https://de.m.wikipedia.org/wiki/Photosynthese), bei der mit Hilfe von Licht organische Verbindungen aus Kohlenstoffdioxid und Wasser synthetisiert werden. Auch in der Atmosphärenchemie, etwa beim Auf- und Abbau von Ozon spielen photochemische Reaktionen eine wichtige Rolle.

https://de.m.wikipedia.org/wiki/Photochemie

###### *Katalyse (indirekt)*

Bei einer [Katalyse](https://de.m.wikipedia.org/wiki/Chemische_Reaktion#Katalyse) findet die Reaktion zweier Stoffe nicht direkt, sondern über einen Umweg statt. Es ist immer ein dritter Stoff, der sogenannte Katalysator, beteiligt, der in die Reaktion eingreift, aber am Ende stets unverändert aus der Reaktion hervorgeht.

https://de.m.wikipedia.org/wiki/Katalyse

###### *Reaktionen in der organischen Chemie*

* In der organischen Chemie gibt es neben den auch bei anorganischen Stoffen ablaufenden Reaktionen wie Oxidationen, Reduktionen oder Säure-Base-Reaktionen eine Vielzahl weiterer Reaktionen, bei denen kovalente Bindungen zwischen Kohlenstoffatomen oder Kohlenstoff- und Heteroatomen (beispielsweise Sauerstoff, Stickstoff, Halogene) gebildet werden.

* Diese werden neben der Unterscheidung zwischen homolytischen und radikalisch ablaufenden Reaktionen, vor allem nach der Art der Strukturänderung eingeteilt. Viele spezielle Reaktionen in der organischen Chemie sind als Namensreaktionen nach ihren Entdeckern benannt.



https://de.m.wikipedia.org/wiki/Chemische_Reaktion#Reaktionen_in_der_organischen_Chemie

**Substitution**

Bei der Substitution wird ein Atom, Molekülteil oder Ligand (in der Komplexchemie, in der Substitutionen ebenfalls möglich sind) gegen einen anderen ausgetauscht. Ein angreifendes Atom oder Molekül nimmt dabei den Platz eines anderen, als Abgangsgruppe abgespaltenen Atoms oder Moleküls ein. Die Bindigkeit des Kohlenstoffatoms ändert sich nicht.



https://de.m.wikipedia.org/wiki/Substitutionsreaktion

**Addition/Eliminierung**

Die Addition und das Gegenstück, die Eliminierung, sind Reaktionen, bei denen sich die Anzahl der Substituenten am Kohlenstoffatom ändert und Mehrfachbindungen gebildet oder gespalten werden. Bei Eliminierungsreaktionen werden Doppel- und Dreifachbindungen aufgebaut, indem an jedem Kohlenstoffatom der Bindung jeweils ein Substituent entfernt („eliminiert“) wird. Für eine Eliminierung muss sich an einem Kohlenstoffatom der fraglichen Bindung eine geeignete Abgangsgruppe befinden, die relativ leicht abgespalten werden kann.

https://de.m.wikipedia.org/wiki/Additionsreaktion

https://de.m.wikipedia.org/wiki/Eliminierungsreaktion

**Weitere organische Reaktionsmechanismen**

https://de.m.wikipedia.org/wiki/Chemische_Reaktion#Weitere_organische_Reaktionsmechanismen

https://de.m.wikipedia.org/wiki/Umlagerung

###### *Biochemische Reaktionen*

https://de.m.wikipedia.org/wiki/Chemische_Reaktion#Biochemische_Reaktionen

* In biochemischen Reaktionen sind Enzyme von zentraler Bedeutung.

* Diese Proteine katalysieren meist speziell eine einzelne Reaktion, so dass Reaktionen sehr exakt gesteuert werden können. Es sind aber auch Enzyme bekannt, die mehrere spezielle Funktionen katalytisch beschleunigen können.

* Die Reaktion findet in einem kleinen Teil des Enzyms, dem Aktiven Zentrum statt, während der Rest des Enzyms überwiegend zur Stabilisierung dient. Das aktive Zentrum liegt in einer Grube oder Furche des Enzyms.

* Für die katalytische Aktivität sind unter anderem Bindungen an das Enzym, die veränderte, hydrophobe, chemische Umgebung und die räumliche Nähe der Reaktanten verantwortlich, während die spezielle Form des aktiven Zentrums für die Selektivität verantwortlich ist.

###### *Nachweisreaktion (Flammfärbung)*

https://de.m.wikipedia.org/wiki/Flammenfärbung

https://de.m.wikipedia.org/wiki/Nachweisreaktion

###### *Verfahrenstechnik*

https://de.m.wikipedia.org/wiki/Gegenstromprinzip_(Verfahrenstechnik)

https://de.m.wikipedia.org/wiki/Verfahrenstechnik

###### *Gase*

https://de.m.wikipedia.org/wiki/Anorganische_Chemie#Gase

### <font color="blue">**Chemisches Rechnen**

##### <font color="blue">*Stöchiometrie*

###### *Molekülmasse, Stoffmenge & molare Masse*

**Die Größe und Masse von Atomen in Einheit u**

> **$1 u=\frac{1}{12} \cdot m\left({ }^{12} C\right)$ für Massen von Atomen oder Molekülen angegeben in u**


Zur Umrechnung von Atommasse u in Gramm mit dem Umrechnungsfaktor $(6,022 \cdot 10^{23})$ g

  * von u nach g: Divide by $6,022 \cdot 10^{23} u$

  * von g nach u: Multiply with $6,022 \cdot 10^{23} u$

> $1 g=\frac{1}{1,66 \, 10^{-24}} u =6,022 \cdot 10^{23} u$

Beispiel: In einen Stecknadelkopf passen 100 Trillionen Eisenatome = 10$^{20}$

* 1980: Erfindung des Rastertunnelmikroskops zur Sichtbarmachung von Atomen

* Atommasse ist sehr klein: ein Wasserstoffatom hat z.B. die Masse von 0,000 000 000 000 000 000 000 001 67 g

* Bezugspunkt fur die Masse ist das Kohlenstoff-Isotop C12 mit dem Wert 12, und davon nimmt man den 12ten Teil = atomare Masseneinheit u

> $1 u=\frac{1}{12} \cdot m\left({ }^{12} C\right)$

* m$_a$ (H) = 1,0079 u

* m$_a$ (O) = 15,9994 u

> **Umrechnung von atomerer Masseneinheit u in Gramm mit Faktor 6,022 $\cdot$ 10$^{23}$. Das ist auch der Zahlenwert der Avogadro-Konstante**

* Wie weiss man nun wir schwer jedes Atom ist? - In jeder Elemententafel steht in der Ecke die Atommasse u

* Source: Video: https://youtu.be/xQaafi2zLik

**Was ist die Masse eines Wassermoleküls H$_2$O?**

> $\begin{aligned} m_{A}\left(H_{2} O\right) &=2 \cdot m_{A}(H)+1 \cdot M_{A}(0) \\ &=2 \cdot 1,01 u+15,99 u \\ &=18,01 u \end{aligned}$

**Wie viel wiegt ein Wassermolekül?**

* Atommasse geteilt durch u (Zahlenwert in der Avogadro-Konstante)

> $18,01:\left(6,022 \cdot 10^{23}\right) =3 \cdot 10^{-23}$ = 0.000 000 000 000 000 000 000 03 g

![ggg](https://raw.githubusercontent.com/deltorobarba/repo/master/sciences_045.png)

**Die Menge von Atomen (Teilchenanzahl N und Stoffmenge Mol in n)**

> **1 Mol = 6,022 $\cdot$ 10$^{23}$ Teilchenzahl N angegeben in Stoffmenge Mol**

Umrechnung von Mol in Teilchenanzahl N:

> N = N$_a$ $\cdot$ n

* mit N$_a$ = Avogadro-Konstante 6,022 $\cdot$ 10$^{23}$ in Einheit $\frac{1}{mol}$

* n = Stoffmenge in mol

* Stoffmenge in Teilchenzahl: Multiply Stoffmenge by Avogadro-Konstante 6,022 $\cdot$ 10$^{23}$

* Teilchenzahl in Stoffmenge: Divide Teilchenzahl by Avogadro-Konstante 6,022 $\cdot$ 10$^{23}$

Beispiel: 1 cm$^2$ Alufolie: darauf befinden sich 60 $\cdot$ 10$^{18}$ Aluminiumatome

* in einem Tropfen Wasser befinden sich 33 $\cdot$ 10$^{20}$ Wassermoleküle

* liegen im Bereich zwischen 10 Trillionen und 10 Trilliarden

* Wie kann man die Teilchenanzahl bestimmen? Zaehlen geht nicht, aber wiegen!

Die Massen von Atomen oder Molekülen werden in der Einheit u angegeben:

> $1 u=\frac{1}{12} \cdot m\left({ }^{12} C\right)$

Zur Umrechnung von Atommasse u in Gramm mit dem Umrechnungsfaktor $(6,022 \cdot 10^{23})$ g

  * von u nach g: Divide by $6,022 \cdot 10^{23} u$

  * von g nach u: Multiply with $6,022 \cdot 10^{23} u$

> $1 g=\frac{1}{1,66 \, 10^{-24}} u =6,022 \cdot 10^{23} u$

*  Beispiele: wie viele Atome sind in einem Stueck Aluminiumfolie?

  * Die Alufolie wiegt 1 Gramm

  * Im Periodensystem der Elemente sieht man wie schwer ein Aluminiumatom ist: 27u

  * Wir müssten daher 1 Gramm durch 27u teilen. Das geht aber nur mit denselben Einheiten.

  * Wir müssen daher erst noch von Gramm in u umrechnen:

> 1 $\cdot$ (6 $\cdot$ 10$^{23}$) u = 6 $\cdot$ 10$^{23}$u

  * Das teilen wir jetzt durch die Masse eines Aluminiumatoms:

  * 6,022 $\cdot$ 10$^{23}$ : 27 = 2,2 $\cdot$ 10$^{22}$ Atome sind in einem Stueck Aluminium von 1 Gramm Masse

* Diese Zahl ist allerdings sehr gross, man braucht also eine spezielle Zahl fur die Teilchenanzahl: man benutzt dafur das Mol.

> 1 Mol = 6,022 $\cdot$ 10$^{23}$ Teilchen

* **Bei Aluminum: wenn ich 1 Gramm Alufolie habe, dann sind das 0,04 Mol Aluminium (Teilchenanzahl N angegeben in Stoffmenge Mol).**

* Source Video: teilchenzahl und stoffmemge: https://youtu.be/WDXYXykdkMQ

**Molare Masse in M und molares Volumen in V**

Wie hängen Stoffmenge n, Masse m und molare Masse M zusammen?

> n = $\frac{m}{M}$

Bei Aluminium: $\frac{1g(Al)}{27g/mol}$ = 0,04 mol

Kann man auch nach m umstellen, zB wie schwer sind 3 mol Wasser?

* Molare Wasser von Wasser (eines Wassermolekuls) ist 18 u, damit ist die molare Masse 18 $\frac{g}{mol}$. 1 Mol Wasser wiegt 18 Gramm. Also:

* m$_{H2O}$ = 18 $\frac{g}{mol}$ $\cdot$ 3 Mol = 54 Gramm

**Molare Masse nicht mit der Atommasse (wie schwer ist ein Atom?) verwechseln!**

* Molare Masse: Wie schwer ist ein Mol einer bestimmten Atomsorte?

  * Und ein Mol waren 6,022 $\cdot$ 10$^{23}$ Atome

* Molare Massen sind sehr viel praktischer (=die Masse eines ganzen Mols an Atomen), weil Atommassen zu winzig sind.

* Beispiel: 1 Gramm Alufolie. Im Periodensystem steht m$_a$(Al) = 27u als Atommasse

* Umrechnung von Atommasse in polare Masse: ist die gleiche Zahl, nur hat sie nicht die Einheit u, sondern die Einheit: Gramm pro Mol

* Die molare Masse von Aluminium ist 27 Gramm pro Mol

> M(Al) = 27 $\frac{g}{mol}$

* **Das heisst 1 Mol Aluminiumatome (ungefähr 6,022 $\cdot$ 10$^{23}$ Aluminiumatome) wiegen 27 Gramm**

* $\frac{1g(Al)}{27g/mol}$ = 0,04 mol




**Sonderfall Gase:**

* Masse durch Volumen V ersetzen!

* Und molare Masse ersetzen durch molares Volumen V$_{m,n}$

* V$_{m,n}$ = Volumen unter Normalbedingungen : Stoffmenge eines Gases = $\frac{V_n}{n}$

* Es ist das Volumen, das genau 6,022 $\cdot$ 10$^{23}$ Gasteilchen enthält in Liter pro mol

* Welches Volumen nehmen 6,022 $\cdot$ 10$^{23}$ Teilchen ein? -

	* Man einigt sich auf eine Norm: 1 Liter enthält immer gleich viele Gasmoleküle, wenn der Druck und die Temperatur gleich bleiben.

	* Bei Druck und Temperatur gehen wir meist vom Normzustand aus: Druck bei 1012 Millibar mbar bzw. hPa und Temperatur 0 Grad Celcius

* Das molare Volumen $V_{m,n}$ = 22,4 L/mol bezogen auf den Normzustand

* In den letzten Jahren hat man den Normzustand auf 20 Grad Celcius geändert, das molare Volumen beträgt nun $V_{m,n}$ = 24 L/mol bezogen auf den Normzustand

* diese Werte gelten fur jeden Gas

* hat was mit dem Avogadrischen Gesetz zu tun: Gasförmige Stoffe enthalten bei gleichem Druck und gleicher Temperatur immer gleich viele Teilchen

* Source Video: Molare Masse und molares Volumen: https://youtu.be/f0CcWMx1_ns

###### *Reaktionsgleichungen (Stöchiometrie)*

![ggg](https://raw.githubusercontent.com/deltorobarba/repo/master/sciences_0780.jpeg)

**Rechnen mit molaren Größen**

molare grösse https://youtu.be/Mp9ss59KoWI

> 2 Na + Cl$_2$ $\rightarrow$ 2 NaCl

* Man hat eine Tonne Natrium. Man will das vollständig in Kochsalz umsetzen. Wie viel Chlor benötig man dafür und wie viele Tonnen Kochsalz kann man damit produzieren?

* Zur Loesung dieser Aufgabe braucht man: Stoffmenge, Masse, Atommasse, Molare Masse, Volumen, Molares Volumen und Avogadro-Konstante

![ggg](https://raw.githubusercontent.com/deltorobarba/repo/master/sciences_043.png)

* Zusazetlich braucht man die Faehigkeit zur Interpretation von Reaktionsgleichungen:

![ggg](https://raw.githubusercontent.com/deltorobarba/repo/master/sciences_044.png)

**Reaktionsgleichung**

*Reaktionsmechanismus*

https://de.m.wikipedia.org/wiki/Donator-Akzeptor-Prinzip

https://de.m.wikipedia.org/wiki/Chemische_Reaktion

https://de.m.wikipedia.org/wiki/Reaktionsmechanismus

*Elementarreaktion*

https://de.m.wikipedia.org/wiki/Elementarreaktion

https://de.m.wikipedia.org/wiki/Reaktionsgleichung

https://www.chemie-biologie.uni-siegen.de/pc/lehre/allcvu/scripte/vorlesung40.pdf

**Stöchiometrie**

https://de.m.wikipedia.org/wiki/Stöchiometrie

https://de.m.wikipedia.org/wiki/Stöchiometrische_Matrix

###### *Formeldarstellung von Molekülen (Strukturformel, Summenformel & Skelettformel)*

https://de.m.wikipedia.org/wiki/Fischer-Projektion

https://de.m.wikipedia.org/wiki/Haworth-Formel

beide von hier: https://de.m.wikipedia.org/wiki/Monosaccharide

![ggg](https://raw.githubusercontent.com/deltorobarba/repo/master/sciences_104.jpg)

*Source: https://de.m.wikipedia.org/wiki/Molekül*

![ggg](https://raw.githubusercontent.com/deltorobarba/repo/master/sciences_172.jpeg)

https://de.m.wikipedia.org/wiki/Strukturformel

* Elektronenformeln (Lewisformel), Valenzstrichformeln, Keilstrichformeln und Skelettformeln angesehen werden.

* Konstitutionsformel,	Summenformel,	Verhältnisformel

* Eine Sonderstellung nehmen zudem Projektionsformeln (z. B. die Fischerprojektion und die Newmanprojektion)

Elektronenformel (Lewisformel)

https://de.m.wikipedia.org/wiki/Elektronenformel

Lewis Modell (mit Valenzstrichformel) vs Molekülorbital-Theorie

* Schwächen des Lewis-Modells: mesomerer Zustand oder als Resonanzhybrid: Elektronenverteilung im Nitrat-Ion kann durch eine Valenzstrichformel offensichtlich nicht angemessen dargestellt werden


https://youtu.be/toQD3nPZQn4

**Summenformel**

* Eine [Summenformel](https://www.chemie.de/lexikon/Summenformel.html), auch Molekülformel (oft auch unpräzise Chemismus), dient in der Chemie dazu, die Anzahl der gleichartigen Atome in einem Molekül oder in der Formeleinheit eines Salzes anzugeben.

* Sie gibt also das Teilchenzahlenverhältnis und damit auch das Stoffmengenverhältnis der Teilchen an, die im Molekül bzw. in der Formeleinheit enthalten sind.

* Die Summenformel einer Verbindung sagt etwas über die Stoffmengenverhältnisse aus, in denen die am Aufbau der Verbindung beteiligten Elemente in der Verbindung vorliegen. Sie bildet damit die Grundlage für alle stöchiometrische Berechnungen

* Summenformeln werden auch bei der Formulierung von chemischen Reaktionsgleichungen benutzt. Dabei werden die Ausgangs- und Endstoffe der Reaktion (Edukte und Produkte) im Reaktionsschema in der anorganischen Chemie in der Regel in Form von Summenformeln angegeben. In der organischen Chemie werden nur selten Summenformeln verwendet, weil sie keine für den Ablauf der chemischen Reaktion wichtigen Informationen enthalten.

https://de.m.wikipedia.org/wiki/Allgemeine_Summenformel

###### *Allotropie (Modifikationen) eines chemischen Elements*

[Allotropie](https://de.m.wikipedia.org/wiki/Allotropie) sind physikalisch polymorphe Aggregatzustände gleicher Stöchiometrie. Als Allotropie bezeichnet man die Erscheinung, wenn ein chemisches Element im gleichen Aggregatzustand in zwei oder mehr Strukturformen auftritt, die sich physikalisch und in ihrer chemischen Reaktionsbereitschaft voneinander unterscheiden. Allotropien werden in der Chemie, Mineralogie und Materialwissenschaft auch als [Modifikationen (Polymorphie)](https://de.m.wikipedia.org/wiki/Polymorphie_(Materialwissenschaft)) eines chemischen Elements bezeichnet.




> **Kohlenstoffallotrope**: Kohlenstoff kommt sowohl als farbloser, extrem harter **Diamant** vor, der ein elektrischer Nichtleiter ist, als auch als schwarzer, weicher **Graphit**, der elektrischen Strom leitet, sowie als Makromoleküle **Fulleren**, Kohlenstoffnanoröhren, **Cyclocarbon**, **Carbin** und als **Graphen** [Source: Allotropie](https://de.m.wikipedia.org/wiki/Allotropie)


##### <font color="blue">*Stereochemie*

###### *Isomerie organischer Verbindungen (Stereochemie & Chiralität)*

> **Isomere Verbindungen sind Verbindungen mit gleicher Summenformel, also gleicher atomarer Zusammensetzung, aber unterschiedlicher Struktur.**

**Isomere**

* [Isomere](https://www.chemie.de/lexikon/Isomer.html) sind chemische Verbindungen der gleichen Summenformel, aber unterschiedlicher chemischer Struktur und teilweise auch mit unterschiedlichen chemischen, physikalischen und biologischen Eigenschaften.

* [Isomerasen](https://www.chemie.de/lexikon/Isomerasen.html) sind die Enzyme der Zelle, die Isomere ineinander umwandeln.

* [Cis-trans-Isomerie](https://www.chemie.de/lexikon/Cis-trans-Isomerie)

* [Tautomerie](https://de.m.wikipedia.org/wiki/Tautomerie)

*Einteilung verschiedener Arten von Isomerie*

![ggg](https://upload.wikimedia.org/wikipedia/commons/thumb/9/94/Isomerism-ball-de.svg/640px-Isomerism-ball-de.svg.png)

*Beispiel: Isomere der Summenformel C6H14:*

* Methylbutan: Methylgruppe ist CH3 (siehe oben Methan) und Butan ist C4H10

* bei 2,2 Dimethylbutan haengt  anstelle eines H-Atom jeweils eine Methylgruppe am zweiten C-Atom,

* oder bei 2,3 Dimethylbutan jeweils eines an der 2. und 3. Stelle

![ggg](https://raw.githubusercontent.com/deltorobarba/repo/master/sciences_111.png)

![ggg](https://raw.githubusercontent.com/deltorobarba/repo/master/sciences_171.jpeg)

**Weitere Links zu Stereochemie & Chiralität**

https://wissensplattform-schueler.de/isomerie/


https://de.m.wikipedia.org/wiki/Stereochemie

https://de.m.wikipedia.org/wiki/Chiralität_(Chemie)

https://de.m.wikipedia.org/wiki/Homochiralit%C3%A4t

Spiegelbildisomerie (Enantiomerie) & Chiralität

https://schulstoff.org/ch/ch11/kohlenhydrate-und-stereochemie.php

* https://www.u-helmich.de/che/lexikon/C/cis-trans-Isomerie.html

* https://w-hoelzel.de/index.php?option=com_content&view=article&id=311&catid=119

###### *Mesomerie (Resonanzstruktur)*

**Mesomerie**

* [Mesomerie](https://de.m.wikipedia.org/wiki/Mesomerie) = Moleküle gleichen Aufbaus in verschiedenen Bindungen

###### *Isomerie: Nomenklatur der Alkene*

**Isomerie bei Alkanen**

* Bei Molekulen aus vier oder mehr Kohlenstoffatomen gibt es verschiedene Moglichkeiten, wie die einzelnen Kohlenstoffatome miteinander verknupft sind.

* Dabei ergeben sich unterschiedliche Eigenschaften (zB unterschiedliche Siedetemperaturen). Gibt es mehrere Moglichkeiten, spricht man von Isomerie.

![ggg](https://raw.githubusercontent.com/deltorobarba/repo/master/sciences_122.png)

*Isomere der Summenformel C6H14:*

* Methylbutan: Methylgruppe ist CH3 (siehe oben Methan) und Butan ist C4H10

* bei 2,2 Dimethylbutan haengt  anstelle eines H-Atom jeweils eine Methylgruppe am zweiten C-Atom,

* oder bei 2,3 Dimethylbutan jeweils eines an der 2. und 3. Stelle

![ggg](https://raw.githubusercontent.com/deltorobarba/repo/master/sciences_111.png)

**Nomenklatur der Alkene (Isomerie)**

Die Endung "-en" fur ein Alken gibt noch nicht an, wo sich im Molekul die Doppelbindugn findet. Zum Beispiel gibt es bei Pentenen 4 Moglichkeiten (hier mit Strukturfomel):

Man sagt dann Pent-1-en, Pent-2-en, Pent-3-en oder Pent-4-en.

(C5H10 hat folgende Isomere, sowie noch weitere, wenn man C als Nebenarme hat, zB 2-Methylbuten. Siehe mehr hier: https://wissensplattform-schueler.de/isomerie/)

![ggg](https://raw.githubusercontent.com/deltorobarba/repo/master/sciences_125.png)




###### *E/Z-Nomenklatur (& Cis-Trans-Isomerie)*

**E/Z-Nomenklatur**

![ggg](https://raw.githubusercontent.com/deltorobarba/repo/master/sciences_170.png)

* Stehen die beiden Substituenten auf der gleichen Seite der π-Ebene (Spur dieser Ebene:--------), nennt man das Stereoisomer cis, stehen sie auf unterschiedlichen Seiten, ist es das trans-Isomer.

* (Da es sich bei Doppelbindungsisomeren um Ste- reoisomere handelt, kann man sie auch Diastereomere nennen.)


* Ab drei Substituenten, von denen aber nicht zwei gleichartige an einer Seite der Doppelbindung stehen dürfen, versagt die cis-/trans-Nomenklatur.

* Dann muss ein neues Kriterium eingeführt werden, das jeweils einem der beiden Substituenten auf jeder Seite (links und rechts) einen Vorrang gibt. Die Stellung dieser vorrangigen Substituenten relativ zur π-Ebene erlaubt dann eine Entscheidung:

  * vorrangige Substituenten auf der gleichen Seite Z (Zusammen)

  * vorrangige Substituenten auf unterschiedlichen Seiten E (Entgegen)

![ggg](https://raw.githubusercontent.com/deltorobarba/repo/master/sciences_0759.png)

![ggg](https://raw.githubusercontent.com/deltorobarba/repo/master/sciences_0760.png)

![ggg](https://raw.githubusercontent.com/deltorobarba/repo/master/sciences_0761.png)

![ggg](https://raw.githubusercontent.com/deltorobarba/repo/master/sciences_0762.png)

###### *Isomerie bei Alkenen (Cis-Trans Stereoisomerie)*

**Isomerie bei Alkenen (Cis-Trans Stereoisomerie)**

* In einem Ethen-Molekül wurden zwei Wasserstoffatome durch zwei Chlor-atome ersetzt, sodass an beiden Kohlenstoffatomen jeweils ein Chloratom gebunden ist.

* Da die C-C-Doppelbindung nicht frei drehbar ist, entstehen zwei mögliche Moleküle:

![ggg](https://raw.githubusercontent.com/deltorobarba/repo/master/sciences_123.png)

> **Liegen beide Chloratome auf der „gleichen Seite", spricht man von der cis-Form, liegen sie auf unterschiedlichen Seiten, spricht man von der trans-Form.**

> **Die cis/trans-Isomerie der Alkene und Cycloalkane ist eine Art der Stereoisomerie.**

![ggg](https://raw.githubusercontent.com/deltorobarba/repo/master/sciences_124.png)



###### *Isomerasen*

Die [Isomerase](https://www.chemie.de/lexikon/Isomerasen.html) ist ein Enzym, das Isomerierungs-Reaktionen, die Umwandlung einer Verbindung in eine isomere Struktur, katalysiert. Isomere sind Moleküle gleicher Summenformel, aber anderer Strukturformel.


[Isomerasen](https://de.m.wikipedia.org/wiki/Isomerasen) sind Enzyme der fünften Enzymklasse laut der systematischen Nomenklatur der Enzymkommission der International Union of Biochemistry (IUB) und katalysieren die Isomerisierungs-Reaktionen, das ist die Umwandlung einer Verbindung in eine isomere Struktur. Isomere sind Moleküle gleicher Summenformel, aber unterschiedlicher absoluter Konfiguration oder Strukturformel. Racemasen katalysieren Racematisierungsreaktionen, bei denen ein Wasserstoff am einzigen chiralen C-Atom einer Verbindung umgelagert wird.

* **Eine wichtige biochemische Isomerisierungsreaktion ist die Aldose-Ketose-Umlagerung.**

* Auch bei Retinal im Auge (Carotinoid / Terpen) : Retinal kommt in zwei Isomerformen vor. Durch Absorption von Licht wird das 11-cis-Retinal in die all-trans-Konfiguration überführt. Dies löst eine Signaltransduktionskaskade aus, durch die das Lichtsignal auf dem Umweg über die atomare Begegnung im Retinal in ein elektrochemisches Signal umgewandelt wird, das Rezeptorpotential der Stäbchenzelle. Fällt die Reizung durch Licht weg, wird das Retinal durch eine Isomerase wieder in die cis-Form umgewandelt.



###### *Anomerer Effekt*

**[Anomerer Effekt](https://de.m.wikipedia.org/wiki/Anomerer_Effekt)**

In der organischen Chemie bezeichnet der anomere Effekt die Tendenz von Atomen in bestimmten Molekülstrukturen eine gewisse räumliche Position bevorzugt einzunehmen, da diese energetisch günstiger ist.

* Fachlich genau formuliert nimmt ein Heteroatom, welches an ein Kohlenstoffatom neben einem Heteroatome **im Cyclohexanring substituiert ist, bevorzugt die axiale Position gegenüber der (sterisch weniger gehinderten!) äquatorialen Position ein, was sterischen Betrachtungen widerspricht**.

  * *Heteroatom = in organisch-chemischen Verbindungen solche Atome, die kein Kohlenstoff oder Wasserstoff sind. Es sind oft, aber nicht ausschließlich, Stickstoff-, Sauerstoff-, Schwefel- oder Phosphoratome*

* Anders ausgedrückt ist bei Vorliegen zweier elektronegativer Substituenten an einem Kohlenstoffatom die **synclinale Anordnung zweier Bindungen gegenüber der antiperiplanaren bevorzugt**. Das Kohlenstoffatom, an das die betreffenden Substituenten gebunden sind, wird als anomerer Kohlenstoff bezeichnet.

**Summary: Anomerer Effekt als besondere Form der Stereoisomerie**

> (-) äquatorialen Position = Antiperiplanare Bindungssituation = sterisch weniger gehindert, aber energetisch nicht bevorzugt

> (+) axiale Position = synclinale Anordnung = energetisch bevorzugt

*Antiperiplanare (1) und synclinale (2) Bindungssituation in cyclischen Halbacetalen. Die entscheidenden Bindungen sind rot markiert (synclinal ist energetisch bevorzugt)*

> ![ggg](https://upload.wikimedia.org/wikipedia/commons/thumb/c/c6/Antiperiplanar_vs._synclinal_V.2.svg/320px-Antiperiplanar_vs._synclinal_V.2.svg.png)

*Löst man reine a-D-Glucose in Wasser, stellt sich ein Gleichgewicht aus 36% a-D-Glucose und 64% ß-D-Glucose ein. Die Gleichgewichtseinstellung ist nur über die offenkettige Form möglich, die in Wasser aber nur zu 0,1 % vorliegt. Eine Bestätigung der Gleichgewichtseinstellung findet man durch die Messung der optischen Aktivität. Der optische Drehwert von [alp = + 52,5° der wässrigen Lösung ergibt sich aus den prozentualen Anteilen der reinen a- und ß-Isomere im Gleichgewicht.*

###### *Keto-Enol-Tautomerie*

[**Keto-Enol-Tautomerie**](https://de.m.wikipedia.org/wiki/Tautomerie#Keto-Enol-Tautomerie)

* Die Tautomerie ist eine Form der Isomerie, bei der zwei Isomere nebeneinander im Gleichgewicht vorliegen und sich ineinander umwandeln können. Anders als die mesomeren Grenzstrukturen sind Tautomere stabile, real existierende Strukturen, die experimentell nachgewiesen werden können.

* **Die Keto-Enol-Tautomerie ist eine Verschiebung der Bindungselektronen mit gleichzeitiger Wanderung eines Wasserstoffatoms vom a-Kohlenstoffatom zum Sauerstoffatom**.

![ggg](https://raw.githubusercontent.com/deltorobarba/repo/master/sciences_253.png)


* Die Lage des Gleichgewichts ist abhängig von der Struktur des Ketons und weiteren funktionellen Gruppen.

* So liegt Aceton nur zu 0,0002 % als Enol vor. Dagegen liegt Pentan-2,4-dion zu 80 % als Enol vor. In diesem Fall kann durch die Wasserstoffbrückenbindung ein Sechsring gebildet und die Struktur auf diese Weise stabilisiert werden.

###### *Ständigkeit (α-, β-, γ-, δ-Stellung)*

[Ständigkeit](https://de.m.wikipedia.org/wiki/Ständigkeit) bezeichnet in der Chemie die Stellung zweier funktionellen Gruppen zueinander innerhalb einer organischen Verbindung.

Hierbei bedeuten

* **α-ständig** oder α-Stellung: die funktionellen Gruppen sitzen am gleichen Kohlenstoffatom (geminal). Beispiele sind α-Hydroxycarbonsäuren oder α-Aminosäuren.

* **β-ständig** oder β-Stellung: die funktionellen Gruppen sitzen an zwei benachbarten Kohlenstoffatomen (vicinal). Beispiele sind β-Alanin, β-Oxocarbonsäuren, wie β-Oxobutansäure (Acetessigsäure).

* **γ-ständig** oder γ-Stellung: die funktionellen Gruppen sitzen an Kohlenstoffatomen, die durch ein weiteres Kohlenstoffatom getrennt sind. Beispiele sind etwa γ-Diketone oder γ-Hydroxybuttersäure (nach IUPAC: 4-Hydroxybutansäure).

* **δ-ständig** oder δ-Stellung: die beiden funktionellen Gruppen sitzen an Kohlenstoffatomen, die durch zwei weitere Kohlenstoffatome getrennt sind. Beispiele sind etwa die δ-Lactone und δ-Thiolactone.

* **ε-ständig** oder ε-Stellung: die beiden funktionellen Gruppen sitzen an Kohlenstoffatomen, die durch drei weitere Kohlenstoffatome getrennt sind. Beispiele sind etwa die ε-Caprolacton oder ε-Caprolactam.[1]

Besondere, nichtsystematische Bedeutung hat die Bezeichnung ω-ständig, für den maximal möglichen Abstand zweier funktioneller Gruppen in einem gegebenen Molekül. Bei ungesättigten Fettsäuren wird das letzte Kohlenstoffatom als ω-ständig bezeichnet, siehe Nomenklatur der ungesättigten Fettsäuren.

Die hier zur Angabe der relativen Stellung von funktionellen Gruppen verwendeten griechischen Buchstaben haben also eine andere Bedeutung als die zur Kennzeichnung stereochemischer Konfigurationen von Kohlenhydraten gebräuchlichen Deskriptoren „α-“ und „β-“ oder die im Rahmen der Steroid-Nomenklatur genutzten Deskriptoren α- und β- zur Beschreibung der beiden Halbräume oberhalb und unterhalb der Ringebene.

*Skeletal formula of butyric acid with the alpha, beta, and gamma carbons marked* Source: [Alpha and beta carbon](https://en.m.wikipedia.org/wiki/Alpha_and_beta_carbon)

![gg](https://upload.wikimedia.org/wikipedia/commons/thumb/8/8f/Butyric_acid_carbons_2.svg/320px-Butyric_acid_carbons_2.svg.png)

*Grundstruktur von α-Aminosäuren (Rest R ist im Fall von Glycin ein H-Atom):*

> ![ggg](https://upload.wikimedia.org/wikipedia/commons/thumb/c/ce/AminoAcidball.svg/320px-AminoAcidball.svg.png)


Beispiel: [Tyrosin (α-Aminosäuren.)](https://de.m.wikipedia.org/wiki/Tyrosin) (abgekürzt Tyr oder Y) ist in seiner natürlichen L-Form eine nichtessentielle proteinogene α-Aminosäure, die in den meisten Proteinen vorkommt: **hier hängen beide funktionellen Gruppen, die Aminogruppe (NH2) und die Carboxy-Gruppe (COOH), an einem C-Atom**

> ![gg](https://upload.wikimedia.org/wikipedia/commons/thumb/4/40/L-Tyrosin_-_L-Tyrosine.svg/320px-L-Tyrosin_-_L-Tyrosine.svg.png)

Zum Vergleich: [β-Alanin](https://de.m.wikipedia.org/wiki/Β-Alanin) - die funktionellen Gruppen sitzen an zwei benachbarten Kohlenstoffatomen

> ![gg](https://upload.wikimedia.org/wikipedia/commons/thumb/3/35/3-aminopropanoic_acid_200.svg/320px-3-aminopropanoic_acid_200.svg.png)

Zum Vergleich: [γ-Aminobuttersäure (GABA)](https://de.m.wikipedia.org/wiki/Γ-Aminobuttersäure)

> ![gg](https://upload.wikimedia.org/wikipedia/commons/thumb/4/4e/Gamma-Aminobuttersäure_-_gamma-aminobutyric_acid.svg/320px-Gamma-Aminobuttersäure_-_gamma-aminobutyric_acid.svg.png)

######*Substitutionsmuster (bei Aromaten: ortho, meta, para / ipso, meso, peri / cine und tele)*

**Bei Benzol-Derivaten wird die relative Stellung [(Ständigkeit)](https://de.m.wikipedia.org/wiki/Ständigkeit) zweier Substituenten durch die Vorsilben ortho, meta und para oder kurz „o-“, „m-“ und „p-“ beschrieben:**

* **ortho-ständig** (entspricht β-ständig) bezeichnet die Stellung an zwei benachbarten Ringatomen, siehe etwa Phthalsäure (ortho-Benzoldicarbonsäure). In der IUPAC-Nomenklatur entspricht das einer 1,2-Substitution.

* **meta-ständig** (entspricht γ-ständig) bezeichnet die Stellung an zwei Ringatomen, die durch ein weiteres Atom getrennt sind, etwa bei Resorcin (meta-Dihydroxybenzol). Ein solches Substitutionsmuster wird in der IUPAC-Nomenklatur 1,3-Substitution genannt.

* **para-ständig** (entspricht δ-ständig) bezeichnet die Stellung an zwei Ringatomen, die durch zwei weitere Atome getrennt sind und sich damit am sechsgliedrigen Benzolring genau gegenüberstehen. Beispiele sind das Hydrochinon (para-Dihydroxybenzol) oder die Terephthalsäure (para-Benzoldicarbonsäure). Dieses Substitutionsmuster wird in der IUPAC-Nomenklatur 1,4-Substitution genannt.

Beispiel: **4. Substituierte Aromate (Benzole)** (Benzolderivate) mit einer [Phenylgruppe](https://de.m.wikipedia.org/wiki/Phenylgruppe) als [Arylgruppe](https://de.m.wikipedia.org/wiki/Arylgruppe), d.h. 1 Benzolring als Grundgerust (Atomgruppe –C6H5). Am Benzolring haengt ein angefügter Substituent, der aber verschiedene Konstitutionsisomere haben kann (wie [Toluol](https://de.m.wikipedia.org/wiki/Toluol), [Xylole](https://de.m.wikipedia.org/wiki/Xylole), [Phenol](https://de.m.wikipedia.org/wiki/Phenol) C6H5–OH, [Nitrobenzol](https://de.m.wikipedia.org/wiki/Nitrobenzol) C6H5–NO2, [Anilin](https://de.m.wikipedia.org/wiki/Anilin) (Aminobenzol) C6H5–NH2, [Hydrochinon](https://de.m.wikipedia.org/wiki/Hydrochinon) HO–C6H4–OH,  [Pikrinsäure](https://de.m.wikipedia.org/wiki/Pikrinsäure) (2,4,6-Trinitrophenol) C6H2(OH)(NO2)3, Benzaldehyde, [Benzoesäure](https://de.m.wikipedia.org/wiki/Benzoesäure), [Brombenzol](https://de.m.wikipedia.org/wiki/Brombenzol), siehe grosse Liste auch unter [Disubstituierte Benzole](https://de.m.wikipedia.org/wiki/Disubstituierte_Benzole))

![ggg](https://raw.githubusercontent.com/deltorobarba/repo/master/sciences_0739.png)

Die [Substitutionsmuster](https://de.m.wikipedia.org/wiki/Substitutionsmuster) sind Teil der chemischen Nomenklatur und beschreiben die relativen Positionen der Substituenten in [aromatischen](https://de.m.wikipedia.org/wiki/Aromaten) Kohlenstoffgerüsten.

> ![ggg](https://upload.wikimedia.org/wikipedia/commons/thumb/e/e0/Substitutionsmuster.svg/199px-Substitutionsmuster.svg.png)

##### <font color="blue">*Hybridisierung in Molekülorbitalen (Delokalisierung & Konjugation)*

###### *Hybridisierung in organischen Verbindungen*


[**Mesomerie**](https://de.m.wikipedia.org/wiki/Mesomerie) = Moleküle gleichen Aufbaus in verschiedenen Bindungen

* Als Mesomerie (auch Resonanz oder Resonanzstruktur) wird in der Chemie das Phänomen bezeichnet, dass die Bindungsverhältnisse in manchen Molekülen oder mehratomigen Ionen nicht durch eine einzige Strukturformel, sondern nur durch mehrere Grenzformeln dargestellt werden können.

[**Delokalisierung**](https://de.m.wikipedia.org/wiki/Delokalisierung) (oft auch π-Elektronen-System)

* wird in der Chemie gesprochen, wenn ein oder mehrere Elektronen in einer Atomgruppe, sprich einem Molekül oder Molekülion, nicht genau lokalisierbar, sondern über die einzelnen Atome verteilt ist.

* Zur Beschreibung dieser Ladungsverteilung nach der VB-Theorie verwendet man mesomere Grenzstrukturen.

* Ein Spezialfall delokalisierter Ladungen findet sich in konjugierten Systemen aromatischer Verbindungen.

  * Ihre π-Elektronen sind nicht in isolierten, zwei Kohlenstoff-Atomen zugehörigen π-Orbitalen lokalisiert, sondern gehören Molekülorbitalen an, die sich über mehrere C-Atome erstrecken;

  * die π-Elektronen sind in einer „Elektronenwolke“ über die Kohlenstoffatome des gesamten aromatischen Systems verteilt.

  * Ein klassisches Beispiel für ein solches aromatisches Molekül mit delokalisierten π-Elektronen ist Benzol.


[**Konjugation**](https://de.m.wikipedia.org/wiki/Konjugation_(Chemie))

* ist die Überlappung eines π-Orbitals mit einem p-Orbital eines sp2-hybridisierten (Kohlenstoff-)Atoms oder weiteren π-Orbitalen

**Hybridisierung mit Pi-Bindung (π-Orbital) & Sigma-Bindung**

> Bei der Hybridisierung geht es um das Verschmelzen von verschiedenen Orbitalen in einem Atom, um sich dann besser mit einem anderen Atom binden zu können. Die Hybridorbitale ermöglichen es dem Molekül dann, einen energetisch günstigeren Zustand anzunehmen.


![ggg](https://raw.githubusercontent.com/deltorobarba/repo/master/sciences_156.png)

![ggg](https://raw.githubusercontent.com/deltorobarba/repo/master/sciences_157.png)

![ggg](https://raw.githubusercontent.com/deltorobarba/repo/master/sciences_158.png)

![ggg](https://raw.githubusercontent.com/deltorobarba/repo/master/sciences_159.png)

![ggg](https://raw.githubusercontent.com/deltorobarba/repo/master/sciences_160.png)

**Die gängigste Form ist die sp3-Hybridisierung**

* Atomorbitale verschmelzen bei der Bindung von Atomen, sie hybridisieren

* Im sp3 Fall hybridisieren 1-s-Orbital und 3 p-Orbitale miteinander

* Ein Molekül bei dem eine sp3-Hybridisierung vorliegt, ist Methan:

> ![ggg](https://upload.wikimedia.org/wikipedia/commons/thumb/5/56/Methan_Lewis.svg/103px-Methan_Lewis.svg.png)

* Kohlenstoff hat nur 2 Aussenelektronen (ihm fehlen 4 weitere bis zur Edelgaskonfiguration)

* Es würde erwartet werden, dass er nur mit 2 Wasserstoffatomen eine gleichwertige Bindung eingehen kann

In der Praxis hat Methan aber 4 nicht unterscheidbare Bindungen:

> ![ggg](https://upload.wikimedia.org/wikipedia/commons/thumb/5/56/Methan_Lewis.svg/103px-Methan_Lewis.svg.png)

![ggg](https://raw.githubusercontent.com/deltorobarba/repo/master/sciences_161.png)


Erläuterung:

* Um Grundzustand hat dieser Kohlenstoff 2 mit Elektronen vollbesetzte s-Orbitale (links)

* Ausserdem kannst Du die 2 Valenzelektronen in die p-Orbitale einzeichnen (Mitte), jeweils 1 Elektron fur 2 p-Orbitale

* Mit diesem Zustand würdest Du erwarten, dass sich nur 2 kovalente Bindungen ausbilden können.

* **Nun kommt es aber zur Hybridisierung der Orbitale. Dabei wird das 2-s und die 2-p Orbital miteinander vermischt. Hierdurch wird das s-Orbital energetisch gehoben, und die drei p-Orbitale ein klein wenig energetisch gesenkt. Das siehst du anhand der Hohe der Orbital bezüglich der Energieskala**.

* So können sich die Elektronen gleichmassig auf die Hybridorbitale aufteilen.

* **Du siehst auch dass jetzt nur jeweils ein Elektron im Orbital ist. Das bedeutet, dass jedes Orbital kann mit einem Wasserstoff-Atom binden und eine kovalente Bindung ausbilden**. So sind durch die Hybridisierung 4 Bindungen möglich geworden anstatt der vorher erwarteten 2:

![ggg](https://raw.githubusercontent.com/deltorobarba/repo/master/sciences_162.png)

Exkurs: Siehe hier die Belegung von Kohlenstoff (identisch zu oben links in der Abbildung). $2s^2$ und $2p^2$ vermischen sich zu 4 und sind damit stabiler (energetisch guenstiger) = Hybridisierung!

![ggg](https://raw.githubusercontent.com/deltorobarba/repo/master/sciences_246.png)


**Ausserdem verändern die neuen Orbital sich in ihrer aussehen Form: sie haben jetzt eine Keulenahnliche Form und stehen in einem Winkel von 109,5 Grad zueinander. Es bildet sich ein Tetraeder.**

![ggg](https://raw.githubusercontent.com/deltorobarba/repo/master/sciences_163.png)

![ggg](https://raw.githubusercontent.com/deltorobarba/repo/master/sciences_164.png)

![ggg](https://raw.githubusercontent.com/deltorobarba/repo/master/sciences_165.png)

![ggg](https://raw.githubusercontent.com/deltorobarba/repo/master/sciences_166.png)

![ggg](https://raw.githubusercontent.com/deltorobarba/repo/master/sciences_167.png)

https://studyflix.de/chemie/aromaten-i-437 Hueckel Regel

https://de.m.wikipedia.org/wiki/Delokalisierung

https://en.m.wikipedia.org/wiki/Pi_bond

https://en.m.wikipedia.org/wiki/Sigma_bond

https://en.m.wikipedia.org/wiki/Antibonding_molecular_orbital

https://studyflix.de/chemie/hybridisierung-2208


###### *Hybridorbital & Aufbau organischer Moleküle*

https://studyflix.de/chemie/hybridisierungsmodell-und-aufbau-organischer-molekule-433

Hybridisierungsmodell im Kohlenstoffatom mit seiner Elektronenkonfiguration:

![ggg](https://raw.githubusercontent.com/deltorobarba/repo/master/sciences_196.png)

* Die 2s und die 2p Schale sind mit je zwei Elektronen nach der Handtaschen Regel und dem Pauli-Prinzip gefüllt

![ggg](https://raw.githubusercontent.com/deltorobarba/repo/master/sciences_197.png)

* Nun nähern sich 2 andere Elemente, die als Bindungspartner fungieren. Sie tun dies aus der sterisch, also bezüglich der Raumausfuellung günstigsten Richtung von 180 Grad:

![ggg](https://raw.githubusercontent.com/deltorobarba/repo/master/sciences_198.png)

* Es kommt hierbei zu einer Vermischung oder auch Hybridisierung zwischen dem 2s Orbital und einem der drei 2p-Orbitale. Diese neue Orbital heisst deshalb sp-Hybridorbital.

* Je ein Elektron wandert in ein sp-Hybridorbital. Die anderen verbleiben in je einem p-Orbital.

* Der s-Anteil und der p-Anteil in einem sp-Hybridorbital betragt damit genau 50%.

![ggg](https://raw.githubusercontent.com/deltorobarba/repo/master/sciences_200.png)

* Dasselbe Prinzip gilt auch fur 3 sich nähernde Bindungspartner

* Hier liegt der Winkel aus streichen Gründen bei 120 Grad.

![ggg](https://raw.githubusercontent.com/deltorobarba/repo/master/sciences_201.png)

* Zwei 2p-Orbital und das 2s-Orbital hybridisieren zu 3 sp2-Orbitalen im 120 Grad Winkel.

* Nur 1 orthogonal 2p-Orbital bleibt dann übrig.

* Der Vermischungsgrad betragt 33% s-Orbitalanteil und 66% p-Charakter (Orbital)

* Drei Elektronen gehen in je ein sp-Hybrid-Orbital und 1 Elektron verbleibt im p-Orbital.

![ggg](https://raw.githubusercontent.com/deltorobarba/repo/master/sciences_202.png)

* Fur 4 ich annähender Bindungspartner haben wir einen Tetraeder mit dem Winkel von 109 Grad.

![ggg](https://raw.githubusercontent.com/deltorobarba/repo/master/sciences_203.png)

* Es entstehen 4 sp3-Hybridorbitale mit 25% s-Orbitalanteil und 75% p-Orbtialanteil aus drei 2p-Orbitalen und einem 2s-Orbital

![ggg](https://raw.githubusercontent.com/deltorobarba/repo/master/sciences_204.png)




**Jetzt können wir uns einen Reaktionsablauf zwischen Wasserstoff und Kohlenstoff anschauen, der zur Bildung der einfachsten organischen Stoffgruppen fuehrt**.

![ggg](https://raw.githubusercontent.com/deltorobarba/repo/master/sciences_205.png)

* Wenn sich ein H-Atom eine C-Atom nähert, führt es zu einer sp-Hybridisierung am C-Atom. Es entsteht ein CH-Fragment.

* Diese ist noch sehr reaktiv, da es noch 3 ungeplante Elektronen enthält.

* Es kann entweder mit noch einem H-Atom oder mit einem weiteren CH-Fragment reagieren.

![ggg](https://raw.githubusercontent.com/deltorobarba/repo/master/sciences_206.png)

* Wenn das CH-Fragment mit einem weiteren CH reagiert, kommt es zur Bildung einer Dreifachbindung zwischen den beiden C-Atomen.

* Eine Sigma-Bindung entsteht durch die Überlappung von zwei sp-Hybridorbitalen, die eine Rotation um die Bindungsachse zulassen.

![ggg](https://raw.githubusercontent.com/deltorobarba/repo/master/sciences_207.png)

* Jeweils eine Pi-Bindung entsteht durch Überlappung der zwei unhybridisierten p-Orbitale, die keine Rotation des Moleküls zulassen.

* Da bei diesem Molekül alle Elektronen gepaart vorkommen, erreichen alle Atome Edelgas-Konfiguration.

* Es liegt also ein stabiles Molekül vor und heisst in diesem Fall Ethin.

* Es laesst sich nicht rotieren, da dies die beiden Pi-Bindungen nicht zulassen.

![ggg](https://raw.githubusercontent.com/deltorobarba/repo/master/sciences_208.png)

* Bei der Reaktion eines CH-Fragments mit noch einem H-Atom, entsteht ein CH2-Fragment.

![ggg](https://raw.githubusercontent.com/deltorobarba/repo/master/sciences_209.png)

* Es hat aber nicht wie am Anfang angenommen einen 180 Grad Winkel, sondern hybridisiert weiter zu einem sp-1,39-Hybridorbital mit einem Winkel von 136 Grad.

* Dieses Fragment bezeichnet man auch als Carben.

* Dies geschieht, weil ein gepaartes Elektron ohne Bindungspartner raumfuellender ist, als die doppelt besetzten Orbital mit Bindungspartner und deshalb mehr Platz braucht.

![ggg](https://raw.githubusercontent.com/deltorobarba/repo/master/sciences_210.png)

* Das Carben reagiert nur mit einem weiteren Carben. Dies fuhrt zur Bildung von Ethen, einem stabilen Molekül, da hier wieder alle Elektronen gepaart sind.

* Aufgrund einer Sigma-Bindung durch Überlappung zweier sp-2-Hybridorbitale und Pi-Bindung durch Überlappung des unhybridisierten p-Orbitals liegt hier eine Doppelbindung vor.

* Auch dieses Molekül kann durch die Doppelbindungen nicht zwischen den beiden C-Atomen rotieren.

* Deshalb liegen alle 6 Atome in einer Ebene und können sich sterisch nicht aus dem Weg gehen.

![ggg](https://raw.githubusercontent.com/deltorobarba/repo/master/sciences_211.png)

* Wenn das Carben mit einem weiteren H-Atom reagiert, fuehrt dies zur Bindung eines CH3-Fragments, in dem das C-Atom sp2-hybridisiert vorliegt und aufgrund eines ungepaarten Elektrons weiterhin reaktiv ist.

![ggg](https://raw.githubusercontent.com/deltorobarba/repo/master/sciences_212.png)

* Dieses reagiert wieder mit einem anderen CH3-Fragment. Es kommt zur Umhybridisierung von sp2 auf sp3, und es entsteht wieder ein stabiles Molekül: Ethan.

* Wegen nur einer einzigen Sigma-Bindung ist die freie Rotation zwischen den C-Atomen möglich, da sich diese Atome am guenstigsten im Raum Norden koennen.

![ggg](https://raw.githubusercontent.com/deltorobarba/repo/master/sciences_213.png)

* Reagiert das CH3-Fragment mit einem weiteren H-Atom, dann entsteht Methan

* Das C-Atom ist sp3-hybridisiert und tetraedisch angeordnet.


![ggg](https://raw.githubusercontent.com/deltorobarba/repo/master/sciences_214.png)


###### *HOMOs und LUMOs*

HOMOs und LUMOs beim Butadien https://www.u-helmich.de/che/Q2/farbe/02/farbigkeit-02.html

Das höchste noch mit Elektronen besetzte MO wird als HOMO bezeichnet. Hiermit ist keine geschlechtliche Orientierung gemeint, sondern der Begriff kommt aus dem Englischen und ist die Abkürzung für "highest occupied Molecular Orbital".

Das niedrigste noch nicht mit Elektronen besetzte MO heißt daher auch nicht HETERO, sondern LUMO, vom englischen "lowest unoccupied Molecular Orbital".

Warum sind diese beiden Molekülorbitale nun von so besonderer Bedeutung für die Farbstoffchemie?

Ganz einfach: Die Elektronen des HOMOs können durch Licht ganz leicht angeregt werden, und in der Regel springen die angeregten Elektronen in das LUMO. Wichtig: Der Spin (also die Drehrichtung) des Elektrons ändert sich dabei nicht.

Beim Butadien ist für diesen "Quantensprung" UV-Licht der Wellenlänge 217 nm erforderlich[1]. Mit anderen Worten: Butadien absorbiert nicht-sichtbares Licht, und da die Komplementärfarbe von "nicht sichtbar" ebenfalls "nicht sichtbar" ist, hat das Butadien keine Farbe.

https://www.u-helmich.de/che/Q2/farbe/02/farbigkeit-01.html

##### <font color="blue">*Atomorbital-Theorie*

###### *Valence Bond Theory (Atomic Orbitals)*

**Valence Bond Theory (Atomic Orbitals)**

> The use of **AOs has led to the [valence bond (VB) method](https://de.m.wikipedia.org/wiki/Valenzbindungstheorie)**, in which the wavefunctions are many-determinant and the AOs centered on different nuclei of the molecule are non- orthogonal.

* VB has played an important role in the development of quantum chemistry, allowing a clear definition of important chemical concepts and the development of sophisticated techniques based on group-theoretic methods.

* Siehje auch: Online Book [Valence bond theory for chemical dynamics](https://onlinelibrary.wiley.com/doi/10.1002/jcc.20529),
[u-helmich.de/che/Q2/orbitalmodell/](https://www.u-helmich.de/che/Q2/orbitalmodell/orbitalmodell-einfach.html), [anthrowiki.at/Atomorbital](https://anthrowiki.at/Atomorbital), Video: [Periodensystem der Elemente I Teil 2 I musstewissen Chemie](https://youtu.be/f5-W87IGLFY)

* Ein Atomorbital ist in den quantenmechanischen Modellen der Atome die räumliche Wellenfunktion eines einzelnen Elektrons in einem quantenmechanischen Zustand, meist in einem stationären Zustand. Sein Formelzeichen ist meist $\varphi$ (kleines Phi) oder $\psi$ (kleines Psi).

  * Das Betragsquadrat $|\psi(\vec{r})|^{2}$ beschreibt als Dichtefunktion die räumliche Verteilung der Aufenthaltswahrscheinlichkeit, mit der das Elektron am Ort $\vec{r}=(x, y, z)$ gefunden werden kann (bornsche Wahrscheinlichkeitsinterpretation der Quantenmechanik).

  * Zusammen mit der Angabe, ob der Spin zu einer festen Achse oder zum Bahndrehimpuls des Elektrons parallel oder antiparallel ausgerichtet ist, beschreibt ein Orbital den Elektronenzustand vollständig.

* Anschaulich stellt man ein Orbital gewöhnlich durch die Oberfläche des kleinstmöglichen Volumens dar, in dessen Inneren sich das Elektron mit großer (z. B. 90%iger) Wahrscheinlichkeit aufhält (s. Abbildung). Man erhält damit Körper, die ungefähr der Größe und Form der Atome entsprechen, wie sie sich in chemischen Molekülen, kondensierter Materie und der kinetischen Gastheorie bemerkbar machen.

  * Die gebräuchlichsten Atomorbitale sind die, die sich für das einzige Elektron des Wasserstoffatoms als Lösungen der Schrödingergleichung des Wasserstoffproblems ergeben und 1926 erstmals veröffentlicht wurden. Sie haben verschiedene Formen, die mit $\psi_{n l m_{l}}(\vec{r})$ bezeichnet werden, wobei der untere Index aus der Hauptquantenzahl $n$, der Bahndrehimpulsquantenzahl $l$ und der magnetischen Quantenzahl $m_{l}$ besteht.

![ggg](https://raw.githubusercontent.com/deltorobarba/repo/master/sciences_0779.jpeg)


*Elektronenkonfiguration*

* Die [Elektronenkonfiguration](https://de.m.wikipedia.org/wiki/Elektronenkonfiguration) gibt im Rahmen des Schalenmodells der Atomhülle die Verteilung der Elektronen auf verschiedene Energiezustände und damit [Aufenthaltsräume (=Atomorbitale)](https://de.m.wikipedia.org/wiki/Atomorbital) an.

* [Quantum Numbers](https://de.wikipedia.org/wiki/Quantenzahl) are used to describe certain quantitites that we are allowed to know at the same time as knowing the energy of a quantum system. **They together completely describe the quantum state of an electron in an atom**!

  * Any eigenstate of the electron (e.g. in the hydrogen atom) is described fully by four quantum numbers** [Source](http://kolibri.teacherinabox.org.au/modules/en-boundless/www.boundless.com/chemistry/definition/quantum-number/index.html)

* Four quantum numbers in atomic physics. The set is: principal quantum number, azimuthal quantum number, magnetic quantum number, and spin quantum number. Together, they describe the unique quantum state of an electron.

* **Die äußerste besetzte Schale ([Valenzschale](https://de.m.wikipedia.org/wiki/Valenzschale)) bestimmt das chemische Verhalten und ist daher maßgeblich für die Einordnung ins Periodensystem.**

* Die Orbitale werden entsprechend den Serien von [Spektrallinien](https://de.m.wikipedia.org/wiki/Spektrallinie) benannt, die ein [angeregtes](https://de.m.wikipedia.org/wiki/Angeregter_Zustand) Elektron aussendet, wenn es in sein ursprüngliches Orbital zurückfällt.

* Gemäß dem [Pauli-Prinzip](https://de.m.wikipedia.org/wiki/Pauli-Prinzip) darf der Zustand keiner zwei Elektronen eines Atoms in allen vier Quantenzahlen übereinstimmen. Mit diesem Prinzip lässt sich zeigen, dass sich die Elektronen auf die verschiedenen erlaubten Zustände und damit auf die [Schalen](https://de.m.wikipedia.org/wiki/Schalenmodell_(Atomphysik)) und Unterschalen verteilen.

* [Heisenberg uncertainty principle](https://en.m.wikipedia.org/wiki/Uncertainty_principle): things we are not allowed to know at the same time, like position and momentum of a particle

* *Die Elektronen in den Orbitalen kannst Du ausserdem unterscheiden. Das sagen Dir die 4 verschiedenen Quantenzahlen. Sie schreiben den Zustand eines Elektrons im Orbital*

* Der Zustand jedes Elektrons der Hülle wird nach dem [Bohr-Sommerfeldschen Atommodell](https://de.m.wikipedia.org/wiki/Bohr-sommerfeldsches_Atommodell) sowie dem Orbitalmodell durch vier Quantenzahlen bestimmt:

> $
\begin{array}{|l|l|l|l|l|}
\hline {\text { Quantenzahl }} & \text { Zeichen } & \text { Wertebereich } & \text { Bezeichnung } & \text { Beispiele } \\
\hline \text { Hauptquantenzahl } & n & 1,2,3, \ldots & \mathrm{K}, \mathrm{L}, \mathrm{M}, \ldots & 3 \\
\hline \text { Nebenquantenzahl } & l & 0, \ldots, \mathrm{n}-1 & \mathrm{~s}, \mathrm{p}, \mathrm{d}, \mathrm{f}, \ldots & 0,1,2 \\
\hline \begin{array}{l}
\text { magnetische } \\
\text { Drehimpulsquantenzahl }
\end{array} & m_{l} & -l, \ldots,+l & \begin{array}{l}
\mathrm{s}, \mathrm{p}_{\mathrm{x}, \mathrm{y}, \mathrm{z},} \\
\mathrm{d}_{\mathrm{yz}, \mathrm{x}, \mathrm{x}, \mathrm{y}_{1}, \mathrm{z}^{2}, \mathrm{z}^{2}-\mathrm{y}^{2}, \ldots}
\end{array} & -2,-1, \pm 0,+1,+2 \\
\hline \begin{array}{l}
\text { magnetische } \\
\text { Spinquantenzahl }
\end{array} & m_{s} & -1 / 2,+1 / 2 & \downarrow, \uparrow & -1 / 2,+1 / 2 \\
\hline
\end{array}
$



###### *Principal Quantum Number (Hauptquantenzahl)*

**Die Hauptquantenzahl n beschreibt das Energieniveau. Du liest sie anhand der Periodennummer ab.**

[*Principal Quantum Number*](https://de.m.wikipedia.org/wiki/Atomorbital#Hauptquantenzahl_n:_Schale) *= Hauptschale = Energy Levels*

> **$n$: principal quantum number** bestimmen main shell (Haupt-Schale), $n_1$ with lowest energy. n = 1, 2, 3, 4 or labeled alphabetically with the letters used in X-ray notation (K, L, M, N, O, P, Q etc). Jede Schale kann gemäß den Beschränkungen von l, $m_{l}$ und $m_{s}$ mit maximal 2n² Elektronen besetzt werden.

* In chemistry and atomic physics, an [electron shell](https://en.m.wikipedia.org/wiki/Electron_shell) may be thought of as an orbit followed by electrons around an atom's nucleus.

* Principle quantum number $n$ corresponds to Bohr's orbit numbers

* In quantum mechanics, the [principal quantum number](https://en.wikipedia.org/wiki/Principal_quantum_number) (symbolized n) is one of four quantum numbers assigned to each electron in an atom to describe that electron's state. Its values are natural numbers (from 1) making it a discrete variable.

* In chemistry, values n = 1, 2, 3, 4, 5, 6, 7 are used in relation to the [electron shell theory](https://en.wikipedia.org/wiki/Electron_shell), **with expected inclusion of n = 8 (and possibly 9) for yet-undiscovered period 8 elements**.

* **In atomic physics, higher n sometimes occur for description of excited states**. Observations of the interstellar medium reveal atomic hydrogen spectral lines involving n on order of hundreds; values up to 766 were detected.

* The shells correspond to the [principal quantum numbers](https://en.m.wikipedia.org/wiki/Principal_quantum_number):

1. n = 1 $\rightarrow$ K-Schale: 2 Elektronen

2. n = 2 $\rightarrow$ L-Schale: 8 Elektronen

4. n = 3 $\rightarrow$ M-Schale: 18 Elektronen

5. n = 4 $\rightarrow$ N-Schale: 32 Elektronen

6. ..

> **The energy level (=principal quantum numbers) dictates how many sub shells there are for electrons to fill**

* n=1 energy level (principal quantum number) has only one sub shell s (azimuthal quantum number), the spherical one (=no orbital angular momentum)

* n=2 energy level (principal quantum number) has two sub shells s und p (azimuthal quantum number): the spherical one and the triple dumbbell one,

* n=3 energy level (principal quantum number) has three different sub shells s, p und d (azimuthal quantum number)

* Video: [Why Real Atoms Don't Look Like This - Quantum Numbers to Understand Atomic Structure](https://youtu.be/EhIbCrl1pGw)

![ggg](https://raw.githubusercontent.com/deltorobarba/repo/master/chem_013.jpg)

Video: https://youtu.be/f5-W87IGLFY

![ggg](https://raw.githubusercontent.com/deltorobarba/repo/master/sciences_0641.png)

Within each energy level there are a set of orbitals.

* The first energy level has one orbital = 2 Electrons

* the second energy level has 1 + 3 more orbitals = 8 Electrons

* the third energy level has 9 orbitals etc = 18 Electrons

![ggg](https://raw.githubusercontent.com/deltorobarba/repo/master/chem_044.png)


![ggg](https://raw.githubusercontent.com/deltorobarba/repo/master/sciences_231.png)

![ggg](https://raw.githubusercontent.com/deltorobarba/repo/master/quantum_141.png)



###### *Azimuthal quantum number $l$ (Neben- oder Bahndrehimpuls-Quantenzahl)*

**Die Nebenquantenzahl l beschreibt die Form des Orbitals. Ausserdem wird sie durch die Hauptquantenzahl n bestimmt.**

> [**$l$: azimuthal quantum number**](https://de.m.wikipedia.org/wiki/Atomorbital#Neben-_oder_Bahndrehimpuls-Quantenzahl_l) bestimmen subshell (Nebenschale). is a quantum number for an atomic orbital that determines its orbital angular momentum and describes the shape of the orbital. Counting: l = 0,1,2,3,.. etc. or s, p, d, f, etc. - die ersten vier Serien heißen aus historischen Gründen s („sharp“), p („principal“), d („diffuse“) und f („fundamental“).

![ggg](https://raw.githubusercontent.com/deltorobarba/repo/master/sciences_232.png)

Pro Energieniveau, das heisst pro Periode im Periodensystem hat ein Atom 1 s-Orbital. Zusätzlich bezeichnet man die verschiedenen s-Orbitale mit einer Nummer, die der jeweiligen Periode entspricht. **In ein s-Orbital passen 2 Elektronen maximal**.

![ggg](https://raw.githubusercontent.com/deltorobarba/repo/master/sciences_239.png)

**Neben den s-Orbitalen gibt es auch die hantelfoermigen p-Orbitale**

Bor ist das erste Element, das ein besetztes p-Orbital hat. Das heisst sie werden besetzt, nachdem die s-Orbitale voll sind.

**Ab Bor gilt, dass ein Atom pro Energieniveau (=Periode / Reihe im Periodensystem) 3 p-Orbitale hat. Die drei p-Orbitale sind energetisch gleich (entartet)**.

P-Orbitale unterscheiden sich in Bezug auf die Ausrichtung entlang der Koordinatenachsen. Deswegen erhalten sie unterschiedliche Bezeichnungen (px, py, pz).

![ggg](https://raw.githubusercontent.com/deltorobarba/repo/master/sciences_241.png)

More about azimuthal quantum number:

![ggg](https://raw.githubusercontent.com/deltorobarba/repo/master/quantum_196.png)

*there is yet another one: f-type, and it comes in sets of 7. ony allowed in n principal quantum number = 4, [Source](https://www.youtube.com/watch?v=ysQfZ48B7rg)*





**Azimuthal Quantum Number im Periodensystem:**

* Im Periodensystem entspricht die Besetzung des s-Orbitals einer neuen Schale dem Sprung in eine neue Periode.

* s-Orbital:	2	Elektronen, umfasst Elemente der 1. und 2. Hauptgruppe sowie Helium

* p-Orbital:	6	Elektronen,	umfasst Elemente der übrigen Hauptgruppenelemente

* d-Orbital:	10	Elektronen,	umfasst Elemente aller Nebengruppenelemente

* f-Orbital:	14	Elektronen,	umfasst alle Lanthanoide und Actinoide

*Das folgende Periodensystem betont die Art der Orbitale, in denen die äußersten Elektronen eines chemischen Elementes stecken (2. Level l : azimuthal quantum number):*

![ggg](https://upload.wikimedia.org/wikipedia/commons/a/a7/Orbitalblöcke.svg)


**Another explication of Azimuthal quantum number**

Azimuthal quantum number (L): Sub Shells with l = 0,1,2,3,.. etc. or s, p, d, f, etc. for sharp, principal, diffuse, fundamental.

![ggg](https://raw.githubusercontent.com/deltorobarba/repo/master/quantum_142.png)

* 3 individual orbitals, each made up of three individual orbitals of which is shaped like dumbbell.

* Each of the dumbbell os perpendicular to each other: one lies along the x-axis, another along y-axis and one on z-axis.

* each of these dumbbell-shaped orbitals can hold 2 electrons. Looking at x-axis: electron can be found at different places,  but most likely to be found somewhere in the dumbbell-shape (it's now orientated)

![ggg](https://raw.githubusercontent.com/deltorobarba/repo/master/quantum_143.png)

So there is a difference now between spherical sub shell and the the triple dumbbell sub shell

> **the energy level dictates how many sub shells there are for electrons to fill**

* n=1 energy level has only one sub shell, the spherical one

* n=2 energy level has two sub shells: the spherical one and the triple dumbbell one,

* n=3 energy level has three different sub shells

![ggg](https://raw.githubusercontent.com/deltorobarba/repo/master/quantum_144.png)

> **What's the difference between the spherical sub shell and the triple dumbbell sub shell?** other than the shape?

* electrons within a particular sub shell have different amounts of orbital angular momentum ! (Not meant: spin, because that's independent of subshell position)

> **We can represent how much angular momentum an electron has in a particular sub shell using the azimuthal quantum number.**

* any electron in any one of the spherical shaped orbitals known as s-orbitals has no angular momentum at all. -> we give each s-orbtial the azimuthal quantum number L=0. here three examples:

  * for an electron in the lowest possible energy level n=1 it must be in the s-subshell (because there is only the sub shell) with azimuthal quantum number L=0

  * for an electron in the second shell and the s-subshell it has the quantum numbers n=2 and l=0

  * but for an electron on the p-subshell it does have a certain amount of angular momentum. Specifically it has 1 lot of h bar of angular momentum and n=2 and l=1

  * for n=3 energy levels (not depicted below) then we could have electrons in the s-subshell with l=0, or p-subshell with l=1 or the d-subshell with l=2 !

![ggg](https://raw.githubusercontent.com/deltorobarba/repo/master/quantum_145.png)



**Bindungen sind durch Orbitalmodell erklaerbar::**

![ggg](https://raw.githubusercontent.com/deltorobarba/repo/master/chem_046.jpg)

> Details zu [Molekülorbitalen (pdf)](https://raw.githubusercontent.com/deltorobarba/repo/master/molekuelorbitale.pdf) der ETH Zurich

###### *Magnetic quantum number $m_l$ (Magnetquantenzahl $m_l$: Neigung des Drehimpulsvektors)*

**Die Magnetquantenzahl $m_l$ beschreibt die räumliche Ausrichtung des Orbitals und ihre Werte werden durch die Nebenquantenzahl l bestimmt.**

> **$m_l$ magnetic quantum numbers**, distinguishes the orbitals available within a subshell, and is used to calculate the azimuthal component of the orientation of orbital in space.

![ggg](https://raw.githubusercontent.com/deltorobarba/repo/master/sciences_233.png)

* The angular momentum quantum number ℓ = 0, 1, 2, ... determines the magnitude of the angular momentum.

  * This leads to a third quantum number, the principal quantum number n = 1, 2, 3, ....

  * Note that the maximum value of the angular momentum quantum number is limited by the principal quantum number: it can run only up to n − 1, i.e. ℓ = 0, 1, ..., n − 1.

[Magnetic quantum number](https://en.wikipedia.org/wiki/Magnetic_quantum_number): the magnetic quantum number distinguishes the orbitals available within a subshell, and is used to calculate the azimuthal component of the orientation of orbital in space.

  * Electrons in a particular subshell (such as s, p, d, or f) are defined by values of ℓ (0, 1, 2, or 3).

  * The value of ml can range from -ℓ to +ℓ, including zero. Thus the s, p, d, and f subshells contain 1, 3, 5, and 7 orbitals each, with values of m within the ranges 0, ±1, ±2, ±3 respectively.

  * Each of these orbitals can accommodate up to two electrons (with opposite spins), forming the basis of the periodic table.

*These orbitals have magnetic quantum numbers $m=-\ell ,\ldots ,\ell$ from left to right in ascending order. The $e^{mi\phi }$ dependence of the azimuthal component can be seen as a color gradient repeating m times around the vertical axis:*
![gg](https://upload.wikimedia.org/wikipedia/commons/thumb/8/8f/Atomic_orbitals_spdf_m-eigenstates.png/640px-Atomic_orbitals_spdf_m-eigenstates.png)

https://de.m.wikipedia.org/wiki/Atomorbital#Magnetquantenzahl_ml:_Neigung_des_Drehimpulsvektors

**Gesamtdrehimpuls $j$ und magnetische Quantenzahl $m_{j}$**: Zu den schweren Atomen hin wird die [Spin-Bahn-Wechselwirkung](https://de.m.wikipedia.org/wiki/Spin-Bahn-Wechselwirkung) stärker. Sie bewirkt die Aufspaltung der Energie einer Unterschale mit bestimmten $n>1, l>0$ in zwei Unterschalen, je nach Wert des [Gesamtdrehimpulses](https://de.m.wikipedia.org/wiki/Atomorbital#Gesamtdrehimpuls_j_und_magnetische_Quantenzahl_mj) $j=l \pm \frac{1}{2}$. Die magnetische Quantenzahl $m_{j}=-j,-(j-1), \ldots,+j$ durchläuft $2 j+1$ Werte. Jedes dieser Orbitale kann von einem Elektron besetzt werden, sodass die Gesamtzahl der Plätze gleich bleibt. In der Bezeichnung wird der Wert für $j$ als unterer Index an das Symbol für $n l$ angefügt, z. B. $2 p_{3 / 2}$.


###### *Spinquantenzahl $m_s$ (Magnetic spin quantum number)*

**Die Spinquantenzahl s beschreibt den Spin eines einzelnen Elektrons.**

> [**$m_s$ spin quantum number**](https://de.m.wikipedia.org/wiki/Atomorbital#Magnetische_Spinquantenzahl_ms), describes the intrinsic angular momentum (or spin angular momentum, or simply spin) of an electron or other particle (The name comes from a physical spinning of the electron about an axis, as proposed by Uhlenbeck and Goudsmit). The value of $m_s$ is the component of spin angular momentum parallel to a given direction (the z–axis), which can be either +1/2 or –1/2 (in units of the reduced Planck constant).

**Nach dem Pauli-Prinzip müssen sich 2 Elektronen in mindestens einer Quantenzahl unterscheiden. Daher haben Elektronen im gleichen Orbital unterschiedliche Spins**.

![ggg](https://raw.githubusercontent.com/deltorobarba/repo/master/sciences_234.png)

Bei den leichteren Atomen braucht man den [Elektronenspin](https://de.m.wikipedia.org/wiki/Elektronenspin) nur in der Form zu berücksichtigen, dass jedes Orbital $\psi_{n l m}$ l von genau einem [Elektronenpaar](https://de.m.wikipedia.org/wiki/Elektronenpaar) besetzt werden kann, dessen zwei Elektronen nach dem Pauli-Prinzip entgegengesetzte magnetische Spinquantenzahlen aufweisen $\left(m_{s}=\pm \frac{1}{2}\right)$.

[Spin quantum number](https://en.wikipedia.org/wiki/Spin_quantum_number): In atomic physics, the spin quantum number is a quantum number (designated ms) which describes the intrinsic angular momentum (or spin angular momentum, or simply spin) of an electron or other particle.

  * The phrase was originally used to describe the fourth of a set of quantum numbers (the principal quantum number n, the azimuthal quantum number l, the magnetic quantum number m, and the spin quantum number ms), which completely describe the quantum state of an electron in an atom.

  * The name comes from a physical spinning of the electron about an axis, as proposed by Uhlenbeck and Goudsmit.

  * **The value of $m_s$ is the component of spin angular momentum parallel to a given direction (the z–axis)**, which can be either +1/2 or –1/2 (in units of the reduced Planck constant).

  * At an elementary level, ms is described as the spin quantum number, and s is not mentioned since its value 1/2 is a fixed property of the electron. **At a more advanced level where quantum mechanical operators are introduced, s is referred to as the spin quantum number, and $m_s$ is described as the spin magnetic quantum number or as the z-component of spin $s_z$.**

![ggg](https://raw.githubusercontent.com/deltorobarba/repo/master/quantum_146.png)


###### *History: From Rutherford's Planetary Model of the Atom to Schroedinger's Wavefunction*

**Step 1: 1911 Ernest Rutherford proposed a planetary model of the atom**

* Solar system and atom were almost identical: moon is held in orbit around the earth due to gravity, the negatively charged electron was held on orbit around a positively charged nucleus (attractive force was electromagnetism instead of gravity)

* It seemed to make sense because Newton's law of universal gravitation $F=G \frac{M m}{r^{2}}$ was almost identical to Coulomb's law of electrostatic force $F=k \frac{Q q}{r^{2}}$

* [Source 1](https://www.youtube.com/watch?v=fP2TAw7NnVU) and [Source 2](https://www.youtube.com/watch?v=azI-_S6g8C8)

**Step 2: The problem with that model**

* when electrons are accelerated, as they would be when constantly changing direction flying around the nucleus (traveling in circular orbits), they create electromagnetic waves according to Maxwell's equations

* spinning electrons would radiate photons constantly causing it to lose energy. would radiate a rainbow of colors, more blue and purple as it came closer to the nucleus until it collapsed into it.

* So using classical laws for the atom would not work if you presumed the Rutherford model

**Step 3: Energy of photons is quantized (Planck 1900)**

* $E = h f$

	* proportionality constant h = 6.6 x 10<sup>-34</sup> J . Sec

	* f = frequency of radiation

* Planck's theory had shown that matter emitted only discretes amounts of radiation with energy E proportional to the frequency f and h

**Step 4: Only orbits that are integer multiples of $\frac{h}{2\pi}$ can exist (Bohr)**

> Niels Bohr combined Rutherford's and Planck's theory: electrons could exist in certain special orbits without radiating energy

* Bohr noticed that Planck's constant $h$ had units of angular momentum. So he hypothesized that only those orbits would be allowed where the angular momentum of the electron is quantized based on Planck's constant:

> Plancks' constant $h$ = 6.6 x 10<sup>-34</sup> J . Sec or: kg m<sup>2</sup> / sec. Angular momentum = mar = kg m<sup>2</sup> / sec

* Bohr guessed that the lowest orbit would have the momentum $\frac{h}{2\pi}$, where the $2 \pi$ comes from the geometry of circular orbits.

* And any orbit could exist as long as it was an integer multiple of those number !! So the next orbit would be 2 x $\frac{h}{2\pi}$, and then 3 x $\frac{h}{2\pi}$ etc.

> **Bohr predicted that the electrons would only emit (radiate) photons when moving from higher orbit to lower orbit, and absorb energy when moving from lower to higher orbits**

> **<font color="red">This is why the states are called Eigenstates and the quantum numbers (for the orbits) are called Eigenvalues, because in this state the quantum system is stable and doesnt loose energy.</font>**

* But Bohr could not explain why electrons would not emit photons all the time, and why these special orbits exist

* But presuming he was right, you could find the size of the orbit of the electron using Coulomb's law $F=k \frac{Q q}{r^{2}}=\frac{m v^{2}}{r}$ $\rightarrow$

	* $k \frac{Q q}{r^{2}}$ is the electric force between electron and proton

	* $\frac{m v^{2}}{r}$ is the centripetal force

	* r is the radius from nucleus to orbit of electron

* It turns out that **the radius of the lowest orbit would be 0.529 X 10^-10 m, about half an angstrom**

	* angular momentum = mvr = $\frac{h}{2 \pi}$

	* r = $\frac{h^{2}}{k q^{2} 4 \pi^{2} m}$

* this could reveal the size of the atom: r = 0.529 x 10^-10 meters

* Knowing this, the energy emitted by the electron as it changes orbits could be calculated and observations of energy emissions confirmed the calculations

	* Total energy = kinetic energy + potential energy

	* Kinetic energy = $=1 / 2 m v^{2}=1 / 2 p^{2} / m$ (in terms of momentum).

	* Potential energy $=-K \frac{g^{2}}{r}$ (from Coulomb's law)

	* $p=\frac{h}{2 \pi}$

	* $\begin{aligned} \text { Total energy } &=\frac{h^{2}}{8 \pi^{2} m}-K \frac{q^{2}}{r} \\ &=-2.17 \times 10^{-18} \text { Joules (Lowest energy state) } \end{aligned}$

* the only problem was: nobody understood why Bohr was right.

**Step 5: An electron is a wave (de Broglie)**

* if a particle has a momentum and it has a wavelength associated with it  because of Planck's constant, then an electron is probably a wave (which also means that solid matter was composed of waves

> <font color="red">**An electron is a circular standing wave!!** (doesnt fly in an orbit) **The circular standing wave can only have an integer number of wavelengths!**

* <font color="red">**Principal quantum number n is wavelenght and higher n = higher wavelength = higher energy**. The the Bohr model emerges with its "orbits". **The energy of an electron is quantized.**


* de Broglie suggested that electrons can only exist in orbits where their waves interfere constructively. And that can only happen if their circumference of the orbit is equal to the wavelength, or twice the wavelength, or any integer times the wavelength.

![ggg](https://raw.githubusercontent.com/deltorobarba/repo/master/sciences_1205.png)

[Source](https://www.youtube.com/watch?v=O6g-7rUgrdg&t=20s)

* This explained why orbits would be at the radii that they are, something that Bohr couldn't do.

* But there were still questions left: what is the nature of these waves? How and why do they exist?

**Step 6: Explaining the nature of waves (Schroedinger)**

* He said: if it's a wave, it can exist anywhere in 3 dimensional space.

Schroedinger formulated the rules to describe the behavior of these waves: $i \hbar \frac{\partial}{\partial t}|\psi(t)\rangle=\hat{H}|\psi(t)\rangle$ (Schroedinger equation of the wave function of atoms)

![ggg](https://raw.githubusercontent.com/deltorobarba/repo/master/quantum_165.png)

* Schroedinger equation could for example describe the Hydrogen atom with more precision than the Bohr model, and all other models in the periodic table which Bohr couldn't


**Step 7: Heisenberg Uncertainty principle: why is it a wave cloud?**

* Large wavelength, like visible light, cannot detect particles, they are too small

* Short wavelengths are smaller but have so Hugh energy that they interact with particles

* Consequence of the Heisenberg uncertainty principle (uncertainty in position x and uncertainty in momentum p): $\Delta x \Delta p \geq \frac{h}{4 \pi}$ (it cannot below a certain constant): we cannot say where they are and how fast they are moving at the same time.

> **So we can't draw a picture of an electron in one place sitting on a circular orbit**

* So we have to guess based on the wave equation, and it tells us that the electron forms a cloud around the nucleus. Shape of the cloud is governed by the wave function.

* the cloud represents the probable position off the electron if you were to measure it. And this cloud exists everywhere from nucleus to very far away. The volume of the atom is not empty - it is filled everywhere with a cloud of electrons.

*Model of the hydrogen atom with electron at lowest energy state:*

![ggg](https://raw.githubusercontent.com/deltorobarba/repo/master/quantum_166.png)

We can even use the Schroedinger wave equation to find the electron probability. Following plot shows the probability of finding the electron at various distances from the nucleus for a hydrogen atom. The highest probability occurs at r = 0.529 x 10^-10 m, what Bohr already calculated.

* so the most probable radius obtained from quantum mechanics is identical to the radius calculated by classical mechanics

![ggg](https://raw.githubusercontent.com/deltorobarba/repo/master/quantum_167.png)

Bohr assumed the electron at this distance all the time, meanwhile Schroedinger just assumed it to be there some of the time:

![ggg](https://raw.githubusercontent.com/deltorobarba/repo/master/quantum_168.png)

The electron cloud is 100,000 larger than the proton cloud.

> **So the idea of discrete spherical protons and neutrons in the nucleus, or electrons orbiting them is fiction.**


###### *Vom Schalenmodell zum Orbitalmodell (Atom und Molekul) inkl. Madelung-Regel, Pauli-Pinzip und Hundsche Regel*

![ggg](https://raw.githubusercontent.com/deltorobarba/repo/master/sciences_235.png)

https://studyflix.de/chemie/orbital-4225


![ggg](https://raw.githubusercontent.com/deltorobarba/repo/master/sciences_236.png)


![ggg](https://raw.githubusercontent.com/deltorobarba/repo/master/sciences_237.png)


![ggg](https://raw.githubusercontent.com/deltorobarba/repo/master/sciences_238.png)


Pro Energieniveau, das heisst pro Periode im Periodensystem hat ein Atom 1 s-Orbital. Zusätzlich bezeichnet man die verschiedenen s-Orbitale mit einer Nummer, die der jeweiligen Periode entspricht. In ein s-Orbital passen 2 Elektronen maximal.

![ggg](https://raw.githubusercontent.com/deltorobarba/repo/master/sciences_239.png)


Schauen wir uns ein Beispiel an:

* Wasserstoff steht in der erste Periode und hat somit nur 1 s-Orbital

* Lithium steht am Anfang der zweiten Periode und braucht zur Verteilung seiner 3 Elektronen noch ein weiteres Orbital.

**Ausserdem nimmt die Grösse des s-Orbitals mit steigender Periode zu.**

![ggg](https://raw.githubusercontent.com/deltorobarba/repo/master/sciences_240.png)


**Neben den s-Orbitalen gibt es auch die hantelfoermigen p-Orbitale**

Bor ist das erste Element, das ein besetztes p-Orbital hat. Das heisst sie werden besetzt, nachdem die s-Orbitale voll sind.

**Ab Bor gilt, dass ein Atom pro Energieniveau (=Periode / Reihe im Periodensystem) 3 p-Orbitale hat. Die drei p-Orbitale sind energetisch gleich (entartet)**.

P-Orbitale unterscheiden sich in Bezug auf die Ausrichtung entlang der Koordinatenachsen. Deswegen erhalten sie unterschiedliche Bezeichnungen (px, py, pz).

![ggg](https://raw.githubusercontent.com/deltorobarba/repo/master/sciences_241.png)


Reminder zum Periodensystem:

![ggg](https://raw.githubusercontent.com/deltorobarba/repo/master/sciences_218.png)


**D-Orbitale (5 Arten) und f-Orbitale (7 Arten) sind komplexer (geometrisch) und spielen erst fur Elemente höherer Ordnungszahlen eine Rolle**. zB sind die d-Orbitale wichtig fur die Nebengruppenelemente.

![ggg](https://raw.githubusercontent.com/deltorobarba/repo/master/sciences_242.png)

![ggg](https://raw.githubusercontent.com/deltorobarba/repo/master/sciences_243.png)



**Um die Orbital richtig zu besetzen, muss man drei Regeln beachten:**

![ggg](https://raw.githubusercontent.com/deltorobarba/repo/master/sciences_244.png)


**Beispiel: Das Kohlenstoffatom hat 6 Elektronen. Deshalb nutzt man 2 s-Orbitale und 3 p-Orbitale**.

![ggg](https://raw.githubusercontent.com/deltorobarba/repo/master/sciences_245.png)

![ggg](https://raw.githubusercontent.com/deltorobarba/repo/master/sciences_246.png)



https://studyflix.de/chemie/orbitalmodell-276

![ggg](https://raw.githubusercontent.com/deltorobarba/repo/master/sciences_215.png)


![ggg](https://raw.githubusercontent.com/deltorobarba/repo/master/sciences_216.png)


![ggg](https://raw.githubusercontent.com/deltorobarba/repo/master/sciences_217.png)


![ggg](https://raw.githubusercontent.com/deltorobarba/repo/master/sciences_218.png)


![ggg](https://raw.githubusercontent.com/deltorobarba/repo/master/sciences_222.png)


Man nennt das s-Orbital auch 1-s-Orbital, weil es fur die erste Periode verwendet wird.

![ggg](https://raw.githubusercontent.com/deltorobarba/repo/master/sciences_219.png)


Lithium braucht ein weiteres Orbital, da es 3 Elektronen hat. Das erste ist schon voll. Dafur verwendet man das 2-s-Orbital:

![ggg](https://raw.githubusercontent.com/deltorobarba/repo/master/sciences_220.png)


![ggg](https://raw.githubusercontent.com/deltorobarba/repo/master/sciences_221.png)


Fur die restlichen 6 Elektronen in der 2. Periode bis zum Element Neon **werden die 2-p-Orbitale verwendet.**

Von den hantelfoermigen p-Orbitalen gibt es 3, die sich in ihrer räumlichen Ausrichtung unterscheiden.

Du musst berücksichtigen, dass die p-Orbitale energetisch gleich sind. Das bedeutet, dass nach der **Hundschen Regel die Orbital zunächst mit 1 Elektron besetzt werden**. Erst wenn jedes Orbital einfach besetzt ist, werden sie nacheinander doppelt besetzt.

![ggg](https://raw.githubusercontent.com/deltorobarba/repo/master/sciences_223.png)


Fur die dritte Periode bleibt die Reihenfolge der Besetzung gleich. Die Bezeichnung der Orbital ändert sich nur.

Bis Magnesium wird das 3s-Orbital besetzt...


![ggg](https://raw.githubusercontent.com/deltorobarba/repo/master/sciences_224.png)


...ab Aluminium werden die energiegleichen 3p-Orbitale zunächst einfach besetzt...


![ggg](https://raw.githubusercontent.com/deltorobarba/repo/master/sciences_225.png)


.. ab Schwefel dann doppelt:

![ggg](https://raw.githubusercontent.com/deltorobarba/repo/master/sciences_226.png)


In der 4. Periode kommt es nun zu einer veränderten Reihenfolge gegenüber den vorherigen Perioden.

Auf das Element Calcium folgt das erste Nebengruppenelement Scandium:

![ggg](https://raw.githubusercontent.com/deltorobarba/repo/master/sciences_227.png)

Die Elektronen der Nebengruppenelemente werden auf die d-Orbitale verteilt, von denen es 5 gibt.

![ggg](https://raw.githubusercontent.com/deltorobarba/repo/master/sciences_228.png)


Bei Scandium wird das zusätzliche Elektron in eines der fünf 3d-Orbitale gesetzt. Das 3d Orbital gehört formal gesehen noch zur dritten Periode, liegt energetisch aber hoher als das 4s-Orbital. Deshalb wird das erst danach besetzt.

![ggg](https://raw.githubusercontent.com/deltorobarba/repo/master/sciences_229.png)


Eine Übersicht über die Reihenfolge der Orbitalbesetzung liefert Dir ein **Energieniveauschema, auch Madelung-Schema genannt.**

![ggg](https://raw.githubusercontent.com/deltorobarba/repo/master/sciences_230.png)


###### *Example 1: Nicht-Relativistisches Wasserstoffproblem mit 1 Atom*

[Wave Equation for the Hydrogen Atom](http://kolibri.teacherinabox.org.au/modules/en-boundless/www.boundless.com/chemistry/textbooks/boundless-chemistry-textbook/introduction-to-quantum-theory-7/quantum-mechanical-description-of-the-atomic-orbital-65/wave-equation-for-the-hydrogen-atom-307-6585/index.html)

* [Lösung der Schrödinger-Gleichung (Wasserstoffproblem)](https://de.wikipedia.org/wiki/Wasserstoffatom#L%C3%B6sung_der_Schr%C3%B6dinger-Gleichung_(Wasserstoffproblem)): Die einfachste Schrödinger-Gleichung für ein geladenes Teilchen im [Coulombfeld](https://de.m.wikipedia.org/wiki/Coulombsches_Gesetz) einer Punktladung wird als Wasserstoffproblem bezeichnet.

* **The hydrogen atom is the simplest one-electron atom and has analytical solutions to the Schrödinger equation.**

* *The solution of the Schrödinger equation for the hydrogen atom uses the fact that the Coulomb potential produced by the nucleus is isotropic—it is radially symmetric in space and only depends on the distance to the nucleus.*

**Using a three-dimensional approach, the following form of the Schrödinger equation can be used to describe the hydrogen atom (Three dimensional Schrödinger equation as applied to the H atom)**:

> $E \psi=-\frac{\hbar^{2}}{2 \mu} \nabla^{2} \psi-\frac{e^{2}}{4 \pi \epsilon_{0} r} \psi$

Compare with time-dependent Schroedinger equation:

> $ E \psi(\vec{r}) =
\left[-\frac{\hbar^{2}}{2 m} \nabla^{2}+V(\vec{r})\right] \psi(\vec{r}) = -\frac{\hbar^{2}}{2 m} \nabla^{2}\psi(\vec{r}) +V(\vec{r}) \psi(\vec{r})
$

where μ is the reduced mass of the electron-proton pair. The wavefunction itself is expressed in spherical polar coordinates:

> $\psi(r, \theta, \phi)=R(r) Y_{l}^{m}(\theta, \phi)=R(r) \Theta(\theta) \Phi(\phi)$

See also: https://www.youtube.com/watch?v=Gu8VS4BN98o

**where R are radial functions and theta (θ) and phi (ϕ) are spherical harmonic terms. The generalized family of solutions is** (family = Also known as a group, a column of elements in the periodic table that share similar reactivity due to their similar valence shell electron configuration):

> $\psi_{n, l, m}(r, \theta, \phi)=\sqrt{\left(\frac{2}{n a_{0}}\right)^{3} \frac{(n-l-1) !}{2 n[(n+l) !]^{3}}} e^{-r / n a_{0}}\left(\frac{2 r}{n a_{0}}\right)^{l} L_{n-l-1}^{2 l+1}\left(\frac{2 r}{n a_{0}}\right) \cdot Y_{l}^{m}(\theta, \phi)$

where $a_0$ is the Bohr radius, L are the generalized Laguerre polynomials, and n, l, and m are the principal, azimuthal, and magnetic quantum numbers, respectively.

**Example: The atomic orbital wavefunctions of a hydrogen atom (Verschiedene Orbitale des Wasserstoffs)** - [Image Source](https://en.wikipedia.org/wiki/Hydrogen_atom#Visualizing_the_hydrogen_electron_orbitals
)

* *The principal quantum number (n) is at the right of each row (which electron shell we are talking about.  $n_1$  is lowest in energy)*,

* *the azimuthal quantum number (ℓ) is denoted by letter at top of each column (which subshell)*

* *Note that there are other shapes that the cloud of the hydrogen atoms could take depending on the energy level and quantum state of the electron (right)*

![ggg](https://raw.githubusercontent.com/deltorobarba/repo/master/sciences_1207.png)


<font color="red">**IMPORTANT: The energy of an electron in a particular atom isn't just related to what shell it's in. It also depends on this interaction between orbital angular momentum and spin angular momentum.**

**Any eigenstate of the electron in the hydrogen atom is described fully by four quantum numbers**

![ggg](https://raw.githubusercontent.com/deltorobarba/repo/master/sciences_1208.png)


**Hydrogen spectral series**

* The [emission spectrum](https://en.m.wikipedia.org/wiki/Emission_spectrum) of atomic hydrogen has been divided into a number of spectral series, with wavelengths given by the [Rydberg formula](https://en.m.wikipedia.org/wiki/Rydberg_formula). These observed spectral lines are due to the electron making transitions between two energy levels in an atom.

* The classification of the series by the Rydberg formula was important in the development of quantum mechanics. The spectral series are important in [astronomical spectroscopy](https://en.m.wikipedia.org/wiki/Astronomical_spectroscopy) for detecting the presence of hydrogen and calculating [red shifts](https://en.m.wikipedia.org/wiki/Redshift).

More info: https://en.m.wikipedia.org/wiki/Hydrogen_spectral_series

![ggg](https://raw.githubusercontent.com/deltorobarba/repo/master/sciences_1206.png)

Images of Hydrogen atoms taen by researchers (indirectly, because measuring a state would make it collapse into one state, you can't see the superposition cloud):

![ggg](https://raw.githubusercontent.com/deltorobarba/repo/master/quantum_169.png)

###### *Example 2: Nicht-Relativistisches Wasserstoffproblem mit 2 Atomen*

**Quantum Mechanics in the Formation of the Hydrogen Gas Molecule**

* goal is always: adopt the lowest energy state (ground state)

* the shape of the electron cloud around the nucleus is determined by the Schroedinger equation, which contains a wavefunction

* wavefunction of the hydrogen atom represents the probable results if we were to measure the its properties (hydrogen atom will be in its ground state)

* if a secind hydrogen atom is introduced, there are several effects happening.

> <font color="red">**The total energy of a two atom system determines bonding**

* to calculate the lowest energy of a two atom system =, or molecule of hydrogen, we have to take into account the following:

    * 1) The kinetic energy of each atom

    * 2) The potential energy between the two protons

    * 3) The potential energy between the two electrons

    * 4) The potential energy between each electron and each proton


> <font color="red">**The sum of the possible outcomes of kinetic and potential energy of this entire system in quantum mechanics is referred to the Hamiltonian $\mathcal{H}$ (to calculate the lowest total energy of a two atom system)**

* Hamiltonian Operator a set of operations **concerning interactions that dictate the state of the system** = Hamiltonian Operator typically describes the total energy of a system or particle.

The Hamiltonian is an (energy) operator corresponding to the energy of a system (and E is the energy eigenvalue):

> $\hat{H}|\psi\rangle=E |\psi\rangle$

* *Once you plug H into the time independent Schroedinger equation you can solve to get possible values for energy.*

<font color="blue">**The Hamiltonian of a 2 atom hydrogen system (interactions between both considerung kinetic and potential energy):**

> $H=-\frac{h^{2}}{2 m_{e}}\left(\nabla_{i}^{2}+\nabla_{2}^{2}\right)-\frac{h^{2}}{2 m_{p}}\left(\nabla_{A}^{2}+\nabla_{2 B}^{2}\right)-\frac{e^{2}}{4 * \pi * \varepsilon_{0} * R_{1 A}}-\frac{e^{2}}{4 * \pi * \varepsilon_{0} * T_{1 B}}-\frac{c^{2}}{4 * \pi * \varepsilon_{0} * r_{2 A}}$
$-\frac{\mathrm{c}^{2}}{4 * \pi+\varepsilon_{0} * r_{2 B}}+\frac{\mathrm{e}^{2}}{4 * \pi * \varepsilon_{0} * \mathrm{r}_{\mathrm{AB}}}+\frac{\mathrm{c}^{2}}{4 * \pi+\varepsilon_{0} * \mathrm{r}_{12}}$

This is not easy to solve but it can be represented by the following graph:

![ggg](https://raw.githubusercontent.com/deltorobarba/repo/master/quantum_170.png)

https://physics.stackexchange.com/questions/98358/finding-the-energy-eigenvalues-of-hydrogen-using-wkb-approach

And 2 hydrogen atoms combine naturally to $H2$ rather than floating around by themselves because their combined energy is lower than for each on individually: (the sharing of electrons by two atoms is called "covalent bond")

![ggg](https://raw.githubusercontent.com/deltorobarba/repo/master/quantum_171.png)

###### *Example 3: Relativistisches Wasserstoffproblem*

**Relativistisches Wasserstoffproblem**

* [Relativistisches Wasserstoffproblem](https://de.wikipedia.org/wiki/Relativistisches_Wasserstoffproblem)

* Das relativistische Wasserstoffproblem ist die Verallgemeinerung des Wasserstoffproblems der nichtrelativistischen Quantenmechanik auf die relativistische Quantenmechanik.

* **Anstatt der Lösung der Schrödinger-Gleichung für das Coulomb-Potential einer Punktladung verlangt die relativistische Verallgemeinerung die Lösung der Dirac-Gleichung**.

* Die Lösung des relativistischen Wasserstoffproblems erklärt vollständig die Feinstruktur des Wasserstoff-Spektrums.

##### <font color="blue">*Molekülorbital-Theorie*

###### *Molecular Orbital Theory*

Die [Molekülorbitaltheorie](https://de.m.wikipedia.org/wiki/Molek%C3%BClorbitaltheorie) (kurz MO-Theorie) bzw. [Molecular orbital theory](https://en.m.wikipedia.org/wiki/Molecular_orbital_theory) ist neben der Valenzbindungstheorie (VB-Theorie) eine von zwei komplementären Möglichkeiten, <font color="blue">**die Elektronenstruktur von Molekülen zu beschreiben**.</font>

> **Beim MO-Verfahren werden über das Molekül delokalisierte Molekülorbitale durch eine Linearkombination ([LCAO-Methode](https://de.m.wikipedia.org/wiki/LCAO-Methode)) der Atomorbitale aller Atome eines Moleküls gebildet.**

* Dabei wird zwischen bindenden und antibindenden Molekülorbitalen unterschieden.

* Das Verfahren wird heute für die meisten quantenchemischen Rechnungen verwendet.

* Zur näherungsweisen Bestimmung der Molekülorbitale dient das [**Rayleigh-Ritz-Prinzip**](https://www.chemie-schule.de/KnowHow/Molekülorbitaltheorie) (Eigenvalue or Expectation Value of the Hamilton operator)

Eine zeitunabhängige $n$-Elektronen-Wellenfunktion hat, wenn der Spin nicht beachtet wird, die allgemeine Form $\Psi\left(\vec{r}_{1}, \ldots \vec{r}_{n}\right)$.

* Das Produkt mit der komplex konjugierten Funktion $\Psi^{*}\left(\vec{r}_{1}, \ldots \vec{r}_{n}\right) \cdot \Psi\left(\vec{r}_{1}, \ldots \vec{r}_{n}\right)$ ist reell und gibt die Wahrscheinlichkeitsdichte wieder, das erste Elektron an der Stelle $\vec{r}_{1}$, das 2-te an der Stelle $\vec{r}_{2}$ usw. zu finden.


* <font color="blue">**Die exakte Wellenfunktion lässt sich für Mehrelektronensysteme analytisch nicht finden. Eine zielführende Vereinfachung ist es, die Elektronen als stochastisch unabhängig anzusehen**</font>.

* Mathematisch bedeutet das, einen Produktansatz zu verwenden $\Psi\left(\vec{r}_{1}, \ldots \vec{r}_{n}\right)=\prod_{i=1}^{n} \Psi_{i}\left(\vec{r}_{i}\right)$.

> Dieser Ansatz ist auch als **Hartree-Produkt** bekannt. <font color="blue">**Die $\Psi_{i}\left(\vec{r}_{i}\right)$ geben die Aufenthaltsbereiche für die einzelnen Elektronen an**</font>. Sie werden als Molekülorbitale bezeichnet.

* Um das Pauli-Prinzip einzuhalten, wird die Wellenfunktion als [Slater-Determinante](https://de.m.wikipedia.org/wiki/Slater-Determinante) (ein antisymmetrisierter Produktansatz) angesetzt. Dabei ist gewährleistet, dass die Wellenfunktion [antisymmetrisch](https://de.m.wikipedia.org/wiki/Antisymmetrische_Funktion) unter Vertauschung zweier ununterscheidbarer Elektronen ist, wie es für Fermionen gelten muss.

* Abgesehen davon, **dass MO-Schemata eine vereinfachte, modellhafte Beschreibung darstellen**, ist zu beachten, dass sie in der MO-Theorie nicht eindeutig bestimmt sind.

  * Entscheidend ist nur die Summe über alle quadrierten Orbitale, die Elektronendichte (das ist auch die Grundlage für die [Dichtefunktionaltheorie DFT](https://de.m.wikipedia.org/wiki/Dichtefunktionaltheorie_(Quantenphysik))). Mathematisch gesprochen ist die Wellenfunktion gegenüber einer unitären Lineartransformation invariant.

  * So lassen sich neben den oft als kanonische MOs bezeichneten Orbitale verschieden andere Darstellungen finden, z. B. Localized Molecular Orbitals (LMOs).

**Molecular orbital theory is the theory that deals with the definition and computation of molecular orbitals (MOs).**

* **The purpose of molecular orbital theory is to obtain approximate solutions of the time-independent Schrödinger equation of molecules—the eigenvalue equation of a molecular Hamiltonian (quantum mechanical energy operator).**

* Solutions of the Schrödinger equation (wave functions) open the door to all kinds of molecular properties of chemical interest. Molecular orbitals are wave functions describing the quantum mechanical "motion" of one electron in the electrostatic field of all the nuclei of a molecule.

* In many-electron molecules the electrostatic field due to the positive nuclei is screened (i.e., weakened) by an average electrostatic field due to the negative electrons.

> The use of **MOs has led, instead to the molecular orbital (MO) method** that has evolved from the one-determinant approximation (one-electron / Density Functional Theory) toward procedures based on many-determinant wavefunctions (many-electron / Hartree–Fock Method).

A typical feature of the MO method is the use of orthogonal orbitals that greatly simplify the expression of the matrix elements required for energy calculations and make this method much easier to work with than the VB method.

https://www.theochem.ru.nl/~pwormer/Knowino/knowino.org/wiki/Molecular_orbital_theory.html

*Molecular Orbitals*

https://en.wikipedia.org/wiki/Quantum_chemistry#Electronic_structure

> https://en.m.wikipedia.org/wiki/Molecular_orbital

> https://en.m.wikipedia.org/wiki/Molecular_orbital_theory

https://de.m.wikipedia.org/wiki/Molekülorbitaltheorie

Toward a quantum computer that calculates molecular energy: https://phys.org/news/2022-03-quantum-molecular-energy.html

Die [**Molekülphysik**](https://de.m.wikipedia.org/wiki/Molekülphysik) ist ein Teilgebiet der Physik, das sich mit der Untersuchung der chemischen Struktur (z. B. Bindungslängen und -winkel), der Eigenschaften (z. B. Energieniveaus) und des Verhaltens (z. B. Reaktionsprozesse) von Molekülen beschäftigt. Daher kann die Molekülphysik auch als Grenzgebiet zwischen Physik und Chemie aufgefasst werden. Untersuchungsobjekte und -methoden entsprechen weitgehend denen der physikalischen Chemie.

Grundlagen sind die Erkenntnisse der Atomphysik und Quantenmechanik. Ein wichtiges Modell zu Berechnung der Moleküleigenschaften ist die Born-Oppenheimer-Näherung. **Das in der Atomphysik wichtige Orbitalmodell wird in der Molekülphysik um [Molekülorbitale](https://en.m.wikipedia.org/wiki/Molecular_orbital) erweitert**.

* a [molecular orbital](https://en.m.wikipedia.org/wiki/Molecular_orbital) is a mathematical function **describing the location and wave-like behavior of an electron in a molecule**.

* This function can be used to calculate chemical and physical properties such as the probability of finding an electron in any specific region.

* The terms atomic orbital and molecular orbital were introduced by Robert S. Mulliken in 1932 to mean one-electron orbital wave functions. At an elementary level, they are **used to describe the region of space in which a function has a significant amplitude**.

* In an isolated atom, the orbital electrons' location is determined by functions called [atomic orbitals](https://en.m.wikipedia.org/wiki/Atomic_orbital). When multiple atoms combine chemically into a molecule, the electrons' locations are determined by the molecule as a whole, so **the atomic orbitals combine to form molecular orbitals**. The electrons from the constituent atoms occupy the molecular orbitals.

* Mathematically, **molecular orbitals are an approximate solution to the Schrödinger equation for the electrons in the field of the molecule's [atomic nuclei](https://en.m.wikipedia.org/wiki/Atomic_nucleus)**.

* They are usually constructed by combining [atomic orbitals](https://en.m.wikipedia.org/wiki/Atomic_orbital) or [hybrid orbitals](https://en.m.wikipedia.org/wiki/Orbital_hybridisation) from each atom of the molecule, or other molecular orbitals from groups of atoms. **They can be quantitatively calculated using the Hartree–Fock or self-consistent field (SCF) methods**.




Molecular orbitals are of three types:

* bonding orbitals which have an energy lower than the energy of the atomic orbitals which formed them, and thus promote the chemical bonds which hold the molecule together;

* antibonding orbitals which have an energy higher than the energy of their constituent atomic orbitals, and so oppose the bonding of the molecule, and

* non-bonding orbitals which have the same energy as their constituent atomic orbitals and thus have no effect on the bonding of the molecule.




A molecular orbital (MO) can be used to **represent the regions in a molecule where an electron occupying that orbital is likely to be found**.

* Molecular orbitals are approximate solutions to the Schrödinger equation for the electrons in the electric field of the molecule's atomic nuclei.

* However calculating the orbitals directly from this equation is far too intractable a problem. Instead **they are obtained from the combination of atomic orbitals, which predict the location of an electron in an atom**.

* A molecular orbital can specify the electron configuration of a molecule:

  * the spatial distribution and energy of one (or one pair of) electron(s).

  * Most commonly a MO is represented as a linear combination of atomic orbitals (the LCAO-MO method), especially in qualitative or very approximate usage. They are invaluable in providing a simple model of bonding in molecules, understood through molecular orbital theory. Most present-day methods in computational chemistry begin by calculating the MOs of the system.

  * **A molecular orbital describes the behavior of one electron in the electric field generated by the nuclei and some average distribution of the other electrons**. In the case of two electrons occupying the same orbital, the Pauli principle demands that they have opposite spin. Necessarily this is an approximation, and highly accurate descriptions of the molecular electronic wave function do not have orbitals (see configuration interaction).



See also example: https://en.wikipedia.org/wiki/Trigonal_planar_molecular_geometry

<font color="blue">*Atomic Orbitals and Molecule Orbitals*

> **Methods proposed for obtaining approximate solutions of the time-independent Schrödinger equation for electrons in a molecule.**

**Since working with complete sets is usually impossible, the quality of any approximate wavefunction depends, in a crucial way, on the choice of the spin-orbitals used in the determinants**.

> The best spin-orbitals are those that yield an accurate wavefunction with the smallest number of determinants.


Two main types of orbitals have been proposed:

* **atomic orbitals (AOs)**, centered on the nuclei of the molecule. Here the **Valence Bond Theor**y is used.

* **molecular orbitals (MOs)** that extend over the whole molecule, being usually constructed as linear combinations of atomic orbitals (LCAO). Here with the **Molecular Orbital Theory (LCAO-MO)** is used.


https://www.youtube.com/watch?v=zH_qF6oG82U


https://www.youtube.com/watch?v=QGyfGCZT110

http://newton.ex.ac.uk/research/qsystems/people/jenkins/mbody/mbody3.html

<font color="blue">*Quantum Many-Body Physics*</font>

![geometry](https://raw.githubusercontent.com/deltorobarba/repo/master/quantum_011.png)

https://www.youtube.com/watch?v=MmpOva4-XPg&list=PLBn8lN0Dcvpla6a6omBni1rjyQJ4CssTP&index=51

* The Schroedinger Equation gives us the time evolution of a quantum system, it includes the wave function

* the wave function is the quantum state of the particle or system of particles

* its time evolution is governed by the Schroedinger Equation

* and we are typically interested in either how a qubit, electron or atom behaves at some later time (prediction ability) - this is very difficult:

* one body problem: hydrogen atom with 1 electron around 1 proton, and the electron feeling the influence of the proton. - time evolution of this system easy to solve, almost exactly in closed form.

* N=2 for helium atom: nucleus of 2 protons and 2 separate electrons orbiting around. No known closed form solution for equations of motion.

* one logical qubit is made of several hundreds to several thousands of physical qubits

* so the quantum many body problem that we want to solve for a typical scenario of a quantum computer (implementing Shor's algorithm for a 2048-bit RSA) is somewhere at N > 4000 logical qubits, which is about 5 mio physical qubits.

* this is the essence of the quantum many body problem as applied to essential quantum computers.

**Many-Body Behavior of Spins**

![geometry](https://raw.githubusercontent.com/deltorobarba/repo/master/quantum_012.png)

https://www.youtube.com/watch?v=7HayIq9OR2E&list=PLBn8lN0Dcvpla6a6omBni1rjyQJ4CssTP&index=52

* lets focus the many body physics on the simplest quantum system which is the quantum number associated with spin or angular momentum for example of the electron.

* this neglects all other complications due to position, momentum, charge, spin orbit coupling, hyperfine splitting. So we focus on on spin.

* spin 1/2 gives us an effective two state quantum system, like up or down

* lets consider only n spins that just occur between Z components of their angular momentum

* hamiltonian covers all the time evolution in the schroedinger equation

* Ising model: sort of simplest model of interacting spins. Typically used not only in physical processes like quantum magnetism, also used to encode optimization problems. Look for lowest energy state governed but this equation.

* you could look into all possible spin configurations, and calculate the energy according to this hamiltonian. But this would be a $2^n$ states.

* in Boltzmann is the partition function: it runs through all possible configurations and is weighed by the Boltzmann weight.

* d wave.. facilitate optimization

* diagonalization to get eigenvalues, but escalates also quickly, max N + 20-40: we can do better than this only with a quantum computer (=quantum supremacy)



**Strategies to Solve the Many-Body Problem**

![geometry](https://raw.githubusercontent.com/deltorobarba/repo/master/quantum_013.png)

https://www.youtube.com/watch?v=uTCeQHzQMdc

*Elektronischer Übergang (Quantensprung)*

Kupfer ist eines der wenigen metallischen Elemente mit einer anderen natürlichen Farbe als Grau oder Silber. Reine Kupferoberflächen sind lachsrot und verfärben sich an der Luft rotbraun. Die charakteristische Farbe von Kupfer ergibt sich aus Interband-Übergängen von im d-Atomorbital vorhandenen Elektronen.

[Interband-Übergängen](https://de.m.wikipedia.org/wiki/Elektronischer_Übergang) (Elektronischer Übergang)

https://de.m.wikipedia.org/wiki/Quantenchemie

https://de.m.wikipedia.org/wiki/Chemische_Graphentheorie

https://en.m.wikipedia.org/wiki/Molecular_graph

https://en.m.wikipedia.org/wiki/HOMO_and_LUMO

Hybdrid Orbitals Explains: https://youtu.be/vHXViZTxLXo

https://youtu.be/pdJeQUd2g_4

https://youtu.be/Ewf7RlVNBSA

https://youtu.be/cPDptc0wUYI

https://youtu.be/UDmrRWeFWwU

https://youtu.be/a8LF7JEb0IA

***VSEPR Theory***

> **The fundamental difference is that valence bond theory describes bonds as local structures, while molecular orbital treats them as more diffuse and spread out over the whole molecule.**

* In VB theory a bond is described by the two adjacent orbitals that go into making the bond. These orbitals are then populated relative to the atoms involved in the relevant bond(s).

* n MO theory all of the contrribtuing orbitals are considered at once and combined, giving orbitals for the whole molecule, rather than just between two atoms. It basically adds up all of the obitals of the atoms in the molecule to make a new set. These orbitals are then poluated to give the over all electronic structure in the molecule.

* In VB theory you could describe methane as consisting of 4 separate C-H sigma bonds. In MO theory you would have 4 orbitals that contribute to those bonds, but they would be spread out over the whole molecule and not pertain to any specific C-H bond.

**What's the difference between VSEPR and the Molecular Orbital Theory?**

* VSEPR is a way for one to determine the shape and the way that certain atoms bond to each other and their angles. MOT is a way to describe the valence electrons of those atoms who are bonded to each other.

* VESPR determines the shape of a molecule based on the number of electron pairs surrounding the central atom. MOT is also a way to determine molecular structure, but the electrons are not stuck in one bond, rather, they are spread out throughout the entire molecule. This is a more accurate representation of the electrons in a molecule.


<font color="blue">*VSEPR Theory*

https://en.m.wikipedia.org/wiki/VSEPR_theory

> Bestimmung der Raumstruktur von Molekülen (VSEPR-Modell)

> VSEPR is a subset of valence bond. It's just a way of figuring out molecular geometries.

* The best place to start is probably VSEPR theory. VSEPR theory is basically concerned with the geometry taken by the bonds in a certain molecule as a function of basic electron-electron repulsion; bonds between atoms (which constitute the sharing of an electron pair basically at this stage) want to be as far from one another as possible. They also want maximum distance from lone pairs and so you get the basic shapes of small molecules.

> BUT, VSEPR theory just places electron pairs in "bonds" or "lone pairs" - it doesn't take into account any real notion of quantum function or in other words, it does not concern itself with the nature of the orbitals that are associated with each atom of the molecule, such as methane having a carbon and 4 hydrogens.

* This is where valence bond theory comes in. After Lewis and Kekule, and with the contributions of quantum theory we knew that electrons were localised discreetly between nuclei in orbitals and it was the orbital interactions (overlaps) which gave rise to chemical bonds. So, atomic orbitals being 1s, 2s 2p x/y/z and molecular orbitals being pi and sigma bonds. Each orbital therefore was calculated to have it's shape ( from electron density calculations, but I'm just an organic chem grad so someone could give a much better explanation about the equations etc) and using the same ideas of electron repulsion etc. However, having orbital shapes allows a better grasp of reactivity, geometry and bond energy of molecules. Like our methane example the contribution of Pauling took into account hybridisation, the mixing of atomic orbitals to account for carbons four bonds, giving 4 sigma molecular orbitals from their overlap and the tetrahedral geometry being a result of equidistance from the 4 orbitals... And each C-H bond is the same energy

* ...but they are not in fact equal! It turns out that the ESCA spectrum of methane shows 2 peaks corresponding to the removal of electrons from the 4 C-H bonds. BUT VB theory predicts these should be equal in energy
This is because of Molecular Orbital Theory (MO). What this model contributes is that in fact, electrons are NOT localised to just bonds between distinct atoms. The orbitals of all the atoms in the molecule are combined in accordance to the sum of all the atomic orbitals to each molecular orbital So, for methane again, we have 4 1s orbitals from methane and the 2s, 2p x/y/z orbitals from carbon. This gives 8 possible MO's (because of the symmetry requirement for orbitals) but crucially, this predicts that one molecular orbital will have NO nodes arising from the 2s and 4 1s all bonding interaction, and 3 MO's which each have 1 node because of the contribution of a p orbital. Giving two different C-H bond energies

Das VSEPR-Modell lässt sich auf Moleküle anwenden, bei denen die an das Zentralatom gebundenen Reste (Atome oder Atomgruppen) nicht allzu groß werden und keine spezifischen Wechselwirkungen aufeinander ausüben.

* Nicht oder nur eingeschränkt anwendbar ist sie auf Übergangsmetall­verbindungen. Vielfach stimmen jedoch auch bei einfachen Molekülen die Bindungswinkel nicht mit dem Modell überein. **Für Verbindungen mit delokalisierten Elektronen kann die Anwendung des Modells ebenfalls mit Schwierigkeiten verbunden sein, hier ist die Hinzuziehung der Molekülorbitaltheorie notwendig.**

* https://de.m.wikipedia.org/wiki/VSEPR-Modell#Grenzen_der_Anwendbarkeit

*Bestimmung der Raumstruktur von Molekülen mit dem Elektronenpaarabstoßungsmodell*



Das Elektronenpaarabstoßungsmodell ist ein sehr einfaches Hilfsmittel zur Bestimmung der räumlichen Struktur von Molekülverbindungen der allgemeinen Zusammensetzung ABx mit x = 1 bis 6. Dabei geht man zweckmäßigerweise in folgenden Schritten vor:


1.	Überprüfen des Bindungstyps (kovalente Bindung).
2.	Aufstellen der Lewis-Formel mit allen Elektronenpaaren.
3.	Bestimmen der Anzahl der gebundenen Atome (B) und der freien Elektronenpaare (E) am Zentralatom.
4.	Ermitteln des Molekültyps und der dazugehörigen Raumstruktur.
5.	Abschätzen der Deformation der Struktur durch freie Elektronenpaare und unterschiedlich gebundene Atome.

Das Elektronenpaarabstoßungsmodell ist nur auf kovalente Verbindungen, nicht auf Ionen- und nur eingeschränkt auf Komplexverbindungen anwendbar.

https://www.lernhelfer.de/schuelerlexikon/chemie-abitur/artikel/bestimmung-der-raumstruktur-von-molekuelen-mit-dem#

###### *Hamiltonians*

***Molecular Hamiltonian***

Mehr-Elektronen-Wellenfunktionen (Molecular Hamiltonians) = Electronic Wavefunction of Many Particle Systems

**One Electron Model**

* the Schrödinger equation is impossible for us to solve for all but the very simplest of systems: You can do it exactly for the single electron in a hydrogen atom

> **you input the potential-energy function created by the electrostatic attraction of the electron to the atomic nucleus, and use the resulting wavefunction to calculate the allowed electron energy levels, spatial distributions and so on**.

**Many Electron Model**

* But once you try to do that for objects with more than one electron, you’re stuck. Two electrons repel one another, and so you can’t figure out the potential experienced by one until you know the spatial distribution of the other.

* The usual solution is to approximate: to pretend that the wavefunction of a two-electron system such as a helium atom or hydrogen molecule can be approximated as a product of two one-electron wavefunctions. This approximation of factorizing the wavefunction gets ever cruder as the object has more electrons.

* Alternatively, you can crunch out a wavefunction by trial and error: making a guess at what it might be and then gradually improving the guess by iterating the Schrödinger equation until you get to a ‘self-consistent’ solution that doesn’t change significantly on subsequent iterations.

* <font color="red">This approach of finding a self-consistent numerical solution for the wavefunction dates back to the famous [**Hartree–Fock method**](https://de.m.wikipedia.org/wiki/Hartree-Fock-Methode) of quantum chemistry in the 1930s.

> Im Orbitalmodell für Atome mit mehreren Elektronen nimmt man an, dass die Elektronen sich unter Berücksichtigung des **Pauli-Prinzips** auf die Orbitale verteilen. Ein solcher Zustand heißt **Elektronenkonfiguration** und stellt oft eine brauchbare Näherung für die Struktur der Atomhülle dar, obwohl diese durch zusätzliche Elektronenkorrelationen noch komplizierter ist.

> Zur Beschreibung von Elektronen in Molekülen werden [Molekülorbitale](https://de.m.wikipedia.org/wiki/Molekülorbitaltheorie) als Linearkombination von Atomorbitalen gebildet. Elektronen in Festkörpern werden durch Orbitale beschrieben, die die Form von [Blochwellenfunktionen](https://de.m.wikipedia.org/wiki/Blochwellenfunktion) haben.

* Die Interpretation von Orbitalen als Wellenfunktionen je eines Elektrons ist nur bei Einzelelektronensystemen eindeutig möglich. Eine Wellenfunktion für N Elektronen kann dann konstruiert werden, indem man N Orbitale in eine [Slater-Determinante](https://de.m.wikipedia.org/wiki/Slater-Determinante) einsetzt.

* Dies garantiert die für [Fermionen](https://de.m.wikipedia.org/wiki/Fermion) notwendige Antisymmetrie der gesamten Wellenfunktion, kann aber darüber hinaus gehende [Elektronenkorrelationen](https://de.m.wikipedia.org/wiki/Quantenverschr%C3%A4nkung) nicht darstellen.

* Um auch die Elektron-Elektron-Wechselwirkung näherungsweise zu berücksichtigen, können die Orbitale durch [Hartree-Fock-](https://de.m.wikipedia.org/wiki/Hartree-Fock-Methode), Kohn-Sham-Rechnungen (siehe: [DFT / Dichtefunktionaltheorie](https://de.m.wikipedia.org/wiki/Dichtefunktionaltheorie_(Quantenphysik)) in der Quantenphysik) oder MCSCF-Rechnungen (MCSCF: Multiconfiguration Self Consistent Field) bestimmt werden.

* Doch stets bleibt gültig, dass anders gewählte Orbitale, wenn sie linear unabhängige [Linearkombinationen](https://de.m.wikipedia.org/wiki/Linearkombination) der ursprünglichen sind, mathematisch die gleiche Slater-Determinante ergeben, sodass man aus einer gegebenen Mehrteilchen-Wellenfunktion nicht eindeutig zurückschließen kann, welches die einzelnen besetzten Orbitale sind.

> The main problem of quantum many-body physics is the fact that the Hilbert space grows exponentially with size. For example, a spin-1/2 chain of length L has $2^L$ degrees of freedom.

**Approaches to Build molecular Hamiltonians**

* Mean-field theory and extensions (e.g. **Hartree–Fock**, Random phase approximation)

* Dynamical mean field theory

* Many-body perturbation theory and Green's function-based methods

* **Configuration interaction**

* Coupled cluster

* Various Monte-Carlo approaches

* **Density functional theory**

* Lattice gauge theory

* Matrix product state

* Neural network quantum states

*Molecular Hamiltonian*

![ggg](https://raw.githubusercontent.com/deltorobarba/repo/master/sciences_0862.png)

https://www.youtube.com/watch?v=B3D_WZ3NbbM&list=PLm8ZSArAXicIijiVIx0yfk2ZOK-16ycji

**Objective**: Looking for a solution for the time-independend Schrödinger-Equation (calculations of molecular wavefunctions):

> $\hat{H}_{0} \Psi(\{\vec{r}\},\{\vec{R}\})=E \Psi(\{\vec{r}\},\{\vec{R}\})$

**Challenge**: Unfortunately, exact analytical solutions of this equation cannot be obtained even for simple molecule.

* **Molecule**: viewed as a system of nuclei and electrons interacting via coulomb forces and internal electromagnetic forces, the latter associated with the intrinsic magnetic moments of the particles.

* **Molecular energetics**: studies bound stationary states of molecules looking, in particular, at electronic structure and properties. The methods used for this are quantum mechanical.

**Solution**: Use methods to approximate electronic wavefunctions (spin orbitals) in molecules (covalent bonds).



> **The purpose of molecular orbital theory is to obtain approximate solutions of the time-independent Schrödinger equation of molecules—the eigenvalue equation of a molecular Hamiltonian** (quantum mechanical energy operator) / calculations of molecular wavefunctions.

> **= Approximate Orbital Energies and Electronic Wavefunctions**

* Solutions of the Schrödinger equation (wave functions) open the door to all kinds of molecular properties of chemical interest.

https://www.theochem.ru.nl/~pwormer/Knowino/knowino.org/wiki/Molecular_Hamiltonian.html

https://www.theochem.ru.nl/~pwormer/Knowino/knowino.org/wiki/Molecular_orbital_theory.html

* The Schrödinger equation can then be solved (usually approximately) for the electronic wavefunction for fixed nuclear distances.

* The plot of electronic energy as a function of nuclear distances is called the potential energy surface.
The P.E. surface represents the potential within which the nuclei move. Motion of nuclei on this surface corresponds to molecular vibration. (Rotation is considered as a special case of vibration in which internuclear distances are fixed.)

*What is the Molecular Hamiltonian*

* The [molecular Hamiltonian](https://en.m.wikipedia.org/wiki/Molecular_Hamiltonian) is the **Hamiltonian operator representing the energy of the electrons and nuclei in a molecule**.

* This operator and the associated Schrödinger equation play a central role in computational chemistry and physics for **computing properties of molecules and aggregates of molecules**, such as thermal conductivity, specific heat, electrical conductivity, optical, and magnetic properties, and reactivity.

* To a very good approximation, **electrons are considered point charges and and nuclei are point masses**.

<font color="blue">*Force field (chemistry)*

https://en.m.wikipedia.org/wiki/Force_field_(chemistry)

> The ultimate goal of computational quantum chemistry is to unravel the quantum effects that determine the structure and properties of molecules. Reaching this goal is challenging since the characteristic energies associated with these effects, e.g., the electronic correlation energy, are typically a tiny fraction of the total energy of the molecule.



https://pennylane.ai/qml/demos/tutorial_quantum_chemistry.html

> Objective: Building molecular Hamiltonians

In principle, it’s easy to write down a complete quantum description of matter.

* You just have to set out the Schrödinger equation for all of its constituent fundamental particles.

* This equation, lets you figure out the mathematical entity called the wavefunction from which all observable properties of the quantum system can be predicted.

**Molecular Dynamics and Limitations of current Quantum Chemistry**

* It might not be ideal, but there is now a veritable industry devoted to finding good, efficient algorithms for calculating approximate solutions to the Schrödinger equation.

* **Quantum-chemical methods can often supply predictions of the properties of molecules – such as bond energies and vibration frequencies – with great accuracy**. But the more accurate you want to be, the more costly the computation.

* That’s why it has been feasible to use fully (albeit still approximate) quantum-mechanical calculations only for rather small chemical systems: molecules with a few atoms. **If you want to simulate an entire protein, you need cruder methods such as [molecular dynamics](https://en.wikipedia.org/wiki/Molecular_dynamics)**, which are based on Newtonian mechanics and at best just build in quantum effects in a rough, heuristic manner.


* [Molecular dynamics](https://en.m.wikipedia.org/wiki/Molecular_dynamics) (MD) is a computer simulation method for analyzing the physical movements of atoms and molecules.

* The atoms and molecules are allowed to interact for a fixed period of time, giving a view of the dynamic "evolution" of the system.

* In the most common version, the trajectories of atoms and molecules are determined by numerically solving Newton's equations of motion for a system of interacting particles, where forces between the particles and their potential energies are often calculated using interatomic potentials or molecular mechanics force fields.

[Moleküldynamik oder Molekulardynamik](https://de.m.wikipedia.org/wiki/Molekulardynamik-Simulation) (MD) bezeichnet Computersimulationen in der molekularen Modellierung, bei denen Wechselwirkungen zwischen Atomen und Molekülen und deren sich daraus ergebende räumliche Bewegungen iterativ berechnet und dargestellt werden. Bei der Modellierung von komplexen Systemen mit einer Vielzahl an beteiligten Atomen werden hauptsächlich Kraftfelder oder semiempirische Methoden verwendet, da der Rechenaufwand zur Anwendung von quantenmechanischen Verfahren (Ab-initio-Methoden) hierbei zu groß wäre.

* Durch die Verwendung der **Dichtefunktionaltheorie** und der stetig steigende verfügbare Rechenleistung werden allerdings zunehmend Ab-initio Molekulardynamik Simulationen (Car-Parrinello-Methode) auch für mittelgroße Systeme möglich.

* Der Begriff Moleküldynamik wird manchmal auch als Synonym für die [Diskrete-Elemente-Methode](https://de.m.wikipedia.org/wiki/Diskrete-Elemente-Methode) (DEM) gebraucht, weil die Methoden sehr ähnlich sind. Die Partikel in DEM müssen aber keine Moleküle sein.

*Molecular dynamics simulations are often used to study biophysical systems. Depicted here is a 100 ps simulation of water:*

![hhh](https://upload.wikimedia.org/wikipedia/commons/f/f4/MD_water.gif)

**Quantum Computing for Quantum Chemistry**

* In particular, a quantum bit (qubit) need not just take the values 1 and 0, but could exist in a quantum superposition of the two states. Superpositions are allowed by the Schrödinger equation: they consist of linear combinations of the wavefunctions of allowed quantum states.

> **It’s common to say that superpositions are in ‘two (or more) states at once’, although more strictly we should say that <font color="red">a quantum system in a superposition of states can be found in any of the superposed states when a measurement is made on it.</font>**

* because qubits are themselves described by quantum wavefunctions, **they can be used to represent – to simulate – quantum objects like electrons**.

* **You no longer need to make simplifying approximations: a system of qubits can represent the electrons in a complex molecule exactly**, and so a simulation on a quantum computer can supply exact predictions without a big computational overhead.

* Many problems in chemistry are optimization or minimization problems: think, for example, of trying to predict the most stable folded state of a protein from its primary sequence of amino acids. Conventional computers have made progress in tackling the protein-folding problem, but it still stretches the capabilities of supercomputers to their limit.

> Much of **computational chemistry is about finding the ground-state energies of molecules**, from which their stability and reactivity can be predicted. **You can get that from the quantum wavefunction** but that’s the very thing you can’t work out classically without a lot of approximation.


The hydrogen molecule was also the subject of a simulation on Google’s quantum computer last year (=2016), in a collaboration between Aspuru-Guzik, quantum engineer John Martinis at Google, and other groups.

* The researchers developed a technique they call a **variational quantum eigensolver**, a kind of quantum version of the iterative minimization used in simulated annealing.

* While the simulation again predicted the molecule’s energy states and bond lengths well, it has the important advantage over the earlier work that Google’s approach can be scaled to larger systems without an exponential escalation in the measures needed for error correction.



**Potential**: If we had more than 200 logical qubits, we could do things in quantum chemistry beyond standard approaches’, he says. ‘And if we had about 5000 such qubits, then the quantum computer would be transformative and disruptive to quantum chemistry.

**Challenge**: To make a ‘logical qubit’ that does its job reliably, you need a good method of error correction. For that, each logical qubit needs to be made up of many physical qubits. Reiher and colleagues estimate that the 100 or so logical qubits they need to crack the nitrogenase mechanism would correspond to around a few hundred thousand to a million physical qubits.

The problem is not quite so bad, though, because the calculations can be done in parallel – you don’t need a single million-qubit device, but just many small devices working together, or indeed one 100-qubit device working through the tasks in sequence. ‘It’s not unreasonable to think that we may have such a moderately sized machine in 5–10 years,’ Reiher says. This division of labour might be the key to doing a lot of chemistry on quantum computers, says Gambetta: to develop what computer engineers call a batching architecture for doing relatively small tasks in a systematic sequence. What’s more, he adds, canny choices in encoding the electrons’ degrees of freedom in the computer’s qubits can take advantage of certain symmetry properties that reduce the number of qubits needed.

[Source: Chemistryworld](https://www.chemistryworld.com/features/quantum-chemistry-on-quantum-computers/3007680.article)

https://en.m.wikipedia.org/wiki/Many-body_problem

<font color="blue">*Repräsentation chemischer Strukturen*

https://de.m.wikipedia.org/wiki/Chemoinformatik

https://en.wikipedia.org/wiki/Computational_chemistry

https://en.m.wikipedia.org/wiki/Quantum_chemistry

***Coulomb Hamiltonian***

* **The molecular Hamiltonian is a sum of several terms**: its major terms are the kinetic energies of the electrons and the Coulomb (electrostatic) interactions between the two kinds of charged particles:

  * <font color="red">The kinetic energy operators for each **nucleus** in the system: $\hat{T}_{n}=-\sum_{i} \frac{\hbar^{2}}{2 M_{i}} \nabla_{\mathbf{R}_{i}}^{2}$</font>

  * <font color="green">The kinetic energy operators for each electron in the system: $\hat{T}_{e}=-\sum_{i} \frac{\hbar^{2}}{2 m_{e}} \nabla_{\mathbf{r}_{i}}^{2}$</font>

  * <font color="green">The potential energy between the electrons and nuclei – the total electron-nucleus Coulombic attraction in the system: $\hat{U}_{e n}=-\sum_{i} \sum_{j} \frac{Z_{i} e^{2}}{4 \pi \varepsilon_{0}\left|\mathbf{R}_{i}-\mathbf{r}_{j}\right|}$</font>

  * <font color="green">The potential energy arising from Coulombic electron-electron repulsions: $\hat{U}_{e e}=\frac{1}{2} \sum_{i} \sum_{j \neq i} \frac{e^{2}}{4 \pi \varepsilon_{0}\left|\mathbf{r}_{i}-\mathbf{r}_{j}\right|}=\sum_{i} \sum_{j>i} \frac{e^{2}}{4 \pi \varepsilon_{0}\left|\mathbf{r}_{i}-\mathbf{r}_{j}\right|}$</font>

  * <font color="green">The potential energy arising from Coulombic nuclei-nuclei repulsions – also known as the nuclear repulsion energy. See electric potential for more details: $\hat{U}_{n n}=\frac{1}{2} \sum_{i} \sum_{j \neq i} \frac{Z_{i} Z_{j} e^{2}}{4 \pi \varepsilon_{0}\left|\mathbf{R}_{i}-\mathbf{R}_{j}\right|}=\sum_{i} \sum_{j>i} \frac{Z_{i} Z_{j} e^{2}}{4 \pi \varepsilon_{0}\left|\mathbf{R}_{i}-\mathbf{R}_{j}\right|}$</font>

* Here $M_{\mathrm{i}}$ is the mass of nucleus $i, Z_{i}$ is the atomic number of nucleus $i$, and $m_{\mathrm{e}}$ is the mass of the electron. The Laplace operator of particle $i$ is:
$\nabla_{\mathrm{r}_{i}}^{2} \equiv \nabla_{\mathrm{r}_{i}} \cdot \nabla_{\mathrm{r}_{i}}=\frac{\partial^{2}}{\partial x_{i}^{2}}+\frac{\partial^{2}}{\partial y_{i}^{2}}+\frac{\partial^{2}}{\partial z_{i}^{2}}$.

> One considers the solution of the eigenvalue (time-independent Schrödinger) equation of the Coulomb Hamiltonian.

* The **Hamiltonian that contains only the kinetic energies of electrons and nuclei, and the Coulomb interactions between them, is known as the Coulomb Hamiltonian**. (the Molecular Hamilton operator that is often referred to as the Coulomb Hamiltonian. This Hamiltonian is a sum of five terms.)

* **From it are missing a number of small terms, most of which are due to electronic and nuclear spin**.

* Although it is generally assumed that the solution of the time-independent Schrödinger equation associated with the Coulomb Hamiltonian will predict most properties of the molecule, including its shape (three-dimensional structure), calculations based on the full Coulomb Hamiltonian are very rare.

* The main reason is that its Schrödinger equation is very difficult to solve. Applications are restricted to small systems like the hydrogen molecule.

***Electronic Hamiltonian***

Clamped Nucleus Hamiltonian = Electronic Hamiltonian

* Almost all **calculations of molecular wavefunctions** are based on the **separation of the Coulomb Hamiltonian** first devised by **Born and Oppenheimer**.

* The **nuclear kinetic energy terms are omitted from the Coulomb Hamiltonian and one considers the remaining Hamiltonian as a Hamiltonian of electrons only**. The stationary nuclei enter the problem only as generators of an electric potential in which the electrons move in a quantum mechanical way.

* The clamped nucleus Hamiltonian describes the energy of the electrons in the electrostatic field of the nuclei, **where the nuclei are assumed to be stationary with respect to an inertial frame**. The form of the **electronic Hamiltonian** is

> $\hat{H}_{\mathrm{el}}=\hat{T}_{e}+\hat{U}_{e n}+\hat{U}_{e e}+\hat{U}_{n n}$

* The coordinates of electrons and nuclei are expressed with respect to a frame that moves with the nuclei, so that the nuclei are at rest with respect to this frame. The frame stays parallel to a space-fixed frame. It is an inertial frame because the nuclei are assumed not to be accelerated by external forces or torques.

* Sometimes it is stated that the nuclei are "at rest in a space-fixed frame". This statement implies that the nuclei are viewed as classical particles, because a quantum mechanical particle cannot be at rest. (It would mean that it had simultaneously zero momentum and well-defined position, which contradicts Heisenberg's uncertainty principle).

* Within this framework the molecular Hamiltonian has been simplified to the so-called **clamped nucleus Hamiltonian**, also called electronic Hamiltonian, that acts only on functions of the electronic coordinates.

***Potential Energy Surface***

* Once the Schrödinger equation of the clamped nucleus Hamiltonian has been solved for a sufficient number of constellations of the nuclei, an **appropriate eigenvalue (usually the lowest)** can be seen as a function of the nuclear coordinates, which leads to a **potential energy surface**.

* **In practical calculations the surface is usually fitted in terms of some analytic functions**.

> In the second step of the Born–Oppenheimer approximation the part of the full Coulomb Hamiltonian that depends on the electrons is replaced by the potential energy surface. This converts the total molecular Hamiltonian into another Hamiltonian that acts only on the nuclear coordinates.

* In the case of a breakdown of the Born–Oppenheimer approximation—which occurs when energies of different electronic states are close—the neighboring potential energy surfaces are needed, see [this article](https://en.m.wikipedia.org/wiki/Born–Oppenheimer_approximation) for more details on this.

* A [potential energy surface](https://en.m.wikipedia.org/wiki/Potential_energy_surface) (PES) describes the energy of a system, especially a collection of atoms, in terms of certain parameters, normally the positions of the atoms. The surface might define the energy as a function of one or more coordinates; if there is only one coordinate, the surface is called a potential energy curve or energy profile. An example is the Morse/Long-range potential.

* It is helpful to use the analogy of a landscape: for a system with two degrees of freedom (e.g. two bond lengths), the value of the energy (analogy: the height of the land) is a function of two bond lengths (analogy: the coordinates of the position on the ground).

> **The potential energy surface then corresponds to a plot of the energy of the system as a function of nuclear coordinates**.

* It can be used to theoretically explore properties of structures composed of atoms, for example, **finding the minimum energy shape of a molecule** or computing the rates of a chemical reaction.

* See also [Enery Profile](https://en.m.wikipedia.org/wiki/Energy_profile_(chemistry)) and [Energy Minimization](https://en.m.wikipedia.org/wiki/Energy_minimization)

*PES for water molecule: Shows the energy minimum corresponding to optimized molecular structure for water- O-H bond length of 0.0958nm and H-O-H bond angle of 104.5:*

![gg](https://upload.wikimedia.org/wikipedia/commons/1/1c/Potential_Energy_Surface_for_Water.png)

###### *Principles*

Franck-Condon Principle

This separation of the electronic and vibrational wavefunctions is an expression of the Born–Oppenheimer approximation and is the fundamental assumption of the Franck–Condon principle.

Hervorzuheben ist, dass die quantenmechanische Formulierung des Franck-Condon-Prinzips das Ergebnis einer Reihe von Annäherungen ist, vor allem der Dipol-Näherung sowie der Born-Oppenheimer-Näherung. Die mit ihrer Hilfe berechenbaren Intensitäten können in der Realität abweichen, beispielsweise dann, wenn zusätzlich magnetische Dipolübergänge oder elektrische Quadrupolübergänge berücksichtigt werden müssen oder die beschriebene Faktorisierung in einen elektronischen sowie Vibrations- und Spin-Anteil nicht hinreichend zulässig ist.

https://physik.cosmos-indirekt.de/Physik-Schule/Franck-Condon-Prinzip

https://en.m.wikipedia.org/wiki/Franck–Condon_principle

Classically, the [Franck–Condon principle](https://en.m.wikipedia.org/wiki/Franck–Condon_principle) is the approximation that an electronic transition is most likely to occur without changes in the positions of the nuclei in the molecular entity and its environment. The resulting state is called a Franck–Condon state, and the transition involved, a vertical transition. The quantum mechanical formulation of this principle is that the intensity of a vibronic transition is proportional to the square of the overlap integral between the vibrational wavefunctions of the two states that are involved in the transition.

*Energy diagram illustrating the Franck–Condon principle applied to the solvation of chromophores. The parabolic potential curves symbolize the interaction energy between the chromophores and the solvent. The Gaussian curves represent the distribution of this interaction energy:*

![ggg](https://upload.wikimedia.org/wikipedia/commons/thumb/c/ce/Solvation-energy-diagram.png/360px-Solvation-energy-diagram.png)

Video: [The Franck-Condon principle](https://www.youtube.com/watch?v=mf_zFHxiY28)

*Born-Oppenheimer Approximation (1927)*

This separation of the electronic and vibrational wavefunctions is an expression of the Born–Oppenheimer approximation and is the fundamental assumption of the Franck–Condon principle.

* Background: why can we use approximate methods?

* there a simplifying feature, valid for the most part of molecules that is the possibility of **separating electronic and nuclear motions as suggested by the Born-Oppenheimer (BO, 1927) approximation**.

* This possibility rests on the fact that mass ratio between electrons and nuclei is roughly $m / M \simeq 10^{-3}-10^{-5}$, while forces acting on both are of comparable magnitude.

* **It follows that the motion of the nuclei is much slower than that of the electrons and the two motions can be treated separately**. Since an energy separation ε is associated with an angular frequency ω = ε / $\hbar$ , one can estimate that during a rotation of the whole molecule, the nuclei make about 100 vibrations around their equilibrium positions and the electrons about 10.000 revolutions.

* The electronic wavefunction $\Phi^{e l}(\{\vec{r}\},\{\vec{R}\})$ which describes the motion of the electrons around a fixed nuclei configuration $\{\vec{R}\}$ depends explicitly on electronic coordinates and parametrically on nuclear coordinates, where parametric dependence means that different sets of nuclear coordinates correspond to different electronic functions.

* The electronic energy $\varepsilon(\{\vec{R}\})$, regarded as a function of the nuclear coordinates and added to the nuclear repulsion energy $V_{n n}$, is called **potential energy surface**, since it is the effective potential that governs the nuclear motion.

> **[Born–Oppenheimer Approximation](https://en.m.wikipedia.org/wiki/Born–Oppenheimer_approximation) is the best known mathematical approximation in molecular dynamics**


Die Born-Oppenheimer-Näherung oder Born-Oppenheimer-Approximation (nach Max Born und J. Robert Oppenheimer) oder adiabatische Näherung ist eine Näherung zur Vereinfachung der Schrödingergleichung von Systemen aus mehreren Teilchen.

https://physik.cosmos-indirekt.de/Physik-Schule/Born-Oppenheimer-Näherung

* it is the assumption that the **wave functions of atomic nuclei and electrons in a molecule can be treated separately**, based on the fact that the nuclei are much heavier than the electrons

* The approximation is **widely used in quantum chemistry to speed up the computation of molecular wavefunctions** and other properties for large molecules.

* There are **cases where the assumption of separable motion no longer holds**, which make the approximation lose validity (it is said to "break down"), but is then **often used as a starting point for more refined methods**.

* In molecular [spectroscopy](https://en.m.wikipedia.org/wiki/Spectroscopy), using the BO approximation means <font color="blue">**considering molecular energy as a sum of independent terms**</font>, e.g.:

> $E_{\text{total}}=E_{\text{electronic}}+E_{\text{vibrational}}+E_{\text{rotational}}+E_{\text{nuclear spin}}$

* These terms are of different orders of magnitude and the nuclear spin energy is so small that it is often omitted.

* The electronic energies $E_{\text{electronic}}$ consist of kinetic energies, interelectronic repulsions, internuclear repulsions, and electron–nuclear attractions, which are the terms typically included when computing the electronic structure of molecules.

* *Example: The benzene molecule consists of 12 nuclei and 42 electrons. The Schrödinger equation, which must be solved to obtain the energy levels and wavefunction of this molecule, is a partial differential eigenvalue equation in the three-dimensional coordinates of the nuclei and electrons, giving 3 × 12 + 3 × 42 = 36 nuclear + 126 electronic = 162 variables for the wave function. The computational complexity, i.e. the computational power required to solve an eigenvalue equation, increases faster than the square of the number of coordinates.*

**The Born-Oppenheimer Approximation states that electronic motion is so fast that
nuclei can be considered fixed**: the mass of a nuclear is so much greater than the mass of an electron, we set the nuclei as fixed, or the kinetic energy of the nuclei are zero (1 proton weighs about 1,800 times more than a single electron, and mist nuclei are even heavier than hydrogen nuclei, i.e. Carbon nucleus is 20,000 times heavier than an electron). We're going to solve of the wavefunction of the electrons at a fixed position of the nuclei.

> The **potential energy surface** then corresponds to a plot of the energy of the system as a function of nuclear coordinates.

See also video: [Quantum Chemistry 10.1 - Hydrogen Molecule Hamiltonian](https://www.youtube.com/watch?v=BBoE6NRRZ8k)

See also Video: [Quantum Chemistry 10.2 - Born-Oppenheimer Approximation](https://www.youtube.com/watch?v=UGB2psWD69s)

![ggg](https://raw.githubusercontent.com/deltorobarba/repo/master/quantum_306.png)

*Rayleigh-Ritz-Principle*

Rayleigh-Ritz-Prinzip (Determining the Eigenvalue or Expectation Value of the Hamilton operator)

**Expectation Value = What is the average value of quantum measurement outcomes?**

> **$\hat{A}\left|u_{n}\right\rangle=\lambda_{n}\left|u_{n}\right\rangle \longrightarrow P\left(\lambda_{n}\right)=\left|\left\langle u_{n} \mid \psi\right\rangle\right|^{2}=\left|c_{n}\right|^{2}$**

We can write the expectation value of $\hat{A}$ in the state $|\Psi\rangle$ as following:

> <font color ="blue">$\langle\hat{A}\rangle=\langle\psi|\hat{A}| \psi\rangle$

Zur näherungsweisen Bestimmung der Molekülorbitale dient das [**Rayleigh-Ritz-Prinzip**](https://www.chemie-schule.de/KnowHow/Molekülorbitaltheorie). Das besagt, dass, wenn man mit einer beliebigen Funktion den Erwartungswert des Hamiltonoperators bildet:

* der **Erwartungswert (expectation value) größer gleich dem Erwartungswert der Eigenfunktion des Hamiltonoperators mit dem geringsten Eigenwert ist.**

* Man muss also in einer Extremwertaufgabe die Funktion mit dem tiefsten Energieerwartungswert auswählen. Diese ist dann wahrscheinlich die beste Näherung.

**Rayleigh–Ritz-Principle & Ritz Method zur Eigenwertbestimmung bzw. Erwartungswert des Hamilton-Operators:** Um die Schrödingergleichung zu lösen, werden die Einteilchen-Wellenfunktionen so variiert, dass die aus der Gleichung entstehende Energie minimal wird. Aufgrund des **Rayleigh-Ritz-Prinzips (=Variationsprinzip zur Bestimmung des kleinsten Eigenwerts eines Eigenwertproblems)** ist diese Energie dann eine obere Grenze für die tatsächliche Energie des Systems.

> Das [Rayleigh-Ritz-Prinzip](https://de.m.wikipedia.org/wiki/Rayleigh-Ritz-Prinzip) (auch Verfahren von Ritz oder Rayleigh-Ritzsches Variationsverfahren) ist ein Variationsprinzip zur Bestimmung des kleinsten Eigenwerts eines Eigenwertproblems.

In der Quantenmechanik besagt das Prinzip, dass für die Gesamtenergie $E_{0}$ des Systems im Grundzustand $\left|\psi_{0}\right\rangle$ (also für den diesbezüglichen Erwartungswert des Hamilton-Operators $\hat{H}$ ) und für beliebige Wellenfunktionen bzw. Zustände $|\psi\rangle$ der Erwartungswert $\langle\psi|\hat{H}| \psi\rangle$ größer oder gleich (gleich im Fall der exakten Grundzustandswellenfunktion) der Grundzustandsenergie des Systems ist:

> $
E_{0} \leq\langle\hat{H}\rangle[\psi]:=\langle\psi|\hat{H}| \psi\rangle, \quad\|\psi\|=1
$

In der Regel ist der Hamilton-Operator dabei nach unten beschränkt und hat an der unteren Grenze des Spektrums einen (nicht entarteten) Eigenwert („Grundzustand"). Die ProbeWellenfunktion kann zwar von der exakten Grundzustandsfunktion erheblich abweichen, wird ihr aber umso ähnlicher, je näher die berechnete Gesamtenergie an der Grundzustandsenergie ist.

*Ritz'sche Variationsverfahren*

Das Ritz'sche Variationsverfahren wendet das Rayleigh-Ritz-Prinzip direkt an. Dazu wird eine Familie von Testvektoren, die über einen Satz von Parametern $\beta$ variiert werden, verwendet. So kann eine (nicht notwendig endliche) Menge von Vektoren $\psi_{n}$ gewählt werden und der Testvektor als Linearkombination dargestellt werden:

> $
\psi_{\vec{\beta}}=\sum_{n} \beta_{n} \psi_{n}
$

Oder man wählt eine Familie von Funktionen, die über einen Parameter variiert werden, wie etwa Gauß-Kurven mit verschiedener Breite $\beta$ :

>$
\psi_{\beta}(x)=\frac{1}{\beta \sqrt{2 \pi}} \cdot \exp \left[-\frac{x^{2}}{2 \beta^{2}}\right]
$

Nun setzt man diese Funktionen in obigen Ausdruck ein und sucht den minimalen Wert von $\langle\hat{H}\rangle\left[\psi_{\beta}\right] .$ Im einfachsten Fall kann dies durch Differentiation nach dem Parameter $\beta$ geschehen:

>$
\frac{\mathrm{d}}{\mathrm{d} \beta}\langle\hat{H}\rangle\left[\psi_{\beta}\right]=0
$

Löst man diese Gleichung, so erhält man für $\beta$ einen Wert, für den die Grundzustandsenergie minimiert wird. Mit diesem Wert hat man eine Näherungslösung, weiß aber nicht, wie gut der Ansatz wirklich ist, weshalb man von ${ }_{n}$ unkontrollierten Verfahren ${ }^{u}$ spricht. Immerhin kann man den Minimalwert als „beste Annäherung" an die tatsächliche Grundzustandsenergie benutzen.

* The [Ritz method](https://en.wikipedia.org/wiki/Ritz_method) is a direct method to find an approximate solution for [boundary value problems](https://en.wikipedia.org/wiki/Boundary_value_problem). The method is named after Walther Ritz, although also commonly called the [Rayleigh–Ritz method](https://en.wikipedia.org/wiki/Rayleigh–Ritz_method) and the [Ritz-Galerkin method](https://en.wikipedia.org/wiki/Galerkin_method).

* In quantum mechanics, a system of particles can be described in terms of an "energy functional" or Hamiltonian, which will measure the energy of any proposed configuration of said particles.

  * It turns out that certain privileged configurations are more likely than other configurations, and this has to do with the eigenanalysis ("analysis of characteristics") of this Hamiltonian system.

  * **Because it is often impossible to analyze all of the infinite configurations of particles to find the one with the least amount of energy, it becomes essential to be able to approximate this Hamiltonian in some way for the purpose of numerical computations.**

* The Ritz method can be used to achieve this goal. In the language of mathematics, it is exactly the [finite element method](https://en.wikipedia.org/wiki/Finite_element_method) used to compute the eigenvectors and eigenvalues of a Hamiltonian system.


https://www.youtube.com/watch?v=w4ijyrdLbNA

https://en.m.wikipedia.org/wiki/Ritz_method

https://en.m.wikipedia.org/wiki/Rayleigh%E2%80%93Ritz_method

*Ising Model*

https://www.youtube.com/watch?v=J-S_wUls314

> **The Ising Model represents the interactions of the spins of electrons in metals**.



* consider a two-dimensional square lattice

* has an electron on every vertex

* spins can only be orientated up or down

* depending on the configuration of the spins in the lattice the energy of the system will change

* Goal: analyse dependence of temperature, energy and spin structure

* every electron only interacts with their nearest neighbors (in this case: 4).

* this interaction will affect positively or negatively to the systems energy

**But why does it change? - Physicists use Hamiltonian to study the energy and behaviour of a system**

> $H=J \cdot \sum s_{i} s_{j}$

* H = Hamiltonian

* J = strenght of interaction

* $\sum$ = sum over all interactions

* $s_{i} s_{j}$ = spin interactions

**We can define the probability of having one spin configuration or other, depending on the system's temperature**:

> $p_{i}=\frac{e^{-\frac{H_{i}}{k T}}}{\sum e^{-\frac{H_{j}}{k T}}}$

* k = Boltzmann constant

* p = probability

* T = temperature

System tend to minimize the energy. This will happen in a way that it will have very different behaviors at low and high temperatures. Spins will align in clusters or not.

* the system has two phases: disordered for hot temperatures and ordered for low temperatures

* between them the system will undergo a phase transition (occurs at a critical temperatures and has specular properties)

**What is the difference between these two phases?**

* first we can define the magnetization as the total angular momentum of all the spins $s_{up}$ = 1 and $s_{down}$ = -1.

> $m=N_{\mathrm{up}}-N_{\text {down }}$

* The total number of up and down spins will be our total magnetization: $s_{up}$ = 5 electrons and $s_{down}$ = 4 electrons:

> m = 5 - 4 = 1 = total magnetization

* disordered phase: m = 0, approx. same number of up and down spins

* ordered phase ≠ 0 $\rightarrow$ we can see how clusters emerge

* The point at which the system changes between phases, i.e. the critical temperature, is called the 'critical point'.

> **The magnetization can be considered the order parameter since it determines whether we are in one phase or the other.**

Simulation on how the spins behave on every phase and its dependence on temperature:

* top: spins changes constantly orientation and on average same up and down spins, high entropy

![ggg](https://raw.githubusercontent.com/deltorobarba/repo/master/quantum_172.png)

**Critical Temperature: What happens at the phase transition?**: Self-similarity / fractal behavior at differen scales!

![ggg](https://raw.githubusercontent.com/deltorobarba/repo/master/quantum_173.png)

* fundamental physics: consider smallest particles

* Ising model: consider different phases and phase transitions - cooperative phenomena. This is impossible to deduce by studying exclusively the bahvior of single particles

* we need to study **many body physics** and look for changes of the collective behavior using simple concepts as the **order parameter** focusing on the interrelation of the parts not only on the parts themselves.

* hence you dont need to solve Schroedinger equation a million times, which is why the ising model is so popular and the beauty of physics of **complex systems**

* Considering the nearest neighbors and focusing on the interaction of the parts started a new way of research: neural networks, social systems, ecosystems, protein folding etc.



---



https://www.youtube.com/watch?v=1CCZkHPrhzk

* [Das Ising-Model](https://de.wikipedia.org/wiki/Ising-Modell) ist ein Gittermodell in der theoretischen Physik. Es beschreibt insbesondere den Ferromagnetismus in Festkörpern (Kristallen). Das Ising-Modell zählt zu den meistuntersuchten Modellen der [statistischen Physik](https://de.wikipedia.org/wiki/Statistische_Physik).

* In dem Modell wird angenommen, dass die Spins, welche das magnetische Moment der Atome oder Ionen bestimmen, nur zwei diskrete Zustände annehmen können (Spinwert $\pm 1$).

* Die Richtung im Raum bleibt aber offen; es handelt sich also um Vektoren (um im klassischen Bild zu bleiben, bzw. quantenmechanisch um Vektoroperatoren).

**Heisenberg-Modell**: Der allgemeine Energieausdruck (oder Hamiltonoperator) für eine solche Situation ist durch das [Heisenberg-Modell](https://de.wikipedia.org/wiki/Heisenberg-Modell) gegeben:

> $\hat{\mathcal{H}}=-\frac{1}{2} \sum_{i, j} J_{i j} \vec{s}_{i} \cdot \vec{s}_{j}-\vec{H} \cdot \sum_{i=1}^{N} \vec{s}_{i} \quad(\text { Heisenberg-Modell })
$

Hierbei bezeichnet
- $\vec{s}_{i}$ einen (mehrkomponentigen) Spin des Atoms am Platz $i$ des Kristallgitters,
- $J_{i j}$ die Kopplungskonstante (Stärke der Austauschkopplungs-Wechselwirkung) zwischen den Spins an den Plätzen $i$ und $j$,
- der Punkt \cdot das Skalarprodukt
- $\vec{H}$ die Stärke des Magnetfeldes.

Beim **Ising-Modell** dagegen wird die Zahl der Spinkomponenten auf Eins reduziert (d.h. parallel oder antiparallel zu einer ausgezeichneten Achse - hier $z$-Achse): $s_{i}^{z}=\pm 1$ :

>$
\hat{\mathcal{H}}=-\frac{1}{2} \sum_{i, j} J_{i j} s_{i}^{z} s_{j}^{z}-H_{z} \sum_{i=1}^{N} s_{i}^{z} \quad(\text { Isingmodell })
$

Oft wird zusätzlich angenommen, dass $J_{ij}$ nur für benachbarte Spins ungleich Null ist. Ist die Austauschkopplung positiv, so spricht man von einer ferromagnetischen Kopplung; ist sie negativ, so wird sie antiferromagnetisch genannt. Bei Ferromagneten bzw. Antiferromagneten dominiert das jeweilige Vorzeichen; bei den Spingläsern kommen beide Vorzeichen gleich häufig vor.

* Durch geeignete Wahl der Wechselwirkungen können u. a. Spingläser (hierbei ist $J_{ij}$ eine Zufallsgröße), verdünnte Magnete mit interessanten kritischen Eigenschaften oder auch räumlich modulierte magnetische Strukturen (hierbei liegen konkurrierende Kopplungen $J_{ij}$ vor, siehe [ANNNI-Modell](https://de.wikipedia.org/wiki/ANNNI-Modell)) modelliert werden.

* Im Allgemeinen beschreibt das Ising-Modell die magnetischen Ordnungen bei tiefen Temperaturen, die bei höheren Temperaturen jedoch durch thermische Fluktuationen aufgebrochen werden, wobei ein Phasenübergang stattfindet. Eine umfassende theoretische Analyse von Phasenübergängen liefert die Theorie der [Renormierungsgruppen](https://de.wikipedia.org/wiki/Renormierungsgruppe), für die Kenneth G. Wilson 1982 den Nobelpreis für Physik erhielt.

Bei der eindimensionalen Ising-Kette mit hinreichend kurzreichweitigen Wechselwirkungen beobachtet man jedoch keinen Phasenübergang. Dies hatte schon Ernst Ising in seiner Doktorarbeit mit Bedauern feststellen müssen. Fälschlicherweise vermutete er, dass dies auch für zwei und mehr Dimensionen zutrifft, was zunächst allgemein akzeptiert wurde.

Rudolf Peierls zeigte jedoch 1936,[4] dass in zwei Dimensionen sehr wohl ein Phasenübergang vorlag. 1941 bestimmten Hendrik Anthony Kramers und Gregory Wannier[5] durch ein Dualitätsargument die kritische Temperatur. Die exakte Lösung des zweidimensionalen Ising-Modells mit Wechselwirkungen zwischen nächsten Nachbarn und bei verschwindendem Magnetfeld wurde erstmals 1944 von Lars Onsager berechnet.

**Solutions**

* Für das dreidimensionale Ising-Modell mit Wechselwirkungen zwischen benachbarten Spins gibt es keine analytisch-exakte Lösung. Seine Eigenschaften kann man jedoch mit Hilfe der Molekularfeldnäherung (oder Landau-Theorie), Monte-Carlo-Simulationen, Reihenentwicklungen oder anderen numerischen Lösungsverfahren berechnen.

* Das Ising-Modell gilt wegen seiner konzeptionellen Einfachheit und seiner vielfältigen Eigenschaften als „Drosophila“ der statistischen Physik. Es hat darüber hinaus Anwendungen in vielen Bereichen der Naturwissenschaften gefunden, bis hin zur Biologie und Hirnforschung. Die nahezu programmatische Aussage von Michael E. Fisher ‚Ising models still thrive‘ (etwa: ‚Ising-Modelle sind noch im Wachsen‘) wird wohl noch für viele Jahre gültig bleiben.

* Verallgemeinerungen des Ising-Modells liefern das Blume-Capel-Modell, das Potts-Modell und das Markow-Netzwerk.

https://de.m.wikipedia.org/wiki/Ising-Modell

https://www.quantamagazine.org/the-cartoon-picture-of-magnets-that-has-transformed-science-20200624/

###### ***Density Functional Theory***

> **Great video**: [Density Functional Theory: Introduction and Applications](https://www.youtube.com/watch?v=HxUZKPg6Jig)

> Video: [How to simulate the universe with DFT](https://youtu.be/55c9wkNmfn0)

> Video: [nanoHUB-U Atoms to Materials L5.3: Density Functional Theory
](https://www.youtube.com/watch?v=3DrqowkhCNE)

https://en.m.wikipedia.org/wiki/Configuration_space_(physics)

Die [Dichtefunktionaltheorie_(Quantenphysik](https://de.m.wikipedia.org/wiki/Dichtefunktionaltheorie_(Quantenphysik)) (DFT) bzw. [Dichtefunktionaltheorie (statistische Physik)](https://de.m.wikipedia.org/wiki/Dichtefunktionaltheorie_(statistische_Physik)) ist ein **Verfahren zur Bestimmung des quantenmechanischen Grundzustandes eines Vielelektronensystems**, das auf der ortsabhängigen Elektronendichte beruht.

Die Dichtefunktionaltheorie wird zur Berechnung grundlegender Eigenschaften von Molekülen und Festkörpern, wie beispielsweise von Bindungslängen und -energien, verwendet.

Die große Bedeutung dieser Theorie liegt darin, dass es mit ihr nicht notwendig ist, die vollständige Schrödingergleichung für das Vielelektronensystem zu lösen, wodurch der Aufwand an Rechenleistung stark sinkt bzw. Berechnungen von Systemen mit deutlich über zehn Elektronen überhaupt erst möglich werden.

> Grundlage der Dichtefunktionaltheorie ist das [Hohenberg-Kohn-Theorem](https://de.m.wikipedia.org/wiki/Hohenberg-Kohn-Theorem).


Instead of using schrödinger equation, you use kohn sham equations: https://en.m.wikipedia.org/wiki/Kohn–Sham_equations

https://www.chemistryworld.com/news/algorithm-out-of-googles-deepmind-finesses-dft-calculations/4014939.article


**Dichtefunktionalmethoden**

* Die Dichtefunktionaltheorie (DFT) ist ein Verfahren zur Bestimmung des Grundzustandes eines Vielelektronensystems, das auf der dreidimensionalen ortsabhängigen Elektronendichte beruht.

> Daher ist es nicht notwendig ist, die Schrödingergleichung für das vieldimensionale Mehrelelektronensystem zu lösen, wodurch sich der Aufwand an Rechenleistung stark verringert bzw. Berechnungen an größeren Systemen möglich werden.

* **Grundlage der Dichtefunktionaltheorie ist das Hohenberg-Kohn-Theorem**.

* Allerdings ist das exakte Funktional, welches die Grundzustandsdichte mit der Eigenenergie des Systems verknüpft unbekannt. In der Praxis ist daher die Wahl eines geeigneten approximiertem Funktionals für die Genauigkeit entscheidend. Die systematische Verbesserbarkeit ist weniger stark ausgeprägt als bei ab-initio-Methoden.

https://de.m.wikipedia.org/wiki/Chemoinformatik#Methoden

*Exkurs: Difference between Density Functional Theory and Ab-initio-Methods (Hartree-Fock)*

> **Hartree-Fock theory completely ignores electron correlation of a system and it assumes the electrons can move independently in a mean field potential. Density functional theory include some electron correlation of a system and there is no mean field approximation in it.**

Both methods are used to describe the quantum states of many-electron systems, e.g. molecules and crystals. Both methods also use the Born-Oppenheimer approximation, in which you first solve for the electronic degrees of freedom.

The Hartree-Fock method assumes that the many-electron wavefunction takes the form of a determinant of single-electron wavefunctions, called a Slater determinant. The problem with this assumption is that a general many-electron wavefunction cannot be expressed as a single determinant. As a result, Hartree-Fock methods do not fully incorporate electronic correlation and the resulting energies tend to be too high. Luckily, if you find a complete basis of single-electron wavefunctions, then you can (in principle) express the exact many-electron wavefunction as a linear combination of all possible determinants made of these wavefunctions. This is called Configuration interaction. There are also many more post-Hartree-Fock techniques which expand on this.

In density functional theory (DFT), the many-electron wavefunction is completely bypassed in favor of the electron density. Thanks to the the Hohenberg-Kohn Theorems, the ground state energy of the system depends uniquely on the electron density, i.e. **the total energy is a functional of the electron density**. Therefore, one can apply the variational principle to minimize the energy with respect to the electron density. The problem with this technique is that no one knows what the energy functional is. At the first level of approximation, one can use the local density approximation (LDA), where you assume that the energy depends locally on the density in the same way it does for a uniform electron gas. Modern methods use generalized gradient approximations (GGAs) where you take into account the gradient of the density to produce a more accurate functional. However, the exact energy functional is still unknown and is a big open problem in computational chemistry and condensed matter physics.

Traditionally, Hartree-Fock and its descendants were considered more reliable than DFT, but this has been changing recently as DFT techniques have become more refined. Moreover, whereas in Hartree-Fock type calculations you need to keep track of the spatial and spin coordinates of all 𝑁
N
 electrons, DFT offers the potential benefit of dealing with only a single function of a single spatial coordinate. For this reason, DFT has been steadily gaining in popularity.

https://www.quora.com/What-is-the-difference-between-density-functional-theory-and-Hartree-Fock

###### ***Hartree-Fock Method***

Hartree–Fock (Ab-initio / Molecular Orbitals: single-electron wavefunctions) - Mean field (Self-Consistent)

> In computational physics and chemistry, the [Hartree–Fock (HF) method](
https://en.m.wikibooks.org/wiki/Density_functional_theory/Hartree–Fock_method) is a method of approximation for the determination of the wave function and the energy of a quantum many-body system in a stationary state.

> The Hartree–Fock method, Density matrix renormalization group, and Ritz method apply the [variational method](https://en.m.wikipedia.org/wiki/Variational_method_(quantum_mechanics)).

**Ab-initio-Methodem**

* Die Güte, mit denen Ab-initio-Verfahren die Eigenschaften von Molekülen berechnen können, hängt vom Basissatz der Atome ab, das heißt, wie gut und mit wie vielen einzelnen Funktionen die Atomorbitale dargestellt werden und in welchem Ausmaß die Elektronenkorrelation berücksichtigt wird.

> Ab-initio-Verfahren, die auch die Elektronenkorrelation berücksichtigen, sind deutlich aufwändiger, liefern jedoch die besten Resultate. Man behilft sich üblicherweise mit einem Kompromiss **und bezieht die Elektronenkorrelation näherungsweise ein**.

* Beispiele für solche Verfahren sind: Møller-Plesset-Störungstheorie, CI (Configuration-Interaction), CC (Coupled Cluster), MCSCF (Multi-Configuration-self-consistent-Field). Ausgangspunkt der meisten ab-initio-Verfahren ist die Hartree-Fock-Methode. Ein Vorteil der ab-initio-Verfahren ist ihre systematische Verbesserbarkeit, da man durch Vergrößerung des Basissatzes und Erhöhung des Grades der Berücksichtigung der Elektronenkorrelation (z. B. CISD, CISDT,...) die Genauigkeit der Ergebnisse systematisch verbessern kann.




Die Hartree-Fock-Methode vereinfacht die Wechselwirkungen der Teilchen untereinander so, dass diese nicht mehr jeweils paarweise untereinander wechselwirken, sondern mit einem [Feld](https://de.m.wikipedia.org/wiki/Feld_(Physik)), das von allen anderen Teilchen im Mittelwert erzeugt wird – dem so genannten **mean field (mittleren Feld)**. Das Feld hängt zwar immer noch vom Verhalten der einzelnen Teilchen ab, die Lösung kann aber jetzt schrittweise berechnet werden:

1. Ein Ausgangszustand wird ausgewählt und daraus das Feld erzeugt.

2. Mit diesem wird dann die Schrödingergleichung für jedes einzelne Teilchen gelöst.

3. Zusammengenommen ergeben die einzelnen Lösungen dann einen neuen Zustand und ein neues Feld.

Dieser Vorgang wird wiederholt, bis sich aufeinanderfolgende Lösungen nur mehr geringfügig unterscheiden, das Feld also zu Lösungen führt, die das Feld selbst konsistent wieder erzeugen. Daraus leitet sich der Begriff self-consistent field ab, der für diesen Teil der Hartree-Fock-Methode verwendet wird.

Als Wellenfunktionen für die behandelten Vielteilchensysteme werden bei Bosonen ein symmetrisches (Hartree-)Produkt von Einteilchenwellenfunktionen verwendet, bei Fermionen (wie Elektronen, Protonen und Neutronen) eine antisymmetrische Kombination dieser Produkte (eine sogenannte Slater-Determinante).

* **Um die Schrödingergleichung zu lösen, werden diese Einteilchenwellenfunktionen so variiert, dass die aus der Gleichung entstehende Energie minimal wird. Aufgrund des [Rayleigh-Ritz-Prinzips](https://de.m.wikipedia.org/wiki/Rayleigh-Ritz-Prinzip) (=Variationsprinzip zur Bestimmung des kleinsten Eigenwerts eines Eigenwertproblems) ist diese Energie dann eine obere Grenze für die tatsächliche Energie des Systems.**

* Die dadurch berechnete Wellenfunktion des gesamten Systems ist allerdings nicht notwendigerweise eine Annäherung der tatsächlichen Wellenfunktion.


* Einfachste Molecular Orbital Methode. Die Hartree-Fock-Methode ist die Basis der Molekülorbitaltheorie.

* Task: Calculating Molecular Orbitals. Describe orbital energies & wavefunctions (of electrons in molecules) of many body systems (Vielteilchensysteme)

* Born–Oppenheimer approximation is inherently assumed: The Born-Oppenheimer Approximation is the assumption that the electronic motion and the nuclear motion in molecules can be separated. It leads to a molecular wave function in terms of electron positions and nuclear positions. https://web.northeastern.edu/afeiguin/phys5870/phys5870/node9.html

* mean field approximation is implied.

Fock Operator Video: https://www.youtube.com/watch?v=3ph5XtkRVl0

**Functional Variation**

https://www.youtube.com/watch?v=Hy3mSGcvVAY

Die Hartree-Fock-Gleichung lautet:

>$
\hat{F}\left|\phi_{m}\right\rangle=\varepsilon_{m}\left|\phi_{m}\right\rangle
$

mit dem Fock-Operator $\hat{F}:$

>$
\hat{F}_{i}=\hat{h}_{i}+\sum_{j=1}^{N_{\text {Elektr. }}}\left(2 \hat{J}_{j}-\hat{K}_{j}\right)
$

* Der Fock-Operator ist ein effektiver Ein-Elektronen-Operator, der die kinetische Energie eines Elektrons $i$, die Coulomb-Anziehung mit dem Kern $\left(\hat{h}_{i}\right)$ und die Coulomb-Abstoßung zu allen anderen Elektronen $j\left(\hat{J}_{j}\right.$ und $\hat{K}_{j}$ ) beschreibt.

* $\hat{J}_{j}$ und $\hat{K}_{j}$ beschreiben als lokaler Coulomb-Operator und Austauschoperator die mittelwertartige Wechselwirkung mit allen anderen Teilchen.

* Der Fock-Operator hängt somit von seinen eigenen Eigenfunktionen (da alle besetzten Orbitale bekannt sein müssen) ab, die Hartree-Fock-Gleichung ist somit nicht-linear, was eine iterative-Lösung notwendig macht.

* Die Lösungen des SCF-Durchlaufs (Self-Consistent-Field-Methode) in HF sind die optimierten Orbitale $\phi _{m}$ als Eigenfunktionen des Fock-Operators mit den dazugehörigen Eigenenergien als Orbitalenergien $\varepsilon _{m}$.

**In the [HF approximation](https://en.m.wikipedia.org/wiki/Hartree–Fock_method), each electron in the molecule is treated as an independent particle that moves under the influence of the Coulomb potential due to the nuclei, and a mean field generated by all other electrons.**

* The optimized coefficients are precisely what we need to build the second-quantized Hamiltonian.

* [Hartree-Fock-Methode](https://de.m.wikipedia.org/wiki/Hartree-Fock-Methode) ist eine probabilistische Parameterschätzung und ermöglicht es, Orbitalenergien und Wellenfunktionen von quantenmechanischen Vielteilchensystemen näherungsweise zu berechnen und ist eine so genannte Ab-initio-Methode, d. h. sie kommt ohne empirische Parameter aus und benötigt nur Naturkonstanten. Sie ist der Ausgangspunkt für Post-Hartree-Fock-Methoden, welche die Genauigkeit der Berechnungen verbessern.

**Die Hartree-Fock-Methode ist die Basis der Molekülorbitaltheorie.**

Die [Hartree-Fock-Methode](https://de.m.wikipedia.org/wiki/Hartree-Fock-Methode) geht von der zeitunabhängigen Schrödinger-Gleichung (hier in Dirac-Notation)

>$
\hat{H}|\psi\rangle=E|\psi\rangle
$

aus, welche die Energie $E$ eines Systems aus der Wellenfunktion $|\psi\rangle$ berechnet, indem die Eigenwerte des Hamilton-Operators $\hat{H}$ zu dieser Wellenfunktion gesucht werden.

* Im Hamilton-Operator werden alle Energiebeiträge der Teilchen und Felder im System sowie deren Wechselwirkungen untereinander beschrieben.

* In vielen praktisch wichtigen Systemen (wie z. B. den Elektronen in einem Molekül) sind die Teilchen miteinander korreliert und beeinflussen sich gegenseitig. Dadurch kann die Schrödinger-Gleichung für solche Systeme nicht mehr exakt, sondern nur noch näherungsweise gelöst werden.

In computational physics and chemistry, the [Hartree–Fock (HF) method](https://en.m.wikipedia.org/wiki/Hartree%E2%80%93Fock_method) is a method of approximation for the determination of the wave function and the energy of a [quantum many-body system](https://en.m.wikipedia.org/wiki/Many-body_problem) <u>**in a stationary state**</u>.

* The Hartree–Fock method often assumes that the exact N-body wave function of the system can be approximated

  * by a single [Slater determinant](https://en.m.wikipedia.org/wiki/Slater_determinant) (in the case where the particles are fermions) or

  * by a single [permanent](https://en.m.wikipedia.org/wiki/Permanent_(mathematics)) (in the case of bosons) of N [spin-orbitals](https://en.m.wikipedia.org/wiki/Atomic_orbital).

* By invoking the [variational method](https://en.m.wikipedia.org/wiki/Calculus_of_variations), one can derive a set of N-coupled equations for the N spin orbitals. A solution of these equations yields the Hartree–Fock wave function and energy of the system.


Self-Consistent Field Method

> **HF-SCF: Wenn die Self-Consistent-Field-Methode (SCF) zur Lösung der Hartree-Fock-Gleichung in der [Hartree-Fock-Methode](https://de.m.wikipedia.org/wiki/Hartree-Fock-Methode) (HF) genutzt wird, werden die erhaltenen Funktionen als SCF-Orbitale bezeichnet.**

Especially in the older literature, the Hartree–Fock method is also called the self-consistent field method (SCF).

Die [Self-Consistent-Field-Methode (SCF)](https://de.m.wikipedia.org/wiki/Self-Consistent-Field-Methode) ist ein iteratives Lösungsverfahren in der Computerchemie. SCF ist eine von vielen Möglichkeiten zur Lösung von Molekülberechnungen. Das self-consistent-field (dt. selbstkonsistentes Feld) ist eine Menge von Funktionen, die eine Eigenwertgleichung löst.

See also Video: [Computational Chemistry 4.27 - Self-Consistent Field](https://www.youtube.com/watch?v=x-O3_t4GVy8)

**The self-consistent field procedure produces solutions to the Hartree-Fock-Roothaan equations for the best possible approximate molecular orbitals for a molecule within the Born-Oppenheimer approximation and Hartree-Fock approximation for a particular choice of atomic orbital basis set and nuclear coordinate configuration**. Da die Fock-Matrix von ihren eigenen Lösungen abhängt muss sie iterativ gelöst werden, dies geschieht mit der SCF-Methode. Ein typischer SCF-Ablauf sieht wie folgt aus:

1. Berechnung aller Ein- und Zweielektronenintegrale.
2. Erzeugung geeigneter MO-Koeffizienten.
3. Bildung der anfänglichen Dichte-Matrix.
4. Bildung der Fock-Matrix als Ein-Elektron-Integrale und der Dichtematrix mal der Zwei-Elektronen-Integrale.
5. Diagonalisierung der Fock-Matrix. Als Ergebnis enthalten die Eigenvektoren die neuen MO-Koeffizienten.
6. Bildung einer neuen Dichte-Matrix. Wenn sich die neue Dichte-Matrix und die anfängliche Dichte-Matrix gleichen kann der SCF-Durchlauf beendet werden, wenn nicht beginnt der Zyklus wieder bei Schritt 4 mit der neuen Dichte-Matrix.

*Mean Field Approximation (Molekularfeldtheorie)*

> **Molekularfeldtheorie (Mean Field Theory): Reducing many-body problem to one-body problem**

* In physics and probability theory, [mean-field theory](https://en.m.wikipedia.org/wiki/Mean-field_theory) (aka MFT or rarely self-consistent field theory) studies the behavior of high-dimensional random (stochastic) models by studying a simpler model that approximates the original by averaging over degrees of freedom (the number of values in the final calculation of a statistic that are free to vary). Such models consider many individual components that interact with each other.

* In MFT, the effect of all the other individuals on any given individual is approximated by a single averaged effect, **thus reducing a [many-body problem](https://en.m.wikipedia.org/wiki/Many-body_problem) to a [one-body problem (two body)](https://en.m.wikipedia.org/wiki/Two-body_problem)**.

Beispiel: Unter Hartree-Fock-Rechnung (beziehungsweise Hartree-Fock-Methode, nach Douglas Rayner Hartree und Wladimir Alexandrowitsch Fock) versteht man eine Methode der Quantenmechanik, in der Systeme **mit mehreren gleichartigen Teilchen in Mean-Field-Näherung behandelt werden.**

https://de.m.wikipedia.org/wiki/Hartree-Fock-Methode

Die Molekularfeldtheorie (engl. mean-field theory) ist eine Näherung, die Systeme miteinander wechselwirkender Teilchen als Systeme freier Teilchen in einem externen Feld betrachtet. Das externe Feld wird dabei als konstant angesehen und berücksichtigt somit nicht, dass jedes Teilchen durch sein Verhalten das Feld lokal verändert (d. h., Fluktuationen werden vernachlässigt).

Obwohl bei dieser Näherung für viele Größen quantitativ ungenaue Werte entstehen, gibt sie zahlreiche qualitative Hinweise auf das Skalenverhalten, also auf die kritischen Exponenten bei Phasenübergängen. Die Molekularfeldtheorie hängt eng mit der Landau-Theorie der Phasenübergänge zusammen.

Die Molekularfeldtheorie wird häufig angewendet in der statistischen Physik oder der statistischen Thermodynamik, u. a. bei der Bestimmung der Permittivität polarisierbarer Medien,[2] im Ising-Modell (Gitter aus N Spins) und in der Van-der-Waals-Theorie (Flüssigkeiten).

https://de.m.wikipedia.org/wiki/Molekularfeldtheorie

https://en.m.wikipedia.org/wiki/Variational_method_(quantum_mechanics)

*Slater Determinants*

Die Interpretation von Orbitalen als Wellenfunktionen je eines Elektrons ist nur bei Einzelelektronensystemen eindeutig möglich. Eine Wellenfunktion für N Elektronen kann dann konstruiert werden, indem man N Orbitale in eine Slater-Determinante einsetzt.

* Dies garantiert die für Fermionen notwendige Antisymmetrie der gesamten Wellenfunktion, kann aber darüber hinaus gehende Elektronenkorrelationen nicht darstellen.


* Um auch die Elektron-Elektron-Wechselwirkung näherungsweise zu berücksichtigen, können die Orbitale durch Hartree-Fock-, Kohn-Sham-Rechnungen (siehe: DFT / Dichtefunktionaltheorie in der Quantenphysik) oder MCSCF-Rechnungen (MCSCF: Multiconfiguration Self Consistent Field) bestimmt werden.

Die [Slater-Determinante](https://de.m.wikipedia.org/wiki/Slater-Determinante) (nach John C. Slater) ist **eine Methode in der Quantenmechanik zur Konstruktion einer Wellenfunktion für Systeme, die aus Fermionen bestehen**. Die entstehende Wellenfunktion entspricht dann den Anforderungen des Pauli-Prinzips und wechselt ihr Vorzeichen, wenn zwei identische Fermionen miteinander vertauscht werden.

> **Die Konstruktion nutzt die Eigenschaften der Determinante einer Matrix, die beim Vertauschen von Zeilen oder Spalten ebenfalls das Vorzeichen wechselt. Die entstandene Wellenfunktion wird oft ebenfalls als Slater-Determinante bezeichnet und z. B. in der Quantenchemie zur Beschreibung von Elektronen in einem Molekül eingesetzt.**

*Fiktive Slater-Determinante für zwei Elektronen, die sich auf einer Linie zwischen zwei Atomen bewegen.*

![ggg](https://upload.wikimedia.org/wikipedia/commons/d/da/Slater-Determinant.png)

*Effekte auf ein Hartree-Produkt und eine Slater-Determinante, wenn zwei Teilchen getauscht werden.*

![ggg](https://upload.wikimedia.org/wikipedia/commons/thumb/b/bc/Hartree-Products_vs_Slater-Determinants.png/477px-Hartree-Products_vs_Slater-Determinants.png)

* **Since the electronic Hamiltonian does not contain any spin operator, its eigenfunctions may depend only on the spatial coordinates of the electrons.**

* However, we know that each electron has an intrinsic angular momentum (spin) and, furthermore, the Pauli principle requires the electronic wavefunctions be antisymmetric under
coordinate permutations.

* Therefore, we want to construct approximate eigenfunctions $\{\Phi\}$ of $\hat{H}_{d}$ that fulfill both these requirements. A general way for obtaining this result is through the following expansion.

> $
\Phi\left(\vec{x}_{1}, \vec{x}_{2}, \ldots, \vec{x}_{N}\right)=\sum_{j} C_{j} \Theta_{j}\left(\vec{x}_{1}, \vec{x}_{2}, \ldots, \vec{x}_{N}\right)
$

* where $\vec{x}$ indicates all the variables needed in referring to a given electron, i.e. the spatial variables $\vec{r}$ and the spin variable $s$

> $\vec{x} \equiv(\vec{r}, s) ; \quad \int d \vec{x} \equiv \sum_{s} \int d \vec{r}
$ $\quad$ (Equ 9)

The wavefunctions $\left\{\Theta_{j}\right\}$ are antisymmetrized spin-orbital products or **Slater determinants**

> $\Theta_{j}\left(\vec{x}_{1}, \vec{x}_{2}, \ldots, \vec{x}_{N}\right)$
$=M_{j} \sum_{P} \epsilon_{P} \hat{P}\left[\psi_{a}\left(\vec{x}_{1}\right) \psi_{b}\left(\vec{x}_{2}\right) \ldots \psi_{w}\left(\vec{x}_{N}\right)\right]$
$=M_{j}\left|\begin{array}{llll}\psi_{a}\left(\vec{x}_{1}\right) & \psi_{b}\left(\vec{x}_{1}\right) & \cdots & \psi_{w}\left(\vec{x}_{1}\right) \\ \psi_{a}\left(\vec{x}_{2}\right) & \psi_{b}\left(\vec{x}_{2}\right) & \cdots & \psi_{w}\left(\vec{x}_{2}\right) \\ \vdots & \vdots & \ddots & \vdots \\ \psi_{a}\left(\vec{x}_{N}\right) & \psi_{b}\left(\vec{x}_{N}\right) & \cdots & \psi_{w}\left(\vec{x}_{N}\right)\end{array}\right|$
$=M_{j} \operatorname{det}\left|\psi_{a}\left(\vec{x}_{1}\right) \psi_{b}\left(\vec{x}_{2}\right) \ldots \psi_{w}\left(\vec{x}_{N}\right)\right|$

* where $\hat{P}$ is an index permutation and $\epsilon_{P}=\pm 1$ its parity, $M$, a normalization constant and $\{\psi(\vec{x})\}$ are spin-orbitals, i.e. products of a spatial function (orbital) $\phi(\vec{r})$ and a spin eigenfunction $\eta_{\pm}(s)$

> $
\begin{array}{l}
\psi(\vec{x})=\phi(\vec{r}) \eta_{\pm}(s) ; \\
\hat{s}_{z} \eta_{\pm}(s)=\pm \frac{\hbar}{2} \eta_{\pm}(s) \rightarrow\left(\begin{array}{l}
\left.\eta_{+}(s) \equiv \alpha(s)\right) \\
\eta_{-}(s) \equiv \beta(s)
\end{array}\right)
\end{array}
$

* If the set of all products of N spin-orbitals is complete, any wavefunction can be exactly represented as in Eq.(9), provided all the determinants that can be obtained from the complete set of spin-orbitals, are included in the expansion.

*Source:* [*MOLECULAR ENERGETICS: VALENCE BOND AND MOLECULAR ORBITAL METHODS.*](https://www.eolss.net/sample-chapters/c06/E6-11-03-02.pdf)

Die Slater-Determinante kann wie folgt geschrieben werden:

> $A_{N}\left[\phi_{1}(1) \phi_{2}(2) \ldots \phi_{N}(N)\right]=\frac{1}{\sqrt{N !}}\left|\begin{array}{cccc}\phi_{1}(1) & \phi_{2}(1) & \ldots & \phi_{N}(1) \\ \phi_{1}(2) & \phi_{2}(2) & \ldots & \phi_{N}(2) \\ \vdots & \vdots & \ddots & \vdots \\ \phi_{1}(N) & \phi_{2}(N) & \ldots & \phi_{N}(N)\end{array}\right|=\left|\phi_{1} \phi_{2} \ldots \phi_{N}\right\rangle$

*Density Matrix Renormalization Group*

For computations for quantum materials and quantum chemistry, for bond dimensions

is a variational method like ritz and hartree fock

Exkurs: density matrix renormalization group

* The [density matrix renormalization group](https://en.m.wikipedia.org/wiki/Density_matrix_renormalization_group) (DMRG) is a numerical variational technique devised to obtain the low-energy physics of quantum many-body systems with high accuracy.

* **As a variational method, DMRG is an efficient algorithm that attempts to find the lowest-energy matrix product state wavefunction of a Hamiltonian**. It was invented in 1992 by Steven R. White and it is nowadays the most efficient method for 1-dimensional systems.

> The density matrix renormalization group (DMRG) is a numerical variational technique devised to obtain the low-energy physics of quantum many-body systems with high accuracy.

* As a variational method, DMRG is an efficient algorithm that attempts to find the lowest-energy matrix product state wavefunction of a Hamiltonian.

* It was invented in 1992 by Steven R. White and it is nowadays the most efficient method for 1-dimensional systems.


https://en.m.wikipedia.org/wiki/Density_matrix_renormalization_group

###### ***Post Hartree–Fock Methods***

In computational chemistry, [post–Hartree–Fock methods](https://en.m.wikipedia.org/wiki/Post%E2%80%93Hartree%E2%80%93Fock) are the set of **methods developed to improve on the Hartree–Fock (HF), or self-consistent field (SCF) method**. **They add electron correlation which is a more accurate way of including the repulsions between electrons** than in the Hartree–Fock method where repulsions are only averaged.

See also Video: [Quantum Chemistry 9.11 - Post Hartree-Fock Methods](https://www.youtube.com/watch?v=oU1zO2bPPB4)


**In general, the SCF procedure makes several assumptions** about the nature of the multi-body Schrödinger equation and its set of solutions:

1. For molecules, the Born–Oppenheimer approximation is inherently assumed. The true wavefunction should also be a function of the coordinates of each of the nuclei.

2. Typically, [relativistic effects](https://de.m.wikipedia.org/wiki/Relativistischer_Effekt) are completely neglected. The momentum operator is assumed to be completely nonrelativistic.

3. The basis set is composed of a finite number of orthogonal functions. The true wavefunction is a linear combination of functions from a complete (infinite) basis set.

4. **The energy eigenfunctions are assumed to be products of one-electron wavefunctions. The effects of electron correlation, beyond that of exchange energy resulting from the anti-symmetrization of the wavefunction, are completely neglected**.

For the great majority of systems under study, in particular for excited states and processes such as molecular dissociation reactions, **the fourth item is by far the most important**. As a result, the term post–Hartree–Fock method is typically used for methods of approximating the electron correlation of a system. Usually, post–Hartree–Fock methods give more accurate results than Hartree–Fock calculations, although the added accuracy comes with the price of added computational cost.

* Configuration interaction (CI)

* Coupled cluster (CC)

* Multi-configuration time-dependent Hartree (MCTDH)

* Møller–Plesset perturbation theory (MP2, MP3, MP4 etc.)

* Quadratic configuration interaction (QCI)

* Quantum chemistry composite methods (G2, G3, CBS, T1. etc.)

Related Methods that use more than one determinant are not strictly post–Hartree–Fock methods, as they use a single determinant as reference, but they often use similar perturbation, or configuration interaction methods to improve the description of electron correlation. These methods include:

* Multi-configurational self-consistent field (MCSCF)

* Multireference single and double configuration interaction (MRCISD)

* N-electron valence state perturbation theory (NEVPT).

*Multi-configuration time-dependent Hartree (MCTDH)*

https://en.m.wikipedia.org/wiki/Multi-configuration_time-dependent_Hartree

*Configuration interaction*

[Configuration interaction (CI)](https://en.m.wikipedia.org/wiki/Configuration_interaction) is a post-Hartree–Fock linear variational method for solving the nonrelativistic Schrödinger equation within the Born–Oppenheimer approximation for a quantum chemical multi-electron system. Mathematically, **configuration simply describes the linear combination of Slater determinants used for the wave function.**

In terms of a specification of orbital occupation (for instance,
$\left.(1 \mathrm{~s})^{2}(2 \mathrm{~s})^{2}(2 \mathrm{p})^{1} \ldots\right)$, interaction means the mixing (interaction) of
different electronic configurations (states).

* Due to the long
$\mathrm{CPU}$ time and large memory required for $\mathrm{Cl}$ calculations, the
method is limited to relatively small systems.

* In contrast to the Hartree-Fock method, in order to account for
electron correlation, $\mathrm{Cl}$ **uses a variational wave function that is
a linear combination of configuration state functions (CSFs)
built from spin orbitals (denoted by the superscript SO)**,

> $\Psi=\sum_{I=0} c_{I} \Phi_{I}^{S O}=c_{0} \Phi_{0}^{S O}+c_{1} \Phi_{1}^{S O}+\ldots$

* where $\Psi$ is usually the electronic ground state of the system.

* If
the expansion includes all possible CSFs of the appropriate
symmetry, then this is a full configuration interaction

*Coupled Cluster (Post Hartree–Fock)*

https://en.m.wikipedia.org/wiki/Coupled_cluster

Ab-initio

### <font color="blue">**Organische Chemie**

##### <font color="blue">*Organische Chemie (Kohlenstoffchemie)*

###### *Organische Chemie*

> [Organische Moleküle](https://de.m.wikipedia.org/wiki/Organische_Chemie) enthalten als Elemente neben Kohlenstoff häufig Wasserstoff, Sauerstoff, Stickstoff, Schwefel, Halogene. Siehe auch [Derivat](https://de.m.wikipedia.org/wiki/Derivat_(Chemie))

![ggg](https://raw.githubusercontent.com/deltorobarba/repo/master/sciences_184.jpeg)

* Begriff  der Organischen Chemie (OC) wird im Laufe der Zeit auf die Chemie des Kohlenstoffs und dessen Verbindungenausgedehnt.

* mehr als 10 Millionen organische Stoffe  sind bekannt

* Nur 2% anorganische Stoffe

* Unterschied OC/AC: chemische Eigenschaften und Reaktionsverhalten unterscheiden sich wesentlich

* umfasst alle Kohlenstoffverbindungen mit Ausnahme der wasserstofffreien Chalkogenidewie Kohlenstoffmonooxid, Kohlenstoffdioxid und Schwefelkohlenstoff und deren Verbindungen (Kohlensäure, Carbonate), sowie der Carbide und Metallcarbonyle

* Stoffe, die nur aus Kohlenstoff-und Wasserstoffatomen aufgebaut sind, nennt man Kohlenwasserstoffe oder Alkane

https://docplayer.org/11199434-Chemie-in-der-einfuehrungsphase-am-cfg.html

###### *Kohlenstoff*

**Verteilung der Elemente in der Erdkruste vs im Holz**

![ggg](https://raw.githubusercontent.com/deltorobarba/repo/master/sciences_194.png)

**Besonderheit des Kohlenstoffs unter den Elementen**

![ggg](https://raw.githubusercontent.com/deltorobarba/repo/master/sciences_195.png)

**Kohlenstoff kann sowohl mit elektropositiveren (z. B. Wasserstoff) als auch mit elektronegativeren Partnern (z. B. Sauerstoff) Verbindungen bilden**. Mit seiner Fähigkeit vier stabile Atombindungen einzugehen ist Kohlenstoff der wichtigste Bestandteil der unzähligen organischen Verbindungen und somit Grundbaustein des Lebens.
Außer bei den salzartigen Carbiden bildet Kohlenstoff in allen seinen Verbindungen polare Atombindungen aus.

* Kohlenstoff als Element der IV. Hauptgruppe bildet fast ausschließlich polare Atombindungen zu anderen Partnern aus. Dabei entsteht eine Vielzahl von Molekülverbindungen, angefangen von den Kohlenwasserstoffen über die ungeheure Vielfalt der anderen organischen Verbindungen bis hin zu den anorganischen Kohlenstoffoxiden.

* Kohlenstoffverbindungen spielen eine wichtige Rolle in der Natur (z. B. Kohlenhydrate, Fette und Eiweiße als Grundbausteine des Lebens), in der Umweltchemie (z. B. das Treibhausgas Kohlenstoffdioxid), im Alltag (z. B. organische Säuren als Konservierungsmittel) und in der Technik (z. B. Erdölprodukte oder Werkstoffe).

* Kohlenstoff ist ein Element der IV. Hauptgruppe. Das Kohlenstoffatom besitzt 6 Protonen und meist 6 Neutronen im Atomkern sowie 6 Elektronen in der Atomhülle. Auf der ersten Schale befinden sich zwei und auf der zweiten Schale vier Elektronen. Diese vier Elektronen der äußeren Schale nennt man Außenelektronen. **Um eine stabile Elektronenanordnung, die Achterschale des Neons, zu erreichen, muss ein Kohlenstoffatom vier Elektronen von Bindungspartnern aufnehmen. Alternativ würde durch Abgabe von 4 Elektronen eine abgeschlossene Schale erreicht werden. Beides ist energetisch ungünstig, daher bildet Kohlenstoff Atombindungen aus, um das Elektronenoktett zu erreichen**. Wie alle Elemente der 2. Periode des PSE kann Kohlenstoff nur maximal vier Atombindungen eingehen.


Zu den anorganischen Stoffen werden traditionell die Elemente und alle Verbindungen gezählt, die keinen Kohlenstoff enthalten.
Dazu kommen einige Ausnahmen von Kohlenstoffverbindungen, die genau wie typische anorganische Stoffe aufgebaut sind oder historisch der Anorganik zugeordnet werden. Hierzu gehören die wasserstofffreien Chalkogenide des Kohlenstoffs (Kohlenstoffmonoxid, Kohlenstoffdioxid, Schwefelkohlenstoff), die Kohlensäure und Carbonate, die Carbide sowie die ionischen Cyanide, Cyanate und Thiocyanate. Die Blausäure gilt als Grenzfall und wird sowohl in der Organik als auch Anorganik behandelt. Obwohl sie traditionell zur anorganischen Chemie gezählt würden, wird sie als Nitril (organische Stoffgruppe) der Ameisensäure aufgefasst.
https://de.m.wikipedia.org/wiki/Anorganische_Chemie#Anorganische_Stoffe

**Flüchtige organische Verbindungen**

https://de.m.wikipedia.org/wiki/Flüchtige_organische_Verbindungen


###### *Kohlenstoffallotrope (Modifikationen des Kohlenstoffs)*

> **Kohlenstoffallotrope**: Kohlenstoff kommt sowohl als farbloser, extrem harter **Diamant** vor, der ein elektrischer Nichtleiter ist, als auch als schwarzer, weicher **Graphit**, der elektrischen Strom leitet, sowie als Makromoleküle **Fulleren**, Kohlenstoffnanoröhren, **Cyclocarbon**, **Carbin** und als **Graphen** [Source: Allotropie](https://de.m.wikipedia.org/wiki/Allotropie)


https://www.scinexx.de/news/technik/ein-moebiusband-aus-kohlenstoff/

![ggg](https://raw.githubusercontent.com/deltorobarba/repo/master/sciences_0793.jpeg)

**Modifikation des chemischen Elements Kohlenstoff: Fullerene, Diamant, Graphit, Lonsdaleit, Chaoit, Kohlenstoffnanoröhren und Graphen.**

* Im **Grafit** sind die Kohlenstoffatome wabenartig angeordnet. Sie bilden „Wabennetze“, die in vielen ebenen Schichten übereinanderliegen. Innerhalb der Sechseckebenen besitzt jedes Kohlenstoffatom nur drei Bindungspartner. Das vierte Außenelektron ist nicht in einer Atombindung fest gebunden und kann sich deshalb frei durch das Gitter bewegen. Diese beweglichen Elektronen sind die Ursache für die elektrische Leitfähigkeit und die tiefschwarze Farbe des Grafits. Die Wechselwirkung zwischen den einzelnen Schichten ist relativ schwach, sodass Grafit relativ weich ist und sogar als Schmiermittel eingesetzt werden kann.

* Anders als im Grafit ist im **Diamant** jedes Kohlenstoffatom mit vier weiteren Kohlenstoffatomen verbunden. Alle vier Außenelektronen sind an Atombindungen beteiligt, und alle Atome sind gleich weit voneinander entfernt. Je vier Atome bilden regelmäßige Tetraeder, die untereinander zu einer sehr stabilen Gitterstruktur verknüpft sind (Bild 3). Die stabilen Bindungen im regelmäßigen Kristallgitter sind der Grund für die extreme Härte des Diamanten. Da die Außenelektronen alle an Atombindungen beteiligt sind, kann Diamant den elektrischen Strom nicht leiten und ist ein Isolator.

> Diamanten aus Präkambrium und aus Erdmantel (150-300km Tiefe riesige Vorkommen), reiner krystalliner Kohlenstoff, härtester Stoff in der Natur, bei 1500 Grad geformt, Nitrogen macht sie gelb, Bor färbt sie blau. 1 Karat = 0,2 Gramm.

###### *Fullerene*

* [**Fullerene**](https://de.m.wikipedia.org/wiki/Fullerene) sind selten auftretende Modifikationen des Kohlenstoffs. Sie bestehen aus Molekülen mit vielen Kohlenstoffatomen, deren Struktur an einen Fußball erinnert (Bild 4). In der Natur wurden Fullerene in Trümmern von Meteoriten und im Weltraum nachgewiesen. Sie wurden erst 1985 durch die amerikanischen Chemiker R. CURL, H. KROTO und R. SMALLEY entdeckt, die für diese Entdeckung den Nobelpreis erhielten.

https://www.scinexx.de/news/technik/ein-netz-aus-fullerenen/

https://www.scinexx.de/news/technik/fulleren-als-ultraschneller-elektronenschalter/

###### *Graphen*

Graphen: hochempfindlicher Sensor etwa für Umweltgifte, ist extrem leitfähig, eignet sich als Elektrodenmaterial in Batterien, als Speichermedium für Solarzellen, als ultraleichter Werkstoff

https://www.spektrum.de/news/materialforschung-warum-graphen-supraleitend-werden-kann/2109165

**Graphen**

* [Graphen](https://de.m.wikipedia.org/wiki/Graphen) ist die Bezeichnung für eine Modifikation des Kohlenstoffs mit zweidimensionaler Struktur, in der jedes Kohlenstoffatom im Winkel von 120° von drei weiteren umgeben ist, sodass sich ein bienenwabenförmiges Muster ausbildet.

> ![ggg](https://upload.wikimedia.org/wikipedia/commons/thumb/7/71/Graphene_structure.svg/255px-Graphene_structure.svg.png)

* Da Kohlenstoff vierwertig ist, müssen dabei je „Wabe“ zwei Doppelbindungen auftreten, die jedoch nicht lokalisiert sind. Es handelt sich um eine Verkettung von Benzolringen, wie sie in aromatischen Verbindungen oft auftritt.

* Graphen ist eine dünne, zweidimensionale Schicht aus Kohlenstoffatomen, welche in einem hexagonalen Gitter angeordnet sind. Es handelt sich um eine Substanz mit vielen verschiedenen Eigenschaften, welche im Nachfolgenden näher erläutert werden sollen. Es ist dabei zu beachten, dass diese Eigenschaften immer mit einer einwandfreien Qualität des Produktes einhergehen.
Graphen besteht aus einer einlagigen Atomschicht und ist somit eines der dünnsten Materialien der Welt. Mit nur 0,77 mg/m² ist es außerdem eine sehr leichte Substanz. Durch die starke Bindung der benachbarten Atome ist es äußerst stabil aber trotzdem biegsam und dehnbar. Seine Zugfestigkeit von 1,25×1011 Pa ist die höchste, die je ermittelt wurde, und rund 125-mal höher als bei Stahl!

* Graphen ist elektrisch leitfähig, hat eine hohe elektrische Dichte (1.000.000 mal so hoch wie die Dichte von Kupfer) sowie eine hoch intrinsische Mobilität. Zudem ist der Widerstand von Graphen bei Raumtemperatur niedriger als bei jedem anderen Material.

* Eine weitere Eigenschaft von Graphen ist die sehr hohe Wärmeleitfähigkeit, welche höher ist als bei Graphit. Graphen leitet Wärme in alle Richtungen und ist somit ein isotroper Leiter. Graphen ist in der Lage 2,3% weißes Licht zu absorbieren, wodurch das Material sehr transparent für das menschliche Auge wird. Liegen mehrere Atomschichten aufeinander, so sieht das Graphen tief schwarz aus.

* Darüber hinaus weist Graphen eine hohe Undurchlässigkeit auf, welche nicht einmal Heliumatome hindurch lässt. Zusätzlich zu allen bereits genannten Eigenschaften ist Graphen sehr flexibel. Eine Biegung von bis zu 20% der Ursprungsgröße ist möglich, ohne das Material zu beschädigen.
Durch die zahlreichen Eigenschaften kann Graphen in vielen unterschiedlichen Anwendungsfeldern zum Einsatz kommen und wird bereits in der Produktion von Sportartikeln, Smartphones, Akkus, Arbeitshandschuhen, Heizfeldern und Kamerasensoren eingesetzt.

https://www.wissenschaft.de/technik-digitales/ein-hauch-von-material/

https://www.graphene-tec.com/de/welche-besonderen-eigenschaften-hat-graphen/

*Ein Vergleich zwischen Graphen und Graphit*

* Graphen und Graphit bestehen beide aus Kohlenstoff, das wohl vielseitigste und wichtigste chemische Elemente.
Die beiden Stoffe unterscheiden sich jedoch in ihren Atom-Dimensionen. Im Graphit liegen die Kohlenstoffatome 3-dimensional vor. Es handelt sich im Prinzip um eine Anhäufung, welche Graphit zu einem Festkörper macht.
Graphen hat dagegen nur eine Atomlage, mit einer Vielzahl von wabenförmig angeordneten Kohlenstoffatomen. Es entsteht eine 2-dimensionale Plättchenform, welche Graphen die vielen besonderen Eigenschaften verleiht.

* Ein weiterer Unterschied der beiden Stoffe ist das Verhalten der Elektronen. Beim Graphit interagieren die Kohlenstoffatome miteinander und verbinden so die Schichten aus Graphit, um ein Objekt zu bilden.

* Bei Graphen ist diese Verbindung nicht vorhanden, wodurch sich einzelne Schichten aus Atomen bilden, die sich wiederum in verschiedene Formen bringen lassen.
Graphen hat durch seine außergewöhnliche Beschaffenheit und die Wandelbarkeit der Atome viele Vorteile gegenüber Graphit. Es ist extrem zugfest und leitet sowohl Elektrizität als auch Wärme. Außerdem ist Graphen ein sehr flexibler und transparenter Stoff.

https://www.graphene-tec.com/de/ein-vergleich-zwischen-graphen-und-graphit/

###### *Organische Elektronik*

https://www.scinexx.de/news/technik/organischer-bipolartransistor-kann-mithalten/

##### <font color="blue">*Kohlenwasserstoffe ohne funktionelle Gruppe (Aliphate)*

###### *Kohlenwasserstoffe*

**Kohlenwasserstoffe**

* [Kohlenwasserstoffe (CmHn)](https://de.m.wikipedia.org/wiki/Kohlenwasserstoffe) bzw. [auf Chemie.de](https://www.chemie.de/lexikon/Kohlenwasserstoffe.html) sind Stoffgruppe von Verbindungen **aus Kohlenstoff und Wasserstoff**

* Kohlenwasserstoffe sind hydrophob (nicht wasserloslich)

* **Einfachste Stoffgruppe der organischen Chemie** mit mehreren Untergruppen. Siehe auch [Stoffgruppen der organischen Chemie](https://de.m.wikipedia.org/wiki/Organische_Chemie#Stoffgruppen_der_organischen_Chemie)

* Kohlenwasserstoffe mit gleicher Summenformel können verschiedene Strukturformeln (Verknüpfungen der C-Atome) aufweisen. Es handelt sich dann um Isomere. Es gibt sie bei den Alkanen ab dem Butan und bei den meisten anderen Kohlenwasserstoffen. Eine besondere Isomerie, die cis-trans-Isomerie tritt mitunter an C=C-Doppelbindungen auf.


![ggg](https://raw.githubusercontent.com/deltorobarba/repo/master/sciences_173.jpeg)

![ggg](https://raw.githubusercontent.com/deltorobarba/repo/master/sciences_0740.png)

> **Gruppe 1:** [**Aliphatische Kohlenwasserstoffe**](https://de.m.wikipedia.org/wiki/Aliphatische_Kohlenwasserstoffe) (Aliphate)

  * [**gesättigte**](https://de.m.wikipedia.org/wiki/Gesättigte_Verbindungen) Verbindungen: C-C-Einfachbindungen

    * kettenförmige (acyclische) Verbindungen: [**Alkane**](https://de.m.wikipedia.org/wiki/Alkane), wie Methan (CH_4), Ethan (C_2H_6), und Propan (C_3H_8)

    * ringförmige (cyclische) Verbindungen: [**Cycloalkane**](https://de.m.wikipedia.org/wiki/Cycloalkane), wie [Cyclopentan](https://de.m.wikipedia.org/wiki/Cyclopentan), [Cyclohexan](https://de.m.wikipedia.org/wiki/Cyclohexan) und [Cycloheptan](https://de.m.wikipedia.org/wiki/Cycloheptan)

      * [**Monocyclen**](https://de.m.wikipedia.org/wiki/Cyclische_Verbindungen#Nomenklatur_nach_Anzahl_der_Ringe)

      * [**Polycyclen**](https://de.m.wikipedia.org/wiki/Polycyclen): verbundene (kondensierte) Polycyclen, verbrückte Polycyclen und [Spiroverbindungen](https://de.m.wikipedia.org/wiki/Spiroverbindung)

  * [**ungesättigte**](https://de.m.wikipedia.org/wiki/Ungesättigte_Verbindungen) Verbindungen: C=C-Doppel- oder Dreifachbindungen (Sehr viele Naturstoffe sind ungesättigte Verbindungen)

    * C=C-Doppelbindungen: [**Alkene**](https://de.m.wikipedia.org/wiki/Alkene) wie Ethen. Polyene sind Verbindungen mit mindestens zwei C=C-Doppelbindungen, wie zum Beispiel 1,3-Butadien

    * C≡C-Dreifachbindungen: [**Alkine**](https://de.m.wikipedia.org/wiki/Alkine) wie Ethin (Acetylen)

> **Gruppe 2:** [**Aromatische Kohlenwasserstoffe**](https://de.m.wikipedia.org/wiki/Aromatische_Kohlenwasserstoffe) - [Arene](https://www.spektrum.de/lexikon/chemie/arene/700) oder [Aromaten](https://de.m.wikipedia.org/wiki/Aromaten) - meist C6-Ringe besitzen wie Benzol. Aromatische Kohlenwasserstoffe finden sich im Erdöl. Siehe auch [chemie.de](https://www.chemie.de/lexikon/Aromaten.html#Einteilung_der_Aromaten)

  * [alkylierte](https://de.m.wikipedia.org/wiki/Alkylierung) Arene ([Alkylbenzole](https://de.m.wikipedia.org/wiki/Alkylbenzole)), siehe auch [Alkylgruppe](https://de.m.wikipedia.org/wiki/Alkylgruppe)

  * Arene mit mehreren [Phenylgruppen](https://de.m.wikipedia.org/wiki/Phenylgruppe) (Polyarylalkane), zum Beispiel [Phenol](https://de.m.wikipedia.org/wiki/Phenol), [Biphenyl](https://de.m.wikipedia.org/wiki/Biphenyl) oder [Benzoesäure](https://de.m.wikipedia.org/wiki/Benzoesäure)

    * siehe auch [Arylgruppe](https://de.m.wikipedia.org/wiki/Arylgruppe) - die einfachste Arylgruppe ist die Phenylgruppe (Ph), (–C6H5).

    * Im Gegensatz zu Arylgruppen wird der Begriff Alkylgruppen verwendet, um einen Rest zu bezeichnen, der nicht aromatisch ist.

  * [anellierte](https://de.m.wikipedia.org/wiki/Anellierung) Arene (PAK)

> Siehe auch [Aromaten (Kohlenwasserstoffe mit Ringstrukturen)](https://de.m.wikipedia.org/wiki/Aromaten)

eine Untergruppe der Aromaten sind die polyzyklischen aromatischen Kohlenwasserstoffe. Bei ihnen handelt es sich um Verbindungen, die aus mehreren aneinander hängenden Benzolringen bestehen. Ein bekannter Vertreter ist das Naphthalin (C10H8).

* einfache Aromaten
* kondensierte Aromaten

> **Heterocyclen**: **biochemische Verbindungen** (Alkaloide, Aminosäuren, Kohlenhydrate, Proteine, Steroide, Terpene, Vitamine)

**Einteilung der Stoffe in der organischen Chemie** [Source](https://commons.wikimedia.org/wiki/File:Einteilung_der_Stoffe-organische_Chemie.svg)

![ggg](https://raw.githubusercontent.com/deltorobarba/repo/master/sciences_176.png)

![ggg](https://raw.githubusercontent.com/deltorobarba/repo/master/sciences_184.jpeg)

https://studyflix.de/chemie/kohlenwasserstoffe-2314

Siehe auch: https://de.khanacademy.org/science/biology/properties-of-carbon/hydrocarbon-structures-and-functional-groups/a/hydrocarbon-structures-and-isomers


**Organylgruppen**

https://de.m.wikipedia.org/wiki/Organylgruppe

###### *Kohlenwasserstoffe: <font color="blue">Alkane (Alkylgruppe)</font> - Aliphatisch, gesättigt (C-C-Einfachbindungen), acyclisch*

> Alkane (Alkyl, **gesättigt** mit C-C-**Einfachbindungen** & kettenförmig)

**Homologe Reihe der Alkane**

![ggg](https://raw.githubusercontent.com/deltorobarba/repo/master/sciences_105.png)

*Organylgruppen der Alkane:*

![ggg](https://raw.githubusercontent.com/deltorobarba/repo/master/sciences_107.png)

**Alkane**

* [Alkane](https://de.m.wikipedia.org/wiki/Alkane) sind eine Gruppe aliphatischer organischer Verbindungen. Siehe auch [Alkan chemie.de](https://www.chemie.de/lexikon/Alkane.html)

> ![ggg](https://upload.wikimedia.org/wikipedia/commons/thumb/4/41/N-Alkanes_General_Formula_V2.svg/320px-N-Alkanes_General_Formula_V2.svg.png)

* Als Alkane (Grenzkohlenwasserstoffe, früher Paraffine) bezeichnet man in der organischen Chemie die Stoffgruppe der gesättigten, acyclischen Kohlenwasserstoffe. Das heißt, ihre Vertreter bestehen nur aus den beiden Elementen Kohlenstoff (C) und Wasserstoff (H), weisen nur Einfachbindungen und keine Kohlenstoffringe auf. Damit sind sie eine Untergruppe der aliphatischen Kohlenwasserstoffe. Für sie gilt die allgemeine Summenformel CnH2n+2 mit n = 1, 2, 3,..




* Das Grundgerüst der Alkane kann ab n = 4 aus unverzweigten (linearen) oder aus verzweigten Kohlenstoffketten bestehen. Die unverzweigten Verbindungen werden als n-Alkane bezeichnet und bilden eine homologe Reihe der Alkane. Die verzweigten Alkane werden Isoalkane (i-Alkane) genannt.

* Gesättigte cyclische Kohlenwasserstoffe haben eine abweichende allgemeine Summenformel und bilden die Gruppe der [Cycloalkane](https://de.m.wikipedia.org/wiki/Cycloalkane) und werden dort beschrieben.

* Eine [Alkylgruppe](https://de.m.wikipedia.org/wiki/Alkylgruppe) ist ein Teil eines Moleküls, der aus miteinander verbundenen Kohlenstoff- und Wasserstoffatomen besteht.

  * Die einfachste Alkylgruppe ist die Methylgruppe –CH3; sie tritt relativ häufig auf.

  * Weitere Beispiele sind die Ethylgruppe –CH2–CH3

  * und die n-Propylgruppe –CH2–CH2–CH3.

  * Die allgemeine Formel für Alkylgruppen in Form einer Kette ist CnH2n+1, die für die ringförmig angeordneten Cycloalkylgruppen CnH2n−1.

* Als [Alkylierung](https://de.m.wikipedia.org/wiki/Alkylierung) wird in der organischen Chemie der Transfer von Alkylgruppen während einer chemischen Reaktion von einem Molekül zum anderen bezeichnet. Dabei können Kohlenstoff-Atome der Ausgangsverbindung, aber auch Heteroatome alkyliert werden.[1] Die entgegengesetzten Reaktionen (also Reaktionen, bei denen Alkylgruppen aus organischen Molekülen abgespalten werden) werden als Desalkylierung (Dealkylierung) bezeichnet. Die zur Alkylierung eingesetzten Verbindungen werden als Alkylanzien bezeichnet.


![ggg](https://raw.githubusercontent.com/deltorobarba/repo/master/sciences_112.png)

**Alkanes' physical states**

![ggg](https://raw.githubusercontent.com/deltorobarba/repo/master/sciences_1351.jpg)

Lube oil: Schmieröl

LPG is a low-carbon fuel

**Vom Petroleum zum Fett (Lipid - Fette und fettige Öle)**: anstatt ein H-Atom, hänge eine OH-Hydroxygruppe jeweils an das C-Atom: aus Propan wird Glycerin. ps: hängt man nur eine OH-Hydroxygruppe ganz an das Ende der C-Kette, dann hat man ein einfachen Alkohol!

Dreiwertiger Alkohol: 1,2,3-Propantriol (Trivialname Glycerin)

> ![gg](https://upload.wikimedia.org/wikipedia/commons/thumb/f/fb/Glycerin_-_Glycerol.svg/320px-Glycerin_-_Glycerol.svg.png)

**Isomerie bei Alkanen**

* Bei Molekulen aus vier oder mehr Kohlenstoffatomen gibt es verschiedene Moglichkeiten, wie die einzelnen Kohlenstoffatome miteinander verknupft sind.

* Dabei ergeben sich unterschiedliche Eigenschaften (zB unterschiedliche Siedetemperaturen). Gibt es mehrere Moglichkeiten, spricht man von Isomerie.

![ggg](https://raw.githubusercontent.com/deltorobarba/repo/master/sciences_122.png)

*Isomere der Summenformel C6H14:*

* Methylbutan: Methylgruppe ist CH3 (siehe oben Methan) und Butan ist C4H10

* bei 2,2 Dimethylbutan haengt  anstelle eines H-Atom jeweils eine Methylgruppe am zweiten C-Atom,

* oder bei 2,3 Dimethylbutan jeweils eines an der 2. und 3. Stelle

![ggg](https://raw.githubusercontent.com/deltorobarba/repo/master/sciences_111.png)

**Alkane - Alkylgruppe - Alkylierung**

* Methan - Methylgruppe - Methylierung

* Ethan - Ethylgruppe - Ethylierung

* Propan - Propylgruppe - Propylierung

* ... etc.

**Methan**

* [Methan](https://de.m.wikipedia.org/wiki/Methan), siehe auch [Methan chemie.de](https://www.chemie.de/lexikon/Methan.html), ist eine chemische Verbindung mit der Summenformel CH4 und der einfachste Vertreter aus der [Stoffgruppe](https://de.m.wikipedia.org/wiki/Stoffgruppe) der [Alkane](https://de.m.wikipedia.org/wiki/Alkane).

> ![ggg](https://upload.wikimedia.org/wikipedia/commons/thumb/f/f2/Methane.Structural_formula.V1.svg/123px-Methane.Structural_formula.V1.svg.png)

* Unter Normalbedingungen ist es ein farb- und geruchloses, brennbares Gas. Methan ist in Wasser unlöslich und bildet mit Luft explosive Gemische. Es verbrennt mit bläulich-heller Flamme in Gegenwart von ausreichend Sauerstoff zu Kohlenstoffdioxid und Wasser.

* Die [Methylgruppe](https://de.m.wikipedia.org/wiki/Methylgruppe) (auch als Methyl-Rest bezeichnet) ist eine der einfachsten Atomanordnungen in der organischen Chemie. Die Formel lautet –CH3, sie besteht also aus einem Kohlenstoff- und drei Wasserstoff-Atomen.

* Sie ist keine eigenständige chemische Substanz, sondern stets Teil eines größeren Moleküls. Sie ist die einfachste Alkylgruppe und findet sich in vielen chemischen Verbindungen. Deren systematische Namen enthalten dann die Silbe „-methyl-“ (z. B. 2-Methylbutan).

* Als [Methylierung](https://de.m.wikipedia.org/wiki/Methylierung) wird in der organischen Chemie der Transfer von Methylgruppen (–CH3) innerhalb einer chemischen Reaktion von einem Molekül auf ein anderes bezeichnet (Donator-Akzeptor-Prinzip). Die Übertragung von Alkylgruppen nennt man allgemein Alkylierung, das Reagenz Alkylierungsmittel. Siehe auch [DNA-Methylierung](https://de.m.wikipedia.org/wiki/DNA-Methylierung)




> Epigenome & methyl groups zur Ein- und Ausschaltung von Eigenschaften in DNA

**Ethan**

* Ethan (standardsprachlich Äthan) ist eine chemische Verbindung, die den Alkanen zugehört (gesättigte Kohlenwasserstoffe).

> ![ggg](https://upload.wikimedia.org/wikipedia/commons/thumb/3/35/Ethan_Lewis.svg/142px-Ethan_Lewis.svg.png)

* Es hat die Summenformel C2H6 und ist ein farb- und geruchloses Gas, das vor allem zu Heiz- und Verbrennungszwecken genutzt wird. Davon abgeleitet ist der Ethyl-Rest (–C2H5). Ethan ist neben Methan ein Hauptbestandteil des Erdgases.

* Die [Ethylgruppe](https://de.m.wikipedia.org/wiki/Ethylgruppe) ist eine Atomanordnung der organischen Chemie mit der Formel –CH2–CH3 bzw. –C2H5 und gehört zu den Alkylgruppen. Sie ist nicht als stabile Verbindung darstellbar und existiert nur als sehr kurzlebiges Intermediat in chemischen Reaktionen. Die Ethylgruppe lässt sich gedanklich durch Abstraktion eines Wasserstoffatoms aus Ethan darstellen.

* Als [Ethylierung](https://de.m.wikipedia.org/wiki/Ethylierung) wird eine chemische Reaktion bezeichnet, die eine Ethylgruppe (–C2H5) in eine organische Verbindung einführt. Sie gehört zu den Alkylierungen.



**Propan**

* [Propan](https://de.m.wikipedia.org/wiki/Propan) ist ein farbloses brennbares Gas und gehört zu den Kohlenwasserstoffen. Es steht in der homologen Reihe der Alkane an dritter Stelle.

> ![ggg](https://upload.wikimedia.org/wikipedia/commons/thumb/9/92/Propane_Structural_Formula_B_V1.svg/226px-Propane_Structural_Formula_B_V1.svg.png)

* Im Labor kann Propan durch eine Addition von Wasserstoff an Propen synthetisiert werden:

* Die [Propylgruppe](https://de.m.wikipedia.org/wiki/Propylgruppe) ist eine Atomanordnung der organischen Chemie mit der Summenformel C3H7. Sie lässt sich gedanklich durch Abstraktion eines Wasserstoffatoms aus Propan ableiten und tritt in zwei isomeren Formen auf.

* Als **Propylierung** wird eine chemische Reaktion bezeichnet, die eine Ethylgruppe (–C3H7) in eine organische Verbindung einführt. Sie gehört zu den Alkylierungen.

**Butan**

* [Butan](https://de.m.wikipedia.org/wiki/Butan) bzw. [chemie.de](https://www.chemie.de/lexikon/Butan.html)

> ![ggg](https://upload.wikimedia.org/wikipedia/commons/thumb/c/cb/Butan_Lewis.svg/222px-Butan_Lewis.svg.png)

* [Butylgruppe](https://de.m.wikipedia.org/wiki/Butylgruppe)

**Höhere Alkane**

* Als [höhere Alkane](https://de.m.wikipedia.org/wiki/Höhere_Alkane) werden Alkane bezeichnet, deren längste Kette mindestens neun Kohlenstoffatome besitzt.

> ![ggg](https://upload.wikimedia.org/wikipedia/commons/thumb/3/3d/N-Alkanes_General_FormulaV1.svg/219px-N-Alkanes_General_FormulaV1.svg.png)

* **Gemische mittlerer und hoherer Alkane werden auch Schmieroele oder Schmierfette bezeichnet.** Begruendung: Zwischen den Molekulen wirken Van der Waals Kraefte. Diese nehmen mit steigender Elektronenanzahl zu, so dass auch Siedetemperaturen der Alkane innerhalb der homologen Reihe steigen. Die Viskositaet steigt dann mit wachsender Kettenleange

> Siehe auch [Höhere Alkane](https://de.m.wikipedia.org/wiki/Höhere_Alkane): Höhere Alkane ab Hexadecan sind die wichtigsten Bestandteile von Heizöl, Schweröl und vielen Schmierölen. Letztere werden häufig auch aufgrund ihrer hydrophoben Eigenschaften als Korrosionsschutz eingesetzt. Da diese Alkane auch häufig bei Raumtemperatur fest sind, werden sie im Paraffin beispielsweise zur Herstellung von Kerzen verwendet.

* Höhere Alkane finden sich auch sehr häufig in den Sekreten, Pheromonen, Allomonen und Wehrsekreten von Tieren. So nutzen einige Insekten solche Substanzen als Lösungsmittel für feste Bestandteile von Wehrsekreten.



###### *Special: Per- und polyfluorierte Alkylverbindungen*

[Per- und polyfluorierte Alkylsubstanzen (v)](https://de.m.wikipedia.org/wiki/Per-_und_polyfluorierte_Alkylverbindungen) sind aliphatische organische Verbindungen, bei denen an mindestens einem Kohlenstoffatom die Wasserstoffatome am Kohlenstoffgerüst vollständig durch Fluoratome ersetzt worden sind.

PFAS sind definiert als fluorierte Stoffe, die mindestens ein vollständig fluoriertes Methyl- oder Methylen-Kohlenstoffatom (ohne daran gebundene H/Cl/Br/I-Atome) enthalten, d. h. bis auf wenige bekannte Ausnahmen ist jeder Stoff mit mindestens einer perfluorierten Methylgruppe (–CF3) oder einer perfluorierten Methylengruppe (–CF2–) ein PFAS.


Per- und polyfluorierte Alkylsubstanzen – kurz PFAS – finden sich fast überall: in Outdoorbekleidung, beschichteten Pfannen, Kosmetik oder Verpackungen für Fast Food. Viele haben Wasser, Schmutz oder auch Fett abweisende Eigenschaften, halten über lange Zeit extremen Temperaturen, großem Druck oder UV-Strahlung stand. Allein in der Medizin stecken sie in Implantaten, Kathetern und Schläuchen. Ihre Bedeutung ist groß. Sie helfen selbst bei der Produktion von Mikrochips.


###### *Kohlenwasserstoffe: <font color="blue">Alkene (Alkenylgruppe)</font> - Aliphatisch, ungesättigt (C=C-Doppelbindungen), acyclisch*

**[Alkene](https://de.m.wikipedia.org/wiki/Alkene) sind ungesättigte Kohlenwasserstoffe mit C=C-Doppelbindungen**:

* Beispiel Ethen:

> ![gg](https://upload.wikimedia.org/wikipedia/commons/thumb/e/e9/Ethene_structural.svg/170px-Ethene_structural.svg.png)

* Vergleiche dazu das **Alkan Ethen (Einfachbindung** zwischen Kohlenstoffatomen):

> ![ggg](https://upload.wikimedia.org/wikipedia/commons/thumb/3/35/Ethan_Lewis.svg/142px-Ethan_Lewis.svg.png)

* Polyene sind Verbindungen mit mindestens zwei C=C-Doppelbindungen, wie zum Beispiel 1,3-Butadien


* **Alkane** reagieren mit Halogenen in einer radikalischen Substitutionsreaktion

* Von einem Alkan-Molekul kann ein Wasserstoff-Molekul abgespalten werden. Dabei entsteht ein **Alken-Molekul**. Man spricht von einer **Dehydrierung**.

Siehe auch: Alken als Grundlage des Naturstoffs Isopren
* https://de.m.wikipedia.org/wiki/1,2-Butadien
* https://de.m.wikipedia.org/wiki/1,3-Butadien
* https://de.m.wikipedia.org/wiki/Isopren

[**Ethen**](https://de.m.wikipedia.org/wiki/Ethen)

[**Propen**](https://de.m.wikipedia.org/wiki/Propen)

[**Butene**](https://de.m.wikipedia.org/wiki/Butene)

https://de.m.wikipedia.org/wiki/1,3-Butadien

https://de.m.wikipedia.org/wiki/Terpene

https://de.m.wikipedia.org/wiki/Isopren

[**Pentene**](https://de.m.wikipedia.org/wiki/Pentene)

[**Hexene**](https://de.m.wikipedia.org/wiki/Hexene)

[Alkenylgruppe](https://de.m.wikipedia.org/wiki/Alkenylgruppe)

**Nomenklatur der Alkene (Isomerie)**

Die Endung "-en" fur ein Alken gibt noch nicht an, wo sich im Molekul die Doppelbindugn findet. Zum Beispiel gibt es bei Pentenen 4 Moglichkeiten (hier mit Strukturfomel):

Man sagt dann Pent-1-en, Pent-2-en, Pent-3-en oder Pent-4-en.

(C5H10 hat folgende Isomere, sowie noch weitere, wenn man C als Nebenarme hat, zB 2-Methylbuten. Siehe mehr hier: https://wissensplattform-schueler.de/isomerie/)

![ggg](https://raw.githubusercontent.com/deltorobarba/repo/master/sciences_125.png)




**Isomerie bei Alkenen**

* In einem Ethen-Molekül wurden zwei Wasserstoffatome durch zwei Chlor-atome ersetzt, sodass an beiden Kohlenstoffatomen jeweils ein Chloratom gebunden ist.

* Da die C-C-Doppelbindung nicht frei drehbar ist, entstehen zwei mögliche Moleküle:

![ggg](https://raw.githubusercontent.com/deltorobarba/repo/master/sciences_123.png)

> **Liegen beide Chloratome auf der „gleichen Seite", spricht man von der cis-Form, liegen sie auf unterschiedlichen Seiten, spricht man von der trans-Form.**

> **Die cis/trans-Isomerie der Alkene und Cycloalkane ist eine Art der Stereoisomerie.**

![ggg](https://raw.githubusercontent.com/deltorobarba/repo/master/sciences_124.png)


Mehr Infos:

* https://www.u-helmich.de/che/lexikon/C/cis-trans-Isomerie.html

* https://w-hoelzel.de/index.php?option=com_content&view=article&id=311&catid=119

###### *Kohlenwasserstoffe: <font color="blue">Alkine (Alkinylgruppe)</font> - Aliphatisch, ungesättigt (C≡C-Dreifachbindungen), acyclisch*

> [**Alkine**](https://de.m.wikipedia.org/wiki/Alkine) sind ungesättigte Kohlenwasserstoffe mit C≡C-**Dreifachbindungen**

* Zum Beispiel **Ethin C2H2** ((Acetylen))

> ![gg](https://upload.wikimedia.org/wikipedia/commons/thumb/1/1e/Structural_formula_of_acetylene.svg/178px-Structural_formula_of_acetylene.svg.png)

Beim Versuch, Kalium in metallischer Form darzustellen, erhielt man Ethin durch Erhitzen von Kaliumsalzen wie Kaliumacetat oder Kaliumcarbonat mit Kohle, gefolgt von der Reaktion des gebildeten Kaliumcarbids mit Wasser.

**Hydrohalogenierung**: An Alkine kann Halogenwasserstoff in der ersten Stufe zu vinylogenen Halogenalken addiert werden.

* Diese reagieren aber nur unter drastischeren Bedingungen weiter zu den Dihalogenen, da diese wenig reaktiv sind. Hierbei entstehen, nach der Markownikoff-Regel 1,1-Dihalogenalkane.

* Die Hydrochlorierung von Ethin zum [Vinylchlorid](https://de.m.wikipedia.org/wiki/Vinylchlorid) war von großer technischer Bedeutung, da Vinylchlorid der monomere Baustein zur Herstellung von [PVC](https://de.m.wikipedia.org/wiki/Polyvinylchlorid) ist.



Vergleiche dazu das **Alken Ethen (Doppelbindung)**

> ![gg](https://upload.wikimedia.org/wikipedia/commons/thumb/e/e9/Ethene_structural.svg/170px-Ethene_structural.svg.png)

Vergleiche dazu das **Alkan Ethen (Einfachbindung)**

> ![ggg](https://upload.wikimedia.org/wikipedia/commons/thumb/3/35/Ethan_Lewis.svg/142px-Ethan_Lewis.svg.png)

https://de.m.wikipedia.org/wiki/Alkinylgruppe

https://de.m.wikipedia.org/wiki/Polyine

###### *Kohlenwasserstoffe: <font color="blue">Cycloaliphaten</font> - Aliphatisch, gesättigt / ungesättigt, cyclisch*

[Alicyclische Verbindungen](https://de.m.wikipedia.org/wiki/Alicyclische_Verbindungen) nicht aromatische ringförmige Kohlenstoffverbindungen.

Nach Art der aliphatischen Verbindung kann unterschieden werden in:

* [Cycloalkane](https://de.m.wikipedia.org/wiki/Cycloalkane) wie zB [Cyclopentan](https://de.m.wikipedia.org/wiki/Cyclopentan), [Cyclohexan](https://de.m.wikipedia.org/wiki/Cyclohexan)
C6 H12 und [Cycloheptan](https://de.m.wikipedia.org/wiki/Cycloheptan)

  * Cyclohexan wird zur Produktion von Kunstfasern wie Nylon verwendet. Außerdem wird es als Lösungsmittel eingesetzt. Cyclohexan kommt im Erdöl vor. Es ist ein wichtiger Ausgangsstoff für organische Synthesen.

  * [Decalin](https://de.m.wikipedia.org/wiki/Decalin): C10 H18 - Verknüpfung von zwei Cyclohexanringen. Als Lösungsmittel (z. B. in Schuhcreme) verwendet

* [Cycloalkene](https://de.m.wikipedia.org/wiki/Cycloalkene)

* [Cycloalkyne](https://de.wikibrief.org/wiki/Cycloalkyne)

Man kann weiterhin differenzieren in:

* [**Monocyclen**](https://de.m.wikipedia.org/wiki/Cyclische_Verbindungen#Nomenklatur_nach_Anzahl_der_Ringe)

* [**Polycyclen**](https://de.m.wikipedia.org/wiki/Polycyclen): verbundene (kondensierte) Polycyclen, verbrückte Polycyclen und [Spiroverbindungen](https://de.m.wikipedia.org/wiki/Spiroverbindung)

*Beispiel: Polycyclische Ringsysteme bei den Cycloalkanen:*

![gg](https://upload.wikimedia.org/wikipedia/commons/0/01/Cycloalkane.png)

https://www.scinexx.de/news/technik/chemie-erster-fuenfring-mit-nur-zwei-%CF%80-elektronen/

* Neben dem in vielen Steroiden und Terpenen vorkommenden Sechsring des Cyclohexans gibt es in der Natur viele weitere Derivate von kleinen Ringen und Makrocyclen. Die Steroide bauen auf dem Cycloalkan Gonan auf.

* Viele Cycloalkane wie Cyclohexan, Methylcyclohexan und Cyclopentan kommen im Erdöl vor. Dort wurden auch geringe Mengen der kristallinen Feststoffe der Diamantoide gefunden, deren einfachster Vertreter das Adamantan ist.

##### <font color="blue">*Kohlenwasserstoffe ohne funktionelle Gruppe (Aromaten)*

Video: [How chemists create "SUPERBENZENE" (Kekulene) | Organic chemistry, synthesis & reaction mechanisms](https://youtu.be/qrBMD_H6jXs)

https://de.m.wikipedia.org/wiki/Helicene

###### *Cyclische Verbindungen (Cyclic compound)*

https://de.m.wikipedia.org/wiki/Cyclische_Verbindungen

![ggg](https://raw.githubusercontent.com/deltorobarba/repo/master/sciences_1386.png)

###### *Kohlenwasserstoffe: <font color="blue">Arene (Arylgruppe)</font> - <font color="red">Carbocyclen-Aromaten</font>, ungesättigt (C=C-Doppelbindungen), cyclisch*

> **Viele biochemische Stoffe sind [Heteroaromate](https://de.m.wikipedia.org/wiki/Heteroaromaten)!** zB. Aspirin (C9 H8 O4) & Paracetamol bei den substituierten Heteroaromaten, [Aromatic_compound](https://en.m.wikipedia.org/wiki/Aromatic_compound) oder Alkaloide (Einfache + kondensierte Heteroaromaten : Arene (Benzole & Aryle wie Phenol) + Amine (Funktionelle Aminogruppe)). <font color="blue">**Cyclische Amine und Imine sind Bestandteile vieler biochemisch wichtiger Verbindungen (Aminosäuren, Enzyme, Nucleinsäure, Farbstoffe, Alkaloide, Vitamine u. a.) und zahlreicher Arzneimittel.** Auch viele kondensierte heterocyclische Systeme gehören in diese Stoffklasse: Indol, Acridin, Chinolin, Isochinolin, Purin, Pteridin, Alloxazin u. a.

* [**Arene (benzoide bzw. aromatische Kohlenwasserstoffe)**](https://de.m.wikipedia.org/wiki/Aromatische_Kohlenwasserstoffe) sind cyclische, planare und durchkonjugierte [Kohlenwasserstoffe](https://de.m.wikipedia.org/wiki/Kohlenwasserstoffe) mit aromatischem System [(Aromaten)](https://de.m.wikipedia.org/wiki/Aromaten), also **durchgehend benachbarte Doppelbindungen haben**

> Arene sind Kohlenwasserstoffe, die alle Kriterien der Aromatizität erfüllen: es sind ebene, zyklische Verbindungen, mit (4n+2) delokalisierten Elektronen.

*  Aromaten erfüllen die [Aromatizitätskriterien](https://de.m.wikipedia.org/wiki/Aromatizität), die sogenannte [Hückelregel](https://de.m.wikipedia.org/wiki/Hückel-Regel): also 4n + 2π Elektronen besitzen

* Die Elektronen der p-Orbitale sind in einem $\pi$-System delokalisiert, d. h. über den gesamten Ring verteilt. Die Delokalisierung führt zu einer Energieabsenkung, der Mesomerieenergie (Resonanzenergie).

> Einfachste Aromaten sind [Benzol](https://de.m.wikipedia.org/wiki/Benzol) und [Toluol](https://de.m.wikipedia.org/wiki/Toluol)

> <font color="blue">Im Gegensatz zu [Heteroaromaten](https://de.m.wikipedia.org/wiki/Heteroaromaten) **besteht der Ring bei aromatischen Kohlenwasserstoffen nur aus Kohlenstoffatomen**</font>

* Unterteilung der aromatischen Kohlenwasserstoffe (Arene): (siehe auch unter [chemie.de](https://www.chemie.de/lexikon/Aromaten.html#Einteilung_der_Aromaten)


**Unterteilung der Aromaten:**

1. Grundbaustein: Einfaches Aromat (Benzole)
2. Konjugierte, monocyclische Kohlenwasserstoffe: Annulene
3. Kondensierte Aromate (Benzole), die Polyzyklische aromatische Kohlenwasserstoffe (PAK)
4. Substituierte Aromate (Benzole) (Benzolderivate) mit einer Phenylgruppe als Arylgruppe
5. Substituierte + kondensierte Aromate (Benzole) (Benzolderivate) - mit mehreren Benzolringen als Grundgeruest, d.h. anstatt 1 Phenylgruppe gibt es komplexere Arylgruppen
6. Einfache + kondensierte **Heteroaromaten** mit ein oder mehreren Heteroatomen als Ringglieder, wie Stickstoff, Sauerstoff oder Schwefel

**Details der [Hückelregel](https://de.m.wikipedia.org/wiki/Hückel-Regel):**

* Ein planares, cyclisch durchkonjugiertes Molekül ist mit [4n+2] π-Elektronen (einer ungeraden Zahl von Elektronenpaaren; Beispiel: n=1, [4·1+2=6], also drei Elektronenpaare) besonders stabil und wird als Aromat bezeichnet.

* Das wichtigste Beispiel ist [Benzol](https://de.m.wikipedia.org/wiki/Benzol), das mit 6 π-Elektronen (n = 1) die Regel erfüllt und als Sechseck außerdem den idealen Bindungswinkel für sp2-hybridisierte Kohlenwasserstoffe bildet. Polycyclische (kondensierte) Aromaten entstehen, wenn man ein oder mehrere Male weitere 4 Kohlenstoffatome an einer Kante hinzufügt, für sie gilt allerdings die Hückelregel nicht mehr (siehe Einschränkungen).

> ![gg](https://upload.wikimedia.org/wikipedia/commons/thumb/d/da/Benzol.svg/100px-Benzol.svg.png)

![ggg](https://raw.githubusercontent.com/deltorobarba/repo/master/sciences_175.png)

![ggg](https://raw.githubusercontent.com/deltorobarba/repo/master/sciences_174.png)

###### *1. Grundbaustein: Einfaches Aromat (Benzole), Benzen C6 H6*

**1. Grundbaustein: Einfaches Aromat (Benzole)**, [Benzen](https://de.m.wikipedia.org/wiki/Benzol) C6 H6

* [Benzol](https://de.m.wikipedia.org/wiki/Benzol) organische Verbindung, einfacher Sechsring mit sechs konjugierten Bindungen (3 Doppel- und 3 Einfachbindungen). Benzol (nach IUPAC Benzen) ist ein flüssiger organischer Kohlenwasserstoff mit der Summenformel C6H6.

* Es besitzt einen charakteristischen aromatischen Geruch, ist farblos, leicht entzündlich und brennt mit stark rußender Flamme. Es mischt sich mit fast allen organischen Lösungsmitteln, jedoch kaum mit Wasser.

* **Benzol ist die Stammverbindung der aromatischen Kohlenwasserstoffe**.


The various representations of benzene.

![ggg](https://upload.wikimedia.org/wikipedia/commons/thumb/9/9a/Benzene_Representations.svg/640px-Benzene_Representations.svg.png)

Major commodity chemicals and polymers derived from benzene [Source](https://en.m.wikipedia.org/wiki/Benzene)

![gg](https://upload.wikimedia.org/wikipedia/commons/thumb/5/54/Benzene_uses.png/800px-Benzene_uses.png)

###### *2. Konjugierte, monocyclische Kohlenwasserstoffe (Annulene)*

**2. Konjugierte, monocyclische Kohlenwasserstoffe**: [Annulene](https://de.m.wikipedia.org/wiki/Annulene)  bzw. [auf englisch](https://en.m.wikipedia.org/wiki/Annulene)

* [Cyclobutadiene](https://en.m.wikipedia.org/wiki/Cyclobutadiene) ([4]annulene)

* [Benzene](https://en.m.wikipedia.org/wiki/Benzene) ([6]annulene)

* [Cyclooctatetraene](https://en.m.wikipedia.org/wiki/Cyclooctatetraene) ([8]annulene)

* [Cyclododecahexaene](https://en.m.wikipedia.org/wiki/Cyclododecahexaene) ([12]annulene)

* [Cyclotetradecaheptaene](https://en.m.wikipedia.org/wiki/Cyclotetradecaheptaene) ([14]annulene)

* [Cyclooctadecanonaene](https://en.m.wikipedia.org/wiki/Cyclooctadecanonaene) ([18]annulene)

* Cyclodocosahendecaene ([22]-annulene)

![ggg](https://raw.githubusercontent.com/deltorobarba/repo/master/sciences_0743.png)

###### *3. Kondensierte Aromate (Polyzyklische aromatische Kohlenwasserstoffe)*

**3. Kondensierte Aromate (Benzole)**, die [Polyzyklische aromatische Kohlenwasserstoffe (PAK)](https://de.m.wikipedia.org/wiki/Polycyclische_aromatische_Kohlenwasserstoffe), auch genannt [anellierte](https://de.m.wikipedia.org/wiki/Anellierung) Arene genannt, d.h. Kohlenwasserstoffe mit mehreren Ringen.

* [Naphthalin](https://de.m.wikipedia.org/wiki/Naphthalin) C10H8 (2 Ringe; zweimal Sechsring)

* [Azulen](https://de.m.wikipedia.org/wiki/Azulen) C10H8 (2 Ringe; Fünf-/Siebenring)

* [Anthracen](https://de.m.wikipedia.org/wiki/Anthracen) C14 H10 (3 Ringe)

* [Phenanthren](https://de.m.wikipedia.org/wiki/Phenanthren) C14 H10 (siehe Gemeinsamkeit & Unterschied zu Anthracen!)

![ggg](https://raw.githubusercontent.com/deltorobarba/repo/master/sciences_0741.png)

###### *4. Substituierte Aromaten (Aspirin und Paracetamol)*

**4. Substituierte Aromate (Benzole)** (Benzolderivate) mit einer [Phenylgruppe](https://de.m.wikipedia.org/wiki/Phenylgruppe) als [Arylgruppe](https://de.m.wikipedia.org/wiki/Arylgruppe), d.h. 1 Benzolring als Grundgerust (Atomgruppe –C6H5). Am Benzolring haengt ein angefügter Substituent, der aber verschiedene Konstitutionsisomere haben kann

  * [Toluol](https://de.m.wikipedia.org/wiki/Toluol)

  * [Xylole](https://de.m.wikipedia.org/wiki/Xylole)

  * [Phenol](https://de.m.wikipedia.org/wiki/Phenol) C6H5–OH

  * [Nitrobenzol](https://de.m.wikipedia.org/wiki/Nitrobenzol) C6H5–NO2

  * [Anilin](https://de.m.wikipedia.org/wiki/Anilin) (Aminobenzol) C6H5–NH2

  * [Hydrochinon](https://de.m.wikipedia.org/wiki/Hydrochinon) HO–C6H4–OH

  * [Pikrinsäure](https://de.m.wikipedia.org/wiki/Pikrinsäure) (2,4,6-Trinitrophenol) C6H2(OH)(NO2)3

  * Benzaldehyde

  * [Benzoesäure](https://de.m.wikipedia.org/wiki/Benzoesäure)

  * [Brombenzol](https://de.m.wikipedia.org/wiki/Brombenzol)

  * siehe grosse Liste auch unter [Disubstituierte Benzole](https://de.m.wikipedia.org/wiki/Disubstituierte_Benzole)

  * **Special:** [Biaryle](https://de.m.wikipedia.org/wiki/Biaryle) wie [Biphenyl](https://de.m.wikipedia.org/wiki/Biphenyl): separaten (isolierten) Ringe anstatt Aromaten mit gemeinsamen Atomen



![ggg](https://raw.githubusercontent.com/deltorobarba/repo/master/sciences_0739.png)

**Benzene derivatives have from one to six substituents attached to the central benzene core.** Examples of benzene compounds with just one substituent are phenol, which carries a hydroxyl group, and toluene with a methyl group. When there is more than one substituent present on the ring, their spatial relationship becomes important for which the arene substitution patterns ortho, meta, and para are devised. For example, three isomers exist for cresol because the methyl group and the hydroxyl group can be placed next to each other (ortho), one position removed from each other (meta), or two positions removed from each other (para). Xylenol has two methyl groups in addition to the hydroxyl group, and, for this structure, 6 isomers exist.



**Representative arene compounds (siehe Aspirin und Paracetamol!)**

![ggg](https://raw.githubusercontent.com/deltorobarba/repo/master/sciences_0909.png)

###### *5. Substituierte + kondensierte Aromate*

**5. Substituierte + kondensierte Aromate (Benzole)** (Benzolderivate) - mit mehreren Benzolringen als Grundgeruest, d.h. anstatt 1 [Phenylgruppe](https://de.m.wikipedia.org/wiki/Phenylgruppe) gibt es komplexere [Arylgruppen](https://de.m.wikipedia.org/wiki/Arylgruppe) wie zB Naphthyl-Rest (fur 2), Anthryl-Rest (fur 3). An einem Benzolring haengt ein angefügter Substituent.

* [1-Naphthol](https://de.m.wikipedia.org/wiki/1-Naphthol)

* [2-Naphthylamin](https://de.m.wikipedia.org/wiki/2-Naphthylamin)

* 9-Anthracencarbaldehyd

* Phenanthren-9-boronsäure

* Siehe auch [Heteroarylgruppe](https://de.m.wikipedia.org/wiki/Heteroarylgruppe)

![ggg](https://raw.githubusercontent.com/deltorobarba/repo/master/sciences_0742.png)

###### *6. Einfache Heteroaromaten (Heterocyclen) + kondensierte Heteroaromaten (Viele Alkaloide)*

**Einführung**

* Siehe [Heterocyclen](https://de.m.wikipedia.org/wiki/Heterocyclen): Ringmoleküle mit mindestens zwei verschiedenen Elementen im Ring

* *Kohlenwasserstoffe: <font color="blue">Arene (Arylgruppe)</font> - <font color="red">Heteroaromaten</font>, ungesättigt (C=C-Doppelbindungen), cyclisch*

* **Einfache + kondensierte [Heteroaromaten](https://de.m.wikipedia.org/wiki/Heteroaromaten)**  mit ein oder mehreren Heteroatomen als Ringglieder, wie Stickstoff, Sauerstoff oder Schwefel. zwei anellierten aromatischen sechsgliedrigen Ringen.

  * Siehe mehr Verbindungen unter [Aromaten](https://de.m.wikipedia.org/wiki/Aromaten)bzw. [Aromatic Compound](https://en.m.wikipedia.org/wiki/Aromatic_compound)

  * Siehe auch [Organic compound](https://en.m.wikipedia.org/wiki/Organic_compound)


* <font color="blue">**Cyclische Kohlenwasserstoffe (insbesondere Heteroaromaten) sind in vielen Naturstoffen verhanden**
  * **Arene (Benzole & Aryle wie Phenol) + [Amine](https://de.m.wikipedia.org/wiki/Amine) (Funktionelle Aminogruppe) = Cyclische Amine** und Imine
  * in vielen Wirkstoffen / sind Bestandteile vieler biochemisch wichtiger Verbindungen (Aminosäuren, Enzyme, Nucleinsäure, Farbstoffe, Alkaloide, Vitamine u. a.) und zahlreicher Arzneimittel.

[**Monocyclische Heteroaromaten**](https://de.m.wikipedia.org/wiki/Heteroaromaten#Monocyclische_Heteroaromaten):

**Dreiring-Heteroaromaten mit einem Heteroatom:**

* [Epoxide](https://de.m.wikipedia.org/wiki/Epoxide) mit 2 Kohlenstoffatomen und dazwischen ein Sauerstoff-Atom, zB [Propylenoxid](https://de.m.wikipedia.org/wiki/Propylenoxid) C3H6O (Dreiring mit Sauerstoffatom). Siehe auch "Umweltschonende Herstellung von Propylenoxid"

![ggg](https://upload.wikimedia.org/wikipedia/commons/thumb/c/c1/Propylene-oxid-2D.svg/120px-Propylene-oxid-2D.svg.png)

**Fünfring-Heteroaromaten mit einem Heteroatom:**

* [Furan](https://de.m.wikipedia.org/wiki/Furan) C4H4O (Fünfring mit Sauerstoffatom)

![ggg](https://upload.wikimedia.org/wikipedia/commons/thumb/3/35/Furan.svg/66px-Furan.svg.png)

* [Thiophen](https://de.m.wikipedia.org/wiki/Thiophen) C4H4S (Fünfring mit Schwefelatom)

![ggg](https://upload.wikimedia.org/wikipedia/commons/thumb/a/af/Thiophen.svg/66px-Thiophen.svg.png)

**Fünfring-Heteroaromaten mit mehreren Heteroatomen:**

* [Pyrrol](https://de.m.wikipedia.org/wiki/Pyrrol) C4H4NH (Fünfring mit Stickstoff- und Wasserstoffatom)

![ggg](https://upload.wikimedia.org/wikipedia/commons/thumb/f/f3/Pyrrol2.svg/66px-Pyrrol2.svg.png)

**Sechsring-Heteroaromaten mit einem Heteroatom:**

* [Pyridin](https://de.m.wikipedia.org/wiki/Pyridin) C5H5N (Sechsring mit Stickstoffatom). Pyridin ist ein Sechsring-Heteroaromaten mit <u>einem Heteroatom</u>, siehe [Pyridin-Alkaloide](https://de.m.wikipedia.org/wiki/Pyridin-Alkaloide),

* wie zB Tabak, das [Nicotin](https://de.m.wikipedia.org/wiki/Nicotin) - haengt man noch ein Benzol ran, erhaelt man Chinolin ! (Benzopyridin und 1-Azanaphthalin gelegentlich anzutreffen. Benzopyridin beschreibt das Molekül als Pyridin, [Source](https://www.chemie-schule.de/KnowHow/Chinolin).

> ![gg](https://upload.wikimedia.org/wikipedia/commons/thumb/d/dc/Pyridin.svg/70px-Pyridin.svg.png)

**Sechsring-Heteroaromaten mit mehreren Heteroatomen:**

* [Pyrimidin](https://de.m.wikipedia.org/wiki/Pyrimidin) - <font color="red">Organische Nukleinbase: Pyrimidin-Basen: Cytosin, Thymin (in der DNA) und Uracil (in der RNA). Pyrimidin-Derivate spielen auch als Diuretika, Antibiotika, Antimetabolika und in der antiviralen Therapie eine Rolle.

> ![ggg](https://upload.wikimedia.org/wikipedia/commons/thumb/f/f2/Pyrimidin.svg/76px-Pyrimidin.svg.png)

*Beziehung zwischen **Pyridin (=Monocyclische Heteroaromaten)**, dem **Chinolin (=Bicyclische Heteroaromaten)** und **Isochinolin** (Heteroatom an anderer Stelle) unter den Heteroaromaten:*

![ggg](https://raw.githubusercontent.com/deltorobarba/repo/master/sciences_0744.png)

[**Bicyclische Heteroaromaten**](https://de.m.wikipedia.org/wiki/Heteroaromaten#Bicyclische_Heteroaromaten):

* [Indol](https://de.m.wikipedia.org/wiki/Indol) ist ein Benzoanellierte Fünfring-Heteroaromaten, siehe [Indol-Alkaloide](https://de.m.wikipedia.org/wiki/Indolalkaloide) (LSD, Ayahuasca, Strychnin)

* [Chinolin](https://de.m.wikipedia.org/wiki/Chinolin) ist ein Benzoanellierte Sechsring-Heteroaromaten, siehe [Chinolin-Alkaloide](https://de.m.wikipedia.org/wiki/Chinolin-Alkaloide)

* [Purin](https://de.m.wikipedia.org/wiki/Purin) ist ein Heteroanellierte Heteroaromaten, siehe [Purin-Alkaloide](https://de.m.wikipedia.org/wiki/Purinalkaloide) (Koffein). <font color="red">Organische Nukleinbase in DNA & RNA: Purin-Basen: Adenin, Guanin:

> ![ggg](https://upload.wikimedia.org/wikipedia/commons/thumb/1/1f/Purines.svg/640px-Purines.svg.png)


*Purine & Pyrimidin in DNA:*

![ggg](https://raw.githubusercontent.com/deltorobarba/repo/master/sciences_078.png)

![ggg](https://raw.githubusercontent.com/deltorobarba/repo/master/sciences_0738.png)

Coffein enthält einen bicyclischen Heteroaromat mit 4 Stickstoffatomen (Xanthin) - siehe [Purin-Alkaloide](https://de.m.wikipedia.org/wiki/Purinalkaloide)

> ![gg](https://upload.wikimedia.org/wikipedia/commons/thumb/a/a1/Koffein_-_Caffeine.svg/220px-Koffein_-_Caffeine.svg.png)

Nikotin enthält einen heteroaromatischen Sechsring (Pyridin). Siehe [Pyridin-Alkaloide](https://de.m.wikipedia.org/wiki/Pyridin-Alkaloide)

> ![ggg](https://upload.wikimedia.org/wikipedia/commons/thumb/8/81/Nikotin_-_Nicotine.svg/224px-Nikotin_-_Nicotine.svg.png)

Struktur von [Morphin](https://de.m.wikipedia.org/wiki/Morphin), Teil der [Isochinolin-Alkaloide](https://de.m.wikipedia.org/wiki/Isochinolin-Alkaloide)

> ![gg](https://upload.wikimedia.org/wikipedia/commons/thumb/3/33/Morphin_-_Morphine.svg/254px-Morphin_-_Morphine.svg.png)


**Heterocyclen allgemein (Heteroaromaten + Heteroaliphaten)**

> [**Heterocyclen**](https://de.m.wikipedia.org/wiki/Heterocyclen) sind [cyclische chemische Verbindungen](https://de.m.wikipedia.org/wiki/Cyclische_Verbindungen) mit ringbildenden Atomen aus mindestens zwei verschiedenen chemischen Elementen - z.B. in der organischen Chemie enthalten sie außer C- Atomen ein oder mehrere Heteroatome als Ringglieder, wie Stickstoff, Sauerstoff oder Schwefel.

* **Ringe aus fünf und sechs Atomen sind am beständigsten**.

* Aliphatische Heterocyclen werden auch kurz als **Heteroaliphaten** bezeichnet.

* [Aromatische](https://de.m.wikipedia.org/wiki/Aromaten) Heterocyclen werden auch kurz als [Heteroaromaten](https://de.m.wikipedia.org/wiki/Heteroaromaten) bezeichnet.

Nach Sättigungsgrad ergibt sich eine Einteilung in:

* [**Alkane**] gesättigte Heterocycloalkane: [Cycloalkane](https://de.m.wikipedia.org/wiki/Cycloalkane)

* [**Alkene**] partiell ungesättigte Heterocycloalkene: [Cycloalkene](https://de.m.wikipedia.org/wiki/Cycloalkene)

* [**Aromaten**] die [Heteroaromaten](https://de.m.wikipedia.org/wiki/Heteroaromaten)

* [**Alkine**] Besonders in [Makrocyclen](https://de.m.wikipedia.org/wiki/Makrocyclische_Verbindungen) sind selten auch Cycloalkine (vergleiche [Alkine](https://de.m.wikipedia.org/wiki/Alkine)) zu finden.

##### <font color="blue">*Kohlenwasserstoffe mit funktioneller Gruppe*

###### *Funktionelle Gruppen & CHNOPS*

![ggg](https://raw.githubusercontent.com/deltorobarba/repo/master/sciences_1353.jpg)

> **[Funktionelle Gruppen](https://de.m.wikipedia.org/wiki/Funktionelle_Gruppe) sind Atome oder Atomgruppen in den Molekülen organischer Verbindungen. Sie bestimmen maßgeblich die chemischen und physikalischen Eigenschaften der Verbindungen.**

* Substanzen, deren Moleküle die gleiche funktionelle Gruppe tragen, bilden eine **Stoffklasse**. Unterscheiden sich die Moleküle der Stoffklasse nur in der Kettenlänge des Alkylrests, bilden sie eine **homologe Reihe**.

* **Viele organische Moleküle tragen nur eine funktionelle Gruppe**, z.B. die Hydroxy-Gruppe bei den verschiedenen Alkoholen. **Es gibt aber auch Verbindungen, die über mehrere funktionelle Gruppen verfügen**. So enthalten beispielsweise Aminosäuren eine Carboxy-Gruppe und eine Amino-Gruppe im Molekül, die beide die Eigenschaften der Verbindungen beeinflussen.

![ggg](https://raw.githubusercontent.com/deltorobarba/repo/master/sciences_177.jpeg)

Das Akronym [CHNOPS](https://de.m.wikipedia.org/wiki/CHNOPS) bezeichnet die sechs wichtigsten Elemente, die in verschiedenen Kombinationen in den meisten Biomolekülen anzutreffen sind. Die einzelnen Buchstaben stehen für die Abkürzungen der Elemente Kohlenstoff, Wasserstoff, Stickstoff, Sauerstoff, Phosphor und Schwefel. Die Bezeichnung CHNOPS fungiert in diesem Zusammenhang auch als Eselsbrücke für den Chemie- und Biologieunterricht.

* **Kohlenwasserstoffe** sind Verbindungen von Kohlenstoff mit Wasserstoff, die in sämtlichen Lebensformen vorkommen und dort das Rückgrat der Biomoleküle bilden, die durch die Verbindung mit funktionellen Gruppen entstehen.

* **Stickstoff N** (Amino-) ist Bestandteil der funktionellen Gruppe der Amine und als solcher Bestandteil sämtlicher Eiweißverbindungen. Des Weiteren ist er essentieller Bestandteil der Desoxyribonukleinsäure und des Chlorophylls.

* **Sauerstoff O** bildet zahlreiche funktionelle Gruppen und ist für die Atmung fast aller Lebewesen von zentraler Bedeutung.

* **Phosphor Ph** ist die Grundlage für den Aufbau von Phospholipiden, einer Klasse von Lipiden, die einen Hauptbestandteil aller Zellmembranen darstellen, sowie Ionen, Proteine und andere Moleküle dort festhalten, wo sie für die Zellfunktion benötigt werden.

  * Phosphatgruppen sind des Weiteren auch ein essentieller Bestandteil des Rückgrats der Nukleinsäuren und nötig, um ATP – die Hauptmoleküle, die als Energiequelle der Zelle in allen Lebewesen genutzt werden, zu bilden.

* **Schwefel S** ist Bestandteil der Aminosäuren Cystein und Methionin.

**Einteilung der Stoffe in der organischen Chemie** [Source](https://commons.wikimedia.org/wiki/File:Einteilung_der_Stoffe-organische_Chemie.svg)

![ggg](https://raw.githubusercontent.com/deltorobarba/repo/master/sciences_176.png)

[**Hydroxygruppe**](https://de.m.wikipedia.org/wiki/Hydroxygruppe) –OH (Alkohole und Phenole)

* ist die funktionelle Gruppe der Alkohole und Phenole und kommt auch in Kohlenhydraten oder bei Hydroxycarbonsäuren als Strukturelement eines Alkohols vor.

* Die Hydroxygruppe ist aber auch ein Teilelement der Carboxygruppe in Carbonsäuren, dort aber gebunden an das C-Atom einer Carbonylgruppe, das einen höheren Oxidationszustand hat.

* Die Hydroxygruppe ist deshalb in Carbonsäuren kein Strukturelement eines Alkohols, sondern hat die sauren Eigenschaften einer Carbonsäure.

> ![ggg](https://upload.wikimedia.org/wikipedia/commons/thumb/b/b1/Hydroxy_Group_Structural_Formulae.svg/144px-Hydroxy_Group_Structural_Formulae.svg.png)

*Hydroxygruppe (blau markiert) als funktionelle Gruppe eines Alkohols. R ist dann eine Alkylgruppe.*

[**Aminogruppe**](https://de.m.wikipedia.org/wiki/Aminogruppe) –NH2

[**Carbonylgruppe**](https://de.m.wikipedia.org/wiki/Carbonylgruppe) -C=O

> **Die Aldehyd- und Ketogruppe sind Carbonylgruppen, da jeweils ein doppelt gebundenes Sauerstoffatom vorliegt.**

* Trägt die Carbonylverbindung lediglich Alkylreste, so wird die Gruppierung auch als Ketogruppe bezeichnet.

* Den einbindigen organischen Rest, der formal aus z. B. einer organischen Säure nach Abspalten der Hydroxygruppe resultiert, nennt man [Acyl-Gruppe](https://de.m.wikipedia.org/wiki/Acylgruppe)

* [**Aldehydgruppe**](https://de.m.wikipedia.org/wiki/Aldehyde) -CHO

* [**Ketongruppe**](https://de.m.wikipedia.org/wiki/Ketone) Verbindungen mit nicht endständiger Carbonylgruppe >C=O

[**Carboxygruppe**](https://de.m.wikipedia.org/wiki/Carboxygruppe) –COOH

[**Ethergruppe**](https://de.m.wikipedia.org/wiki/Ether) –O–

###### *Halogenkohlenwasserstoffe (-Fl, -Cl, -Br, -I)*

> Halogenkohlenwasserstoff = Halogene + Aliphate oder Aromate

* [**Halogenkohlenwasserstoffe**](https://de.m.wikipedia.org/wiki/Halogenkohlenwasserstoffe) sind Kohlenwasserstoffe, bei denen mindestens ein Wasserstoffatom durch eines der Halogene **Fluor, Chlor, Brom oder Iod** ersetzt wurde.

* Halogenkohlenwasserstoffe sind lipophile (fettlösliche) Substanzen, die vielfach hervorragende organische Lösungsmittel sind.

* Ein wichtiger Vertreter ist der Kunststoff [Polyvinylchlorid](https://de.m.wikipedia.org/wiki/Polyvinylchlorid) (PVC).

* Daneben [Chloroform](https://de.m.wikipedia.org/wiki/Chloroform) oder [Tetrachlormethan](https://de.m.wikipedia.org/wiki/Tetrachlormethan) als bedeutende [Lösungsmittel](https://de.m.wikipedia.org/wiki/Lösungsmittel)

*Struktur verschiedener Klassen halogenierter Kohlenwasserstoffe, von oben nach unten: Halogenalkan, Halogenalken, Halogenaromat. Halogenatom blau markiert:*

![gg](https://upload.wikimedia.org/wikipedia/commons/thumb/d/dc/Halocarbon_General_Formulae_V.1.png/155px-Halocarbon_General_Formulae_V.1.png)

[**Halogenalkane**](https://de.m.wikipedia.org/wiki/Halogenkohlenwasserstoffe#Halogenalkane)

* [Fluoralkane](https://de.m.wikipedia.org/wiki/Fluoralkane), zB Primäres Fluoralkan:

![gg](https://upload.wikimedia.org/wikipedia/commons/thumb/3/30/Prim-Fluoroalkanes_Structural_Formula_V1.png/320px-Prim-Fluoroalkanes_Structural_Formula_V1.png)

* [Chloralkane](https://de.m.wikipedia.org/wiki/Chloralkane)

* [Bromalkane](https://de.m.wikipedia.org/wiki/Bromalkane)

* [Iodalkane](https://de.m.wikipedia.org/wiki/Iodalkane)


[**Halogenalkene**](https://de.m.wikipedia.org/wiki/Halogenkohlenwasserstoffe#Halogenalkene)

[**Aromatische Halogenkohlenwasserstoffe (Halogenaromaten)**](https://de.m.wikipedia.org/wiki/Halogenkohlenwasserstoffe#Aromatische_Halogenkohlenwasserstoffe)

* (auch Halogenarene) sind Derivate von Aromaten oder Arenen, bei denen ein oder mehrere Wasserstoffatome durch Elemente der 7. Hauptgruppe (Halogene) des Periodensystems ersetzt wurden.

* Die meisten aromatischen Halogenverbindungen sind hochgiftig und werden oder wurden als Insektizide oder Fungizide eingesetzt. Viele davon sind mittlerweile verboten.

###### *Hydroxy-Gruppe (-OH) (Alkohole & Phenole)*

![ggg](https://raw.githubusercontent.com/deltorobarba/repo/master/sciences_179.png)

**Alkohole**

> Alkohole = Hydroxygruppe + Aliphate (Alkanolen, Alkenolen, Alkinolen)

![ggg](https://raw.githubusercontent.com/deltorobarba/repo/master/sciences_178.png)

**Homologe Reihe der Alkane**

![ggg](https://raw.githubusercontent.com/deltorobarba/repo/master/sciences_105.png)

**Alkohole basierend auf Alkanenen:**

![ggg](https://raw.githubusercontent.com/deltorobarba/repo/master/sciences_109.png)


> [**Alkohole**](https://de.m.wikipedia.org/wiki/Alkohole) ist eine organische Verbindung mit einer oder mehreren an **aliphatische Kohlenstoffatome** gebundenen Hydroxygruppen

![ggg](https://upload.wikimedia.org/wikipedia/commons/thumb/4/4d/Alkohol_-_Alcohol.svg/115px-Alkohol_-_Alcohol.svg.png)

* Oben: Funktionelle Gruppe der Alkohole ist die blau markierte Hydroxygruppe. **R ist ein Alkyl-, Alkenyl- oder Alkinyl-Rest**, jedoch kein Aryl-Rest, Acyl-Rest oder ein Heteroatom.

* Unten links im Bild ist der organische Teil des Moleküls, den so genannten Alklyrest, rechts in blau ist die funktionelle Gruppe des Alkohols, die Hydroxygruppe mit ihrer stark polaren O-H-Bindung.

* **Beispiel unten ist eine Ethylgruppe (C2H5) aus den Alkanen + eine OH-Hydroxygruppe. Das ist dann [Ethanol](https://de.m.wikipedia.org/wiki/Ethanol) genannt.**

![ggg](https://raw.githubusercontent.com/deltorobarba/repo/master/sciences_108.png)

* **Beide Teile des Moleküls bestimmen die physikalischen Eigenschaften der Alkohole**.

  * Die Alkylgruppe verleiht den Alkoholen teilweise Eigenschaften der Alkane,. **Je länger die Ketten, desto hydrophober / lipophiler ist der Alkohol**.

  * während die OH-Gruppe den Alkoholen zum Teil Eigenschaften des Wassers verleiht.

  * Welche Gruppe einen stärkeren Einfluss hat, hängt von der Größe des Alkylrestes und von der Anzahl der OH-Gruppen ab sowie von weiteren Faktoren wie beispielsweise den Verzweigungsgrad des Moleküls und der Stellung der OH-Gruppe(n).

* Um die physikalischen Eigenschaften der kurzkettigen Alkohole besser zu verstehen, vor allem die gute Wasserlöslichkeit und die hohen Siedetemperaturen, muss man sich intensiv mit dem Aufbau und den Eigenschaften des Wassers beschäftigt haben. [Source](http://www.u-helmich.de/che/Sek2/Organik/Klassen/Alkohole/2_eigenschaften/wasser.html)

Quelle: https://chemiezauber.de/inhalt/basic-5-kl-10-kohlenstoffverbindungen-organische-chemie-2/alkohole/homologe-reihe-der-n-alkanole.html


> [Glycerin](https://de.m.wikipedia.org/wiki/Glycerin)  ist der Trivialname und die gebräuchliche Bezeichnung von Propan-1,2,3-triol. Glycerin ist ein [Zuckeralkohol (Alditole)](https://de.m.wikipedia.org/wiki/Alditole) und der einfachste dreiwertige Alkohol, ein Triol. Glycerin ist in allen natürlichen Fetten und fetten Ölen – z. B. Pflanzenölen – chemisch gebunden als [Fettsäureester (Carbonsäureester)](https://de.m.wikipedia.org/wiki/Carbonsäureester) [(Triglyceride)](https://de.m.wikipedia.org/wiki/Triglyceride) vorhanden und spielt eine zentrale Rolle als Zwischenprodukt in verschiedenen Stoffwechselprozessen. Als Lebensmittelzusatzstoff trägt es das Kürzel E 422.



Siehe auch [**minderwertige Alkohole**](https://de.m.wikipedia.org/wiki/Alkohole#Wertigkeit_der_Alkohole), die [https://de.m.wikipedia.org/wiki/Polyole](Polyole): Gruppe von organischen Verbindungen, die mehrere Hydroxygruppen enthalten -  mit den einfachsten Vertretern Monoethylenglykol und Glycerin!

> ![](https://upload.wikimedia.org/wikipedia/commons/thumb/1/1f/Pentaerythritol.svg/320px-Pentaerythritol.svg.png)

**Unterschiedliche Arten an Alkoholen**


*Einwertige und mehrwertige Alkohole*

Trägt in Alkohol-Molekül nur eine Hydroxylgruppe, handelt es sich um einen einwertigen Alkohol, bei mehreren OH-Gruppen um einen mehrwertigen Alkohol.

Dreiwertiger Alkohol: 1,2,3-Propantriol (Trivialname Glycerin)

> ![gg](https://upload.wikimedia.org/wikipedia/commons/thumb/f/fb/Glycerin_-_Glycerol.svg/320px-Glycerin_-_Glycerol.svg.png)

[**Alkanole**](https://de.m.wikipedia.org/wiki/Alkanole): Stoffgruppe, die sich von den Alkanen (gesättigten Kohlenwasserstoffen) ableitet. Als funktionelle Gruppe besitzen alle Alkanole eine oder mehrere Hydroxylgruppe(n).

**Eine Hydroxylgruppe:**

* Alkanole mit einer cyclischen Alkylgruppe

  * [Methanol](https://de.m.wikipedia.org/wiki/Methanol)

  * [Ethanol](https://de.m.wikipedia.org/wiki/Ethanol)

  * [Propanole](https://de.m.wikipedia.org/wiki/Propanole)

  * [Butanole](https://de.m.wikipedia.org/wiki/Butanole)

  * [Fettalkohole](https://de.m.wikipedia.org/wiki/Fettalkohole), Fettalkohole finden sich in natürlichen Wachsen

* Alkanole mit einer cyclischen Alkylgruppe nennt man [**Cycloalkanole**](https://de.m.wikipedia.org/wiki/Cycloalkanole), Beispiel: [Cyclohexanol](https://de.m.wikipedia.org/wiki/Cyclohexanol)

**Zwei oder mehr Hydroxylgruppen:**

* Alkanole mit zwei Hydroxygruppen nennt man [**Alkandiole**](https://de.m.wikipedia.org/wiki/Alkandiole), einfachstes Beispiel: [Ethylenglycol](https://de.m.wikipedia.org/wiki/Ethylenglycol)

* Alkanole mit drei Hydroxygruppen sind **Alkantriole**, einfachstes Beispiel ist [Glycerin](https://de.m.wikipedia.org/wiki/Glycerin).

![ggg](https://raw.githubusercontent.com/deltorobarba/repo/master/sciences_113.png)

*Primäre, sekundäre und tertiäre Alkohole*

Betrachten wir das Kohlenstoffatom, das die Hydroxy|gruppe bindet. Befinden sich an diesem Kohlenstoffatom in weiteres Kohlenstoffatom und ansonsten nur Wasserstoffatome, spricht man von einem primären Alkohol. Falls sich zwei weitere Kohlenstoffatome darn befinden, heißt es sekundärer Alkohol.
bei drei Kohlenstoffatomen tertiärer Alkohol.

* Primäre Alkohole: werden zu Aldehyden oxidiert

* sekundäre Alkohole: werden zu Ketonen oxidiert

* tertiäre Alkohole: werden nicht oxidiert

![ggg](https://raw.githubusercontent.com/deltorobarba/repo/master/sciences_126.png)

![ggg](https://raw.githubusercontent.com/deltorobarba/repo/master/sciences_127.png)


**Alkohole als Lösungsmittel**

* Allgemein gilt der Grundsatz "Gleiches löst sich in Gleichem". Somit lösen sich Stoffe in den Lösungsmitteln, die in ihrem Aufbau und somit auch in ihrer Polarität ähnlich sind. Bei Alkohol ist das anders.

* In Experimenten zeigt sich, dass **Ethanol sowohl mit Wasser als auch mit Benzin mischbar ist**. Das ist an sich sehr ungewöhnlich, **denn Wasser und Benzin lösen sich (fast!) überhaupt nicht miteinander**. Also muss das Molekül von Ethanol etwas Besonderes an sich haben, so dass es in der Lage ist, sich mit zwei so unterschiedlichen Stoffen vermischen zu können.

* Um das Lösungsverhalten eines Stoffes gegenüber Wasser zu charakterisieren, benutzt man die beiden Begriffe "hydrophil" und "hydrophob".

  * Ein hydrophiler Stoff ist somit sehr gut wasserlöslich. **Stoffe, die sich gut mit Wasser mischen, sind immer polar (so wie das Wasser) oder elektrisch geladen.**

  * Man bezeichnet einen Stoff als hydrophob, wenn er schlecht oder gar nicht in Wasser löslich ist. **Das sind immer unpolare oder elektrisch neutrale Moleküle. Z. B. Fette, Öle und Alkane sind hydrophobe Stoffe**. Ist ein Stoff fettlöslich, so bezeichnet man ihn auch als lipophil (griechisch lipos = Fett).


**Die Erklärung für das Lösungsverhalten von Ethanol**

* Diese liefert uns sein molekularer Aufbau. Das Lösungsverhalten dieser Verbindung wird also durch die unterschiedlichen Eigenschaften der beiden Molekülteile bestimmt.

> **Zum einen enthält sein Molekül eine Hydroxylgruppe (OH-Gruppe), die dem Wassermolekül ähnlich ist.**

* **Die Fähigkeit, sich in Wasser zu lösen, verdankt das Ethanol dieser funktionellen Gruppe**. Deshalb bezeichnet man diesen Alkohol auch als einen hydrophilen (wasserliebenden) Stoff.

* Doch woran liegt das? Die Bindung zwischen dem Wasserstoff- und Sauerstoffatom ist polar, weil das Sauerstoffatom die Bindungselektronen aufgrund seiner hohen Elektronegativität zu sich zieht und es nun eine höhere Elektronendichte hat. Somit gibt es eine positive Teilladung (δ+) am Wasserstoffatom und eine negative (δ-) am Sauerstoff. **Ethanolmoleküle sind also Dipole - wie die Wassermoleküle. Beide Moleküle weisen die gleiche Polarität auf und sind deshalb ineinander löslich.**

* Obendrein können beide Verbindungen **Wasserstoffbrücken** untereinander ausbilden, wodurch Anziehungskräfte zwischen den Molekülen entstehen. Das ist ein weiterer Aspekt, der die Vermischung begünstigt. Ethanolmoleküle können jedoch nur eine Brücke zu ihrem Nachbarn ausbilden, Wassermoleküle hingegen zwei.

> **Zum anderen besitzt das Ethanol eine Alkylgruppe, einen benzinartigen Kohlenwasserstoffrest.**

* Zwischen diesem Rest und Benzin herrschen **Van der Waals-Kräfte**; das sind **Anziehungskräfte zwischen unpolaren Molekülen**.

* Die Kohlenwasserstoffe sind daher auch hydrophob (wasserabstoßend). Dank der Alkylgruppe löst Ethanol auch unpolare Stoffe, wie z. B. Fette.

* Ethanol kann in gewissem Umfang Wasser und Benzin "verbinden". Es bilden sich auf diese Weise sogar ternäre Gemische. Hierauf beruht auch eine [Entwässerungsmethode für Alkohole.](https://www.chemieunterricht.de/dc2/r-oh/alk-dest.htm)

> Ethanol hat demnach sowohl einen hydrophilen als auch lipophilen Teil, was sein hervorragendes Lösungsverhalten erklärt.

![ggg](https://raw.githubusercontent.com/deltorobarba/repo/master/sciences_110.png)


**Je länger die Ketten, desto hydrophober / lipophiler ist der Alkohol**

* Nur die ersten drei Alkohole sind in Wasser in jedem Mischungsverhältnis löslich (Methanol, Ethanol, Propanol)

* Butanol zum Beispiel zeigt bereits schon eine Mischungslücke. Das erkennt man daran, dass sich beim Vermischen der beiden Flüssigkeiten zwei Phasen bilden: Eine wasserreiche Phase (unten) und eine butanolreiche Phase (oben).

* Besonders langkettige Alkohole (Fettsäure-Alkohole) verhalten sich vollständig wasserabweisend. Wenn man sie jedoch in Benzin löst und auf Wasser gießt, dann bildet sich eine pelzartige Schicht von Alkoholmolekülen auf der Wasseroberfläche - der Alkohol verhält sich wie ein Tensid. Die OH-Gruppe taucht ins Wasser, die lipophile Kohlenwasserstoffkette zeigt in die unpolare Luft

Source: https://www.chemieunterricht.de/dc2/r-oh/alk-loemi.htm

[**Alkenole**](https://de.m.wikipedia.org/wiki/Alkenole) (mit einer C=C-Doppelbindung) - Stoffgruppe von **ungesättigten** Kohlenwasserstoffen mit eine oder mehrere Hydroxylgruppe(n) als funktionelle Gruppe

[**Alkinole**](https://de.m.wikipedia.org/wiki/Alkinole) (mit einer C≡C-Dreifachbindung) - Stoffgruppe von **ungesättigten** Kohlenwasserstoffen mit eine oder mehrere Hydroxylgruppe(n) als funktionelle Gruppe

**Phenole**

> Phenole = Hydroxygruppe + Aromate (Arene)

![ggg](https://raw.githubusercontent.com/deltorobarba/repo/master/sciences_179.png)

![ggg](https://raw.githubusercontent.com/deltorobarba/repo/master/sciences_180.png)

[**Phenole**](https://de.m.wikipedia.org/wiki/Phenole) sind Verbindungen, die aus einem aromatischen Ring (Arene) und einer oder mehreren daran gebundenen Hydroxygruppen bestehen. Nach der chemischen Nomenklatur werden Phenole durch Anhängen der Nachsilbe -ol oder Voranstellen der Vorsilbe Hydroxy- bezeichnet.


> ![ggg](https://upload.wikimedia.org/wikipedia/commons/thumb/8/8b/Phenol2.svg/70px-Phenol2.svg.png)

[*Phenol*](https://de.m.wikipedia.org/wiki/Phenol)

>![ggg](https://upload.wikimedia.org/wikipedia/commons/thumb/e/ed/Brenzcatechin.svg/129px-Brenzcatechin.svg.png)
[*1,2-Dihydroxybenzol (Brenzcatechin)*](https://de.m.wikipedia.org/wiki/Brenzcatechin)



###### *Amino-Gruppe (-NH2)*

![ggg](https://raw.githubusercontent.com/deltorobarba/repo/master/sciences_0735.jpg)

[Stickstoff](https://de.m.wikipedia.org/wiki/Stickstoff): N

[Ammoniak](https://de.m.wikipedia.org/wiki/Ammoniak): (NH3)

[Amine (Aminogruppen)](https://de.m.wikipedia.org/wiki/Amine): (R-NH2, R-NH-R, R-NR2, R-NR3+)

[Nitrate](https://de.m.wikipedia.org/wiki/Nitrate): (NO3)

Das einfachste aromatische Amin [Anilin](https://de.m.wikipedia.org/wiki/Anilin) ist flüssig.

Cyclische Kohlenwasserstoffe (insbesondere Heteroaromaten) sind in vielen Naturstoffen verhanden!
* Arene (Benzole & Aryle wie Phenol) + Amine (Funktionelle Aminogruppe) = Cyclische Amine und Imine
* in vielen Wirkstoffen / sind Bestandteile vieler biochemisch wichtiger Verbindungen (Aminosäuren, Enzyme, Nucleinsäure, Farbstoffe, Alkaloide, Vitamine u. a.) und zahlreicher Arzneimittel.

> Als [Amine](https://de.m.wikipedia.org/wiki/Amine) werden organische Verbindungen bezeichnet, die letztlich Derivate des Ammoniaks (NH3) sind und bei denen ein, zwei oder alle drei Wasserstoffatome des Ammoniaks durch [Alkylgruppen](https://de.m.wikipedia.org/wiki/Alkylgruppe) (Alkangruppen) oder [Arylgruppen](https://de.m.wikipedia.org/wiki/Arylgruppe) ersetzt sind oder sich zu mehr oder weniger unterschiedlichen heterocyclischen Ringsystemen zusammengeschlossen haben.
* Arene (Benzole & Aryle wie Phenol): Cyclische Kohlenwasserstoffe in vielen Naturstoffen verhanden, in Verbindung mit Aminen: "Funktionelle Aminogruppe" -
*!Cyclische Amine und Imine sind Bestandteile vieler biochemisch wichtiger Verbindungen (Aminosäuren, Enzyme, Nucleinsäure, Farbstoffe, Alkaloide, Vitamine u. a.) und zahlreicher Arzneimittel.

> Als [Aminogruppe](https://de.m.wikipedia.org/wiki/Aminogruppe) wird in der organischen Chemie die funktionelle Gruppe (-NH2) der primären Amine und Aminosäuren verstanden.

* Als Aminogruppe wird eine basische funktionelle Gruppe bezeichnet, die aus einem Stickstoff- und zwei Wasserstoffatomen besteht und damit die Formel -NH2 hat. Sie ist ein primäres Amin.

* Aminogruppen sind Bestandteile vieler organischer Moleküle, unter anderem Aminosäuren, Nukleotide, Vitamine und Alkaloide. [Soirce](https://flexikon.doccheck.com/de/Aminogruppe)

![ggg](https://raw.githubusercontent.com/deltorobarba/repo/master/sciences_181.png)

**Primäre, sekundäre und tertiäre Amine** mit der blau markierten funktionellen Gruppe. R1 bis R3 ist ein Organyl-Rest (Alkyl-Rest, Aryl-Rest, Alkylaryl-Rest etc.), jedoch kein Acyl-Rest:

> ![ggg](https://upload.wikimedia.org/wikipedia/commons/thumb/0/04/Prim_sec_tert_Amines_General_Formulae.png/320px-Prim_sec_tert_Amines_General_Formulae.png)

* Die Aminogruppe ist eine basische funktionelle Gruppe, da das freie Elektronenpaar am Stickstoff-Atom – analog zu dem im Ammoniak-Molekül – ein Proton von einer Säure HA aufnehmen kann. Dadurch geht sie in ein Ammoniumsalz über.

* Viele Arzneistoffe sind Ammoniumsalze (Hydrofluoride, Hydrochloride, Hydrobromide, Hydroiodide, Maleate, Acetate etc.), da diese stabiler (länger lagerfähig) und häufig besser resorbierbar sind als die freien Amine.

> [Ammoniak](https://de.m.wikipedia.org/wiki/Ammoniak): Gasförmige chemische Verbindung mit der Summenformel NH3

* [Amine](https://de.m.wikipedia.org/wiki/Amine) können als Substitutionsprodukte des Ammoniaks aufgefasst
werden. Nach der Anzahl der im NH -Molekül durch andere Gruppen
ersetzten H-Atome unterscheidet man primäre, sekundäre und tertiäre
Amine. Die Substitutionsbezeichnungen beziehen sich auf das N-
Atom; demzufolge ist das tertiär-Butylamin ein primäres Amin. Falls der
Stickstoff vier Substituenten trägt, spricht man von (quartären) Ammoni
um-Verbindungen

* Unter Di- und Triaminen versteht man aliphatische oder aromatische
Kohlenwasserstoff-Verbindungen, die im Molekül zwei oder drei NH2-
Gruppen besitzen.

* Cyclische Amine gehören zu der umfangreichen Substanzklasse der hete-
rocyclischen Verbindungen.

  * Es sind ringförmige Kohlenwas-
serstoffe (zumeist 5- und 6-Ringe), in denen eine oder mehrere CH- bzw.
CH:-Gruppen durch >NH bzw. ›N- ersetzt sind. Es gibt gesättigte, par-
tiell ungesättigte und aromatische Systeme.

  * <font color="blue">**Cyclische Amine und Imine sind Bestandteile vieler biochemisch wichtiger Verbindungen (Aminosäuren, Enzyme, Nucleinsäure, Farbstoffe, Alkaloide, Vitamine u. a.) und zahlreicher Arzneimittel.** Auch viele kondensierte heterocyclische Systeme gehören in diese Stoffklasse: Indol, Acridin, Chinolin, Isochinolin, Purin, Pteridin, Alloxazin u. a.

* Große Bedeutung und weite Verbreitung haben Amine auch deshalb,
weil viele Verbindungen funktionelle Gruppen besitzen, die sich formal
von den Aminen ableiten

**Aminosäuren (AS)**

> Carbonsäure (COOH) + Aminogruppe (NH2) = Aminosäure

* unüblich aber genauer auch Aminocarbonsäuren, veraltet Amidosäuren genannt, sind chemische Verbindungen mit einer Stickstoff (N) enthaltenden **Aminogruppe** und einer Kohlenstoff (C) und Sauerstoff (O) enthaltenden **Carbonsäuregruppe.**

* Aminosäuren kommen in allen Lebewesen vor. Sie sind die Bausteine von Proteinen (Eiweiß) und werden frei bei der Zerlegung von Proteinen (Proteolyse).

* Essentielle Aminosäuren kann ein Organismus nicht selber herstellen, sie müssen daher mit der Nahrung aufgenommen werden.

*Grundstruktur von α-Aminosäuren (Rest R ist im Fall von Glycin ein H-Atom)*

> ![ggg](https://upload.wikimedia.org/wikipedia/commons/thumb/c/ce/AminoAcidball.svg/320px-AminoAcidball.svg.png)



* [Amine](https://de.m.wikipedia.org/wiki/Amine) sind  organische Verbindungen, die letztlich Derivate des Ammoniaks (NH3) sind und bei denen ein, zwei oder alle drei Wasserstoffatome des Ammoniaks durch **Alkylgruppen** oder **Arylgruppen** ersetzt sind ([=Aromatische Kohlenwasserstoffe](https://de.m.wikipedia.org/wiki/Aromatische_Kohlenwasserstoffe)) oder sich zu mehr oder weniger unterschiedlichen heterocyclischen Ringsystemen zusammengeschlossen haben.

* Amide

* Diazoniumsalze

* Nitroverbindungen, beispielsweise TNT

* Nitrile

**Carbonsäureamide**

Amide: https://de.m.wikipedia.org/wiki/Carbonsäureamide

* organische Verbindungen aus Ammoniak und Carbonsäuren

https://de.m.wikipedia.org/wiki/Acrylamid

https://www.efsa.europa.eu/de/topics/topic/acrylamide

###### *Ether-Gruppe (-O-)*

[**Ether**](https://de.m.wikipedia.org/wiki/Ether)

> ![ggg](https://upload.wikimedia.org/wikipedia/commons/thumb/6/6d/FunktionelleGruppen_Ether.svg/126px-FunktionelleGruppen_Ether.svg.png)

Ales Ether werden in der Chemie organische Verbindungen bezeichnet, die als funktionelle Gruppe eine Ethergruppe – ein Sauerstoffatom, das mit zwei Organylresten substituiert ist – besitzen

https://www.lernhelfer.de/schuelerlexikon/chemie/artikel/waschmittel

Nichtionische Tenside (Beispiel: Fettalkoholpolyglykolether) besitzen einen polaren Teil, der im Gegensatz zu den ionischen Tensiden, nicht elektrisch geladen ist, **sondern mehrere Ethergruppen enthält**. Diese bewirken hier die Löslichkeit im polaren Lösungsmittel (z.B. Wasser).

Auf Grund der Unabhängigkeit der Waschwirkung vom pH-Wert, einer geringeren Schaumbildung sowie einer verbesserten Waschwirkung im Temperaturbereich von 30 bis 60°C gegenüber Alkylbenzolsulfonaten steigern die nichtionischen Tenside ihren Marktanteil. Jedoch verhindern der hohe Preis und die aufwändige Produktion dieser Tenside einen stärkeren Einsatz solcher waschaktiver Substanzen.



###### *Carbonyl-Gruppe (-C=O/-R)*

https://de.m.wikipedia.org/wiki/Carbonylgruppe

![gg](https://upload.wikimedia.org/wikipedia/commons/thumb/5/5a/Carbonyl_Group_V.2.png/263px-Carbonyl_Group_V.2.png)

###### *Ketone (-C=O/-R) - Teil der Carbonyl-Gruppe*

> Ketone = Ketongruppe + Alkane

[**Ketone**](https://de.m.wikipedia.org/wiki/Ketone)

* Alkan (methylgruppe, kohlenwasserstoff) + Keton (funktionelle gruppe CO als Ketogruppe, Art einer Carbonylgruppe)

* Im Gegensatz zu Aldehyden lassen sich Ketone nicht oxidieren

> ![ggg](https://upload.wikimedia.org/wikipedia/commons/thumb/6/60/Ketone_Structural_Formula_V.2.svg/111px-Ketone_Structural_Formula_V.2.svg.png)

* werden zB in Nagellack- oder Fleckentfernern verwendet

* Ketone sind zB im Hormon Testosteron

* Aceton dient als universelles Loesungsmittel

[**Propanon (Aceton)**](https://de.m.wikipedia.org/wiki/Aceton)

> ![ggg](https://upload.wikimedia.org/wikipedia/commons/thumb/a/ab/Aceton_Strukturformel.svg/181px-Aceton_Strukturformel.svg.png)

[**Butanon**](https://de.m.wikipedia.org/wiki/Butanon)

> ![ggg](https://upload.wikimedia.org/wikipedia/commons/thumb/c/c3/Butan-2-one_200.svg/320px-Butan-2-one_200.svg.png)

[**Pentanone**](https://de.m.wikipedia.org/wiki/Pentanone)

[**Hexanone**](https://de.m.wikipedia.org/wiki/Hexanone)

###### *Aldehydgruppe (-C=O/-H) - Teil der Carbonyl-Gruppe, Teil der Ketone*

https://de.m.wikipedia.org/wiki/Aldehyde

https://de.m.wikipedia.org/wiki/Acetaldehyd

https://de.m.wikipedia.org/wiki/Acetaldehydsyndrom

**Aldehyde**

> Aldehyde = Aldehydgruppe + Aliphate oder Arene (wie Alkanale)

[**Aldehyde**](https://de.m.wikipedia.org/wiki/Aldehyde) sind eine Klasse organischer Verbindungen mit der funktionellen Gruppe −CHO (Aldehydgruppe oder auch Formylgruppe) + Organyl-Rest (Alkyl-, Aryl-, Alkenyl-Rest etc.)

* Aldehyde = alcoholus dehydrogenatus, „dehydrierter Alkohol“ oder „Alkohol, dem Wasserstoff entzogen wurde“

> ![gg](https://upload.wikimedia.org/wikipedia/commons/thumb/9/95/FunktionelleGruppen_Aldehyde.svg/114px-FunktionelleGruppen_Aldehyde.svg.png)

*Allgemeine Struktur eines Aldehyds. Der Rest R kann ein Wasserstoffatom oder ein **Organyl-Rest (Alkyl-, Aryl-, Alkenyl-Rest etc.)** sein. Die Aldehydgruppe (Formylgruppe) ist blau gekennzeichnet.*

> ![ggg](https://upload.wikimedia.org/wikipedia/commons/thumb/6/63/Simple_Aldehydes_Structural_Formulae_V.1.svg/320px-Simple_Aldehydes_Structural_Formulae_V.1.svg.png)

*Beispiele: Formaldehyd (Methanal, links), Acetaldehyd (Ethanal, Mitte) und Propionaldehyd (Propanal, rechts) mit blau gekennzeichneter Aldehydgruppe (Formylgruppe)*

[**Methanal (Formaldehyd)**](https://de.m.wikipedia.org/wiki/Formaldehyd)

> ![gg](https://upload.wikimedia.org/wikipedia/commons/thumb/0/0c/Structural_formula_of_formaldehyde.svg/114px-Structural_formula_of_formaldehyde.svg.png)

[**Ethanal (Acetaldehyd)**](https://de.m.wikipedia.org/wiki/Acetaldehyd)

[**Propanal (Propionaldehyd)**](https://de.m.wikipedia.org/wiki/Propionaldehyd)

[**Butanal**](https://de.m.wikipedia.org/wiki/Butanal)

###### *Carboxy-Gruppe (-C=O/-OH)*

[**Carbonsäuren**](https://de.m.wikipedia.org/wiki/Carbonsäuren)

* Carbonsäuren = Carboxygruppe (COOH) + Organylrest

* Leiten sich Carbonsäuren von Alkanen ab, werden sie auch Alkansäuren genannt

* **Langkettige Carbonsäuren** sind an der Bildung von Fetten (Lipiden) beteiligt. Daher werden sie auch **Fettsäuren** genannt.

* Carbonsäuren werden durch Oxidation von Alkoholen (Alkanolen) oder von Aldehyden (Alkanalen) hergestellt

> ![ggg](https://upload.wikimedia.org/wikipedia/commons/thumb/2/28/Carboxylic_Acid_General_Structure_V.1.svg/173px-Carboxylic_Acid_General_Structure_V.1.svg.png)

https://www.seilnacht.com/Lexikon/carbons.html

[**Methansäure (Ameisensäure)**](https://de.m.wikipedia.org/wiki/Ameisensäure)

* C H2 O2

> ![ggg](https://upload.wikimedia.org/wikipedia/commons/thumb/f/f7/Formic_acid.svg/166px-Formic_acid.svg.png)

[**Ethansäure (Essigsäure)**](https://de.m.wikipedia.org/wiki/Essigsäure)

* C2 H4 O2

> ![gg](https://upload.wikimedia.org/wikipedia/commons/thumb/d/d6/Kyselina_octová.svg/169px-Kyselina_octová.svg.png)

[**Propansäure (Propionsäure)**](https://de.m.wikipedia.org/wiki/Propionsäure)

* C3 H6 O2

> ![ggg](https://upload.wikimedia.org/wikipedia/commons/thumb/6/68/Propionsäure_Skelett.svg/320px-Propionsäure_Skelett.svg.png)

[**Butansäure (Buttersäure)**](https://de.m.wikipedia.org/wiki/Buttersäure)

* Buttersäure ist der Trivialname der Butansäure, einer Carbonsäure und gleichzeitig der einfachsten Fettsäure.

* C4 H8 O2

> ![gg](https://upload.wikimedia.org/wikipedia/commons/thumb/7/76/Buttersäure_Skelett.svg/320px-Buttersäure_Skelett.svg.png)

![ggg](https://raw.githubusercontent.com/deltorobarba/repo/master/sciences_182.jpeg)

###### *Methylengruppe (R-CH2-R)*

https://de.m.wikipedia.org/wiki/Methylengruppe

![gg](https://upload.wikimedia.org/wikipedia/commons/thumb/f/f5/Methylene_group_expanded.svg/115px-Methylene_group_expanded.svg.png)

![gg](https://upload.wikimedia.org/wikipedia/commons/thumb/0/05/Methylene_Group_General_Formulae_V.2.svg/514px-Methylene_Group_General_Formulae_V.2.svg.png)

###### *Hydroxymethylgruppe (-CH2–OH)*

https://de.m.wikipedia.org/wiki/Hydroxymethylgruppe

![gg](https://upload.wikimedia.org/wikipedia/commons/thumb/2/2a/Hydroxymethyl_Group_General_Formulae.png/149px-Hydroxymethyl_Group_General_Formulae.png)

###### *Ester (Carboxy-Gruppe -COOH + Hydroxy-Gruppe -OH)*

[**Ester**](https://de.m.wikipedia.org/wiki/Ester)

> **Ester = Carbonsäure (Cabroxy-Gruppe) + Alkohole (Hydroxy-Gruppe)**

* **Die Veresterung ist eine typische, auch vielfach technisch genutzte Reaktion der Carbonsäuren**

* sind ein Typ von organischen Verbindungen aus einer Säure und einem Alkohol. Ein Ester entsteht durch eine Reaktion aus Carbonsäure mit Alkohol

* Organylgruppe + funktionelle Gruppe der Ester (COOC) oder auch von Carbonsäureester

> ![ggg](https://upload.wikimedia.org/wikipedia/commons/c/ce/Ester_Teile.png)

*Essigsäureethylester als Beispiel für einen Carbonsäureester, der säurekatalysiert aus Essigsäure und Ethanol synthetisiert wurde.*


###### *Kohlenstoff mit Phosphor (-H3PO4)*

https://de.m.wikipedia.org/wiki/Phosphorsäureester


Man unterscheidet zwischen Monoester, Diester und Triester: Monoester, Diester und Triester der ortho-Phosphorsäure (von links nach rechts)

![gg](https://upload.wikimedia.org/wikipedia/commons/thumb/5/53/Phosphorsäureester.png/799px-Phosphorsäureester.png)

ADP, DNA, sarin

Phosphor ist zum Beispiel Bestandteil unserer DNA. Und abgesehen davon auch von "ATP", Adenosin-Tri-Phosphat. ATP ist ein Nukleotid und der wichtigste Energieträger in unseren Zellen. Ohne Phosphor, kein Leben.

**Phosphorverbindungen**
* Phosphorsäureester

* Phosphine, beispielsweise Triphenylphosphin

> Phosphate sind die Salze und Ester der [Orthophosphorsäure](https://de.m.wikipedia.org/wiki/Phosphorsäure) (H3PO4)

https://de.m.wikipedia.org/wiki/Phosphate

* In der menschlichen Ernährung spielt Phosphat eine wesentliche Rolle im Energiestoffwechsel und im Knochenumbau. Es verbindet sich mit Calcium zum festen [Calciumapatit](https://de.m.wikipedia.org/wiki/Hydroxylapatit). Der Phosphatspiegel steht im engen Zusammenhang mit dem Calciumspiegel. Die Bedeutung von Phosphat für das Auftreten von Hyperaktivität bei Kindern gilt als widerlegt.

* Mit Ausnahme der Alkali- und Ammonium-Verbindungen sind die meisten Phosphate schlecht wasserlöslich.

* Phosphate können Verbindungen mit Schwermetallen eingehen. Diese Eigenschaft macht die Verwendung von Phosphaten problematisch, da die Phosphate aus dem Klärschlamm Schwermetalle mobilisieren können. Zum überwiegenden Teil enthalten Lagerstätten von Phosphatverbindungen auch Schwermetalle, wie z. B. Cadmium und Uran.

Siehe: https://de.m.wikipedia.org/wiki/Calcium-_und_Phosphathaushalt

###### *Kohlenstoff mit Schwefel (zB -SH)*

**Schwefelverbindungen**

![gg](https://upload.wikimedia.org/wikipedia/commons/thumb/9/92/Mercapto_Group_General_Formulae.png/320px-Mercapto_Group_General_Formulae.png)

*Allgemeine Struktur von Thiolen mit der blau markierten Thiolgruppe (Mercaptogruppe)*

* [Alkanthiole](https://de.m.wikipedia.org/wiki/Thiole): Da die S–H-Bindung schwächer ist als die O–H-Bindung, sind Thioalkohole stärker sauer als die analogen Alkohole und bilden bereits in wässriger NaOH-Lösung Thiolate. Ferner lassen sich Thiole im Gegensatz zu Alkoholen oxidativ zu Disulfiden dimerisieren; dies ist wichtig bei Proteinen, in denen zwei Aminosäure-Ketten über Cystein-Einheiten verknüpft sein können, wobei Cystin-Derivate entstehen.


* Sulfide
* Disulfide
* Ester der Schwefelsäure
* Sulfone
* Sulfoxide
* Thionamide
* Thiolester
* Thiosäure

**Exkurs: Lipidperoxidation und Alkanthiole**

* Unter [Lipidperoxidation](https://de.m.wikipedia.org/wiki/Lipidperoxidation) versteht man die oxidative Degradation von Lipiden. Bei diesem Prozess gibt ein Lipid Elektronen an reaktive Radikale ab. Die dadurch in Gang gesetzte radikalische Kettenreaktion kann bei Lipiden in der Zellmembran zur Zellschädigung führen.

* zB Kupfer: Experimente legen nahe, dass die Kontaktabtötung durch einen Mechanismus abläuft, bei dem der Metall-Bakterien-Kontakt die Zellhülle schädigt, was wiederum die Zellen anfällig für weitere Schäden durch Kupfer macht.

* Die keimreduzierende Wirkung entsteht genauer gesagt dadurch, dass Kupferionen sich an **Thiolgruppen** von Proteinen binden und Lipide der Zellmembran peroxidieren, was zur Bildung von freien Radikalen führt, welche die DNA und Zellmembranen schädigen.

http://www.chemgapedia.de/vsengine/vlu/vsc/de/ch/16/im/antioxsys/antioxsys.vlu/Page/vsc/de/ch/16/im/antioxsys/lipidoxidation.vscml.html

###### *Metallorganische Chemie*

*Kohlenstoff mit Metallen*

Die [metallorganische Chemie](https://de.m.wikipedia.org/wiki/Metallorganische_Chemie) in der ursprünglichen Definition ist die Chemie der Verbindungen, in denen ein organischer Rest oder eine organische Verbindung direkt an ein Metallatom gebunden ist. Diese Verbindungen werden als metallorganische oder auch organometallische Verbindungen bezeichnet.



Wichtige Einteilungsschemata für die metallorganischen Verbindungen sind:

* nach dem Metall, zum Beispiel:

  * Alkalimetallorganyle, z. B. das für Synthesen verwendete [Butyllithium](https://de.m.wikipedia.org/wiki/Butyllithium), metallorganische Verbindung des Elements Lithium: C4H9Li

  * Erdalkalimetallorganyle, z. B. Magnesiumorganyle wie die oben erwähnten Grignard-Reagenzien

  * Organyle zur Borgruppe, z. B. Aluminiumorganyle wie DIBAL oder welche, die selbstentzündlich sind und in Flammenwerfern verwendet wurden

  * Münzmetallorganyle, z. B. organische Kupferverbindungen, Carboplatin

* nach typischen Strukturelementen, zum Beispiel:

  * Sandwich-Komplexe – Sie enthalten ein Metallatom oder -ion eingebettet zwischen zwei aromatischen Ringen. Das Bis-cyclopentadienyl-Eisen oder auch [Ferrocen](https://de.m.wikipedia.org/wiki/Ferrocen) ist die bekannteste Sandwich-Verbindung

  * Carben-Komplexe – Sie zeichnen sich durch eine Metall-Kohlenstoff-Doppelbindung aus.

* nach der Wertigkeit des Metalls, vor allem bei den Nebengruppenmetallen

[Neue Wege zum Stickstoff](https://www.faz.net/aktuell/wissen/ein-blick-auf-neue-wege-zur-stickstoff-fixierung-18616371.html)

Ammoniak ist ein Grundpfeiler unserer Landwirtschaft. Doch die aktuellen Verfahren zur Synthese sind alles andere als optimal. Das könnte sich bald ändern.

Was manchen Mikroorganismen seit Jahrmilliarden scheinbar mühelos gelingt, kann der Mensch nur unter großem Aufwand: Mehr als ein Prozent der globalen Energieproduktion steckt die Menschheit in die Synthese von Stickstoffverbindungen. Ausgangsmaterial ist der Luftstickstoff, der reichlich vorhanden ist. Rund 78 Prozent der Erdatmosphäre bestehen aus diesem Gas. Das Produkt der sogenannten Stickstoff-Fixierung ist in der Regel Ammoniak (NH3). Aus diesem wird insbesondere Kunstdünger hergestellt, ohne den es keine Landwirtschaft gäbe, die acht Milliarden Menschen ernähren kann.

Die [erste technische Stickstoff-Fixierung](https://www.basf.com/global/de/who-we-are/history/chronology/1902-1924/1913.html) gelang den späteren Nobelpreisträgern Fritz Haber und Carl Bosch im Jahr 1913. Seitdem wird über das Haber-Bosch-Verfahren aus Wasserstoff (H2) und Stickstoff Ammoniak hergestellt. Damit die Reaktion abläuft, bedarf es neben geeigneter Katalysatoren allerdings hoher Temperaturen und Drücke von etwa 500 °C und 200 bar – und damit sehr viel Energie. Denn Luftstickstoff liegt als „N2“ vor, also Molekülen aus jeweils zwei Stickstoffatomen. Für die Bildung anderer Stickstoffverbindungen muss die Bindung zwischen den beiden Atomen im N2-Molekül gebrochen werden. Allerdings zählt diese zu den stärksten chemischen Bindungen überhaupt. Die eingangs erwähnten Mikroorganismen verfügen nun über evolutionär optimierte Bio-Katalysatoren, sogenannte Nitrogenasen, um den Luftstickstoff zu fixieren und damit auch allen anderen Organismen in den Nahrungsketten nutzbar zu machen. Sollte die menschliche Chemie nicht auch in der Lage sein, sich ihre Stickstoffverbindungen effizienter zu beschaffen?

Eine Möglichkeit, die industrielle Stickstoff-Fixierung energiesparender zu gestalten, wäre eine erleichterte Abtrennung des Ammoniaks aus dem Haber-Bosch-Reaktor. Denn bisher muss es unter Kühlung verflüssigt und anschließend destilliert werden. Das Aufheizen und Abkühlen verbraucht erhebliche Energiemengen. Hier könnte die neue Entwicklung einer kalifornischen Forschergruppe um Jeffrey R. Long weiterhelfen: [Die Wissenschaftler stellen im Fachjournal „Nature“ ein neues Material vor, das effizient Ammoniak bindet, speichert und bei Bedarf wieder freisetzt. Es handelt sich um ein sogenanntes MOF (für metal organic framework), eine metall-organische Gerüstverbindung, durch die sich Gase frei bewegen können.](https://www.nature.com/articles/s41586-022-05409-2) An bestimmten Stellen des MOFs sitzen Kupferatome, die gerne Ammoniak-Moleküle binden und in die umliegenden Gerüststrukturen einbauen. Andere Gase hingegen strömen ungestört wieder aus den Poren raus. Das Ammoniak lässt sich später durch Unterdruck und Erhitzen wieder heraus spülen und das Speichermaterial dadurch regenerieren. Wie die Forscher berichten, könnte ihr MOF eingesetzt werden, um das Ammoniak unmittelbar aus dem Herstellungsprozess zu entfernen - ohne Destillation, ohne Verflüssigung und damit ohne Abkühlung.

Speichern, binden, freisetzen

Andere Forschungsanstrengungen zielen auf neue Methoden, den Stickstoff chemisch einzufangen. Bis vor ein paar Jahren untersuchte man hier vor allem Metalle auf ihre Eignung als Katalysatoren – also als Materialien, die an einer chemischen Reaktion teilnehmen, sie befördern oder erst ermöglichen, ohne dabei aber selbst verbraucht zu werden. Doch mittlerweile haben Wissenschaftler große Teile des Periodensystem der chemischen Elemente für die Stickstoff-Fixierung erschlossen: Selbst Elemente wie das leichte Nichtmetall Bor vermögen mit Luftstickstoff zu reagieren – und ermöglichen sogar bisher unbekannte Reaktionen wie die [direkte Verkettung zweier N2-Moleküle](https://www.science.org/doi/10.1126/science.aav9593). Die aussichtsreichsten Verfahren nutzen allerdings nach wie vor Metalle. Das Fernziel dieser ganzen Forschung ist die Entwicklung neuer Katalysatoren, die entweder bei niedrigeren Temperaturen wirken oder alternative Energiequellen wie Strom oder Licht nutzen können. Weiterhin sollte ein Katalysator möglichst preiswert, in großen Mengen verfügbar, selbst ungiftig und chemisch robust sein. Einen wichtigen Beitrag für die Katalysator-Forschung leisten inzwischen computergestützte Simulationen. Sie können als eine Art virtuelles Labor dabei helfen, besonders viele Kandidaten in kurzer Zeit zu untersuchen. Auf diese Weise lässt sich die Zahl der Laborexperimente stark reduzieren, was Kosten spart und gleichzeitig die Umwelt schont. Weiterhin [ermöglichen Computer-Verfahren tiefe Einblicke die Wirkungsweise von Katalysatoren](https://pubs.acs.org/doi/10.1021/acscatal.9b01537) und unterstützen somit ihre gezielte Entwicklung.

Irgendwo muss die Energie herkommen

Ein Beispiel für eine [photokatalytische, also durch Licht vermittelte, Stickstoff-Fixierung](https://www.nature.com/articles/s41467-022-34984-1) stellen japanische Chemiker um Yoshiaki Nishibayashi in „Nature Communications“ vor. Ihr Verfahren nutzt zwei Katalysatoren: Einer aus Iridium fängt sichtbares Licht ein und treibt einen anderen aus Molybdän an, der nun die eigentliche Stickstoff-Fixierung übernimmt. Eine Besonderheit dieses Prozesse ist, dass statt elementaren Wasserstoffs ein organisches Molekül namens Dihydroacridin als eine Art Wasserstoffträger eingesetzt wird, um den Stickstoff in Ammoniak zu überführen. Leider eignet sich Dihydroacridin nicht für eine großtechnische Synthese und es fallen große Mengen eines unverwertbaren Nebenprodukts an. Die Arbeit zeigt jedoch, dass ein solcher Prozess grundsätzlich auch ohne Wasserstoff und ohne hohe Temperaturen funktionieren kann. Andere Arbeiten wiederum konzentrieren sich auf die Verbesserung der Prozessführung. [Eine Gruppe der Max-Planck-Gesellschaft für Kohleforschung in Mühlheim an der Ruhr hat ein Verfahren entwickelt](https://onlinelibrary.wiley.com/doi/10.1002/anie.202112095), das sogar bei Raumtemperatur und Normaldruck Ammoniak erzeugt. Als Reaktor nutzen sie eine Kugelmühle, in der die mechanische Reibung der Kugeln die chemische Reaktion antreibt. Doch was im Labor funktioniert, lässt sich nicht immer eins zu eins in der Industrie umsetzen und der Weg einer vielversprechenden Entwicklung bis hin zur großtechnischen Anwendung ist meist lang. So auch hier. Erst in den Neunzigerjahren wurde ein abgewandeltes Verfahren in Betrieb genommen, das statt Fritz Habers ursprünglichen Eisen-Katalysators einen aus Ruthenium nutzt.

Solche Anstrengungen lohnen sich, auch deswegen, weil Ammoniak viel mehr sein kann als Rohstoff für Dünger. Das Gas lässt sich auch als Wasserstoffspeicher oder in speziellen Brennstoffzellen zur Stromerzeugung nutzen. Seine geringe Entflammbarkeit und hohe Energiedichte machen es zu einem aussichtsreichen Energieträger, beispielsweise zu einem [modernen Treibstoff für die Schifffahrt.](https://www.ikem.de/ammoniak-in-der-schifffahrt/) Mittlerweile ist die jährliche Produktion auf 180 Million Tonnen Ammoniak gestiegen. Der Bedarf wird zukünftig sicherlich weiter steigen – deshalb stürzen sich immer mehr Konzerne wie [Siemens](https://www.siemens-energy.com/uk/en/offerings-uk/green-ammonia.html) oder [Thyssenkrupp](https://www.thyssenkrupp-industrial-solutions.com/power-to-x/en/green-ammonia) auf die Herstellung von grünem Ammoniak, also solchem, der nicht unter der Verwendung fossiler Brennstoffe gewonnen wurde. Ein Weg dorthin bestünde natürlich darin, die enorme Prozessenergie für die Haber-Bosch-Synthese aus nachhaltigen Quellen zu beziehen. Besser wäre es sicher, dieses nunmehr 110 Jahre alte Verfahren zur durch eines zu ersetzen, das den Stickstoff effizienter fixiert.

##### <font color="blue">*Kunststoffe*

[Kunststoff der Strom leitet (Polymer)](https://www.gamestar.de/artikel/stromleitender-kunststoff-neues-material,3386440.html)

###### *Monomere*

[Monomere](https://de.m.wikipedia.org/wiki/Monomer) sind niedermolekulare, reaktionsfähige Moleküle, die sich zu unverzweigten oder verzweigten Polymeren zusammenschließen können.

* **Monomere können Einzelsubstanzen, aber auch Gemische unterschiedlicher Verbindungen sein**.

* Im ersten Fall bilden sich Homopolymere, im zweiten Copolymere (Heteropolymere).

In der Polymerchemie sind Monomere prinzipiell alle chemischen Verbindungen, bei denen über Polyreaktionen Polymere gebildet werden können.

> **Monomere sind oft Moleküle mit einer C=C-Doppelbindung oder mit mindestens zwei funktionellen Gruppen pro Molekül**.

* Auch ringförmige Strukturen wie [Caprolactam](https://de.m.wikipedia.org/wiki/Caprolactam) können als Monomere für Polyreaktionen herangezogen werden.

> **Monomere können durch Kettenpolymerisation, Polykondensation oder Polyaddition zu Polymeren verknüpft werden.**





**Typische Monomere für Kunststoffe**

*Caprolactam*

![ggg](https://upload.wikimedia.org/wikipedia/commons/thumb/a/a9/Strukturformel_Caprolactam.svg/140px-Strukturformel_Caprolactam.svg.png)

*Ethen (Ethylen)*

![ggg](https://upload.wikimedia.org/wikipedia/commons/thumb/e/e9/Ethene_structural.svg/170px-Ethene_structural.svg.png)

*Propen (Propylen)*

![gg](https://upload.wikimedia.org/wikipedia/commons/thumb/c/cd/Propene_Structural_Formula_V1.svg/161px-Propene_Structural_Formula_V1.svg.png)

*Vinylchlorid (VC)*

![gg](https://upload.wikimedia.org/wikipedia/commons/thumb/8/89/Vinylchloride_Structural_Formula_V1.svg/134px-Vinylchloride_Structural_Formula_V1.svg.png)

*Acrylamid*

![ggg](https://upload.wikimedia.org/wikipedia/commons/thumb/6/6b/Acrylamide_Structural_Formula_V1.svg/211px-Acrylamide_Structural_Formula_V1.svg.png)

###### *Polymere*

![ggg](https://raw.githubusercontent.com/deltorobarba/repo/master/sciences_0937.png)


Ein [Polymer](https://www.chemie.de/lexikon/Polymer.html) ist eine chemische Verbindung, die aus Ketten- oder verzweigten Molekülen (Makromolekül) besteht, die aus gleichen oder gleichartigen Einheiten (den sogenannten Monomeren) bestehen.

* Das Adjektiv **polymer bedeutet entsprechend aus vielen gleichen Teilen aufgebaut**.

* Obwohl mit diesem Begriff meist kovalent gebundene Stoffe beschrieben werden, die auch als Makromoleküle bezeichnet werden, ist es oft vorteilhaft, auch Salze und Metalle in die Definition mit einzubinden.

* Auch hier ist der gesamte Stoff aus vielen gleichen Teilchen, den Formeleinheiten aufgebaut.



[**Polymerchemie**](https://www.chemie.de/lexikon/Polymerchemie.html)

* Die **Kettenbildung**, das heißt die Verbindung einzelner Monomere, geschieht durch Polyreaktionen. Dabei verbinden sich die Monomere zu Polymeren. Polyreaktionen sind z. B.

  * **Polymerisation**

  * **Polykondensation**

  * **Polyaddition**

* Polymere aus unterschiedlich gebauten Monomeren nennt man Heteropolymere oder Copolymere.

> **Die meisten [Kunststoffe](https://www.chemie.de/lexikon/Kunststoff.html) sind Polymere, bei denen der Kohlenstoff für die molekulare Kettenbildung sorgt**.

* Man unterscheidet:

  * **isotaktische** Polymere, bei denen alle Substituenten einer Polymerkette die gleiche stereoelektronische Konformation haben, wie z. B. isotaktisches Polystyrol mit Konfiguration R-R-R-R-R-… oder S-S-S-S-S-…

  * Als **syndiotaktisch** bezeichnet man Polymere, deren Substituenten abwechselnd aus R und S bestehen.

  * Bei **ataktischen** Polymeren sind die Substituenten wahllos geordnet (eine Art Racemat =  ein äquimolares Gemisch von zwei Enantiomeren).


*Unterscheidung von Polymeren anhand von* [*Polypropylen:*](https://www.spektrum.de/lexikon/chemie/polypropylen/7351)

![ggg](https://raw.githubusercontent.com/deltorobarba/repo/master/sciences_0843.png)

![ggg](https://raw.githubusercontent.com/deltorobarba/repo/master/sciences_0844.png)

**Einteilung der Polymere**

*Polymere können je nach der Anzahl der Grundmonomere eingeteilt werden. Wichtig ist jedoch, dass mindestens eine monomere Substanz die Kette aufbaut.*

* **Homopolymer**: Im einfachsten Fall besteht das Polymer nur aus einer Monomerart, z. B. PE, PP, PVC und PA6.

* **Copolymere** sind aus verschiedenen Monomeren aufgebaut, wie z. B. Polyester, Polyurethane und auch einige Polyamide (PA 66).

*Außerdem unterscheidet man **organische Polymere** und **anorganische Polymere**. Organische Polymere lassen sich weiter unterteilen in:*

* **Biopolymere** oder natürliche Polymere: Sie sind die Grundbausteine der lebenden Organismen.
  * Proteine (Enzyme, Haare, Seide)
  * DNA (Erbsubstanz)
  * RNA
  * Kohlenhydrate (Zellulose, Holz, Papier, Stärke, Chitin)
  * Polyhydroxyalkanoate (Biopolyester als Energie- und Kohlenstoff-Speicher von Bakterien)

* **Chemisch modifizierte Polymere** entstehen durch die Weiterverarbeitung von Biopolymeren, z. B. Nitrocellulose, Celluloid oder Stärkederivate

* **Synthetische Polymere**: Zu den synthetischen Polymeren, die durch Polymerisationsverfahren industriell oder im Labormaßstab hergestellt werden, gehören Polyethylen, Polystyrol, Polyvinylchlorid, etc.

  * **Synthetische Polymere auf <u>Kohlenstoffbasis</u> ([Kunststoffe](https://www.chemie.de/lexikon/Kunststoff.html)!):**
   * PE (Polyethylen)
   * PP (Polypropylen)
   * PVC (Polyvinylchlorid)
   * PS (Polystyrol); besser bekannt in geschäumtem Zustand als Styropor® (BASF AG)
   * PTFE (Polytetrafluorethylen); Handelsname: Teflon® (E. l. Du Pont de Nemours and Company)
   * PMMA (Polymethylmethacrylat); Handelsname: Plexiglas® (Evonik Röhm GmbH)
   * PA (Polyamid), z. B. PA66 (Handelsname: Nylon®) oder PA6 (Handelsname: Perlon®)
   * Polyester, zu dieser Produktgruppe gehören auch PC (Polycarbonat) (Handelsname: Makrolon® (Bayer AG)) und PET (Polyethylenterephthalat)
   * PEG (Polyethylenglykol)
   * Dendrimere; stark verzweigte Strukturen mit Selbstähnlichkeit
  * **Synthetische Polymere auf anderer Basis**:
    * Auch das Silizium ist in der Lage, stabile Verbindungen mit sich selbst einzugehen. Dabei entstehen Silikone.



Daneben gibt es noch **Geopolymere**: ein anorganisches Zweikomponentensystem, das aus einer reaktiven Feststoffkomponente (Siliziumoxid, Aluminiumoxid) und einer alkalischen Aktivierungslösung entsteht. In mehreren Reaktionsschritten bildet sich ein alumosilikatisches Polymer (Kategorie: Anorganische Chemie)

**Nach ihren physikalischen Eigenschaften unterteilt man die Polymere in:**

* Die Thermoplaste,

* die Elastomere und

* die Duroplaste.

###### *Beispiel: PU-Schaum (Polyurethane)*

https://de.m.wikipedia.org/wiki/Polyurethane

Je nach Vernetzungsgrad und/oder eingesetzter Isocyanat- oder OH-Komponente erhält man Duroplaste, Thermoplaste oder Elastomere. Mengenmäßig sind Polyurethanschaumstoffe, als Weich- oder Hartschaum am wichtigsten. Polyurethane werden jedoch auch als Formmassen zum Formpressen, als Gießharze (Isocyanat-Harze), als (textile) elastische Faserstoffe, Polyurethanlacke und als Polyurethanklebstoffe verwendet.

https://www.pu-schaum.center/wissenswertes/pu-im-alltag/

Zwei große Kunststoffgruppen werden heute durch Polyaddition nach diesem Verfahren hergestellt: die Polyurethane und die Epoxidharze.

[Isocyanate](https://de.m.wikipedia.org/wiki/Isocyanate) sind die wesentlichen Ausgangsstoffe für die Herstellung von Polyurethanen.

[GAFF-AIC: reoptimisation of the GAFF force field for realistic densities and viscosities in aromatic isocyanates](https://www.tandfonline.com/doi/abs/10.1080/08927022.2023.2178234?journalCode=gmos20)

###### *Beispiel: Epoxidharze*

https://de.m.wikipedia.org/wiki/Epoxidharz

https://de.m.wikipedia.org/wiki/Harz_(Material)#Kunstharz

###### *Kunststoffe*

https://innovatives-brandenburg.de/de/nachhaltig-heute/kunststoff-recycling/ein-wegweiser-durch-die-plastik-welt

https://www.chem2do.de/c2d/de/silicone/im_vergleich__kunststoffe/_klassifizierung.jsp

https://www.chemieseiten.de/?p=866

https://www.sofatutor.com/chemie/videos/thermoplaste-duroplaste-elastomere-basiswissen#wo-ist-der-unterschied-zwischen-thermoplasten-duroplasten-und-elastomeren

http://www.mannheimer-schulen.de/lilo/2005-2006/chemie/dat/thermo_duro_elasto.html

https://www.chemie.de/lexikon/Kunststoff.html

https://www.u-helmich.de/che/Q2/kunststoffe/index.html

**Kunststoffe: Synthetische Polymere auf <u>Kohlenstoffbasis</u>:**
  * PE (Polyethylen)
  * PP (Polypropylen)
  * PVC (Polyvinylchlorid)
  * PS (Polystyrol); besser bekannt in geschäumtem Zustand als Styropor® (BASF AG)
  * PTFE (Polytetrafluorethylen); Handelsname: Teflon® (E. l. Du Pont de Nemours and Company)
  * PMMA (Polymethylmethacrylat); Handelsname: Plexiglas® (Evonik Röhm GmbH)
  * PA (Polyamid), z. B. PA66 (Handelsname: Nylon®) oder PA6 (Handelsname: Perlon®)
  * Polyester, zu dieser Produktgruppe gehören auch PC (Polycarbonat) (Handelsname: Makrolon® (Bayer AG)) und PET (Polyethylenterephthalat)
  * PEG (Polyethylenglykol)
  * Dendrimere; stark verzweigte Strukturen mit Selbstähnlichkeit


> Kunststoffe sind synthetische, organische, makromolekulare Polymere (Stoffe), und damit Werkstoffe

**Einteilung der Kunststoffe anhand der thermischen und mechanischen Eigenschaften**

Eine Ordnung hinsichtlich der Werkstoffeigenschaften unterscheidet:

* die weichen **Thermoplaste**, die bei Erwärmung zähflüssig oder flüssig werden und so leicht formbar sind

* die harten, mechanisch gut belastbaren **Duroplaste**, die sich bei Erwärmung zersetzen

* die weichen, elastisch verformbaren **Elastomere**

![ggg](https://raw.githubusercontent.com/deltorobarba/repo/master/sciences_295.png)

![ggg](https://raw.githubusercontent.com/deltorobarba/repo/master/sciences_185.jpeg)

*Monomere in Thermoplasten: enthalten eine Doppelbindung oder zwei funktionelle Gruppen im Molekül*

![ggg](https://raw.githubusercontent.com/deltorobarba/repo/master/sciences_0827.png)

*Monomere in Duroplasten: enthalten zwei Doppelbindungen oder drei funktionelle Gruppen im Molekül*

![ggg](https://raw.githubusercontent.com/deltorobarba/repo/master/sciences_0824.png)

*Monomere in Elastomeren: enthalten zwei oder drei funktionelle Gruppen im Molekül*

![ggg](https://raw.githubusercontent.com/deltorobarba/repo/master/sciences_0828.png)

![ggg](https://raw.githubusercontent.com/deltorobarba/repo/master/sciences_0823.png)

*Quelle: https://chemie-am-auto.de/kunststoffe/verwendung.html*

**Einteilung der Kunststoffe nach dem Reaktionstyp ihrer Synthese aus Monomeren**

* Die synthetischen organischen makromolekularen Werkstoffe werden im weiteren Sinn auch als **Plaste, Kunstfasern und Elastomere** bezeichnet.

* Synthetische und natürliche Polymere bestehen aus Makromolekülen mit Molekülmassen von mehr als 10000 g • mol-1.

* Sie werden aus kleinen Molekülen (Monomeren) synthetisiert, sodass in der Struktur eines Polymers immer wiederkehrende gleichartige Bausteine auftreten.

* Je nach Art der verwendeten Monomere können kettenförmige oder vernetzte Makromoleküle entstehen. Als ein Ordnungsprinzip für die ungeheure Vielfalt der Makromoleküle bietet sich der Reaktionstyp an, nach dem diese Bausteine chemisch miteinander verbunden sind:

![ggg](https://raw.githubusercontent.com/deltorobarba/repo/master/sciences_294.png)

**Beispiele für Kunststoffe vom Covestro Meeting**:

• polyurethan

• isocyanate

• polyol

• metallverbundelemente

• pentane insulator

• polycarbonate stoffklasse

###### ***Duroplaste*** *(Kunstharze, Polyester, Polyamide wie Nylon) - basiert auf Phenol und Methanal (Formaldehyd) - **kovalente Bindungen**, meist Polykondensation (manche: Polyaddition)*

Siehe Beispiele in [Kunstharze](https://de.m.wikipedia.org/wiki/Harz_(Material)#Chemische_Unterteilungen)

https://de.m.wikipedia.org/wiki/Polyamide

https://de.m.wikipedia.org/wiki/Polyester

***Die [Duroplaste](https://de.m.wikipedia.org/wiki/Duroplaste) sind sehr harte Kunststoffe, die mit Werkzeugen bearbeitet werden können. Meistens uber Polykondensation hergestellt, manche auch durch Polyaddition.***

> **Beispiel**: Fiberglas am Autodachträger, Topf- und Pfannengriffe, als Bremsbeläge für Autos, als Steckdosen, als Gehäuse für elektrische Geräte, Polyester in Textilien, Lacken etc.

* **Basierend auf Monomere: Phenol und Methanal (Formaldehyd)** - enthalten zwei Doppelbindungen oder drei funktionelle Gruppen im Molekül

> ![gg](https://upload.wikimedia.org/wikipedia/commons/thumb/8/8b/Phenol2.svg/70px-Phenol2.svg.png)

> ![gg](https://upload.wikimedia.org/wikipedia/commons/thumb/0/0c/Structural_formula_of_formaldehyde.svg/114px-Structural_formula_of_formaldehyde.svg.png)

* **Duroplaste sind Polymere, die in einem Härtungsprozess aus einer Schmelze oder Lösung der Komponenten durch eine Vernetzungsreaktion hervorgehen.**

* Diese irreversible Reaktion wird meist durch Erhitzen bewirkt (daher auch der englische Fachterminus thermosets), kann aber auch durch Oxidationsmittel, energiereiche Strahlung oder Einsatz von Katalysatoren initiiert bzw. beschleunigt werden.

* Eine Erwärmung von Duroplasten führt nicht zu einer plastischen Verformbarkeit, sondern lediglich zu deren Zersetzung. Ausgehärtete Duroplaste sind meist hart und spröde sowie im weitergehenden Fertigungsprozess nur noch mechanisch bearbeitbar.

* Wegen ihrer mechanischen und chemischen Beständigkeit auch bei erhöhten Temperaturen werden sie häufig für Elektroinstallationen verwendet.

**Beispiel: Phenoplaste**

Phenoplaste bestehen aus ausgehärteten Phenolharzen, die man durch die Synthese von [Phenolen](https://de.m.wikipedia.org/wiki/Phenole) mit [Aldehyden](https://de.m.wikipedia.org/wiki/Aldehyde) erhält.

[Phenoplast](https://de.m.wikipedia.org/wiki/Phenoplast) gibt es zum Beispiel auf der Basis von [Phenol](https://de.m.wikipedia.org/wiki/Phenol) und [Formaldehyd](https://de.m.wikipedia.org/wiki/Formaldehyd).

*Formaldehyd:*

> ![gg](https://upload.wikimedia.org/wikipedia/commons/thumb/0/0c/Structural_formula_of_formaldehyde.svg/114px-Structural_formula_of_formaldehyde.svg.png)

*Phenol (Neben Phenol werden auch Verbindungen wie [3-Cresol](https://de.m.wikipedia.org/wiki/Kresole), [3,5-Xylenol](https://de.m.wikipedia.org/wiki/Xylenole) oder [Resorcin](https://de.m.wikipedia.org/wiki/Resorcin) verwendet):*

> ![gg](https://upload.wikimedia.org/wikipedia/commons/thumb/8/8b/Phenol2.svg/70px-Phenol2.svg.png)

*Phenoplast (**Methylenphenolgruppe** als Strukturelement bei einem klassischen Phenoplast auf der Basis von Phenol und Formaldehyd. Der Phenoplast ist hoch vernetzt und damit ein Duroplast):*

> ![gg](https://upload.wikimedia.org/wikipedia/commons/thumb/f/f7/Methylenphenolgruppe.svg/211px-Methylenphenolgruppe.svg.png)


**Herstellung**

*Schritt 1: Durch eine elektrophile Substitution von Phenol werden säurekatalysiert Vorprodukte gebildet, die abhängig von der eingesetzten Formaldehydmenge ein bis drei Hydroxymethylgruppen (–CH2–OH) tragen. Als saure Katalysatoren werden u. a. Salzsäure oder Oxalsäure eingesetzt. Die Substitution erfolgt nur in ortho- oder para-Stellung des Phenols. Es bildet sich o-Hydroxymethylphenol (1) und p-Hydroxymethylphenol (2):*

> ![gg](https://upload.wikimedia.org/wikipedia/commons/thumb/9/91/Phenolharz0-1.svg/512px-Phenolharz0-1.svg.png)

*Bei einem Überschuss an Formaldehyd und unter basischen Bedingungen können sich Verbindungen mit bis zu drei Hydroxymethylgruppen bilden. Durch katalysierte Polykondensation dieser Phenol-Derivate bilden sich die Harze. Je nach gewünschtem Ergebnis werden die Vorkondensate dann mit sauren oder basischen Kondensationsmitteln versetzt.*

*Schritt 2 - **[Novolake](https://de.m.wikipedia.org/wiki/Novolak): In saurer Umgebung** bilden sich aus den Phenylalkoholen durch Kondensation über Methylengruppe (–CH2–) verknüpfte Oligomere, die sogenannten Novolake. Formaldehyd und Phenole werden im Verhältnis 4:5 oder weniger Formaldehyd umgesetzt:*

> ![gg](https://upload.wikimedia.org/wikipedia/commons/thumb/3/31/NovolakeSynthese.svg/640px-NovolakeSynthese.svg.png)

*Strukturformel eines Novolaks:*

> ![gg](https://upload.wikimedia.org/wikipedia/commons/thumb/e/ef/Novolak.png/640px-Novolak.png)

*Schritt 2 - **[Resole](https://de.m.wikipedia.org/wiki/Resol_(Kunstharz)): Mit basischen Kondensationsmitteln** bilden sich dagegen schmelzbare und in vielen Lösemitteln lösliche Harze, die Resole. Durch die meist eingesetzten größeren Formaldehydmengen (bis 2,5:1) werden neben Methylengruppen auch Ethergruppen gebildet.*

> ![gg](https://upload.wikimedia.org/wikipedia/commons/thumb/b/b8/ResolV2.svg/554px-ResolV2.svg.png)

*Resole neigen zur Selbsthärtung durch weitere Kondensation und bilden die Zwischenstufe Resitol. Werden die Vorkondensate erhitzt, erhält man unter weiterer Abspaltung von Wasser vernetzte Polymere, die unschmelzbare und unlösliche Endstufe, das Resit.*

**Resole: sind schmelzbare, lösliche Phenoplaste, die reaktive [Methylolgruppen (Hydroxymethylgruppe)](https://de.m.wikipedia.org/wiki/Hydroxymethylgruppe) enthalten und bei der <u>Kondensation von Phenolen mit Formaldehyd unter basischen Bedingungen entstehen</u>. Die Benzolringe sind hierbei über Methylengruppen sowie über Etherbrücken miteinander verbunden**

*Schritt 3: Nach einer Aushärtung eines Harzes bilden sich engmaschig vernetzte Polymere, da die Phenolgruppen mit bis zu drei Methylengruppen untereinander verbunden sind. Mögliche Struktur eines Phenoplast. Gestrichelte Linien deuten die Fortsetzung des Makromoleküls an:*

> ![gg](https://upload.wikimedia.org/wikipedia/commons/thumb/c/c6/PhenolharzHart.svg/694px-PhenolharzHart.svg.png)

**Weitere Beispiele für Duroplaste**: "Duropaste" ist eine "Sammelbezeichnung für alle aus härtbaren Harzen hergestellten Kunststoffe". Zu den Duroplasten gehören Kunststoffe wie zum Beispiel:

[Polyester (wiki)](https://de.m.wikipedia.org/wiki/Polyester), [Polyester](https://www.u-helmich.de/che/Q2/kunststoffe/verbindungen/polyester.html), [Polyester (PES)](https://www.chemie.de/lexikon/Polyester.html), in Textilien, Lacken etc.. Beispiel: Wiederholeinheiten von Polyestern, die aus Dicarbonsäuren und Dihydroxyverbindungen hergestellt werden:

> ![gg](https://upload.wikimedia.org/wikipedia/commons/thumb/5/58/Polyester-WE-Dicarbonsaure.svg/320px-Polyester-WE-Dicarbonsaure.svg.png)

Siehe auch [Ungesättigte Polyesterharze](https://de.m.wikipedia.org/wiki/Ungesättigte_Polyesterharze). Allgemeine Struktur eines gehärteten UP-Harzes:

> ![gg](https://upload.wikimedia.org/wikipedia/commons/thumb/9/95/UP-Gehärtet.svg/640px-UP-Gehärtet.svg.png)

[Aminoplaste](https://de.m.wikipedia.org/wiki/Aminoplast)  (auch: Aminoharze oder Amidharze) sind relativ niedermolekulare, aushärtbare Kunstharze, die durch Polykondensation von Carbonylverbindungen (oft Formaldehyd) und Verbindungen mit NH-Gruppen, wie beispielsweise Harnstoff (Harnstoffharze, UF), Melamin (Melaminharze, MF) oder Dicyandiamid (Dicyandiamidharze, DD) gewonnen werden

> ![gg](https://wikimedia.org/api/rest_v1/media/math/render/svg/46dc0e5fef13ffd3d98c0892ccafc955b89befff)

[Polyurethane (wiki)](https://de.m.wikipedia.org/wiki/Polyurethane) (Je nach Vernetzungsgrad und/oder eingesetzter Isocyanat- oder OH-Komponente erhält man Duroplaste, Thermoplaste oder Elastomere. Mengenmäßig sind Polyurethanschaumstoffe, als Weich- oder Hartschaum am wichtigsten. Polyurethane werden jedoch auch als Formmassen zum Formpressen, als Gießharze (Isocyanat-Harze), als (textile) elastische Faserstoffe, Polyurethanlacke und als Polyurethanklebstoffe verwendet.), [Polyurethane](https://www.u-helmich.de/che/Q2/kunststoffe/verbindungen/polyurethane.html), [Polyurethanharze](https://www.chemie.de/lexikon/Polyurethan.html) für Lacke und Oberflächenbeschichtungen

* Wiederholeinheit bei linearen Polyurethanen, die aus einem Diol und Diisocyanat hergestellt wurden. Die Urethan-Gruppen sind blau gekennzeichnet. R1 steht für den „Rest“ des zur Synthese eingesetzten Diols (HO–R1–OH), R2 für den „Rest“ des Diisocyanats (OCN–R2–NCO):

> ![gg](https://upload.wikimedia.org/wikipedia/commons/thumb/3/3f/Polyurethane-allg.svg/380px-Polyurethane-allg.svg.png)

Polyurethane entstehen durch die [Polyadditionsreaktion](https://de.m.wikipedia.org/wiki/Polyaddition) von [Polyisocyanaten](https://de.m.wikipedia.org/wiki/Polyisocyanate) mit mehrwertigen Alkoholen, den [Polyolen](https://de.m.wikipedia.org/wiki/Polyole). Die Verknüpfung erfolgt durch die Reaktion einer Isocyanatgruppe (–N=C=O) eines Moleküls mit einer Hydroxygruppe (–OH) eines anderen Moleküls unter Bildung einer Urethangruppe (–NH–CO–O–). Im Gegensatz zur Polykondensation erfolgt keine Abspaltung von Nebenprodukten.

> ![gg](https://upload.wikimedia.org/wikipedia/commons/thumb/f/fa/Addition_zu_Polyurethan.svg/544px-Addition_zu_Polyurethan.svg.png)

[Formaldehydharze (Harnstoffharz)](https://de.m.wikipedia.org/wiki/Harnstoffharz),

Kettenverlängerung durch Kondensationsreaktion unter Ausbildung von Methylengruppen:
> ![gg](https://upload.wikimedia.org/wikipedia/commons/thumb/f/f5/Harnstoffharz_Kondensation.svg/799px-Harnstoffharz_Kondensation.svg.png)

Mögliche Struktur eines ausgehärteten Harnstoffharzes:

> ![gg](https://upload.wikimedia.org/wikipedia/commons/thumb/d/d7/Harnstoffharz_ausgehaertet.svg/529px-Harnstoffharz_ausgehaertet.svg.png)



Praktisch alle [Kunstharze, chemie](https://www.chemie.de/lexikon/Harz_%28Material%29.html#Kunstharz/), [Kunstharz, wiki](https://de.m.wikipedia.org/wiki/Harz_(Material)#Kunstharz)

  * beispielsweise [Epoxide](https://www.chemie.de/lexikon/Epoxide.html), [Epoxidharze](https://www.u-helmich.de/che/Q2/kunststoffe/verbindungen/epoxidharze.html)

* [Gießharz](https://de.m.wikipedia.org/wiki/Gießharz), aus dem man die durchsichtigen Plastikblöcke herstellt, in dem sich dann Gegenstände wie Muscheln, Seepferdchen etc.

* Auch im Bereich Autozubehör werden Duroplaste verwendet. Beispielhaft für ein Produkt aus Duroplast ist der Dachträger – genauer gesagt bestehen diese Träger sogar aus [**glasfaserverstärktem Kunststoff** (GFK)](https://de.m.wikipedia.org/wiki/Glasfaserverstärkter_Kunststoff). GFK ist auch unter den Namen **Fiberglas** bekannt und besteht aus einem Duroplast-Kunststoff und Epoxidharz, Polyesterharz oder Polyamid.

![ggg](https://raw.githubusercontent.com/deltorobarba/repo/master/sciences_0824.png)

* Duroplaste, manchmal auch Duromere genannt, sind engmaschig vernetzte hochpolymere Kunststoffe, die dauerhaft formbeständig sind.

* **Durch Erwärmen können Duroplaste also <u>nicht</u> verformt werden** (siehe dagegen Thermoplaste), was sowohl Vorteile wie auch Nachteile mit sich bringt.

* **Vorteile von Duroplasten**

  * Da Duroplaste sich in der Wärme nicht verformen, sind sie recht **temperaturbeständig**.

  * Daher eignen sie sich gut als Topf- und Pfannengriffe, als Bremsbeläge für Autos, als Steckdosen, als Gehäuse für elektrische Geräte und so weiter.

  * Aufgrund ihrer molekularen Struktur sind Duroplaste sehr hart und können sogar mit Werkzeugen bearbeitet werden. Man kann zum Beispiel Löcher in einen solchen Kunststoff bohren oder ihn schleifen oder sägen.

* **Nachteile von Duroplasten**

  * Da Duroplaste sich in der Wärme nicht verformen, müssen sie bereits bei der Herstellung ihre endgültige Form erhalten, **ein späteres Verformen durch Erwärmen ist nicht mehr möglich**.

  * Ein Nachteil der Härte und Sprödigkeit von Duroplasten ist die Tatsache, dass sie leicht zerbrechen können.

* Duroplaste gehören zu den ältesten Kunststoffen überhaupt. Bereits 1905 hat Leo Hendrik Baekeland einen [Phenoplasten](https://de.m.wikipedia.org/wiki/Phenoplast) hergestellt und 1909 unter dem Namen Bakelit vermarktet. Die Gehäuse alter Telefone und Radios bestanden zum Beispiel aus diesem Duroplasten.


**Struktur von Duroplasten**

* Duroplaste stellen engmaschige dreidimensionale Netzwerke von Molekülen dar.

* Die Monomere der Duroplaste sind **trifunktionell, haben also drei funktionelle Gruppen**, so das stabile Quervernetzungen über kovalente Bindungen möglich werden.

* Eine Folge dieser engmaschigen Vernetzung ist, dass die Moleküle beim Erwärmen nicht mehr aneinander vorbeigleiten können, so wie das bei den Thermoplasten der Fall ist. Erhitzt man einen Duroplast zu stark, brechen die Moleküle auseinander, und der Kunststoff zersetzt sich.





![ggg](https://raw.githubusercontent.com/deltorobarba/repo/master/sciences_0822.png)

**Herstellung von Duroplasten**

* Die meisten Duroplaste werden durch **Polykondensation** hergestellt, einige aber auch durch **Polyaddition**.

* Oft werden zunächst lineare Kettenmoleküle synthetisiert, die dann mit Hilfe bestimmter Zusätze noch dreidimensional vernetzt werden. Manche trifunktionellen Monomere bilden aber auch sofort ein dreidimensionales Netzwerk.

###### ***Thermoplaste*** *(Teflon, PVC, PET, Plexiglas) - basiert auf Ethen - **keine** kovalenten Bindungen (**van der Vaals Kraefte**)*

***Thermoplaste sind Kunststoffe, die sich in der Wärme verformen. Sie können leicht recycelt und in neue Formen gegossen werden.***

> **Beispiel**: Teflon, Plexiglas, PVC, Jogurtbecher, Einkaufstaschen, PET Getränkeflaschen

* **Basiert auf Monomer: Ethen** - enthalten eine Doppelbindung oder zwei funktionelle Gruppen im Molekül

> ![gg](https://upload.wikimedia.org/wikipedia/commons/thumb/e/e9/Ethene_structural.svg/170px-Ethene_structural.svg.png)


https://www.chemie.de/lexikon/Thermoplast.html

https://www.chemie.de/lexikon/Polyamide.html#Perlon

**Bekannte Thermoplaste**

Bekannte Kunststoffe aus der Klasse der Thermoplaste sind
* [Polyethylen (PE)](https://www.u-helmich.de/che/Q2/kunststoffe/verbindungen/polyethylen.html)
* [Polypropylen (PP)](https://www.u-helmich.de/che/Q2/kunststoffe/verbindungen/polypropylen.html)
* [Polyvinylchlorid (PVC)](https://www.u-helmich.de/che/Q2/kunststoffe/verbindungen/pvc.html)
* [Polystyrol (PS)](https://www.u-helmich.de/che/Q2/kunststoffe/verbindungen/polystyrol.html)

Weitere Thermoplaste sind
* [Polytetrafluorethen](https://www.u-helmich.de/che/Q2/kunststoffe/verbindungen/teflon.html) (**Teflon**),
* Polyacrylnitril (PAN),
* [Polyethylenterephthalat (PET)](https://www.u-helmich.de/che/Q2/kunststoffe/polykondensation-01.html#PET)
* [Polymethacrylsäuremethylester](https://www.u-helmich.de/che/Q2/kunststoffe/verbindungen/plexiglas.html) (PMMA, **Plexiglas**).

Auch die [Polyurethane](https://www.u-helmich.de/che/Q2/kunststoffe/verbindungen/polyurethane.html) gehören zu den Thermoplasten.

> [„Plastikfresser“ im menschlichen Speichel: Enzym aus Speichelmikroben kann PET-Kunststoffe zersetzen](https://www.scinexx.de/news/biowissen/plastikfresser-im-menschlichen-speichel-entdeckt/) - In ersten Tests konnte dieses neuentdeckte Enzym PET-Plastik besser zersetzen als andere natürliche und gentechnisch-optimierte PET-[Hydrolasen](https://de.m.wikipedia.org/wiki/Hydrolasen) (Hydrolasen sind Enzyme, die Ester, Ether, Peptide, Glycoside, Säureanhydride oder C-C-Bindungen in reversibler Reaktion hydrolytisch spalten.)

*Das im menschlichen Speichel entdeckte Enzym MG8 kann den Kunststoff PET in seine Grundeinheiten zerlegen:*

![ggg](https://raw.githubusercontent.com/deltorobarba/repo/master/sciences_0906.png)

![ggg](https://raw.githubusercontent.com/deltorobarba/repo/master/sciences_0827.png)

* Thermoplaste sind Kunststoffe, die sich in der Wärme verformen.

* Das liegt daran, dass Thermoplaste aus **linearen oder nur schwach verzweigten Polymerketten** bestehen, die sich gegenseitig nur schwach anziehen (van-der-Waals-Kräfte).

* Daher lassen sich diese Ketten leicht gegeneinander verschieben, was die Verformbarkeit der Thermoplaste erklärt

* Je nach Erwärmungsgrad hat man verschiedene Eigenschaften:

  * **fester** Zustand: bei Zimmertemperatur ist ein Joghurtbecher zwar etwas elastisch, man kann ihn leicht zusammendrücken, aber wenn man loslässt, nimmt er wieder seine ursprüngliche Form an.

  * **thermoelastischer** bei leichter Erwärmung kann einen Joghurtbecher verformen, aber er nimmt aich hier danach wieder seine alte Form an.

  * **thermoplastischen** Temperaturbereich: Bei diesen Temperaturen kann man den Joghurtbecher dauerhaft verformen. Lässt man den Becher wieder los, behält er die neue Form bei, auch wenn er sich wieder abkühlt. Bei diesen Temperaturen kann man also Thermoplaste verformen.

  * **flüssiger** Temperaturbereich:Erhitzt man den Joghurtbecher noch stärker, wird er **verflüssigt**. Nun könnte man den Kunststoff in eine neue Form gießen und einen völlig neuen Gegenstand herstellen. Dies wird in der Technik bei der Herstellung von Kunststoffprodukten oder auch beim Recycling von alten Kunststoffen angewandt.

  * Eine noch stärkere Erhitzung führt nun **<u>nicht</u> etwa in den gasförmigen Zustand**, wie man sich vielleicht denkt, sondern der Kunststoff zersetzt sich unter Freisetzung von viel **Wärmeenergie**. Ein Thermoplast könnte also auch "thermisch verwertet" werden - eine andere Bezeichnung wäre "Müllverbrennung".

* Thermoplaste sind zwar Reinstoffe (also keine Stoffgemische), haben aber trotzdem keinen definierten Schmelzpunkt.



**Thermoplaste als Werkstoffe**

Thermoplaste (Plastomere) sind wichtige Werkstoffe. Unter diesem Aspekt werden sie in folgende Gruppen unterteilt:

* **Standardkunststoffe**
  * Hierzu gehören die "einfachen" Thermoplaste wie Polyethylen, Polypropylen, Polyvinylchlorid und Polystyrol.
  * Der Einsatzbereich dieser Kunststoffe liegt bei maximal 90 ºC. **Einkaufstaschen** sind ein gutes Beispiel für den Einsatz dieser Standardkunststoffe.

* **Technische Kunststoffe**
  * Unter diesem Begriff versteht man Thermoplaste, die bei höheren Temperaturen von bis zu 150 ºC eingesetzt werden können.
  * Beispiele für technische Kunststoffe sind **PET** (Polyethylenterephthalat) und Polycarbonat.

* **Hochleistungskunststoffe**
  * Diese Kunststoffe sind bei sehr hohen Temperaturen bis 260 ºC verwendbar.
  * **Teflon** ist ein solcher Kunststoff, der zum Beispiel für die Beschichtung von Bratpfannen eingesetzt wird.

**Struktur von Thermoplasten**

* Thermoplaste sind aus **langen und unverzweigten bzw. nur wenig verzweigten Ketten von Makromolekülen** aufgebaut.

* Diese langen Ketten werden im festen Zustand durch **van-der-Waals-Kräfte** zusammengehalten, bei bestimmten Thermoplasten auch durch **Wasserstoffbrücken-Bindungen**.

* Erwärmt man einen Thermoplasten, geraten die Moleküle in Schwingungen und **dadurch werden diese lockeren intermolekularen Bindungen gelöst**.

* Die langen Ketten können daher leicht aneinander vorbei gleiten.



*Links im Bild sieht man den festen Thermoplast mit den intermolekularen Bindungen (rot gezeichnet), rechts ist der flüssige Bereich dargestellt. In der Mitte sieht man den thermoelastischen oder thermoplastischen Bereich; der Übergang zwischen diesen Bereichen ist sehr flexibel:*

![ggg](https://raw.githubusercontent.com/deltorobarba/repo/master/sciences_0821.png)

![ggg](https://raw.githubusercontent.com/deltorobarba/repo/master/sciences_0820.png)

**Amorphe und teilkristalline Thermoplaste** (Bild oben ganz rechts)

* Oft wird noch zwischen amorphen und teilkristallinen Thermoplasten unterschieden. Bei den amorphen Thermoplasten liegen die Ketten im festen Zustand völlig ungeordnet, kreuz und quer ineinander verknäult vor.
* Bei den teilkristallinen Thermoplasten dagegen finden sich Bereiche, in der die langen Molekülketten fast parallel zueinander verlaufen.
* Hier können zwischenmolekulare Kräfte besonders gut wirken, daher sind die Thermoplaste in diesen kristallinen Bereichen besonders hart und stabil.
* Allgemein kann man sagen: Je mehr kristalline Bereiche ein Thermoplast enthält, desto härter und spröder ist er.

###### ***Elastomere*** *(Gummi, Kautschuke) - basiert Ethen und Propen (Propylen) und Isopren, **kovalente Bindungen**, meist Polymerisation*

Genau wie Duroplaste  sind Elastomere ebenfalls durch kovalente Bindungen  zwischen den einzelnen Molekülen miteinander vernetzt. Nur ist die Dichte der Vernetzungspunkte bei letzteren wesentlich geringer. https://studyflix.de/chemie/elastomere-1664

***Elastomere schließlich sind elastische Kunststoffe mit Eigenschaften wie Gummi, aber wesentlich beständiger***.

> **Beispiel**: Reifen, Gummibänder, Dichtungsringe, Vulkanisate von Naturkautschuk und Silikonkautschuk.

* **Basieren auf Monomere: Ethen und Propen (Propylen)** - enthalten zwei oder drei funktionelle Gruppen im Molekül

> ![gg](https://upload.wikimedia.org/wikipedia/commons/thumb/e/e9/Ethene_structural.svg/170px-Ethene_structural.svg.png)

> ![gg](https://upload.wikimedia.org/wikipedia/commons/thumb/b/b6/Propene-2D-flat.svg/254px-Propene-2D-flat.svg.png)

* **Zu den Elastomeren gehören alle Arten von vernetztem Kautschuk**. Die Vernetzung erfolgt beispielsweise durch Vulkanisation mit Schwefel, mittels Peroxiden, Metalloxiden oder Bestrahlung.

* Die Elastomere sind weitmaschig vernetzt und daher flexibel. Sie werden beim Erwärmen nicht weich und sind in den meisten Lösemitteln nicht löslich. Daher werden sie für Hygieneartikel oder Chemikalienhandschuhe verwendet. Die Gummimischung von Autoreifen ist ebenfalls ein Elastomer, das seine Eigenschaften durch Vulkanisation erhält.

Important basic Monomer [**Isopren**](https://de.m.wikipedia.org/wiki/Isopren) (2-Methyl-1,3-butadien):

> ![ggg](https://upload.wikimedia.org/wikipedia/commons/thumb/4/4d/Isoprene_Structural_Formulae_V.1.svg/279px-Isoprene_Structural_Formulae_V.1.svg.png)

**Beispiele für Elastomere:**

[Naturkautschuk](https://de.m.wikipedia.org/wiki/Naturkautschuk) (NR)

* Naturkautschuk ist ein Polymerisat aus dem Monomer [**Isopren**](https://de.m.wikipedia.org/wiki/Isopren) (2-Methyl-1,3-butadien) und hat eine fast einheitliche Struktur mit cis-1,4-Verknüpfung. Es wird den [Polyterpenen](https://de.m.wikipedia.org/wiki/Terpene#Polyterpene) zugeordnet.

* Als Beispiel einige Polyterpene: obere Reihe Polyprene, untere Reihe Polyprenole:

> ![gg](https://upload.wikimedia.org/wikipedia/commons/thumb/1/10/Polyterpene.svg/640px-Polyterpene.svg.png)

* Kautschuke mit ungesättigter Hauptkette, wie z. B. Naturkautschuk oder Styrol-Butadien-Kautschuk, zählen zur **R-Gruppe**.

* Die mittlere molare Masse des Naturkautschuks liegt bei etwa 500.000 bis 2 Millionen g·mol−1 außerordentlich hoch.

* *Struktur von Naturkautschuk: Die Isopreneinheiten haben aus Perspektive der Hauptkette eine cis-Struktur:*

> ![ggg](https://upload.wikimedia.org/wikipedia/commons/thumb/6/63/Naturkautschuk.svg/180px-Naturkautschuk.svg.png)

**Zusammenhang mit Elastomere mit Isopren bzw. Terpenen sowie Naturkautschuk und Synthesekautschuk am Beispiel von [Butadien-Kautschuk (Polybutadien)](https://de.m.wikipedia.org/wiki/Polybutadien):**

* Polybutadien ist eine Sammelbezeichnung für **Homopolymere**, die aus 1,3-Butadien hergestellt werden. Polybutadien zählt zu den **Synthesekautschuken** und wird auch als **Butadien-Kautschuk** (Kurzzeichen BR vom englisch butadiene rubber) bezeichnet.

* = Butadien-Kautschuk oder künstlicher Kautschuk

* hergestellt nach dem Kunststoff-Prinzip der **Polymerisation** von [Ethen](https://de.m.wikipedia.org/wiki/Ethen)-Derivaten

* wie zum Beispiel [1,3-Butadien](https://de.m.wikipedia.org/wiki/1,3-Butadien), auch genannt Vinylethylen, Vinylethen

* 2-Methyl-1,3-butadien auch genannt [**Isopren**](https://de.m.wikipedia.org/wiki/Isopren), die Grundeinheit der [Terpene](https://de.m.wikipedia.org/wiki/Terpene), ist ein Derivat des Butadiens. Isopren:

> ![gg](https://upload.wikimedia.org/wikipedia/commons/thumb/4/4d/Isoprene_Structural_Formulae_V.1.svg/279px-Isoprene_Structural_Formulae_V.1.svg.png)

* **Isoprenoide sind auch Lipide, wie Baumwachse & Harze** - da die Verbindung zu Kautschuk??

* Siehe auch [Synthesekautschuk](https://de.m.wikipedia.org/wiki/Synthesekautschuk): Um 1860 konnte Charles Hanson Greville Williams aus [Naturkautschuk](https://de.m.wikipedia.org/wiki/Naturkautschuk) **Isopren** destillieren und die Summenformel C5H8 bestimmen (Naturkautschuk = Pflanzensaft / ein gummiartiger Stoff im Milchsaft (Latex) vieler verschiedener Kautschukpflanze

*Biosynthese der Terpene:*

![gg](https://upload.wikimedia.org/wikipedia/commons/thumb/6/68/Terpene_Biosynthese.svg/800px-Terpene_Biosynthese.svg.png)

[Synthesekautschuk](https://de.m.wikipedia.org/wiki/Synthesekautschuk)

* Als Synthesekautschuk bezeichnet man elastische Polymere, aus denen Gummi hergestellt wird und die auf der Basis petrochemischer Rohstoffe erzeugt werden.

* Die wichtigsten Synthesekautschuke stellen dabei Styrol-Butadien-Kautschuk und EPDM dar.

* Neben Synthesekautschuk existieren Naturkautschuktypen, vor allem auf der Basis des Milchsafts (Latex) des Kautschukbaumes (Hevea brasiliensis).

* Unterteilung und Beispiele:

  * EPDM gehört zu den **Synthesekautschuken** mit gesättigter Hauptkette (nach DIN ISO 1629: **M-Gruppe**).

  * Kautschuke mit ungesättigter Hauptkette, wie z. B. Naturkautschuk oder Styrol-Butadien-Kautschuk, zählen hingegen zur **R-Gruppe**.




![ggg](https://raw.githubusercontent.com/deltorobarba/repo/master/sciences_0846.png)

[Styrol-Butadien-Kautschuk](https://de.m.wikipedia.org/wiki/Styrol-Butadien-Kautschuk) (SBR) - C12 H14

* Styrol-Butadien-Kautschuk (auch Styrol-Butadien-Copolymer, Kurzzeichen SB) ist ein **Copolymer** aus 1,3-Butadien und Styrol.

* SBR ist die am meisten hergestellte Variante des **Synthesekautschuks**. Im Jahr 2012 sind weltweit rund 5,4 Mio. Tonnen verbraucht worden.

* Kautschuke mit ungesättigter Hauptkette, wie z. B. Naturkautschuk oder Styrol-Butadien-Kautschuk, zählen zur **R-Gruppe**.

* SBR ist heute der meistverwendete Synthesekautschuk, der unter Verwendung verschiedener Füllerstoffe wie beispielsweise Silica in der Herstellung von Reifen – besonders der Lauf­flächen –, Dichtungen und Transportbändern Anwendung findet.

> ![gg](https://upload.wikimedia.org/wikipedia/commons/thumb/f/f8/Styrol-Butadien-Kautschuk.svg/282px-Styrol-Butadien-Kautschuk.svg.png)



[Acrylnitril-Butadien-Kautschuk](https://de.m.wikipedia.org/wiki/Acrylnitril-Butadien-Kautschuk) (NBR) - [CH2CH(CN)]x(CH2CH=CHCH2)y


* Acrylnitril-Butadien-Kautschuk, kurz auch Nitrilkautschuk, Kurzzeichen AB und NBR (Nitrile Butadiene Rubber), ist ein **Copolymer** aus Acrylnitril und 1,3-Butadien und zählt zu den [Synthesekautschuken](https://de.m.wikipedia.org/wiki/Synthesekautschuk).

* Vulkanisate des Kautschuks haben eine hohe Beständigkeit gegen Mineralöle, Fette und Kohlenwasserstoffe.

* Der Werkstoff wird z. B. unter den Markennamen Perbunan (früher Buna N) oder Europrene N vertrieben. Darüber hinaus gibt es noch weitere Marken von AB-Kautschuken. Verwendung auch als Nitril-Handschuh, wie er in der Medizin verwendet wird.

> ![ggg](https://upload.wikimedia.org/wikipedia/commons/thumb/1/13/Acrylnitril-Butadien-Copolymer.svg/272px-Acrylnitril-Butadien-Copolymer.svg.png)

[Ethylen-Propylen-Dien-Kautschuk](https://de.m.wikipedia.org/wiki/Ethylen-Propylen-Dien-Kautschuk) (EPDM)

* Ethylen-Propylen-Dien-Kautschuke (Kurzzeichen EPDM, Ethylen-Propylen-Dien; M-Gruppe) sind Terpolymere aus Ethylen, Propylen und einem nicht näher festgelegten Dien. Ist ein **Copolymer**.

  * EPDM gehört zu den **Synthesekautschuken** mit gesättigter Hauptkette (nach DIN ISO 1629: **M-Gruppe**).

  * Kautschuke mit ungesättigter Hauptkette, wie z. B. Naturkautschuk oder Styrol-Butadien-Kautschuk, zählen hingegen zur **R-Gruppe**.

* EPDM-Kautschuke besitzen dafür Doppelbindungen in den Seitenketten und sind daher ebenfalls mit Schwefel vulkanisierbar.

* Das gummielastische Material EPDM wird industriell vielseitig verwendet, z. B. für Profile im Automobil- und Gerätebau, für Förderbänder, Schläuche für Wasch- und Spülmaschinen, Dichtungen in Wasserarmaturen, für Kabelummantelungen etc. Bei Gebäuden dienen EPDM-Dichtungsbahnen zur Abdichtung der Sohle, Keller, Tür- und Terrassenelemente sowie Dächer. Außerdem können sie als Teichfolie zur Abdichtung von Folienteichen eingesetzt werden (Schwimmteiche und Wasserbecken).

[Chloropren-Kautschuk](https://de.m.wikipedia.org/wiki/Chloropren-Kautschuk) (CR)

* Chloropren-Kautschuk, auch Polychloropren oder Chlorbutadien-Kautschuk, ist ein **Synthesekautschuk**, der unter anderem im Automobilbau und für wärmedämmende Sportbekleidung eingesetzt wird.

* Im deutschen Sprachraum ist er unter dem Namen **Neopren** bekannt.

* Die Herstellung erfolgt durch Polymerisation von 2-Chlor-1,3-butadien (Chloropren). Die Abkürzung nach ISO 1043 (1975) für Chloropren-Kautschuk ist CR.

* *Ausschnitt aus einem rein trans-1,4-verknüpften Polymer:*

![ggg](https://upload.wikimedia.org/wikipedia/commons/thumb/5/5a/Polychloropren.svg/458px-Polychloropren.svg.png)




[Butadien-Kautschuk (Polybutadien)](https://de.m.wikipedia.org/wiki/Polybutadien) (BR)

* Polybutadien ist eine Sammelbezeichnung für **Homopolymere**, die aus 1,3-Butadien hergestellt werden. Polybutadien zählt zu den **Synthesekautschuken** und wird auch als **Butadien-Kautschuk** (Kurzzeichen BR vom englisch butadiene rubber) bezeichnet.

* In Bezug auf den globalen Verbrauch ist Polybutadien der zweitwichtigste synthetische Kautschuk und machte im Jahr 2012 etwa ein Viertel des Marktvolumens aus.

* Verwendung: Zur Verbesserung der Eigenschaften und der Verarbeitbarkeit wird Polybutadien mit Naturkautschuk oder Styrol-Butadien-Kautschuk SBR versetzt. Es findet Verwendung in KFZ-Reifen und Golfballkernen.

* Weitere Notes und Verbindungen zu Isoprenen:

  * = Butadien-Kautschuk oder künstlicher Kautschuk
  * hergestellt nach dem Kunststoff-Prinzip der **Polymerisation** von [Ethen](https://de.m.wikipedia.org/wiki/Ethen)-Derivaten
  * wie zum Beispiel [1,3-Butadien](https://de.m.wikipedia.org/wiki/1,3-Butadien), auch genannt Vinylethylen, Vinylethen
  * 2-Methyl-1,3-butadien auch genannt [**Isopren**](https://de.m.wikipedia.org/wiki/Isopren), die Grundeinheit der Terpene, ist ein Derivat des Butadiens.
  * Isoprenoide sind auch Lipide, wie Baumwachse & Harze - da die Verbindung zu Kautschuk??
  * Siehe auch [Synthesekautschuk](https://de.m.wikipedia.org/wiki/Synthesekautschuk): Um 1860 konnte Charles Hanson Greville Williams aus [Naturkautschuk](https://de.m.wikipedia.org/wiki/Naturkautschuk) **Isopren** destillieren und die Summenformel C5H8 bestimmen (Naturkautschuk = Pflanzensaft / ein gummiartiger Stoff im Milchsaft (Latex) vieler verschiedener Kautschukpflanze

![ggg](https://raw.githubusercontent.com/deltorobarba/repo/master/sciences_0828.png)

* Elastomere sind Kunststoffe, die so sowohl bei Zug- wie auch bei Druckbelastung elastisch verformen und dann wieder ihre ursprüngliche Gestalt einnehmen können.

* Elastomere werden als Material für Reifen, Gummibänder etc. eingesetzt.

* Die verschiedenen Kautschuk-Varianten (Naturkautschuk, künstlicher Kautschuk, Siliconkautschuk usw.) sind die bekanntesten Elastomere.

* Neben der Elastizität (siehe unten vertiefend) gibt es noch weitere Eigenschaften von Elastomeren:

  * sie sind **nicht schmelzbar**. Man kann also einen Elastormer nicht durch Erwärmung verformbar machen, so wie dies bei den Thermoplasten der Fall ist.

  * Gegen Kälte sind die meisten Elastormere empfindlich; bei großer Kälte verlieren sie ihre elastischen Eigenschaften und **können eventuell sogar gefrieren und zerbrechen**.



![ggg](https://raw.githubusercontent.com/deltorobarba/repo/master/sciences_0825.png)

**Elastizität**

* **Die Elastizität erklärt sich aus dem molekularen Aufbau eines Elastomers**.

* Im nicht-gestreckten Grundzustand sind die Polymerketten stark verknäult.

  * Beim Anlegen einer Zugspannung strecken sich die Ketten, allerdings ist dieser Zustand energetisch ungünstiger als der verknäulte.

  * Wenn die Zugspannung wegfällt, also zum Beispiel beim Loslassen eines langgezogenen Gummibandes, nehmen die Ketten wieder den energetisch günstigeren verknäulten Zustand ein. Ähnlich wie bei einer Spiralfeder ist der verknäulte Zustand noch weiter zusammenpressbar, das Knäuel wird dann noch dichter.

  * Allerdings ist auch dieser Zustand energetisch ungünstig, so dass beim Weglassen des Drucks wieder der energetisch günstigere Knäuelzustand eingenommen wird. Mit Hilfe folgender Graphik kann man sich das besser klar machen:

![ggg](https://raw.githubusercontent.com/deltorobarba/repo/master/sciences_0826.png)

* In der Mitte sehen wir den Grundzustandes eines Gummiballs. Vier der Polymerketten sind farblich hervorgehoben und dabei stark vergrößert.

* Je größer die Zugbelastung, desto mehr strecken sich die Polymerketten, was aber den Energiegehalt des Gummiballs vergrößert. Wird der Gummiball zusammengepresst (Druckbelastung), so lagern sich die Polymerketten noch enger zusammen, und der Energiegehalt steigt wieder an.

**Thermoelastische Elastomere**

* Das sind besondere Elastormere, die zusätzlich zur Elastizität auch thermoplastische Eigenschaften haben, sich also bei Erwärmung verformen lassen. Solche Kunststoffe werden hergestellt, indem man einen Elastomer mit einem Thermoplasten "vermischt". Ein bekanntes Beispiel ist der Styrol-Butadien-Elastomer.

* Die Polystyrol-Anteile sind für die thermoplastischen Eigenschaften verantwortlich, während die Polybutadien-Anteile für die elastischen Eigenschaften zuständig sind.

* Auf der Chemgapedia-Seite "Struktur der thermoplastischen Styrol-Butadien-Elastomere" findet sich eine anschauliche Animation, die zeigt, wie sich ein "thermoplastisches Styrol-Butadien-Elastomer unter Einwirkung einer äußeren Zugspannung" verhält.

###### *Primär-, Sekundär- und Tertiärstruktur*

> **Die Feststellung, aus welchen Atomen ein Makromolekül aufgebaut ist, sagt noch nichts über seinen wirklichen Aufbau aus.** Polyethylen, Polypropylen, Kautschuk, Polystyren und andere **bestehen alle nur aus Kohlenstoff- und Wasserstoffatomen, haben jedoch durchaus unterschiedliche Eigenschaften**.

* Entscheidend ist, wie und in welchem Verhältnis die verschiedenen Atome miteinander verbunden sind.

* Zur besseren Veranschaulichung des Zusammenhangs zwischen Struktur und Eigenschaften der Makromoleküle betrachtet man **verschiedene Strukturebenen, indem man das komplexe Polymer gedanklich zerlegt: Primärstrukturen, Sekundärstrukturen und Tertiärstrukturen.**.

* *Die Auswirkung der zwischenmolekularen Kräfte auf die Sekundär- und Tertiärstruktur ist am eingehendsten bei den Eiweißen untersucht.*

**Primärstruktur**

* Die Primärstruktur erfasst die elementaren atomaren Bausteine und deren chemische Verknüpfung.

* Die Sekundärstruktur beschreibt die regelmäßige räumliche Anordnung der Makromoleküle, die durch zwischenmolekulare Kräfte bewirkt wird.

* Wie sich mehrere Makromoleküle zu mikroskopisch oder gar mit bloßem Auge sichtbaren Aggregaten zusammenfinden, stellt die Tertiärstruktur dar.

*Monomere im Makromolekül*

* Die Anzahl, Art und Anordnung der Monomere beeinflussen die Werkstoffeigenschaften wesentlich. Die Anzahl der Monomere bestimmt die Größe der Makromoleküle und ermöglicht enorme Variationsmöglichkeiten ihrer Gestalt und Ordnung.

* Unterschiedliche Substituenten an gleichartigen Monomeren beeinflussen maßgeblich das chemische Verhalten und damit die Materialeigenschaften. So hängt die chemische Beständigkeit von Kunststoffen davon ab, ob nur relativ stabile C-C-Bindungen im Makromolekül vorliegen oder z. B. hydrolyseanfällige Ester- bzw. Amidbindungen.

* Auch die Polarität der funktionellen Gruppen der Monomere beeinflusst die Eigenschaften. So weist z.B. Polyvinylchlorid infolge der zwischen dem negativ polarisierten Chlor- und dem positiv polarisierten Kohlenstoffatom wirkenden Dipolkräfte eine deutlich höhere Erweichungstemperatur auf als Polyethylen und ist zudem auch spröder.

* Beim Polyacrylnitril ist die C-CN-Bindung noch stärker polarisiert, sodass hier besonders starke Dipolwechselwirkungen auftreten. Daraus resultiert die Ausbildung einer Leiterstruktur, die für die große Festigkeit verantwortlich ist. Der Kunststoff ist daher nicht schmelzbar, die Zersetzungstemperatur liegt mit 350°C knapp über der Glasübergangs- bzw. Glastemperatur.

* Der Grad der Verzweigung in der Polymerstruktur hängt sowohl von der Art der verwendeten Monomere als auch von den Bedingungen der Synthese ab.

![ggg](https://raw.githubusercontent.com/deltorobarba/repo/master/sciences_309.png)


* Aus Monomeren mit zwei reaktiven Gruppen erhält man lineare oder verzweigte Makromoleküle. Tri- oder höherfunktionelle Monomere, die auch als Vernetzer bezeichnet werden, führen zu dreidimensional vernetzten Makromolekülen.

* Vernetzte Polymere quellen bei Lösungsmittelkontakt nur an, sind jedoch immer unlöslich.

> Der Grad der Verzweigung und die zwischenmolekularen Wechselwirkungen bestimmen die thermischen und mechanischen Eigenschaften der Plaste, Kunstfasern und Elastomere.

![ggg](https://raw.githubusercontent.com/deltorobarba/repo/master/sciences_310.png)


**Copolymere**

* Copolymere oder Mischpolymere entstehen, wenn zwei oder mehr unterschiedliche Monomere in eine Polymerkette eingebaut sind.

* Meist werden Monomere mit ganz unterschiedlichen Strukturen verknüpft. Auf diese Weise lassen sich die günstigen Eigenschaften von Polymeren, die nur aus einem Monomer aufgebaut sind, kombinieren. Abhängig von der Abfolge (Sequenz) der im Makromolekül enthaltenen Monomere A und B unterscheidet man zwischen verschiedenen Arten von Copolymeren.

* Die meisten Bausteine aus Kunststoff werden aus ABS, einem Copolymer aus Acrylnitril, Buta-1.3-dien und Styren, hergestellt. ABS ist formstabil, extrem hart und dauerhaft einfärbbar.

![ggg](https://raw.githubusercontent.com/deltorobarba/repo/master/sciences_311.png)


*Isomerie und Werstoffeigenschaften*

* Die Primärstruktur und damit die Werkstoffeigenschaften der Kunststoffe werden auch durch die Konstitution und Konfiguration der Moleküle wesentlich geprägt.

![ggg](https://raw.githubusercontent.com/deltorobarba/repo/master/sciences_312.png)


* Ein Beispiel ist die infolge der symmetrischen Anordnung energetisch stabilere 1,3-Stellung der Substituenten in einem Ethenpolymer. Noch offensichtlicher wird die Auswirkung der Isomerie für die Bildung von kristallinen Strukturen im Feststoff am Beispiel von 1,3-Dienen.

![ggg](https://raw.githubusercontent.com/deltorobarba/repo/master/sciences_313.png)


* Das Verhältnis von cis- zu trans-konfigurierten Doppelbindungen hat genau wie das von 1,2- und 1,4-Polymerisaten entscheidenden Einfluss auf die Materialeigenschaften. Naturkautschuk entspricht dem ausschließlich cis-konfigurierten Polyisopren, daher rühren auch dessen elastische Eigenschaften. Demgegenüber findet man im kristallinen unelastischen Guttapercha ausschließlich das trans-1,4-Polyisopren.

  * Guttapercha dient zur Isolierung von Unterseekabeln und zur Herstellung chemikalienbeständiger Kitte.

> Als Kautschuk wird heute oft Synthesekautschuk anstelle von Naturkautschuk eingesetzt. Durch Vulkanisation wird das Produkt gummielastisch und vielseitig einsetzbar.

* Auch die unterschiedliche Anordnung der Seitengruppen an tertiären, asymmetrischen Kohlenstoffatomen führt zu anderen Eigenschaften der Kunststoffe. Diese Form der Stereoisomerie von Makromolekülen bezeichnet man als Taktizität.

* Normalerweise entstehen bei einer Polymerisation (/S.374ff.) bei hohen Temperaturen und Drücken taktische Produkte ohne regelmäßige sterische Anordnung. Diese Hochdruckpolymere haben geringe Dichten, sind weich, elastisch oder gummiartig.

* Die katalytisch bei geringen Temperaturen und Drücken erzeugten iso- und syndiotaktischen Polymere wie Polypropylen weisen ein hohes Maß an Kristallinität und damit auch eine höhere Dichte auf. Erst dadurch wird eine Verarbeitung zu Rohren, Folien oder Fasern möglich.

* Die stereospezifische Niederdrucksynthese wurde erst durch die Entdeckung spezieller metallorganischer Katalysatoren durch K. W. ZIEGLER(1898-1973) und G. NATTA (1903 bis 1979) möglich.

![ggg](https://raw.githubusercontent.com/deltorobarba/repo/master/sciences_314.png)


**Sekundärstruktur**

* Zwischen den Molekülen der Makromoleküle bilden sich je nach Art der in den Monomeren vorliegenden Atomgruppen Van-der-Waals-Kräfte, Dipol-Dipol-Wechselwirkungen oder Wasserstoffbrückenbindungen aus.

* Die zwischenmolekularen Kräfte wachsen mit der Größe und dem Ordnungsgrad der Moleküle. Allerdings schwanken die Molekülgrößen und Ordnungsgrade und damit auch die zwischenmoleklaren Kräfte stark.

> Die Stärke der Sekundärbindungen und damit die Bedeutung der Sekundärstruktur für die Eigenschaften der Werkstoffe hängt von der Größe der Moleküle, dem Ordnungsgrad, der Molekülart sowie den äußeren Bedingungen ab.

* Dies führt beim Erwärmen dazu, dass die Anziehungskräfte in einem mehr oder weniger breiten Temperaturfenster überwunden werden und Kunststoffe daher keine definierte Schmelztemperatur aufweisen. Die Temperatur, bei der ein Kunststoff in den weichen plastischen Zustand übergeht, bezeichnet man als Glasübergangs- bzw. Glastemperatur.

> Ein Maß für die Molekülgröße ist der Polymerisationsgrad, der die Anzahl der Monomere in einem Makromolekül angibt.



![ggg](https://raw.githubusercontent.com/deltorobarba/repo/master/sciences_315.jpeg)


**Tertiärstruktur**

* Die Tertiärstruktur der Kunststoffe beschreibt den Zusammenhang zwischen der Kristallinität und den Eigenschaften der Kunststoffe.

**Polyamide**

* aliphatische Polymere mit sich regelmäßig wiederholenden Amidbindungen

* Kondensationsprodukt

https://de.wikipedia.org/wiki/Polyamide#Nylon

Nylon (Polyhexamethylenadipinsäureamid)

https://www.u-helmich.de/che/Q2/kunststoffe/index.html

###### ***Polymerisation*** *(Polymere)*

**Die Polymerisation liefert viele bekannte Kunststoffe**

> Der Großteil der heutigen Kunststoffe wird durch Polymerisation gewonnen, allein Polyethylen (PE) hat einen Marktanteil von ca. 30%.

* Nach dem **Prinzip der Polymerisation von Ethen-Derivaten** werden folgende wichtige Kunststoffe synthetisiert:

  * Polyacrylnitril aus Acrylnitril-Monomeren
  * Polyethylen aus Ethen-Monomeren
  * Polypropylen aus Propen-Monomeren
  * Polyvinylchlorid (PVC) aus Chlorethen-Monomeren (Vinylchlorid)
  * Polystyrol aus Styrol-Monomeren
  * Teflon aus Tetrafluorethen-Monomeren
  * Plexiglas aus Methylmethacrylat-Monomeren
  * Orlon aus Acrylnitril-Monomeren

* In der Kunststoff-Industrie spielen aber auch Butadien und Butadien-Derivate eine wichtige Rolle:

  * Polybutadien aus Butadien-Monomeren (Buta-1,3-dien)
  * Synthetischer Kautschuk aus Isopren-Monomeren (2-Methyl-buta-1,3-dien)
  * **Neopren** aus Chloropren-Monomeren (2-Chlor-2-methyl-buta-1,3-dien)

> **Thermoplaste**: Teflon (Polytetrafluorethen), Polyethylen (PE), Polyvinylchlorid (PVC), Polymethacrylsäuremethylester (PMMA, Plexiglas), Polystyrol. **Elastomere**: Polybutadien aus Butadien-Monomeren (Buta-1,3-dien), Synthetischer Kautschuk aus Isopren-Monomeren (2-Methyl-buta-1,3-dien), Neopren aus Chloropren-Monomeren (2-Chlor-2-methyl-buta-1,3-dien).

> Bei der Polymerisation werden ungesättigte Monomere - oft Derivate des Ethens oder Propens - einfach aneinander gehängt. Die Doppelbindungen "klappen auf", und schon sind die Monomere miteinander verbunden.

* Siehe auch [Kettenpolymerisation](https://de.m.wikipedia.org/wiki/Kettenpolymerisation) und als Beispiel [PVC - Polyvinylchlorid](https://de.m.wikipedia.org/wiki/Polyvinylchlorid)

* Die Polymerisation ist einer von drei möglichen Mechanismen, nach denen sich Monomere zu einem Polymer vereinigen können. Die beiden anderen Mechanismen sind die Polyaddition und die Polykondensation.

* Von einer Polymerisation spricht man, wenn sich die Monomere quasi durch "Aufklappen" ihrer C=C-Doppelbindungen untereinander verknüpfen. Meistens spielen dabei Radikale eine besondere Rolle, wie die folgende Abbildung zeigt.



*Grundprinzip der radikalischen Polymerisation*

![ggg](https://raw.githubusercontent.com/deltorobarba/repo/master/sciences_0829.png)

* Hier ist das Grundprinzip der radikalischen Polymerisation dargestellt. In dem Bild sieht man vier Alken-Moleküle R-CH=CH2 (Doppelbindung zwischen den beiden ersten C-Atomen) sowie ein Starter-Radikal. Die Alken-Moleküle sind stark vereinfacht dargestellt; wichtig sind hier nur die C=C-Doppelbindungen.

* Bei der Polymerisation "klappen" die Doppelbindungen auf, wodurch einfach besetzte Kugelwolken bzw. Orbitale entstehen, die dann die Verbindung zu benachbarten Alken-Einheiten herstellen. Das zuletzt "angebaute" Alken-Molekül hat wieder ein einfach besetztes Orbital ("einsames Elektron"), so dass sich ein weiteres Alken-Molekül anlagern kann. Einzelheiten dazu siehe: radikalische Polymerisation.



**Polymerisation (Polymere)**

> Unter einer Polymerisation versteht man eine sich vielfach wiederholende gleichschrittige Reaktionsfolge - Kettenreaktion genannt, bei der sich einfache Ausgangsstoffe (Monomere) über reaktive Doppelbindungen miteinander verbinden. In der stark exothermen Reaktion **werden keine Nebenprodukte abgespalten**.

* Die Aktivierung der Doppelbindung kann auf verschiedenen Wegen erreicht werden, so z.B. durch Energiezufuhr (Wärme, Bestrahlung, Ultraschall) oder durch Verwendung stofflicher Initiatoren, die sich leicht zu Radikalen oder reaktiven lonen umsetzen lassen und so die Polymerisation starten.

* Von technischem Interesse sind insbesondere radikalbildende Initiatoren, wie Peroxide und Azoverbindungen, sowie ionische Initiatoren.

* Zunehmend gewinnen Metall-Komplex-Initiatoren an Bedeutung, da sich mit ihrer Hilfe eine ausreichende Kontrolle der stereochemischen Abläufe erreichen lässt

* **Radikalische Polymerisation**

* Radikale, also reaktive Teilchen, die ein freies, ungepaartes Elektron besitzen, sind die Initiatoren der radikalischen Polymerisationen.

* Solche Radikale werden in der Regel in einem der eigentlichen Reaktion vorgelagerten Schritt durch Einwirkung von sichtbarem Licht oder anderer energiereicher Strahlung bzw. Wärme auf leicht spaltbare Peroxide oder Azoverbindungen gewonnen.

* Im Labor greift man dafür häufig auf Dibenzoylperoxid (DBO) oder Azobisisobutyronitril (AIBN) zurück, während im industriellen Maßstab aus wirtschaftlichen Gründen preiswertere Verbindungen verwendet werden.



![ggg](https://raw.githubusercontent.com/deltorobarba/repo/master/sciences_302.png)


Das gebildete Startradikal löst in der Folge eine Kettenreaktion aus, die im Wesentlichen durch drei Schritte gekennzeichnet ist:

1. In der Startreaktion (auch als Kettenstart bezeichnet) spaltet das Startradikal die Doppelbindung des Monomers, indem es sich an dieselbe addiert und so ein neues Radikal bildet.

2. Dieses neue Radikal reagiert in der folgenden Kettenwachstumsreaktion mit einem weiteren Monomer, wobei wieder ein Radikal entsteht, das erneut mit einem Monomer reagiert. Der Wachstumsprozess endet erst, wenn..

3. ..in der der Abbruchreaktion entweder zwei Radikale miteinander kombinieren oder aber durch Disproportionierung (d.h. Bildung eines Alkans und eines Alkens durch Übergang eines Wasserstoffatoms) ihre Radikaleigenschaft verlieren und so keine weitere Wachstumsreaktion mehr möglich ist. Da die Abbruchreaktionen rein statistisch erfolgen, bilden sich Makromoleküle unterschiedlicher Kettenlänge.

![ggg](https://raw.githubusercontent.com/deltorobarba/repo/master/sciences_303.png)

> Mit X= H erhält man als Produkt das Polyethylen (PE), mit X = CI das Polyvinylchlorid (PVC), mit X = CN das Polyacrylnitril (PAN) und X= C6H, liefert das Polystyrol (PS).

* Der Ablauf bleibt prinzipiell gleich, sodass eine Vielzahl von Ethenderivaten polymerisierbar ist. Man kann auch verschiedene Monomere miteinander umsetzen, woraus dann Copolymere oder Mischpolymere  entstehen.

* Bei der Polymerisation erfolgt die Kettenwachstumsreaktion in der Regel so, dass sich das stabilere Kettenradikal bildet. D.h. die Addition von Monomeren, wie Monochlorethen oder Propen (mit unsymmetrisch substituierten Doppelbindungen) erfolgt normalerweise in der sogenannten Kopf-Schwanz-Stellung (1,3-Stellung der Substituenten X im Produkt), da das substituierte C-Atom im Kettenwachstumsschritt immer das freie Elektron trägt.

* Der Verlauf der Polymerisation wird in hohem Maße durch den Einsatz spezieller Katalysatoren beeinflusst, aber auch die Temperatur der exothermen Reaktion steuert die Eigenschaften des Polymers.



Regelung der Kettenlänge bei der Polymerisation

* Die Kettenlänge der gebildeten Polymere kann durch die Menge der verwendeten Startersubstanz ebenso wie durch die Temperatur beeinflusst werden. Je höher die Temperatur ist bzw. je mehr Startradikale zur Verfügung stehen, umso schneller verläuft die Reaktion, entsprechend kürzer werden die gebildeten Ketten.
* Werden die Ketten zu lang, besteht die Gefahr eines explosiven Verlaufs der Polymerisation, da die Reaktionswärme in dem mit wachsender Kettenlänge zunehmend viskosen Polymerisat nur schlecht abgeführt werden kann.
* Durch Zusatz geeigneter Inhibitoren oder Regler kann die Kettenlänge ebenfalls gezielt beeinflusst werden. Generell eignen sich ungesättigte Substanzen mit hoher Elektronendichte (insbesondere aromatische Systeme) als Inhibitoren, da der Elektronenmangel eines entstehenden Radikals durch mesomere oder induktive Effekte stabilisiert wird.
* Inhibitoren sind Stoffe wie Amine, Phenole und Chinone, die relativ stabile und damit reaktionsträge Radikale bilden, die die wachsende Kette nicht fortpflanzen können. Durch Disproportionierung oder Rekombination werden die Inhibitoren weiter stabilisiert.
* [Thiole](https://de.m.wikipedia.org/wiki/Thiole) sind Regler des Kettenwachstums

![ggg](https://raw.githubusercontent.com/deltorobarba/repo/master/sciences_304.png)


lonische Polymerisation

* Eine Polymerisation kann auch durch lonen initiiert werden. Man unterscheidet zwischen kationischer und anionischer Polymerisation.
* Für die kationische Polymerisation können als Initiatoren starke Brönsted-Säuren wie Perchlorsäure HCIO4 eingesetzt werden. Meist werden die Kationen jedoch aus Lewis-Säuren wie Aluminiumtrichlorid AlCl3 gewonnen. Die Säuren binden zunächst ein Wassermolekül, wobei ein Proton freigesetzt wird.
* Bei der Startreaktion wird das Proton an die Doppelbindung des Monomers addiert. Dabei entsteht ein Carbo-Kation, das mit einem weiteren Alkenmolekül reagiert und so das Kettenwachstum sichert. Durch Abspaltung eines Protons oder durch Anlagern eines Anions (aus dem Startkatalysator) wird das Kettenwachstum abgebrochen.
* Man setzt dazu auch Basen oder geeignete Anionen zu. Ein wichtiges Produkt der kationischen Polymerisation ist Butylkautschuk, ein Copolymer aus 2-Methylpropen und 2-Methylbuta-1,3-dien.

* Die Brönsted-Säure A-H überträgt ihr Proton an das Monomer unter Bildung eines Kations. Die starke Base B reagiert mit dem Monomer zu einem Carbanion.

![ggg](https://raw.githubusercontent.com/deltorobarba/repo/master/sciences_305.png)


* Die anionische Polymerisation wird durch Addition einer starken Base oder eines reaktionsfähigen Metalls wie Natrium ausgelöst, wobei ein Carbanion entsteht, welches die Kettenreaktion fortsetzt. Man muss folglich mit extrem reinen Substanzen arbeiten, da schon kleinste Verunreinigungen (z. B. Wasser oder Alkanol) ein Proton freisetzen können, das zum Kettenabbruch führt.


![ggg](https://raw.githubusercontent.com/deltorobarba/repo/master/sciences_306.png)


###### ***Polyaddition*** *(Polyethen, Polyurthane) - eine funktionelle Gruppe*

> Bei der Polyaddition schließlich **addieren elektrophile Monomere an die C=C-Doppelbindung von Alken-Derivaten**. Dabei kommt es zur Umlagerung von H-Atomen. Die so hergestellten Kunststoffe bestehen also stets aus zwei verschiedenen Monomeren, die sich regelmäßig abwechseln.

![ggg](https://raw.githubusercontent.com/deltorobarba/repo/master/sciences_0935.png)


*Als typisches Beispiel für eine Polyaddition wird in der Literatur oft die Reaktion von Butan-1,4-diol mit Hexamethylen-1,6-diisocyanat dargestellt:*

![ggg](https://raw.githubusercontent.com/deltorobarba/repo/master/sciences_0830.png)


* Bei dieser speziellen Additionsreaktion handelt es sich um eine nucleophile Addition. Das C-Atom der Isocyanat-Gruppe ist durch die benachbarten elektronegativen N- und O-Atome positiv polarisiert, so dass das negativ polarisierte O-Atom der Hydroxygruppe des Alkohol nucleophil angreifen kann.

* In der mittleren Zeile der Abbildung 1 sehen wir ein hypothetisches Zwischenprodukt (fünfwertiges C-Atom, dreiwertiges O-Atom), in der letzten Zeile das Endprodukt dieser Addition.

* Man achte auf die Umlagerung des blau gekennzeichneten H-Atoms. Diese Umstrukturierung ist typisch für Polyadditionen. Bei einer Polymerisation (radikalisch, kationisch oder anionisch) bleiben die Strukturen der Monomere erhalten, es werden keine Atome umgruppiert.

* Bei einer Polyaddition dagegen werden die Strukturen der Monomere verändert. Im Gegensatz zu einer Polykondensation werden allerdings keine niedermolekularen Nebenprodukte (zum Beispiel Wasser) freigesetzt.

> Bei einer Polyaddition werden zwar keine Moleküle abgespalten, die Monomere werden aber umstrukturiert.

**Unterschied Polymerisation zu Polyaddition (andere Arten):**

* Bei einer Polymerisation bildet sich aus zwei Monomeren ein Dimer, dann lagert sich ein drittes Monomer an, und es entsteht ein Trimer. So geht das immer weiter, die Kette wird kontinuierlich länger.

* Bei einer Polyaddition bilden sich auch zunächst Dimere, Trimere und so weiter, auch Oligomere (4-10 Monomere) und kurze Polymere entstehen. Allerdings wachsen diese nicht immer weiter, sondern mehrere Oligomere oder kurze Polymere lagern sich zu längeren Polymeren zusammen, die sich wieder mit anderen Oligomeren oder Polymeren zu noch längeren Polymeren zusammenlagern.

* Die folgende Abbildung zeigt Polymerisation und Polyaddition im schematischen Vergleich:

![ggg](https://raw.githubusercontent.com/deltorobarba/repo/master/sciences_0831.png)


**Polyaddition**

* Die Polyaddition ist gekennzeichnet durch die Addition einer bi- oder polyfunktionellen Verbindung an ein anderes Molekül, das als reaktives Strukturelement Doppelbindungen oder cyclische Gruppen aufweist.

* Dabei kommt es immer zur Wanderung eines Wasserstoffatoms von der funktionellen Gruppe der addierten Verbindung zu einem Atom des ungesättigten bzw. cyclischen Monomers. Bei der Polyaddition entstehen keine Reaktionsnebenprodukte.

* Zwei große Kunststoffgruppen werden heute durch Polyaddition nach diesem Verfahren hergestellt: die **Polyurethane** und die **Epoxidharze**.

**Polyurethane**

* Urethane sind Amide der Kohlensäure. Bei der Polyaddition werden Urethanbindungen (Ester-Amid-Bindungen: -NH-CO-O-) während der Reaktion zwischen Diisocyanaten und Diolen gebildet.

  * Dabei wird die Hydroxylgruppe an die Isocyanat (-N=C=O)-Gruppe addiert, und zwar ausschließlich an die Kohlenstoff-Stickstoff-Bindung.

* Das Wasserstoffatom wandert anschließend vom Sauerstoffatom der Hydroxylgruppe zum Stickstoffatom der Isocyanatgruppe.

  * Da Isocyanate mit Wasser zu Kohlenstoffdioxid reagieren, kann man durch Wasserzugabe ein Aufschäumen der Polyurethane erreichen.

* Polyurethanschaumstoffe kann man in verschiedenen Härtegraden gewinnen.

* Die „elastische" Stoßstange eines Autos kann ebenso aus Polyurethan gefertigt werden, wie eine dünne Kabelummantelung.



![ggg](https://raw.githubusercontent.com/deltorobarba/repo/master/sciences_307.png)


* Genau wie die Hydroxylgruppe der Alkohole kann auch die Aminogruppe von Aminen an das Isocyanat addiert werden. Dabei entstehen als Reaktionsprodukte Polyharnstoffe bzw. Kohlensäurediamide (R-NH-CO-NH-R'). Diese Reaktion wird für die gezielte Vernetzung der Polyurethane genutzt, indem man bei der Synthese einen Überschuss Diisocyanat zufügt, das mit der reaktiven Aminogruppe eines linearen, kettenförmigen Polyurethans reagiert.

> Kaum eine andere Kunststoffgruppe eröffnet so vielfältige Einsatzgebiete. Je nach verwendetem Ausgangsstoff kann man lineare (Diol und Diisocyanat) oder vernetzte (mindestens eine Komponente ist trifunktionell) Polyurethane erhalten, die für viele Anwendungen in Schaumstoffen, Elastomeren, Lacken, Klebstoffen, Fasern usw. eingesetzt werden.

* Ob weich, hart, offen- oder geschlossenporig, sogar mit einer Porengröße nach Wunsch können sie gefertigt werden, abhängig von der Beschaffenheit und dem Mengenverhältnis der eingesetzten Monomere.

* Für Armaturenabdeckungen oder Armlehnen in Pkws werden schäumungsfähige Reaktionsgemische (Diisocyanat und Diol unter Wasserzusatz) in temperierten Formwerkzeugen unter Druck ausgehärtet. Dabei entsteht ein Werkstück aus Polyurethan, dessen Dichte infolge des Schäumdrucks von innen nach außen stetig zunimmt. Diese Integralschaum-Formstücke besitzen sofort eine feste, schützende Außenhaut und haben gleichzeitig einen energieabsorbierenden, elastischen Kern.

* Hautnah spürt jeder von uns die Vorteile dieser Werkstoffe, etwa wenn man an die hochabriebsfeste Schuhsohle denkt, die direkt an den Schaft des Sportschuhs geschäumt wurde.

> Uberhaupt sind die Polyurethane aus der Welt des Sports nicht mehr wegzudenken, sei es für die Konstruktion eines Bootsrumpfs oder den Kern eines Skis, dem der Polyurethan-Schaumstoff erst die erforderliche Dämpfungseigenschaft bei geringem Gewicht verleiht.

* Selbst ein moderner Fußball verdankt seine präzise Flugbahn einer Oberfläche aus Millionen von elastischen, gasgefüllten Mikrokügelchen auf Polyurethanbasis.


![ggg](https://raw.githubusercontent.com/deltorobarba/repo/master/sciences_308.png)


**Epoxidharze**

* Ein weiteres Produkt der Polyaddition sind die Epoxidharze.

* Diese haben sich bei der Herstellung glasfaserverstärkter Formteile ebenso bewährt wie für Klebstoffe oder Anstrichstoffe.

###### ***Polykondensation*** *(Polyester & Polyamide) - mindestens zwei funktionelle Gruppen, meist Duroplaste*

**Polykondensation**

> Verbindendes Merkmal der Polykondensation ist die Verknüpfung von Monomeren, **die <u>mindestens zwei reaktionsfähige funktionelle Gruppen</u> tragen**, zu einem Makromolekül. Dabei werden **einfache, niedermolekulare Nebenprodukte wie Wasser, Alkohole, Ammoniak oder Chlorwasserstoff abgespalten**.

* Werden bifunktionelle Monomere mit zwei funktionellen Gruppen eingesetzt, so entstehen lineare, unverzweigte Polymere (**Thermoplaste**).

* Bei der Verwendung von polyfunktionellen Monomeren mit mehr als zwei reaktiven Gruppen erhält man dagegen verzweigte oder gar dreidimensional vernetzte Polymere (**Duroplaste**).

* Damit eine Polykondensation stattfinden kann, also der Zusammentritt vieler Moleküle unter Abspaltung von Wasser oder anderen kleinmolekularen Produkten, müssen die Monomere zwei funktionelle Gruppen haben, die in der Lage sind, Atome oder Atomgruppen abzugeben.

* Geradezu prädestiniert dafür sind OH-Gruppen und COOH-Gruppen sowie NH2-Gruppen.

> **Bei der Polykondensation werden Monomere mit OH-, NH2-, Cl- oder COOH-Gruppen unter Abspaltung von Wasser oder Chlorwasserstoff HCl aneinander gehängt**.

![ggg](https://raw.githubusercontent.com/deltorobarba/repo/master/sciences_0936.png)


> **Als Kondensation bezeichnet man allgemein eine chemische Reaktion, bei der sich zwei Moleküle unter Abspaltung eines kleineren Produktes zu einem größeren Endprodukt vereinigen.**

*Meistens handelt es sich bei dem abgespaltenen Produkt um Wasser. Ein einfaches Beispiel für eine solche Kondensation aus der Biochemie ist die Kondensation von zwei Aminosäuren zu einem Dipeptid:*

![ggg](https://raw.githubusercontent.com/deltorobarba/repo/master/sciences_0832.png)


> Weitere Einzelheiten beim Thema "Aminosäuen, Peptide und Proteine". **In der Biochemie bzw. Zellbiologie kommen überhaupt recht viele Kondensationsreaktionen und auch Polykondensationen vor**, wenn sich zum Beispiel Tausende von Glucose-Molekülen zu einem Stärke-Molekül zusammenschließen oder Zehntausende von Nucleodiden zu einem DNA-Strang.

> [Enzym zersetzt Plastik in Rekordzeit - Aus dem Kompost isoliertes Enzym ermöglicht schnelles und effizientes PET-Recycling](https://www.scinexx.de/news/biowissen/enzym-zersetzt-plastik-in-rekordzeit/)

**Polykondensation: Polyester**

  * Ausgangsstoffe sind im einfachsten Fall eine beliebige Dicarbonsäure und ein Diol. Daraus entsteht zunächst ein **bifunktioneller Ester mit einer Hydroxy- (OH) und einer Carboxy-Gruppe (COOH)** (Ester = Carbonsäure (Cabroxy-Gruppe) + Alkohole (Hydroxy-Gruppe))

  * Durch **vielfache Wiederholung der Veresterung** an diesen beiden funktionellen Gruppen bildet sich ein linearer Polyester, wobei Wasser als Nebenprodukt anfällt.

  * **Aus Terephthalsäure und Ethandiol erhält man Polyterephthalsäureethylester bzw. Polyethylenterephthalat (PET).**

  * Der Kunststoff Polyethylenterephthalat (PET) wurde in den Jahren 1939 bis 1946 von WHINFIELD und DICKSON in England entwickelt.

  * Für die Synthese nutzt man den Methylester der Terephthalsäure und setzt diesen mit Ethandiol um. Als Nebenprodukt dieser Umesterung genannten Reaktion fällt Methanol an.

  * Anstelle der Ester kann man auch die reaktiveren Carbonsäurechloride verwenden und diese mit einem Diol umsetzen. In diesem Fall entsteht Chlorwasserstoff als Nebenprodukt. Letztlich kommt es bei der Polyestersynthese nur darauf an, Monomere mit zwei reaktiven funktionellen Gruppen immer wieder miteinander durch eine Esterbindung zu verknüpfen.

> Polyester sind Kunststoffe, die aus dem Alltag nicht mehr wegzudenken sind. Nicht nur Textilfasern wie Trevira, Diolen und so weiter sind Polyester, sondern auch PET-Getränkeflaschen (Aufmacherbild), CDs, DVDs und BlueRays werden aus Polyestern hergestellt.

* Es gibt grundsätzlich zwei Möglichkeiten, einen Polyester herzustellen. Ein Ester ist ja eine Verbindung aus einer Sauerstoffsäure und einem Alkohol. Ein Polyester besteht aus vielen Säure- und Alkohol-Molekülen, die abwechselnd miteinander verbunden sind. Jedes Alkohol-Molekül ist mit zwei Säure-Molekülen verbunden, muss daher also zwei OH-Gruppen besitzen.

* Jedes Säure-Molekül ist mit zwei Alkohol-Molekülen verbunden, und muss folglich zwei COOH-Gruppen besitzen. Wir können also einen ganz einfachen Polyester herstellen (zumindest in der Theorie), wenn wir einen zweiwertigen Alkohol (z.B. Ethan-1,2-diol) mit einer Dicarbonsäure (z.B. Oxalsäure) koppeln.








*Das Bild zeigt einen "Minipolyester" aus vier Molekülen Glycol (Ethan-1,2-diol) und drei Molekülen Oxalsäure:*

![ggg](https://raw.githubusercontent.com/deltorobarba/repo/master/sciences_0833.png)


*Diese Abbildung zeigt das verallgemeinerte Grundprinzip der Polyesterbildung. Hierbei haben nur 5 Säure- und 4 Alkohol-Moleküle reagiert, bei einem "echten" Polyester kondensieren Tausende solcher Monomere*

![ggg](https://raw.githubusercontent.com/deltorobarba/repo/master/sciences_0834.png)


> Besitzt die Carbonsäure drei Carboxylgruppen (wie zum Beispiel Zitronensäure) oder der Alkohol drei OH-Gruppen (wie zum Beispiel Glycerin), so können sich sogar verzweigte Polyester bilden, die dann natürlich viel stabiler sind als unverzweigte Polyester.



**PET**

* Einer der bekanntesten Polyester ist PET, die Abkürzung für Polyethylenterephthalat.

* Diese Verbindung ist ein Polymer aus Ethan-1,2-diol (Ethylenglycol) und der Terephthalsäure, einem Benzolring mit zwei Carboxylgruppen.

* Hier sieht man ein "Mini-PET-Molekül", bestehend aus vier Molekülen der Dicarbonsäure und drei Molekülen des Diols. PET ist ein thermoplastischer Kunststoff, der vor allem durch die PET-Flaschen berühmt geworden ist, in denen heute fast alle Getränke verkauft werden.

* Aus dem PET-Kunststoff kann man aber auch dünne Fasern ziehen (Thermoplast!), die dann unter dem Handelsnamen Trevira oder Diolen als Textilfasern bekannt sind.

![ggg](https://raw.githubusercontent.com/deltorobarba/repo/master/sciences_0835.png)


**Polyamide**

* Polyamide entstehen, wenn eine Dicarbonsäure mit einem Diamin, einem zweifachen Amin verestert wird.

* Hier sehen wir das Grundprinzip dieser Polymerisation am Beispiel eines "Mini-Polyamids" aus fünf Diol-Molekülen und vier Diamid-Molekülen:

![ggg](https://raw.githubusercontent.com/deltorobarba/repo/master/sciences_0836.png)


**[Polyamide](https://de.m.wikipedia.org/wiki/Polyamide) wie Nilon oder Perlon**

* siehe auch: https://www.chemie.de/lexikon/Polyamide.html

* [Perlon](https://de.m.wikipedia.org/wiki/Polyamide#Perlon) - chemisch [Polycaprolactam](https://de.m.wikipedia.org/wiki/Polycaprolactam), Ausgangsstoff: [Caprolactam](https://de.m.wikipedia.org/wiki/Caprolactam)

* Die Bezeichnung Polyamide wird üblicherweise als Bezeichnung für synthetische, technisch verwendbare thermoplastische Kunststoffe verwendet und grenzt diese Stoffklasse damit von den chemisch verwandten Proteinen ab. Fast alle bedeutsamen Polyamide leiten sich von primären Aminen ab, d. h. in ihren Wiederholeinheiten kommt die funktionelle Gruppe -CO-NH- vor. Daneben existieren auch Polyamide von sekundären Aminen (-CO-NR-, R = organischer Rest). Als Monomere für die Polyamide finden besonders Aminocarbonsäuren, Lactame und/oder Diamine und Dicarbonsäuren Verwendung.

* Bekannte Polyamid-Kunststoffe sind die Textilfasern Perlon und Nylon, aus denen nicht nur Bekleidung hergestellt wird, sondern auch dicke Seile und Taue sowie Bodenbeläge. Bei der Bildung des Polyamids entstehen übrigens Peptidbindungen, die mit denen der Proteine identisch sind.

* Die Kunstfaser Nylon (1934 zum ersten Mal hergestellt) entsteht durch Polykondensation der Dicarbonsäure Adipinsäure (1,6-Hexandisäure) und des Diamids Hexamethylendiamin (1,6-Diaminohexan), also recht großen Monomeren mit je sechs C-Atomen.

* Im Folgenden sieht man die Bildung von Nylon aus 1,6-Hexandisäure und 1,6-Diaminohexan. Nylon ist ein Thermoplast.

  * Der Kunststoff wird im flüssigen Zustand durch feine Düsen gepresst und härtet dann zu Kunstfasern aus. Dabei wird der Kunststoff mit flüssigem Stickstoff gekühlt.

  * Die so erhaltenen Fasern werden dann vorsichtig (langsam) gedehnt, bis das handelsfertige Nylon entsteht. Bei diesem "Strecken" richten sich die Polymerketten mehr oder weniger parallel aus, zwischen den H-Atomen der ehemaligen Aminogruppen und den O-Atomen der ehemaligen Carboxylgruppen können sich H-Brücken bilden, so dass der gestreckte Nylonfaden deutlich stabiler ist als das Rohprodukt.

![ggg](https://raw.githubusercontent.com/deltorobarba/repo/master/sciences_0837.png)

* Hier sieht man die H-Brücken zwischen zwei dieser gestreckten Polymerfäden:

![ggg](https://raw.githubusercontent.com/deltorobarba/repo/master/sciences_0838.png)


**Polycarbonate**

* Polycarbonate gewinnt man aus Phosgen, einem Chlorderivat der Kohlensäure, bei dem beide OH-Gruppen durch Cl-Atome ersetzt sind, und einem Bisphenol, also einem Alkan-Derivat mit zwei Phenol-Gruppen.

* Der einfachste Bisphenol ist Bisphenol A, der aus Phenol und Aceton synthetisiert werden kann.

![ggg](https://raw.githubusercontent.com/deltorobarba/repo/master/sciences_0839.png)

* Das Polykondensationsprodukt der beiden Edukte wird als Polycarbonat bezeichnet, hauptsächlich deswegen, weil das Phosgen ein Kohlensäure-Derivat ist und weil die Salze der Kohlensäure Carbonate heißen. Bei der Kondensationsreaktion werden die beiden Chlor-Atome abgegeben, sie reagieren mit den H-Atomen der OH-Gruppen zu Chlorwasserstoff.

![ggg](https://raw.githubusercontent.com/deltorobarba/repo/master/sciences_0840.png)


* Hier sehen wir einen Ausschnitt aus einem solchen Polycarbonat-Polymer. Polycarbonate sind einerseits mechanisch sehr stabil, andererseits können sie sehr durchsichtig hergestellt werden. Daher eignen sie sich hervorragend für Brillen oder CDs und DVDs. Sogar Schutzhelme werden aus Polycarbonaten hergestellt.






---



![ggg](https://raw.githubusercontent.com/deltorobarba/repo/master/sciences_297.png)


> Da es sich bei der Veresterung um eine Gleichgewichtsreaktion handelt, kommt der Entfernung des flüchtigen Nebenprodukts aus dem System große Bedeutung zu, um hohe Umsatzraten zu erzielen.


![ggg](https://raw.githubusercontent.com/deltorobarba/repo/master/sciences_299.png)


* Grosse Bedeutung haben auch lineare Polyester aus Kohlensäureestern wie Diphenylcarbonat und aromatischen Diolen wie dem 2,2-Di-(4-hydroxypheny/)propan, die z.B. unter dem Handelsnamen Makrolon® bekannt sind.

* Aus Makrolon® können praktisch unzerbrechliche transparente Platten hergestellt werden, die sich als Werkstoff für Brillengläser und Motorradhelme ebenso eignen wie für die Datenspeicherung auf einer CD.

* **Verwendet man ungesättigte Monomere, so bilden sich bei der Kondensation ungesättigte Polyester, die als Rohstoffe für Lack- und Gießharze oder in faserverstärkter Form als Werkstoffe dienen können. Entscheidend ist das Vorliegen von Doppelbindungen im Polymer, sodass eine Polymerisation als Folgereaktion ablaufen kann**.

* [Polyethylenterephthalat (PET)](https://de.m.wikipedia.org/wiki/Polyethylenterephthalat) ist ein Polyester der [Terephthalsäure C8H6O4](https://de.m.wikipedia.org/wiki/Terephthalsäure). Großtechnisch erfolgt die Synthese meist durch Schmelzkondensation von sehr reiner Terephthalsäure mit Ethandiol (12-Dihydroxyethan) in einer Stickstoffatmosphäre unter erhöhtem Druck. Das entstehende Wasser wird bei einer Temperatur zwischen 220 und 260°C aus dem Reaktionsgemisch abdestilliert, wodurch die Ausbeute nach dem Prinzip von LECHATELIER auf die Seite des Polyesters verschoben wird.


![ggg](https://raw.githubusercontent.com/deltorobarba/repo/master/sciences_298.png)


* Weltweit werden heute rund 40 Mio. t PET produziert. Hauptverwendungszweck ist die Erzeugung von Fasern, Folien und die Produktion von Flaschen aller Art. Der Polyester verdankt seinen Siegeszug in der Lebensmittelindustrie der Tatsache, dass er farblos und von hoher Lichtdurchlässigkeit ist.

* Sortenreines PET ist sehr gut recyclingfähig und wird hauptsächlich zu Textilfasern verarbeitet. Textilien auf PET-Basis sind knitterfrei, scheuerfest, form- und chemikalienbeständig, wetterfest sowie lichtecht. Unter verschiedenen Handelsnamen werden PET-Fasern für Ober- und Unterbekleidung ebenso eingesetzt wie als Füllmaterial von Jacken, Schlafsäcken, oder für Zelte und Markisen. Auch bei der Herstellung von Sicherheitsgurten oder als Trägermaterial für Kinofilme findet der vielseitige Polyester Verwendung.

**Polykondensation: Polyamide**

* Polyamide wie Nylon und Perlon® werden in vielfältiger Weise als Synthesefasern und als technische **Thermoplaste** genutzt. Aus den Fasern stellt man Freizeitbekleidung, Bodenbeläge oder Taue her.

* Aber auch Maschinenbauteile, Folien oder Heizöltanks werden aus Polyamiden mit thermoplastischen Eigenschaften gefertigt.

* Analog zu den Polyestern werden die Polyamide durch Polykondensation von Dicarbonsäuren mit Diaminen gewonnen.

> Genau wie bei den Proteinen sind die Monomere hier über Amidbindungen (-CO-NH-) miteinander verknüpft.

* Das älteste synthetische Polyamid, Nylon, wird aus 1,6 Diaminohexan und Hexandisäure - oder auch dem reaktiveren Hexandisäurechlorid - hergestellt.

* Da Monomere mit 6 Kohlenstoffatomen im Molekül eingesetzt werden, weist das Produkt auf beiden Seiten der Amidbindung jeweils 6 Kohlenstoffatome auf. Daher spricht man von einem Polyamid 6,6.




![ggg](https://raw.githubusercontent.com/deltorobarba/repo/master/sciences_300.png)


* Nylon:

![ggg](https://raw.githubusercontent.com/deltorobarba/repo/master/sciences_301.png)


###### *Degradation von Kunststoffen*

https://de.m.wikipedia.org/wiki/Degradation_von_Kunststoffen

> [„Plastikfresser“ im menschlichen Speichel: Enzym aus Speichelmikroben kann PET-Kunststoffe zersetzen](https://www.scinexx.de/news/biowissen/plastikfresser-im-menschlichen-speichel-entdeckt/) - In ersten Tests konnte dieses neuentdeckte Enzym PET-Plastik besser zersetzen als andere natürliche und gentechnisch-optimierte PET-[Hydrolasen](https://de.m.wikipedia.org/wiki/Hydrolasen) (Hydrolasen sind Enzyme, die Ester, Ether, Peptide, Glycoside, Säureanhydride oder C-C-Bindungen in reversibler Reaktion hydrolytisch spalten.)

*Das im menschlichen Speichel entdeckte Enzym MG8 kann den Kunststoff PET in seine Grundeinheiten zerlegen:*

![ggg](https://raw.githubusercontent.com/deltorobarba/repo/master/sciences_0906.png)

###### *Organische Elektronik*

Organische Elektronik

https://de.m.wikipedia.org/wiki/Organische_Elektronik

Organische Leuchtdiode (OLED)

https://de.m.wikipedia.org/wiki/Organische_Leuchtdiode

Organische Halbleiter (Polymer-Halbleiter)

https://www.scinexx.de/news/technik/organischer-transistor-stapelt-hoch/

 (Nature, 2023; doi: 10.1038/s41586-022-05592-2)

https://de.m.wikipedia.org/wiki/Organische_Halbleiter

##### <font color="blue">*Tenside*

###### *Tenside*

> Seife: eine Seite unpolar (für Öle und Fette) und andere Seite polar (für Wasser)



**Definition und Anwendungfall von Tensiden**

Als [**Tenside**](https://de.m.wikipedia.org/wiki/Tenside) (lat. tensio Spannung) werden Stoffe genannt, die die [Oberflächenspannung](https://de.m.wikipedia.org/wiki/Oberflächenspannung) von Wasser herabsetzen. Zu ihnen zählen:

* Seifen

* synthetische Tenside bzw. Waschmittel (auch: Detergentien)

> Tenside bewirken, dass zwei eigentlich nicht miteinander mischbare Flüssigkeiten, wie zum Beispiel Öl und Wasser, fein vermengt werden können.

* Unter Tensiden versteht man auch waschaktive Substanzen (Detergentien), die in Waschmitteln, Spülmitteln und Haarwaschmitteln enthalten sind. **In Reinigungsmittelformulierungen liegt der Tensidgehalt meist bei 1–40 %**.

* **Tenside vs Seifen**: Moderne Tenside wurden in der ersten Hälfte des 20. Jahrhunderts entwickelt und haben das traditionell verwendete Tensid Seife (Natrium- und Kaliumsalze von Fettsäuren) ergänzt und in einigen Anwendungsbereichen weitgehend verdrängt.

* Beim Einsatz in der Lebensmitteltechnik oder Kosmetik werden Tenside als **Emulgatoren** bezeichnet.

* Tenside werden auch als Benetzungs-, Lösch- und Dispergiermittel eingesetzt.


**Anwendungsfall**: Gießt man Speiseöl auf Wasser, bilden sich zwei durch eine Berührungsfläche bzw. Grenzfläche getrennte Schichten aus, die man Phasen nennt. Durch Schütteln entstehen vorübergehend Öltröpfchen in Wasser.

* Das trübe Gemisch ist eine aus zwei **Phasen** bestehende Öl-in-Wasser-(O/W)-**Emulsion**, die sich rasch wieder trennt.

* Triebkraft für die Trennung der Phasen ist ihre **Grenzflächenspannung**.

> Grenzflächenspannung ist die Kraft, die an der Berührungsfläche zwischen zwei Phasen dahingehend wirkt, dass die ausgebildete Grenzfläche so klein wie möglich ist. (siehe auch Variationsrechnung, Seifenblasen etc).

> Um eine Emulsion zu stabilisieren, d.h. **die Entmischung zu verzögern**, muss die **Grenzflächenspannung erniedrigt werden**.

* Dies geschieht durch Zugabe von Verbindungen, die mit beiden Phasen mischbar sind und sich an der Grenzfläche anreichern. **Solche Verbindungen nennt man grenzflächenaktive Stoffe oder Tenside.**

> Textilgewebe, insbesondere aus Chemiefasern, haben oft einen stark hydrophoben Charakter. Fetthaltiger Schmutz haftet fest auf den lipophilen Fasern und kann mit Wasser allein nicht entfernt werden.

* Tensidmoleküle bestehen aus einem lipophilen (Fett liebenden) und einem hydrophilen (Wasser liebenden) Teil.

  * In der Regel ist der lipophile Teil ein **langkettiger unpolarer Alkylrest** (Alkane)

  * während der **hydrophile Teil eine polare Endgruppe, z.B. eine Carboxy-Gruppe** sein kann.

* Aufgrund dieser Struktur sind Tenside bifunktionelle (**amphiphile**) Verbindungen, die Gegensätze Lipophilie und Hydrophilie sind nebeneinander im selben Molekül ausgeprägt. Sie können sich daher mit Wasser und Öl mischen und an der Grenzfläche anreichern.

![ggg](https://raw.githubusercontent.com/deltorobarba/repo/master/sciences_266.jpeg)

*Reminder: Carboxy-Gruppe COOH:*

> ![ggg](https://upload.wikimedia.org/wikipedia/commons/thumb/2/28/Carboxylic_Acid_General_Structure_V.1.svg/173px-Carboxylic_Acid_General_Structure_V.1.svg.png)




> [Glycerin](https://de.m.wikipedia.org/wiki/Glycerin)  ist der Trivialname und die gebräuchliche Bezeichnung von Propan-1,2,3-triol. Glycerin ist ein [Zuckeralkohol (Alditole)](https://de.m.wikipedia.org/wiki/Alditole) und der einfachste dreiwertige Alkohol, ein Triol. Glycerin ist in allen natürlichen Fetten und fetten Ölen – z. B. Pflanzenölen – chemisch gebunden als [Fettsäureester (Carbonsäureester - siehe Carboxy-Gruppe !)](https://de.m.wikipedia.org/wiki/Carbonsäureester) (siehe auch [Triglyceride)](https://de.m.wikipedia.org/wiki/Triglyceride) vorhanden und spielt eine zentrale Rolle als Zwischenprodukt in verschiedenen Stoffwechselprozessen. Als Lebensmittelzusatzstoff trägt es das Kürzel E 422.


> [**Carbonsäuren**](https://de.m.wikipedia.org/wiki/Carbonsäuren) ??

**Dispersion, Mizellen & Emulgiervermögen von Tensiden**

* Tenside fördern aber nicht nur die Benetzbarkeit von Schmutz und seine Ablösung. **Unter Aufwendung von mechanischer Energie und Wärme in der Waschmaschine lassen sich abgelöste Feststoffe in wässriger Lösung dispergieren, d. h. fein verteilen**.

* Zur Stabilisierung dispergierter Systeme dienen Tenside als Emulgatoren. Die Dispersion einer Flüssigkeit in einer anderen Flüssigkeit heißt Emulsion. Werden Feststoffe in einer Flüssigkeit verteilt, nennt man dieses Zweiphasensystem Suspension.

* Micellen (lat.: mica - Körnchen) können unterschiedliche Formen (z.B. Kugel- oder Stabmicellen) annehmen und erreichen eine Partikelgröße von 1-1000 nm. Lösungen, die solche kolloiden Teilchen enthalten, weisen den sogenannten Tyndall-Effekt auf.

![ggg](https://raw.githubusercontent.com/deltorobarba/repo/master/sciences_269.png)


**Die überragende wirtschaftliche Bedeutung von Tensiden liegt heute in der Verwendung in Wasch- und Reinigungsmitteln. Die Waschwirkung der Tenside beruht auf mehreren Effekten:**

1. der Verringerung der Grenzflächenspannung und der damit verbundenen Erhöhung der Benetzbarkeit der Fasern

2. dem Dispergier- und dem Emulgiervermögen der Tenside

###### *Emulgatoren*

**Tenside als Emulgatoren**

* Als Emulgatoren finden Tenside vielfältige Anwendungen im Alltag. Sie sind nicht nur Bestandteile von Waschmitteln, sondern auch von **Haarshampoos, Körperlotionen und vielen Kosmetika**.

* Eine jedermann bekannte Bedeutung haben Emulgatoren schon seit alters her bei der Zubereitung von Speisen. **Die berühmte Sauce Mayonnaise ist eine durch den in Eigelb enthaltenen Emulgator Lecithin stabilisierte Öl-Wasser-Mischung**.

* Komplexere Systeme werden erhalten, wenn eine gasförmige Phase in eine Emulsion eingearbeitet wird, wie beim Schlagen von Schlagsahne.

* Besonders gut untersucht sind Emulsionen, die als **Salben auf der menschlichen Haut** angewendet werden sollen, um Arzneistoffe auf erkrankte Haut aufzubringen oder durch die Haut zu transportieren.

* Auch Salben ohne Wirkstoffe können durch physikalische Effekte der **Tenside zur Wundheilung** oder zum Schutz und zur Regeneration der Haut beitragen.

###### *Seifen*

> Lineare (geradlinige) [Detergenzien](https://de.m.wikipedia.org/wiki/Detergens) können von Bakterien aufgespalten werden, während verzweigkettige Varianten nicht so leicht biologisch abbaubar sind.

**Seifen**

> [Glycerin](https://de.m.wikipedia.org/wiki/Glycerin)  ist der Trivialname und die gebräuchliche Bezeichnung von Propan-1,2,3-triol. Glycerin ist ein [Zuckeralkohol (Alditole)](https://de.m.wikipedia.org/wiki/Alditole) und der einfachste dreiwertige Alkohol, ein Triol. Glycerin ist in allen natürlichen Fetten und fetten Ölen – z. B. Pflanzenölen – chemisch gebunden als [Fettsäureester (Carbonsäureester)](https://de.m.wikipedia.org/wiki/Carbonsäureester) [(Triglyceride)](https://de.m.wikipedia.org/wiki/Triglyceride) vorhanden und spielt eine zentrale Rolle als Zwischenprodukt in verschiedenen Stoffwechselprozessen. Als Lebensmittelzusatzstoff trägt es das Kürzel E 422.


**Seifen (Verseifung von Fetten)**

* Durch Kochen von Fetten mit Holzasche wird schon seit Jahrtausenden **eine Klasse von grenzflächenaktiven Verbindungen mit der dargestellten typischen Struktur gewonnen: die Seifen**.

* Sie entstehen durch **Hydrolyse von Fetten, z.B. mit Natronlauge**. Dabei bilden sich die Natriumsalze der Fettsäuren wie das als Hauptbestandteil der Kernseife bekannte Natriumstearat.

* **Aber: Die historische Verknüpfung von Seifen und Waschmittel ist überholt. In modernen Waschmitteln werden Seifen nicht mehr eingesetzt**.

  * Als Waschmittel und zur Hautreinigung ist das klassische Tensid Seife nur bedingt geeignet. Seifen sind Salze schwacher Säuren und starker Basen.

  * Sie reagieren daher alkalisch und greifen die Haut ebenso an wie das zu reinigende Gewebe. Durch Bildung schwer löslicher Salze mit im Waschwasser gelösten Ca2+- und Mg2t-lonen werden zudem Kalk- bzw. Magnesiumseifen gebildet.

  * Diese schlagen sich auf der Wäsche nieder und führen zu Wäschestarre, Vergrauung und einem unangenehmen Geruch. Schon nach kurzer Zeit müssen Textilien ersetzt werden.


* Seifen sind Natrium- und Kaliumsalze höherer Fettsäuren. Die Herstellung bezeichnet man als Fettverseifung.

* Verwendet man Natriumhydroxid bekommt man Kernseife, bei Kaliumhydroxid die Schmierseife.

* Die Fettsäuren werden mit einem modernen Hochdruckverfahren hergestellt: Fett und überhitzter Wasserdampf reagieren bei Temperaturen von 100-250°C und einem Druck von 60 bar zu den Fettsäuren und Glycerin. Nach Abtrennung des Glycerins liegen die Ionen vor.

![ggg](https://raw.githubusercontent.com/deltorobarba/repo/master/sciences_151.jpg)

* Fette und Öle mischen sich aufgrund ihrer unpolaren Molekülstruktur nicht mit Wasser, sind aber in vielen polaren Lösungsmitteln löslich.

> **Durch Natron- oder Kalilauge können die Fette in Glycerin und die Salze der Fettsäuren gespalten werden**.

* Die Reaktion entspricht einer alkalischen Esterhydrolyse und wird **Verseifung** genannt.

* Die Ursache liegt darin, dass die Natriumsalze der Fettsäuren seit Jahrhunderten als Kernseife und die Kaliumsalze als Schmierseife genutzt werden.

> Durch den lipophoben Kopf und den lipophilen Schwanz bilden die Salze der Fettsäuren Micellen und können so Schmutz und Fettpartikel einschließen

![ggg](https://raw.githubusercontent.com/deltorobarba/repo/master/sciences_262.png)


**Eigenschaften von Seifen**

* Die Fettsäuren werden mit einem modernen Hochdruckverfahren hergestellt: Fett und überhitzter Wasserdampf reagieren bei Temperaturen von 100-250°C und einem Druck von 60 bar zu den Fettsäuren und Glycerin. Nach Abtrennung des Glycerins liegen die Ionen vor.

Die Eigenschaften der Seife werden durch den Bau des amphiphilen Seifenanions (mit dem hydrophoben Alkylrest und der hydrophilen Carboxylatgruppe) bestimmt:

* Herabsetzung der Oberflächenspannung und damit besseres Benetzungsvermögen

* Mizellenbildung

* Ablösen einer Fettverschmutzung von einer Faser

Die Umnetzung mit Seifenanionen führt dazu, dass die wasserunlöslichen Schmutzpartikel als Kerne einer Mizelle in Lösung gebracht werden. Die nach außen gerichteten negativen Ladungen verhindern durch elektrostatische Abstoßung, dass sich in Lösung gebrachte Schmutzpartikel wieder zusammenlagern oder erneut auf die Textilfasern aufziehen.

![ggg](https://raw.githubusercontent.com/deltorobarba/repo/master/sciences_152.jpg)

**Nachteile von Seifen**

![ggg](https://raw.githubusercontent.com/deltorobarba/repo/master/sciences_153.jpg)

###### *Synthetische Tenside & Waschmittel*

**Synthetische Tenside bzw. Waschmittel**

**Synthetische Waschmittel: Anionische Tenside**

* Beispiele für anionische Tenside ist die Gruppe des Alkylbenzolsulfonats (ABS), die sehr schwache Basen sind. Sie besitzen zwischen acht und sechzehn Methylgruppen.

* Die Vorteile dieses Tensids sind, dass der pH-Wert neutral bleibt, sich keine schwerlöslichen Kalkseifen bilden und es eine Waschaktivität auch im sauren Bereich gibt.

Aufgrund der Nachteile von Seifen (aus der Verseifung) **wurden nach dem Strukturprinzip der Seifen eine Reihe von synthetischen Tensiden mit unpolarem Alkylrest und polarer Endgruppe entwickelt**.

* Neben den klassischen, anionischen Tensiden kennt man heute kationische, nichtionische und zwitterionische Tenside.

* Diese haben gegenüber den klassischen Seifen den Vorteil, dass sie mit Ca2+- und Mg2+-Ionen keine schwer löslichen Salze bilden.

**Differenzierung der Tenside**

* Alle Tenside sind aus einem unpolaren und einem polaren Teil (funktionelle Gruppen) aufgebaut (siehe Polarität). Als unpolarer Teil dient meist eine [Alkylgruppe](https://de.m.wikipedia.org/wiki/Alkylgruppe) oder eine [Alkylbenzolgruppe](https://de.m.wikipedia.org/wiki/Alkylbenzole).

![ggg](https://raw.githubusercontent.com/deltorobarba/repo/master/sciences_268.png)

* Der polare Teil kann verschieden aufgebaut sein (siehe Tabelle):

![ggg](https://raw.githubusercontent.com/deltorobarba/repo/master/sciences_154.jpg)

**Zusammensetzung moderner Waschmittel**

* Waschvorgang ist ein komplizierter Prozess.

  * **Die Verschmutzungen der Wäsche können sehr unterschiedlicher Natur sein und wie Eiweiße von Tensiden nur schlecht gelöst werden**.

  * Auch die Textilfasern Wolle, Cellulose und Kunstfasern haben unterschiedliche Strukturen und Eigenschaften.

  * Daneben hängt das Waschergebnis von der Härte des Wassers und einigen optischen Effekten wie der Stabilität des Wäschefarbstoffs oder der Weißtönung der Wäsche ab.

* Waschmittel sind daher ein komplexes Stoffgemisch, wobei sich die **Zusammensetzung von Universal- und Spezialwaschmitteln unterscheidet**.

> **Als waschaktive Substanzen sind 15 -25 % anionische und nichtionische Tenside enthalten.**

* Klassische **anionische Tenside**

  * sind z.B. Schwefelsäuremonoalkylester, die sich wie die Seifen ebenfalls aus Fettsäuren gewinnen lassen. Durch Reduktion entstehen zunächst Fettalkohole, die sich unter Einwirkung von Schwefelsäure verestern lassen.

  * Mit Natronlauge erhält man daraus neutrale Monoalkylsulfate mit hoher Waschkraft, die mit Ca2+- und Mg2t-lonen kaum schwer lösliche Salze bilden.

* **Nichtionische Tenside**

  * sind noch weniger härteempfindlich und hautfreundlicher.

  * Moderne Vertreter wie [**Alkylpolyglucoside**](https://de.m.wikipedia.org/wiki/Alkylpolyglycoside) (Plantaren®) sind die waschaktiven Verbindungen der nächsten Generation: Sie werden aus nachwachsenden Rohstoffen (Fettalkoholen und Kohlenhydraten) hergestellt, sind nicht toxisch **und vollständig biologisch abbaubar**.

* **Kationische Tenside**:

  * zum Beispiel: Alkyl Quats wie [Benzalkoniumchlorid](https://de.m.wikipedia.org/wiki/Benzalkoniumchlorid)

  * Als kationische Tenside bezeichnet man Tenside, die eine **positiv geladene funktionelle Gruppe** besitzen.

  * Wie jedes Tensid sind auch die kationischen Tenside aus einem polare und einem unpolaren Teil aufgebaut. Die polare Gruppe ist immer eine **quarternäre Ammonium-Einheit**. Der unpolare Teile besteht aus verschiedenen Alkylresten.

  * In Waschmitteln werden kationische Tenside zusätzlich zu anionischen Tensiden als **Reinigungsverstärker** eingesetzt. Sie haben besonders hervorragende fettlösende Eigenschaften.

  *  Anders als anionische Tenside können kationische Tenside dabei auch in **sauren Milieus und hartem Wasser verwendet** werden, ihre Reinigungskraft ist jedoch schwächer.

  * In Weichspülern kommen sie als **Antistatika und Avivagemittel** (Nachbehandlung von gefärbter Seide, um Griff und Glanz zu verbessern) zum Einsatz.

  * Einige kationische Tenside haben biozide Eigenschaften und werden in **Reinigungsmitteln** zur Desinfektion eingesetzt.



**Der zweite Hauptbestandteil der Waschmittel mit einem Anteil von bis zu 30 % sind Wasserenthärter bzw. Builder**.

  * Ein multifunktioneller Enthärter ist Pentanatriumtriphosphat. Es bildet mit Caz+- und Mg2+-lonen stabile Chelatkomplexe, stellt einen günstigen pH-Wert von 9,5 in der Waschlauge ein, unterstützt die Schmutzablösung bei polarem Schmutz und **erleichtert das Ausspülen der Wäsche**.

  * Phosphate sind jedoch nicht unbegrenzt als Rohstoffe verfügbar. Außerdem belasten phosphathaltige Waschlaugen die Gewässer, da sie zur [Eutrophierung](https://de.m.wikipedia.org/wiki/Eutrophierung) (Anreicherung von Nährstoffen in einem Ökosystem) beitragen.

  * Deshalb werden heute in Waschmitteln hauptsächlich **Zeolithe** (z.B. Sasil®: Na12(A/02) 12 (SiO) 12 : 27 H20) zur Wasserenthärtung verwendet. Diese sind ökologisch unbedenklich, unterstützen aber, anders als das Pentanatriumtriphosphat, nicht die Schmutzablösung.

![ggg](https://raw.githubusercontent.com/deltorobarba/repo/master/sciences_270.png)

**Bleichmittel**

> Wäscheschmutz besteht aus vielen verschiedenen Bestandteilen. Die Tenside in Waschmitteln können daher keine ausreichende Reinigung gewährleisten.

* Bleichmittel wir **Natriumperborat** entwickeln bei erhöhter Temperatur **Wasserstoffperoxid**.

  * Dieses oxidiert Verschmutzungen wie Obstflecken oder Tinte und wirkt der Vergrauung der Fasern entgegen.

  * **Tetraacetylethylendiamin** (TAED) ist ein Bleichmittelaktivator, der schon bei niedriger Temperatur das wirksame Bleichmittel Peressigsäure bildet. Die Wäsche wird dadurch schon bei unter 60°C sauber.

* **Natürliche oder modifizierte Enzyme wie Proteasen, Lipasen oder Cellulase** beschleunigen die Entfernung schwer löslicher Verschmutzungen von Eiweißen, Fetten und Kohlenhydraten. Die Enzyme sind aber nur bei Waschtemperaturen bis 60 °C wirksam.

* Neue biologisch abbaubare Polymere unterstützen die Dispersionswirkung der Tenside und tragen dazu bei, die erneute Anlagerung von Schmutz an die Faser zu verhindern. **Dadurch werden weniger anionische Tenside in den Waschmitteln benötigt**.

* Darüber hinaus enthalten Waschmittel eine Vielzahl weiterer Bestandteile, um neben hygienischer Sauberkeit auch für Farbbrillanz, Langlebigkeit und angenehme Griffigkeit der Wäsche zu sorgen.

**Petrochemische und oleochemische Synthesewege für Tenside**


![ggg](https://upload.wikimedia.org/wikipedia/commons/thumb/d/d1/Synthesewege-tenside.png/469px-Synthesewege-tenside.png)

Weitere Quellen:

https://schulstoff.org/ch/ch11/lipide-und-tenside.php

https://de.m.wikipedia.org/wiki/Tenside

http://www.unterrichtsmaterialien-chemie.uni-goettingen.de/material/11-12/V11-507.pdf

https://www.gym8-lehrplan.bayern.de/contentserv/3.1.neu/g8.de/id_27476.html

https://blog.donau-chemie-group.com/blog-posts/Tenside

https://www.chemie.de/lexikon/Tenside.html

https://www.chemie-schule.de/KnowHow/Tenside

http://www.wasser-wissen.de/abwasserlexikon/l/las.htm

https://www.chemie.de/lexikon/Anionische_Tenside.html

https://de.m.wikipedia.org/wiki/Alkylbenzolsulfonate

##### <font color="blue">*Materialwissenschaften (Werkstoffe)*

###### *Material Sciences & Conventional Materials (Werkstoffe)*

*Festkörperphysik & Festkörperchemie*

https://de.m.wikipedia.org/wiki/Festkörperphysik

https://de.m.wikipedia.org/wiki/Festkörperchemie

Eine scharfe Abgrenzung der Festkörperchemie von der Festkörperphysik und der Werkstoffwissenschaft sowie der Mineralogie, der Metallurgie und der Kristallographie ist nicht ohne weiteres möglich.

**Material Sciences** (Materialwissenschaft und Werkstofftechnik)

https://de.m.wikipedia.org/wiki/Materialwissenschaft_und_Werkstofftechnik

**Werkstoffe**

> **Conventional materials like metals, insulators, semiconductors, composites etc.**

https://de.m.wikipedia.org/wiki/Werkstoff

> Werkstoffe sind feste Materialien wie Holz, Stein, Metalle oder Kunststoffe, die zur handwerklichen oder industriellen Herstellung von Gegenständen oder Bauwerken genutzt werden.

* Die Orientierung wird durch Einteilung in Werkstoffklassen erleichtert.

* Heute geht es vorrangig darum, Werkstoffe so fortzuentwickeln, dass sie den speziellen Anforderungen der Ingenieure und Verbraucher entsprechen und gleichzeitig möglichst umweltschonend in Produktion und Nutzung sind.

* Dabei spielt die chemische Modifikation synthetischer und natürlicher Werkstoffe eine zentrale Rolle.

![ggg](https://raw.githubusercontent.com/deltorobarba/repo/master/sciences_296.png)

**Verbundwerkstoffe (Composites)**

https://de.m.wikipedia.org/wiki/Verbundwerkstoff

https://de.m.wikipedia.org/wiki/Faserverbundwerkstoff

**Nichtleiter (Insulators)**

https://de.m.wikipedia.org/wiki/Nichtleiter

* Viele Stoffe sind Nichtleiter, einer der bekanntesten Vertreter ist reiner **Kohlenstoff** in der Modifikation Diamant. Aber auch zahlreiche Kohlenstoffverbindungen zählen zu den Nichtleitern, beispielsweise Bernstein oder verschiedene **Kunststoffe**. Letztere werden unter anderem für die Isolation von Kabeln oder für Gehäuse verwendet. Weitere Nichtleiter sind **Keramikwerkstoffe, Glas oder auch Silikone.**

* **Nichtionisierte, trockene Gase, wie Argon, Sauerstoff oder auch normale trockene Luft**, sind ebenfalls Nichtleiter. Generell ist die Anwesenheit von Wasser für viele natürliche Stoffe bzw. Stoffgemische (z. B. **Holz**), die den elektrischen Strom von sich aus nicht bedeutend leiten, dafür verantwortlich, dass diese zum Leiter werden. Denn destilliertes oder deionisiertes Wasser gilt zwar als Isolator, da aber immer einige Wassermoleküle dissoziiert sind, stehen Ionen zur Verfügung, die den elektrischen Strom leiten und Wasser zu einem schlechten Isolator machen. Bei normalem Leitungswasser oder Wasser in Seen kommen noch die gelösten Salze (Metall- und Nichtmetallionen) usw. dazu. Diese erhöhen die Leitfähigkeit enorm und machen Wasser dadurch zu einem Leiter.

* **Salze im festen Zustand sind – trotz ihres Ionenaufbaus – meist Nichtleiter**. Die Bindungskräfte zwischen den Ionen sind zu groß, als dass sich genug Ionen ausreichend frei bewegen könnten. Werden Salze geschmolzen, ändert sich das. Die Ionen sind nun nicht mehr so fest an ihren Nachbarionen gebunden und so können Salzschmelzen den elektrischen Strom durch Ionenleitung gut transportieren.

**Halbleiter (Semiconductors)**

* https://studyflix.de/elektrotechnik/halbleiter-2138

* https://de.m.wikipedia.org/wiki/Halbleiter

* Elementhalbleiter Silicium und Germanium

* https://de.m.wikipedia.org/wiki/Halbmetalle
Galliumnitrid

* https://de.m.wikipedia.org/wiki/III-V-Verbindungshalbleiter

***Exkurs: Warum eignet sich Silizium gut als Halbleiter?***

Silicium ist ein klassischer [Halbleiter](https://www.halbleiter.org/waferherstellung/silicium/), seine Leitfähigkeit liegt also zwischen der von Leitern und Nichtleitern.

Funktionsweise von elektrischen Halbleitern

https://www.net4energy.com/wiki/halbleiter

* Einfluss auf elektrische Halbleiter kann zum Beispiel eine steigende Temperatur nehmen. Die Elektronen fangen an zu schwingen, ab und zu löst sich eines aus seiner Bindung zum Atom. An der Stelle, wo sich das Elektron befand, entsteht eine positiv geladene Lücke. Wandert ein benachbartes Elektron in diese Lücke, führt dies im Endeffekt zu einem Stromfluss, da sich die positive Ladung weiterbewegt.

* **Die Leitfähigkeit von elektrischen Halbleitern beruht sowohl auf negativen als auch positiven Ladungsträgern.** Bei Erzeugung eines elektrischen Felds wandern die negativ geladenen Elektronen zum Pluspol und die positiv geladenen Löcher beziehungsweise Defektelektronen zum Minuspol. Elektrische Halbleiter sind vor allem aufgrund der sogenannten Dotierung bedeutsam für die Elektrotechnik. Dabei werden Fremdatome in kleiner Menge in den elektrischen Halbleiter eingebracht. Diese bringen entweder zusätzliche Elektronen oder Löcher mit ein.

* Werden Elektronen beziehungsweise Donatoren eingebracht, ergibt sich eine n-dotierte Halbleiterstruktur. In Fällen, in denen Löcher beziehungsweise Akzeptoren zugefügt werden, resultiert eine p-Dotierung. Als Beispiel für einen Halbleiter sei der Stoff Silizium angeführt. Silizium besitzt vier Außen- beziehungsweise Valenzelektronen. Deshalb kann sich ein Kristallgitter mit einer kovalenten Bindung bilden.

* **Werden nun zum Beispiel Phosphor-Atome zugefügt, die fünf Außenelektronen besitzen, bleibt ein freies Elektron übrig. Die Leitfähigkeit des Siliziums wird dadurch erhöht.** Elektrische Halbleiter sind die Grundlage für zahlreiche Bauelemente, zum Beispiel für Transistoren und Dioden. Neben

* **Silizium gibt es weitere Halbleitermaterialien wie Germanium und Galliumarsenid**.

###### *Intelligente Werkstoffe (Smart / Responsive Materials)*

**Smart Materials (Responsive Materials)**

Unlike conventional materials like metals, insulators, semiconductors, composites etc., the smart materials are designed properties that can be engineered in several ways. Metamaterials appear as a class of smart materials, which are fabricated from assemblies of multiple elements from composite materials, such as metals or plastics metamaterials might exhibit specific electro-magnetic or acoustic properties, not observed in the materials commonly found in nature.

https://aip.scitation.org/doi/abs/10.1063/1.5050728?journalCode=apc

https://en.m.wikipedia.org/wiki/Smart_material

https://de.m.wikipedia.org/wiki/Intelligenter_Werkstoff

https://materialsscience.peersalleyconferences.com/tracks/smart-hybrid-materials

###### *Hybrid material*

**Hybrid material**

https://www.uni-saarland.de/lehrstuhl/kickelbick/forschung/hybridmaterialien-und-nanokomposite.html

* [Hybrid materials](https://en.m.wikipedia.org/wiki/Hybrid_material) are composites consisting of two constituents at the nanometer or molecular level. Commonly one of these compounds is inorganic and the other one organic in nature.

* [Sol-Gel-Prozess](https://de.m.wikipedia.org/wiki/Sol-Gel-Prozess)

* Häufig mit Silanen: "An organic moiety containing a functional group that allows the attachment to an inorganic network, e.g. a trialkoxy**silane** group, can act as a network modifier because in the final structure the inorganic network is only modified by the organic group. Phenyltrialkoxy**silanes** are an example for such compounds; they modify the silica network in the sol–gel process via the reaction of the trialkoxy**silane** group"

* https://phys.org/news/2022-11-quantum-phase-hybrid-materials.amp

* https://www.elsevier.com/books/nickel-titanium-smart-hybrid-materials/thomas/978-0-323-91173-3


###### *Metamaterialien*

**Metamaterialien**

Metamaterialien
* Polarisation von Licht
* Für elektromagnetische Wellen und Schallwellen
* Brechung der geometrischen und zeitlichen Symmetrien
* Über Nanofabrikation
* Für bessere Kommunikation ohne Rauschen zB. Auch Bildgebung, Gebäudetechnik, Akustik, Mechanik, Elektronik

Metamaterialien sind künstlich hergestellte Werkstoffe mit optischen, elektrischen oder magnetischen Eigenschaften, die in der Natur nicht vorkommen. Ein spannendes Beispiel für ihre Anwendung sind sogenannte Tarnkappen: Man arbeitet an Metamaterialien, die Objekte unsichtbar machen. Dazu stellen Forscher den Brechungsindex des Tarnmaterials gerade so ein, dass es für das Licht günstiger ist, um das getarnte Objekt herumzulaufen anstatt vom ihm abgelenkt oder gestreut zu werden.

https://www.weltderphysik.de/gebiet/materie/metamaterialien/

https://de.m.wikipedia.org/wiki/Metamaterial

https://www.scinexx.de/news/physik/metamaterial-als-terahertz-booster/

machine learning for metamaterials
combinatorial mechanical metamaterials: https://techxplore.com/news/2022-11-floppy-ai-properties-complex-metamaterials.amp
https://techxplore.com/news/2022-05-physics-ai-insights-material-properties.html

https://www.spektrum.de/magazin/metamaterialien-tarnkappen-fuer-licht-und-schall/2099862


###### *Nanotechnologie (Nanocomposites)*

**Nanotechnologie (Nanocomposites)**

https://de.m.wikipedia.org/wiki/Nanotechnologie

https://de.m.wikipedia.org/wiki/Nanowissenschaften

https://de.m.wikipedia.org/wiki/Oberflächenchemie

https://en.m.wikipedia.org/wiki/Nanocomposite

https://phys.org/news/2022-06-topology-machine-reveal-hidden-relationship.html

https://phys.org/news/2022-11-low-cost-terahertz-camera.amp

![ggg](https://raw.githubusercontent.com/deltorobarba/repo/master/sciences_0795.jpeg)

![ggg](https://raw.githubusercontent.com/deltorobarba/repo/master/sciences_0796.jpeg)

###### *Smart Polymer*

**Smart Polymer**

https://en.m.wikipedia.org/wiki/Smart_polymer

https://en.m.wikipedia.org/wiki/Temperature-responsive_polymer

https://en.m.wikipedia.org/wiki/Coordination_polymer

https://phys.org/news/2022-11-material-heavy.html

https://en.m.wikipedia.org/wiki/Electroactive_polymer

###### *OLED (Organische Leuchtdiode)*

**OLED (Organische Leuchtdiode)**

https://de.m.wikipedia.org/wiki/Organische_Leuchtdiode

Blue OLED:

https://t3n.de/news/blaue-quantenpunkte-japanische-forschende-arbeiten-am-bildschirm-der-zukunft-1513087/amp/

###### *Self-healing materials*

**Self-healing materials**

https://en.m.wikipedia.org/wiki/Self-healing_material

https://www.golem.de/news/materialforschung-forscher-entwickeln-selbstheilenden-verbundwerkstoff-2211-169406.html

Forscher entwickeln selbstheilenden Verbundwerkstoff

Faserverstärkte Verbundwerkstoffe, aus denen etwa Windräder gefertigt werden, können sich auflösen. Forscher haben ein Verfahren entwickelt, mit dem sich die Werkstoffe selbst reparieren.

* Laminierte Verbundwerkstoffe bestehen aus Fasermatten, beispielsweise von Glas- und Kohlefasern, die durch einen Kunststoff verbunden sind. Schäden treten meist durch Delaminierung auf, wenn sich eine oder mehreren Schichten ablösen. Das Team von der North Carolina State University in Raleigh, dem Israel Institute of Technology in Haifa und der University of Houston hat ein Verfahren entwickelt, um das zu verhindern.

* Dazu wird auf eine Fasermatte ein thermoplastisches Heilmittel gedruckt. Zudem werden in den Verbundwerkstoff dünne Heizschichten eingebettet. Wird an die Heizschichten elektrischer Strom angelegt, erwärmen diese sich. Dadurch schmilzt das Heilmittel und schließt Risse oder kleine Schäden im Verbundwerkstoff.

* Wir suchen jetzt nach Partnern in der Regierung und der Industrie, die uns dabei helfen, diese polymerbasierten Verbundwerkstoffe für spezifische Anwendungen maßzuschneidern.

https://www.nature.com/articles/s41467-022-33936-z

![ggg](https://raw.githubusercontent.com/deltorobarba/repo/master/sciences_0797.jpeg)

###### *Thermoelectric Material*

**Thermoelectric Material**

https://en.wikipedia.org/wiki/Thermoelectric_materials

https://scitechdaily.com/scientists-create-crystals-that-generate-electricity-from-heat/amp/

###### *Smart Glass*

**Smart Glass**

https://techxplore.com/news/2022-11-smart-windows-polarize-sunlight-energy.html

https://physicsworld.com/a/black-holes-could-reveal-their-quantum-superposition-states-new-calculations-reveal/

* Traditional lenses, like contact lenses or those found in magnifying glasses and microscopes, bend rays of light coming from different directions by altering their phase so that they hit the same location in space—the focal point.

* Time lenses, on the other hand, "bend" light beams in similar ways—but they **alter the phase of light beams in time rather than space**. In this way, different colors of light, which travel at different speeds, are re-timed so that they each hit the focal plane at the same time.

###### *Liquid-crystal display (LCD)*

**Liquid-crystal display (LCD)**

https://en.m.wikipedia.org/wiki/Liquid-crystal_display

###### *Flüssigkristallpolymer (Polymer Liquids)*

**Flüssigkristallpolymer (Polymer Liquids)**

https://de.m.wikipedia.org/wiki/Flüssigkristallpolymer

https://phys.org/news/2022-11-experimental-validates-theory-molecular-diffusion.amp

###### *Topological Materials (Topological Insulator)*

**Topological Materials (Topological Insulator)**

https://de.m.wikipedia.org/wiki/Topologischer_Isolator

The most interesting topological materials are topological insulators, which are materials that are insulating in the bulk, but conducting on the surface. https://physicsworld.com/a/a-periodic-table-for-topological-materials/

*topological quantum matter*

https://phys.org/news/2022-11-boundaries-topological-quantum.amp

https://en.wikipedia.org/wiki/Topological_order

https://www.edx.org/course/topological-states-of-matter

###### *Tetrataenit*

**Tetrataenit**

https://m.winfuture.de/amp/news/133017

https://de.wikipedia.org/wiki/Tetrataenit

###### *Quantum*

https://www.slashgear.com/1094648/bosch-deploys-quantum-computers-to-find-alternative-ev-materials/

###### *Spin Glass*

https://de.m.wikipedia.org/wiki/Spin-Glas

https://www.forbes.com/sites/chadorzel/2021/10/06/whats-a-spin-glass-and-why-does-it-matter-the-nobel-prize-in-physics-2021/

https://phys.org/news/2022-11-mathematics-dont-behavior-glasses.amp

##### <font color="blue">*Silicium & Siliciumverbindungen (Silicone, Silicate, Sand, Gläser & Keramiken) - Non-Organic*

**Silicium**

* [Silicium](https://de.m.wikipedia.org/wiki/Silicium) chemisches Element mit dem Symbol Si und der Ordungszahl 14

* Reines Silicium kann nur industriell hergestellt werden und ist von großer Bedeutung als Halbleiter in elektronischen Bauelementen und in der Solartechnik.

* Elementares Silicium findet in unterschiedlichen Reinheitsgraden Verwendung in der Metallurgie (Ferrosilicium), der Photovoltaik (Solarzellen) und in der [Mikroelektronik](https://de.m.wikipedia.org/wiki/Mikroelektronik) (Halbleiter, [Computerchips](https://de.m.wikipedia.org/wiki/Integrierter_Schaltkreis)).


  * Demgemäß ist es in der Wirtschaft gebräuchlich, elementares Silicium anhand unterschiedlicher Reinheitsgrade zu klassifizieren:

    * **Simg** (metallurgical grade, Rohsilicium, 98–99 % Reinheit),

    * **Sisg** (solar grade, Solarsilicium, Verunreinigungen kleiner 0,01 %)

    * **Sieg** (electronic grade, Halbleitersilicium, Verunreinigungen kleiner 10−9).

  * Für Solarzellen ist die Reinheit des Materials in seiner gesamten Stärke wichtig, um eine möglichst lange Ladungsträger-Lebensdauer zu gewährleisten, für viele Anwendungen in der Mikroelektronik müssen nur die oberen Schichten von etwa 20 bis 30 µm hochrein sein.

* Silicium ist ein klassischer [Halbleiter](https://www.halbleiter.org/waferherstellung/silicium/), seine Leitfähigkeit liegt also zwischen der von Leitern und Nichtleitern.




**Siliciumverbindungen**

* [Silicium und Siliciumverbindungen](https://www.lernhelfer.de/schuelerlexikon/chemie-abitur/artikel/silicium-und-siliciumverbindungen#)

* Allerdings kommt es nur in Form von Siliciumdioxid und davon abgeleiteten Verbindungen, z. B. den **Silicaten**, vor. Viele Gesteine und Mineralien wie **Quarz**, Feldspat oder Tonerden sind Silciumverbindungen.

* Siliciumdioxid und Silicate sind die Hauptbestandteile von **Glas**, finden aber auch andere vielfältige Anwendungen, z. B. als Schleifmittel oder technische Katalysatoren.

* Auch leistungsfähige **keramische** Werkstoffe sind Verbindungen des Siliciums

* Und die als Dichtungsmaterial bekannten **Silicone** sind Verbindungen des Siliciums. **Silikone** sind eine Klasse von chemischen Verbindungen des Siliciums: Synthetische Polymere auf anderer Basis: **Auch das Silizium ist in der Lage, stabile Verbindungen mit sich selbst einzugehen. Dabei entstehen Silikone.**

**Kieselsäuren**

* Als [Kieselsäuren](https://de.m.wikipedia.org/wiki/Kieselsäuren) werden die Sauerstoffsäuren des Siliciums bezeichnet.

* Im deutschen Sprachraum wird für Siliciumdioxid statt der korrekten Bezeichnung pyrogenes Siliciumdioxid oder gefälltes Siliciumdioxid die Bezeichnung Kieselsäure benutzt oder in letzter Zeit auch das aus dem Englischen übernommene Silica.

**Siliciumdioxid**

* [Siliciumdioxid](https://de.m.wikipedia.org/wiki/Siliciumdioxid) ist ein Oxid des Siliciums mit der Summenformel SiO2.

* Siliciumdioxid bildet als Teil von **Silicaten** wie z. B. [Feldspat](https://de.m.wikipedia.org/wiki/Feldspat), [Tonmineralen](https://de.m.wikipedia.org/wiki/Tonminerale) oder in freier Form als Quarz den Hauptbestandteil der Erdkruste und somit auch die häufigste Siliciumverbindung.

* $\hookrightarrow$ [Quarz](https://de.m.wikipedia.org/wiki/Quarz): gesteinsbildendes Mineral, Siliciumdioxid in freier Form

* $\hookrightarrow$ [Sand](https://de.m.wikipedia.org/wiki/Sand): Der Großteil der weltweiten Sandvorkommen besteht aus Siliciumdioxid (Quarz), denn dieses ist in der oberen kontinentalen Erdkruste nicht nur häufig, sondern aufgrund seiner Härte und seiner chemischen Widerständigkeit auch besonders verwitterungsbeständig. Siliciumdioxid ist der Hauptbestandteil aller Quarz-Gläser.

**Silicate**

* [Silicate](https://de.m.wikipedia.org/wiki/Silicate) sind Salze und Ester der Ortho-Kieselsäure. Siliciumdioxid ist Teil von Silicaten!

* $\hookrightarrow$ [Glas](https://de.m.wikipedia.org/wiki/Glas): Im Alltagsgebrauch bezeichnet Glas meist transparente Silikatgläser wie Trink- und Fenstergläser, die überwiegend aus Quarzsand bzw. Siliciumdioxid bestehen und zu den anorganischen Gläsern gehören.

* $\hookrightarrow$ [Keramik](https://de.m.wikipedia.org/wiki/Keramik#Silikat-Rohstoffe)

* $\hookrightarrow$ [Ton](https://de.m.wikipedia.org/wiki/Tonminerale)

* $\hookrightarrow$ [Feldspat](https://de.m.wikipedia.org/wiki/Feldspat)

* [Sanidin](https://de.m.wikipedia.org/wiki/Sanidin) ist ein häufig vorkommendes Mineral aus der Mineralklasse der „Silikate und Germanate“.


**Silicone**

* Not to be confused with the chemical element silicon! Silicone nicht verwechseln mit English "Silicon", was im deutschen "Silizium" heisst (Halbleiter-Element, siehe oben)

* [Silicone](https://www.lernhelfer.de/schuelerlexikon/chemie/artikel/silicone) sind **siliciumorganische Kunststoffe**, in denen die Siliciumatome noch organische Reste tragen. Sie ähneln in ihrer Struktur organisch modifiziertem Quarz, weisen eine ähnliche Beständigkeit auf und haben zudem die Flexibilität von Kunststoffen.

* [Silikone](https://de.m.wikipedia.org/wiki/Silikone): Gruppe synthetischer Polymere, bei denen Siliciumatome über Sauerstoffatome verknüpft sind

* Produktklassen: Silikonflüssigkeit und Silikonfett, Silikonkautschuk und Silikonelastomere, Silikonharz, Fluorsilikone und Hochtransparente Silikone

**Besondere Anwendungen**

* https://www.spektrum.de/magazin/ideales-glas-die-perfekte-unordnung/2066868

* https://www.scinexx.de/news/physik/glas-verbluefft-physiker/

**Silane**

* [Silane](https://de.m.wikipedia.org/wiki/Silane) sind eine chemische Stoffgruppe von Silizium und Wasserstoff

* Finden Anwendung zB im [Sol-Gel-Prozess](https://de.m.wikipedia.org/wiki/Sol-Gel-Prozess) fur Hybrid Materials

##### <font color="blue">*Metallorganische Chemie*

* [Metallorganische Chemie](https://de.m.wikipedia.org/wiki/Metallorganische_Chemie): 

  * [Ferrocen](https://de.m.wikipedia.org/wiki/Ferrocen), ein klassischer Vertreter der metallorganischen Chemie. Halbstrukturformel [Fe(C5H5)2]. Ferrocen und seine Derivate finden Anwendung in Bereichen wie der Sensorik, der Katalyse und der Medizin und sie lassen sich zu [Polyferrocenen](https://de.m.wikipedia.org/wiki/Polyferrocene) polymerisieren, einer Klasse von Ferrocen enthaltenden, temperaturstabilen Polymeren.

![ggg](https://upload.wikimedia.org/wikipedia/commons/thumb/e/e9/Ferrocene.svg/103px-Ferrocene.svg.png)

* [Biometallorganische Chemie](https://de.m.wikipedia.org/wiki/Biometallorganische_Chemie): ist ein Teilgebiet der Chemie, das sich mit der Untersuchung von biologisch aktiven Molekülen befasst, die eine direkte Kohlenstoff–Metall- oder Kohlenstoff–Halbmetallbindung aufweisen. Dabei kann es sich um klassische Übergangsmetallkomplexe mit Liganden wie Kohlenstoffmonoxid, Ethylen oder Cyclopentadien handeln oder Biomoleküle wie Aminosäuren, Zuckern oder Antikörpern, die eine direkte Kohlenstoff-Metallbindung aufweisen und eine Rolle bei biologischen Prozessen spielen.

  * [Vitamin B12 (Cobalamine)](https://de.m.wikipedia.org/wiki/Cobalamine): essentielle organometallische Verbindungen mit Cobaltatom

* https://phys.org/news/2022-11-chemists-artificial-photosynthesis-ten-efficient.amp

* https://de.m.wikipedia.org/wiki/Metallorganische_Gerüstverbindung